# SAM John McCormick Purchase and Scrap Report for: 
# P\&S Transportation


In [1]:
Enterprise_Number = '4990319'
Customer_Name = 'P&S Transportation'
Customer_Name_Underscores = 'P&S_Transportation'

# And for graphs, let's define the customer colors with variables:
customer_color_1 = '#005f9f'
customer_color_2 = '#A6ACAF'

# Create a template on BFI for customer and add the template name below:
bfi_website_saved_NRT_Report_Name_for_Webscrape = 'FOR_SAMS_P_AND_S_TRANSPORTATION_NRT_SUMMARY'
Customer_Logo_Image_Name = 'P_and_S_Transportation_logo.png'
Customer_Logo_Image_Name_for_pdf = 'P_and_S_Transportation_logo.png'

# Define list of vendors, vendor numbers, and list out the tire sizes manually:
vendor_names_list = ['Bridgestone', 'Continental', 'Yokohama']
vendor_numbers = ['007', '021', '005']

list_of_tire_sizes = ['295/75R22.5', '11R22.5']

# Input the pdsize codes corresponding to the list of tire sizes.
list_of_pdsize_codes = ['%022500295750%', '%022500011000%']

### Note!  If need help finding that correct size, you can just use this query below in the database to find the corresponding pdsize codes:
# -- JUST REPLACE PDDESCRIP = _____ with a tire size between the % signs below.

# SELECT P.PDDESCRIP, P.PDSIZE FROM TMSALE S
# INNER JOIN TMPROD P ON S.TSVN = P.PDVENDOR AND S.TSPC = P.PDCLASS AND S.TSPD = P.PDNUMBER AND S.TSLC = P.PDLOCATION
# WHERE PDDESCRIP LIKE '%215/85R16%'
# GROUP BY PDDESCRIP, PDSIZE


zipped_tire_sizes_and_pdsize_codes = list(zip(list_of_tire_sizes, list_of_pdsize_codes))
number_of_tire_sizes = len(list_of_tire_sizes)


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from pyzipcode import ZipCodeDatabase
zcdb = ZipCodeDatabase()

from shutil import copyfile
from splinter import Browser
from bs4 import BeautifulSoup as soup
from datetime import datetime
from shutil import copyfile
import os
import shutil
import time
import re
import plotly.graph_objects as go


# PLOTLY INTERACTIVE CREDS:
import chart_studio

# Get environment variables
username = os.getenv('plotly_chart_studio_username')
api_key = os.environ.get('plotly_chart_studio_api_key')

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
import chart_studio.plotly as py
import chart_studio.tools as tls


import warnings
warnings.filterwarnings('ignore')

# Before we pull the data, we will use the datetime module to create time-related variables for use in the queries later.

In [3]:
# ========================================================================================================================
# ========================================================================================================================
# ========================GET VARIABLES FOR CURRENT=======================================================================
# ========================================================================================================================
# ========================================================================================================================


# Define a variable as "now" which we'll use to determine specific current varaibles (Current_Year and Current_Month):
now = datetime.now()
print("now:", now)

# Get Current_Year in yyyy integer format:
Current_Year = int(now.strftime("%Y"))
print(f"Current_Year (actual): {Current_Year}")
# Get Current_Month in yyyymm integer format:
Current_Month = int(now.strftime("%Y%m"))
print(f"Current_Month (actual): {Current_Month}")
print('-----------')




# Get "Last_Year" in yyyy integer format:
Last_Year = Current_Year -1
print(f'Last_Year: {Last_Year}')
print('----------')

# Get Last_Month in yyyymm integer format, 
# Note, if the current month is January (yyyy01), we cannot just subtract yyyy01 - 1 to get Last_Month as yyyy12 because it will give us yyyy00
# So we'll perform a special calculation for only when the current month is January to get last month in yyyy12 format:
if f'{Current_Month}' == f'{Current_Year}01':
    Last_Month = Last_Year * 100 + 12
    print(f"Last_Month: {Last_Month}")
    print('----------')
# For any other month, we'll just subtract Current_Month - 1 to get Last_Month in yyyymm integer format
else:
    Last_Month = Current_Month - 1
    print(f"Last_Month: {Last_Month}")
    print('----------')


# Get last month's full name as a "Last_Month_Descriptive" so we can use it in graph titles, file names, and anything else we need.
# First we need to get the "Last_Month_One_or_Two_Digit" to determine where to split the month name string to get month.
list_for_month_digits = [['01', 12], ['02', 1], ['03', 2], ['04', 3], ['05', 4], ['06', 5], ['07', 6], ['08', 7], ['09', 8], ['10', 9], ['11', 10], ['12', 11]]
for current_month_two_digit_string, last_month_one_or_two_digit_integer in list_for_month_digits:
    if f'{Current_Month}' == f'{Current_Year}{current_month_two_digit_string}':
        Last_Month_One_or_Two_Digit = last_month_one_or_two_digit_integer
        print(f'Last_Month_One_or_Two_Digit: {Last_Month_One_or_Two_Digit}')

# Note: With python indexing starting a 0, we'll need to subtract 1 from the Last_month_One_or_Two_digit variable to achieve the desired name. 
Last_Month_Descriptive = "January February March April May June July August September October November December".split()[Last_Month_One_or_Two_Digit-1]
print(f'Last_Month_Descriptive: {Last_Month_Descriptive}')
print('----------')

# We'll also create a variable for shorter Month name descriptions.  
# For example, we'll use this for the first 'banner' title of the home page of the website and pdf:
Last_Month_Descriptive_Short = "JAN FEB MAR APR MAY JUN JUL AUG SEPT OCT NOV DEC".split()[Last_Month_One_or_Two_Digit-1]
print(f'Last_Month_Descriptive_Short: {Last_Month_Descriptive_Short}')

# Finally, create a variable to list all the month name abbreviations as strings:
Months_Strings = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]
print('-----------')


# =================================================================================================================================

# Now, ONLY if the month is January, we will redefine "Last_Year" and "Current_Year" as they will need to represent the two years
# PRIOR to what year we are in now.  This is because we'll only use completed months and otherwise there would be no data to display for the current year yet
if f'{Current_Month}' == f'{Current_Year}01':
    Current_Year = int(now.strftime("%Y")) - 1
    Last_Year = int(now.strftime("%Y")) - 2

print(f'Last_Year for queires: {Last_Year}')
print(f'Current_Year for queires: {Current_Year}')



# Depending on the chosen Year in the format YYYY, we will create variables for each month accordingly.
# For example, if we had selected Current_Year = 2021, Jan will become 202101.  This is useful because the
# Maddenco database has the dates formatted in this manner YYYYMM and we will use these to determine if the month has completed or not.
# If it has, we will collect data, if not, we will take other actions.

Jan = Current_Year * 100 + 1
Feb = Current_Year * 100 + 2
Mar = Current_Year * 100 + 3
Apr = Current_Year * 100 + 4
May = Current_Year * 100 + 5
Jun = Current_Year * 100 + 6
Jul = Current_Year * 100 + 7
Aug = Current_Year * 100 + 8
Sept = Current_Year * 100 + 9
Oct = Current_Year * 100 + 10
Nov = Current_Year * 100 + 11
Dec = Current_Year * 100 + 12

# We can also define a list of these numerical Months:
# Again, these will be in format like Jan = 202101.
Months_Current_Year = [Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sept, Oct, Nov, Dec]
print("Months:")
for x in Months_Current_Year:
    print(f'    ', x)
print("---------")

# Let's also do this for last year dates if needed:
Jan_Last_Year = Last_Year * 100 + 1
Feb_Last_Year = Last_Year * 100 + 2
Mar_Last_Year = Last_Year * 100 + 3
Apr_Last_Year = Last_Year * 100 + 4
May_Last_Year = Last_Year * 100 + 5
Jun_Last_Year = Last_Year * 100 + 6
Jul_Last_Year = Last_Year * 100 + 7
Aug_Last_Year = Last_Year * 100 + 8
Sept_Last_Year = Last_Year * 100 + 9
Oct_Last_Year = Last_Year * 100 + 10
Nov_Last_Year = Last_Year * 100 + 11
Dec_Last_Year = Last_Year * 100 + 12

now: 2022-01-14 11:24:21.054451
Current_Year (actual): 2022
Current_Month (actual): 202201
-----------
Last_Year: 2021
----------
Last_Month: 202112
----------
Last_Month_One_or_Two_Digit: 12
Last_Month_Descriptive: December
----------
Last_Month_Descriptive_Short: DEC
-----------
Last_Year for queires: 2020
Current_Year for queires: 2021
Months:
     202101
     202102
     202103
     202104
     202105
     202106
     202107
     202108
     202109
     202110
     202111
     202112
---------


# HERE WE WILL CREATE ALL THE NECESSARY BLANK HTML FILES.

## We will add hmtl to each of these blank files as we go along to create each page

In [4]:
# Create all the blank html files, each of which we will add to as we go along.
# Note, if you run this notebook out of order or if the files already exist in the current folder from being run and created in the past, 
# when you run this cell each will be overwritten as blank:

# HOME PAGE
with open('index.html', 'w') as new_file:
    new_file.write('')

# ----------------------------------------------------------------------------------------------------------
# SALES BY STORES
# Here, we'll have a dropdown... and the following items will become list items in the html header

# Now the these pages under the dropdown begin:
# First dropdown item
with open('Service_Sales_in_Units_by_Store.html', 'w') as new_file:
    new_file.write('')
# Second
with open('Service_Sales_in_Dollars_by_Store.html', 'w') as new_file:
    new_file.write('')
# Third
with open('Retread_Sales_by_Store.html', 'w') as new_file:
    new_file.write('')
# Fourth
with open('Cap_and_Casing_Sales_by_Store.html', 'w') as new_file:
    new_file.write('')

for vendor_name in vendor_names_list:    
# depending on how many vendors we'll look at, we'll add more down here:
# Fifth, Sixth... depending on how many vendors were specified
    with open(f'{vendor_name}_New_Tires_Sales_by_Store.html', 'w') as new_file:
        new_file.write('')


# ----------------------------------------------------------------------------------------------------------
# Second Drop down menu on website will have these 2 pages:

# SALES BY TIRE SIZE
# If we just wanted one table, we could 
with open('Sales_Table_by_Tire_Size_All_Vendors.html', 'w') as new_file:
    new_file.write('')

# # There will also be an extra dropdown because we'll make an extra table JUST for vendors by size!
with open('Sales_Table_by_Tire_Size_by_Vendor.html', 'w') as new_file:
    new_file.write('')

# ----------------------------------------------------------------------------------------------------------
# These next 3 html pages will each be the pages with graphs:
with open('Sales_Breakdown_Column_Charts.html', 'w') as new_file:
    new_file.write('')

with open('Trend_Analysis_Line_Charts.html', 'w') as new_file:
    new_file.write('')

with open('BFI_Scraps_Analysis.html', 'w') as new_file:
    new_file.write('')

# Finally we'll add a downloads page:
with open('Data_Downloads.html', 'w') as new_file:
    new_file.write('')


# And we'll only use this html page to generate a pdf off of it:
with open('pdf.html', 'w') as new_file:
    new_file.write('')


# Build the HEADER string to setup the nav bars for every webpage

In [5]:
# For our first dropdown on the website, we'll need to add specified vendors as list items.
# So we'll do this dynamically.  For example, if the specified vendors are Bridgestone and Continental, this code would
# auto-add those two list items into the string_for_header string as dropdown items.

string_for_each_vendor_to_add_as_dropdown_list_items = ''

for index, each in enumerate(vendor_names_list):
    string_for_each_vendor_to_add_as_dropdown_list_items = string_for_each_vendor_to_add_as_dropdown_list_items + f'<li><a href="{vendor_names_list[index]}_New_Tires_Sales_by_Store.html">{vendor_names_list[index]} Tires Sold (Units)</a><li>'



# Now create a string which we'll use as the header for every page's nav bar of the website.

string_for_header = f"""
<header>
        <nav>
            <ul>
                <li><a href="index.html">Home</a></li>
                
                <li><a href="#">Sales by Store</a>
                    <ul>
                        <li><a href="Service_Sales_in_Units_by_Store.html">Services Sold (Units)</a></li>
                        <li><a href="Service_Sales_in_Dollars_by_Store.html">Services Sold (Dollars)</a></li>
                        <li><a href="Retread_Sales_by_Store.html">Retreads Sold (Units)</a><li>
                        <li><a href="Cap_and_Casing_Sales_by_Store.html">Cap+Casings Sold (Units)</a><li>
                        {string_for_each_vendor_to_add_as_dropdown_list_items}
                    </ul>
                </li>
                
                <li><a href="#">Sales by Tire Size</a>
                    <ul>
                        <li><a href="Sales_Table_by_Tire_Size_All_Vendors.html">All Vendors</a></li>
                        <li><a href="Sales_Table_by_Tire_Size_by_Vendor.html">Specified Vendors</a></li>
                    </ul>
                </li>
                <li><a href="Sales_Breakdown_Column_Charts.html">Sales Breakdown</a></li>
                <li><a href="Trend_Analysis_Line_Charts.html">Sales Trends Analysis</a></li>
                <li><a href="BFI_Scraps_Analysis.html">BFI Scraps Trend Analysis</a></li>
                <li><a href="Data_Downloads.html">Data Downloads</a></li>
            </ul>
        </nav>
    </header>
"""

## Dymanically build the bottom portion of the home page which displays the vendors and bandag logo:

In [6]:
string_for_vendors_and_bandag_logo_display_on_home_page = ''

if len(vendor_names_list) <= 1:

    string_for_vendors_and_bandag_logo_display_on_home_page = string_for_vendors_and_bandag_logo_display_on_home_page + f"""
            <div class = "vendors_and_bandag_logo_container_for_website">
                <img class= "{vendor_names_list[0]}_logo_for_website" src= "./Images_logos/{vendor_names_list[0]}_logo.png">
                <img class= "bandag_logo_for_website_IF-ODD-NUMBER-OF-VENDORS" src= "./Images_logos/Bandag_logo.svg">
            </div>
            """
elif len(vendor_names_list) == 2:
        string_for_vendors_and_bandag_logo_display_on_home_page = string_for_vendors_and_bandag_logo_display_on_home_page + f"""
    <div class = "vendors_and_bandag_logo_container_for_website">
    <img class= "{vendor_names_list[0]}_logo_for_website" src= "./Images_logos/{vendor_names_list[0]}_logo.png">
    <img class= "{vendor_names_list[1]}_logo_for_website" src= "./Images_logos/{vendor_names_list[1]}_logo.png">
    </div>
    <div class = "vendors_and_bandag_logo_container_for_website">
    <img class= "bandag_logo_for_website" src= "./Images_logos/Bandag_logo.svg">
    </div>
    """
elif len(vendor_names_list) == 3:
        string_for_vendors_and_bandag_logo_display_on_home_page = string_for_vendors_and_bandag_logo_display_on_home_page + f"""
    <div class = "vendors_and_bandag_logo_container_for_website">
    <img class= "{vendor_names_list[0]}_logo_for_website" src= "./Images_logos/{vendor_names_list[0]}_logo.png">
    <img class= "{vendor_names_list[1]}_logo_for_Website" src= "./Images_logos/{vendor_names_list[1]}_logo.png">
    </div>
    <div class = "vendors_and_bandag_logo_container_for_website">
    <img class= "{vendor_names_list[2]}_logo_for_Website" src= "./Images_logos/{vendor_names_list[2]}_logo.png">
    <img class= "bandag_logo_for_website_IF-ODD-NUMBER-OF-VENDORS" src= "./Images_logos/Bandag_logo.svg">
    </div>
    """





In [7]:
# Build Home Page formatted for the website)
# This will mostly just be to display which customer we're dealing with but also to add logos, description of the report at hand etc.

Create_html = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>



<body class = 'container'>
{string_for_header}
    <!--------------------------------------HOME PAGE----------------------------------------------------------------->

    <br>
    <br>
    <br>
    <!-- STM LOGO -->
    <div class="STM_logo_for_website">
        <img src=".\Images_logos\STM_logo.svg">
    </div>
    <br>
    <br>
    <br>
    <br>
    <!--YEAR TO DATE BANNER text-->
    <h1 class="first_banner_text_for_website">YEAR TO DATE - {Last_Month_Descriptive_Short} {Current_Year}</h1>
    <br>
    <br>
    <br>
    <br>
    <br>
    <!-- Customer LOGO -->
    <div class="Customer_logo_image_for_website">
        <img src='./Images_logos/{Customer_Logo_Image_Name}'>
    </div>
    <br>
    <br>
    <!--  Second Blue Banner Text -->
    <h1 class="second_banner_text_for_website">PURCHASE AND SCRAP PROFILE</h1>
    <br>
    <!--  VENDORS AND BANDAG LOGOS-->
{string_for_vendors_and_bandag_logo_display_on_home_page}
</body>

</html>

"""
# ------------------------------------------------------------------------------------------------------------------------------
with open('index.html', 'w') as f:
    f.write(Create_html)

In [8]:
# Connect to DB2 Databse:
import pyodbc
cnxn = pyodbc.connect("DSN=MaddenCo;UID=U212READO;PWD=stmffe")
cursor = cnxn.cursor()

### ====================================================
### +++++ SERVICES by STORE ++++++
### ======================================================

### Get a df of all the existing stores:

In [12]:
# GET ALL THE STORES:
Query = f"""SELECT DISTINCT(TSST), TRIM(SSNAMSTORE) FROM TMSALE T
INNER JOIN SYST S ON S.SSNUMSTORE = T.TSST
WHERE SSNAMSTORE LIKE '%#%'
AND SSNAMSTORE NOT LIKE '%WHOLESALE%'
AND SSNAMSTORE NOT LIKE '%RETREAD%'
AND SSNAMSTORE NOT LIKE '%CONSIGNMENT%'
AND SSNAMSTORE NOT LIKE '%TRAINING%'
AND TSYP = '{Last_Month}'
AND SSYNACTIVE = 'Y'
ORDER BY TSST
"""
#--------------------------------------------------------------------
# Each Month values:

# cursor.execute(Query)
cursor.execute(Query)
values = cursor.fetchall()

all_stores_df = pd.DataFrame(values, columns = ['store_name_and_num'])
# df = df['store_name_and_num'].str[1].reset_index(drop=True)


all_stores_df['store_name_and_num'] = all_stores_df['store_name_and_num'].str[1]

list_of_store_names = all_stores_df['store_name_and_num'].to_list()
all_stores_df

,store_name_and_num
0,TALLAHASSEE #140
1,PENSACOLA #142
2,PANAMA CITY #144
3,MOSSY HEAD #145
4,GAINESVILLE GA #157
...,...
110,FOUNTAIN VALLEY #709
111,CARSON #710
112,EL CAJON #745
113,LAS VEGAS #750


### For each of the services for which we'll collect data (Flat Repairs, Tire Rotations, etc), 
### we'll start with 2 df templates, one for Units, the other for dollars.

In [13]:
# Create a df template for Flat Repairs of all the stores for units df (one row):
Flat_Repairs_ALL_STORES_for_Excel_in_Units_df = pd.DataFrame({'': ['FLAT REPAIRS']})

# Create a df template for Flat Repairs of all the stores for dollars df (one row):
Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df = pd.DataFrame({'': ['FLAT REPAIRS']})

# Uncomment either to view either:
Flat_Repairs_ALL_STORES_for_Excel_in_Units_df
Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df


,
0,FLAT REPAIRS


## Loop last year and current year to the Flat Repair in Units df (we'll do dollars next):
#### NOTE:  This df will have EVERY store (not just top 10) and both current and last year's data and will be added later to "DATA DOWNLOADS" page of the website.  Each df will be its own excel sheet in an excel workbook we'll make later.

#### We will also use each of the 2 (units and $) full dfs to further filter them for the website itself which will display only the top 10 stores and current year (unless it's Jan then we'll display only last year). Begin with loop to create the full dfs that will be used later for excel download.

In [14]:
Last_Year

2020

In [15]:
years = [Last_Year, Current_Year]


for year in years:
    print('-------------------------------------------')
    print(f'year: {year}')
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)

    # ================================================================================================================================
    # +++++++++++++++++++++++++++++++++++++ Flat Repairs by Store in Units +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ================================================================================================================================
    # Flat Repair:
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '60' fo all "Tire Repair Services" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('60', '70')
    -- Select Vendor 084 for "TIRE SERVICE/LABOR
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('413', '413T', '126', '127', '128', '129', '126F', '127F', '212', '181', 
    '1001116', '1001258018', '10012165', '100130024', '10015525', '100160025', '10019524', '155IND', 
    '108', '513', '107', '111', '109', '100180033', '10020525', '100240035', '10026525', '10030051', 
    '100356533', '114', '114T', '118', '118T', '115', '115T', '116', '116T', '117', '117T', '119', 
    '119T', '113', '113T', '131', '105', '104LT', '105ATT', '100MWUPS', '100', '103', '100TE', '104', 
    '104LOOSE', 'FR', '100ATT', '104UPS', '100UPS', '103UPS', '141UPS', '155', '155T', '513T', '106ATT', 
    '104P', '106')
    AND TSCODDEL <> 'D'
    """
    Flat_Repairs_ALL_STORES = {}
    #--------------------------------------------------------------------
    # Each Month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        cursor.execute(f"""{Base_Query}
        AND TSYP = {month}""")
        value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Flat_Repairs_ALL_STORES[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Flat_Repairs_ALL_STORES.values():
        Sum_YTD = Sum_YTD + i
    Flat_Repairs_ALL_STORES['YTD'] = Sum_YTD 


    Each_Month_List = ["YTD", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]

    for each_month in Each_Month_List:
        Flat_Repairs_ALL_STORES_for_Excel_in_Units_df[f'{each_month} {year}'] = [
                                        # Flat Repairs
                                        int(Flat_Repairs_ALL_STORES[each_month])
                                        ]

# ====================================================================================================================
# NOW that we have a row for "FLAT REPAIRS" total for all stores at the top of the df, we will add each store's flat repairs to the df:

years = [Last_Year, Current_Year]

    
# Flat repairs for each store
for index, store in enumerate(list_of_store_names): 
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE T
    INNER JOIN SYST S ON S.SSNUMSTORE = T.TSST
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '60' fo all "Tire Repair Services" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('60', '70')
    -- Select Vendor 084 for "TIRE SERVICE/LABOR
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('413', '413T', '126', '127', '128', '129', '126F', '127F', '212', '181', 
    '1001116', '1001258018', '10012165', '100130024', '10015525', '100160025', '10019524', '155IND', 
    '108', '513', '107', '111', '109', '100180033', '10020525', '100240035', '10026525', '10030051', 
    '100356533', '114', '114T', '118', '118T', '115', '115T', '116', '116T', '117', '117T', '119', 
    '119T', '113', '113T', '131', '105', '104LT', '105ATT', '100MWUPS', '100', '103', '100TE', '104', 
    '104LOOSE', 'FR', '100ATT', '104UPS', '100UPS', '103UPS', '141UPS', '155', '155T', '513T', '106ATT', 
    '104P', '106')
    AND TSCODDEL <> 'D'
    AND SSNAMSTORE = '{store}'
    """
    Flat_Repairs_EACH_STORE_last_year = {}
    Flat_Repairs_EACH_STORE_current_year = {}
    for year in years:
        Months_Numerical = []
        for x in range(1,13):
            Months_Numerical.append(year*100 + x)
    #--------------------------------------------------------------------
        # Each Month values:
        #--------------------------------------------------------------------
        # Each Month values:
        for i, month in enumerate(Months_Numerical):
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            if year == Last_Year:
                Flat_Repairs_EACH_STORE_last_year[f"{Month_Name} {year}"] = int(value)
            else:
                Flat_Repairs_EACH_STORE_current_year[f"{Month_Name} {year}"] = int(value)

        if year == Last_Year:
            Sum_YTD = 0
            for i in Flat_Repairs_EACH_STORE_last_year.values():
                Sum_YTD = Sum_YTD + i
            Flat_Repairs_EACH_STORE_last_year[f'YTD {year}'] = int(Sum_YTD)

        elif year == Current_Year:
            Sum_YTD = 0
            for i in Flat_Repairs_EACH_STORE_current_year.values():
                Sum_YTD = Sum_YTD + i
            Flat_Repairs_EACH_STORE_current_year[f'YTD {year}'] = int(Sum_YTD)            

    if year == Current_Year:
        new_line = {"": store, 
        f"YTD {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"YTD {Last_Year}"], 
        f"JAN {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"JAN {Last_Year}"], 
        f"FEB {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"FEB {Last_Year}"], 
        f"MAR {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"MAR {Last_Year}"], 
        f"APR {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"APR {Last_Year}"], 
        f"MAY {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"MAY {Last_Year}"], 
        f"JUN {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"JUN {Last_Year}"], 
        f"JUL {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"JUL {Last_Year}"], 
        f"AUG {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"AUG {Last_Year}"], 
        f"SEPT {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"SEPT {Last_Year}"], 
        f"OCT {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"OCT {Last_Year}"], 
        f"NOV {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"NOV {Last_Year}"], 
        f"DEC {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"DEC {Last_Year}"],
        f"YTD {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"YTD {Current_Year}"], 
        f"JAN {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"JAN {Current_Year}"], 
        f"FEB {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"FEB {Current_Year}"], 
        f"MAR {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"MAR {Current_Year}"], 
        f"APR {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"APR {Current_Year}"], 
        f"MAY {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"MAY {Current_Year}"], 
        f"JUN {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"JUN {Current_Year}"], 
        f"JUL {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"JUL {Current_Year}"], 
        f"AUG {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"AUG {Current_Year}"], 
        f"SEPT {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"SEPT {Current_Year}"], 
        f"OCT {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"OCT {Current_Year}"], 
        f"NOV {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"NOV {Current_Year}"], 
        f"DEC {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"DEC {Current_Year}"] 
        }

        Flat_Repairs_ALL_STORES_for_Excel_in_Units_df = Flat_Repairs_ALL_STORES_for_Excel_in_Units_df.append(new_line, ignore_index=True)


# Finally, make sure that the current month's values are NOT included in the current year's YTD total in the df
list_to_loop = [['01', "JAN", 15], ['02', 'FEB', 16], ['03', 'MAR', 17], ['04', 'APR', 18], ['05', 'MAY', 19], ['06', 'JUN', 20], ['07', 'JUL', 21], ['08', 'AUG', 22], 
['09', 'SEPT', 23], ['10', 'OCT', 24], ['11', 'NOV', 25], ['12', 'DEC', 26]]

for month_number_string, month_name, column_number_for_dashes in list_to_loop:
    if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
        Flat_Repairs_ALL_STORES_for_Excel_in_Units_df[f'YTD {Current_Year}'] = Flat_Repairs_ALL_STORES_for_Excel_in_Units_df[f'YTD {Current_Year}'] - Flat_Repairs_ALL_STORES_for_Excel_in_Units_df[f'{month_name} {Current_Year}']


# Sort by the current year's YTD:
Flat_Repairs_ALL_STORES_for_Excel_in_Units_df = Flat_Repairs_ALL_STORES_for_Excel_in_Units_df.sort_values(by=f"YTD {Current_Year}", ascending=False)

# Reset Index
Flat_Repairs_ALL_STORES_for_Excel_in_Units_df = Flat_Repairs_ALL_STORES_for_Excel_in_Units_df.reset_index(drop=True)

# Finally we will make DASHES for any incomplete month column:
zipped_list_of_current_year_columns_and_yyyymm = list(zip(Flat_Repairs_ALL_STORES_for_Excel_in_Units_df.columns[15:], Months_Numerical))
# now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
    # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
    if numerical_month >= Current_Month:
        Flat_Repairs_ALL_STORES_for_Excel_in_Units_df[col] = '-'


# View the df:
pd.set_option('display.max_columns', None)
Flat_Repairs_ALL_STORES_for_Excel_in_Units_df

                                        

-------------------------------------------
year: 2020
-------------------------------------------
year: 2021


,,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,FLAT REPAIRS,28,6,1,1,5,0,1,3,0,0,2,1,8,40,4,4,3,7,4,3,2,6,1,2,4,0
1,COLUMBIA #255,1,1,0,0,0,0,0,0,0,0,0,0,0,10,0,2,0,2,1,0,1,1,0,0,3,0
2,MERIDIAN #265,3,0,0,0,0,0,0,0,0,0,0,0,3,8,0,2,2,3,1,0,0,0,0,0,0,0
3,AUSTIN #465,1,0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,1,0,0,1,0,1,0,0,0
4,LAFAYETTE TTC #310,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,SULPHUR #315,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
112,LAFAYETTE RETAIL #311,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
113,NEW ORLEANS #307,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
114,GONZALES #306,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Now build the same exact df but for DOLLARS for Flat Repairs:

In [16]:
years = [Last_Year, Current_Year]


for year in years:
    print('-------------------------------------------')
    print(f'year: {year}')
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)

    # ================================================================================================================================
    # +++++++++++++++++++++++++++++++++++++ FLAT REPAIRS BY STORE IN DOLLARS ++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ================================================================================================================================
    # Build first row of df to show totals of all stores both years:
    Base_Query = f"""
    SELECT IFNULL(SUM(TSSALES),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '60' fo all "Tire Repair Services" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('60', '70')
    -- Select Vendor 084 for "TIRE SERVICE/LABOR
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('413', '413T', '126', '127', '128', '129', '126F', '127F', '212', '181', 
    '1001116', '1001258018', '10012165', '100130024', '10015525', '100160025', '10019524', '155IND', 
    '108', '513', '107', '111', '109', '100180033', '10020525', '100240035', '10026525', '10030051', 
    '100356533', '114', '114T', '118', '118T', '115', '115T', '116', '116T', '117', '117T', '119', 
    '119T', '113', '113T', '131', '105', '104LT', '105ATT', '100MWUPS', '100', '103', '100TE', '104', 
    '104LOOSE', 'FR', '100ATT', '104UPS', '100UPS', '103UPS', '141UPS', '155', '155T', '513T', '106ATT', 
    '104P', '106')
    AND TSCODDEL <> 'D'
    """
    Flat_Repairs_ALL_STORES_dollars = {}
    #--------------------------------------------------------------------
    # Each Month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        cursor.execute(f"""{Base_Query}
        AND TSYP = {month}""")
        value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Flat_Repairs_ALL_STORES_dollars[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Flat_Repairs_ALL_STORES_dollars.values():
        Sum_YTD = Sum_YTD + i
    Flat_Repairs_ALL_STORES_dollars['YTD'] = Sum_YTD 


    Each_Month_List = ["YTD", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]

    for each_month in Each_Month_List:
        Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df[f'{each_month} {year}'] = [
                                        # Flat Repairs
                                        float(Flat_Repairs_ALL_STORES_dollars[each_month])
                                        ]

# ====================================================================================================================
# NOW that we have a row for "FLAT REPAIRS" total for all stores at the top of the df, we will add each store's flat repairs to the df:

years = [Last_Year, Current_Year]

    
# Flat repairs for each store
for index, store in enumerate(list_of_store_names): 
    Base_Query = f"""
    SELECT IFNULL(SUM(TSSALES),0) FROM TMSALE T
    INNER JOIN SYST S ON S.SSNUMSTORE = T.TSST
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '60' fo all "Tire Repair Services" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('60', '70')
    -- Select Vendor 084 for "TIRE SERVICE/LABOR
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('413', '413T', '126', '127', '128', '129', '126F', '127F', '212', '181', 
    '1001116', '1001258018', '10012165', '100130024', '10015525', '100160025', '10019524', '155IND', 
    '108', '513', '107', '111', '109', '100180033', '10020525', '100240035', '10026525', '10030051', 
    '100356533', '114', '114T', '118', '118T', '115', '115T', '116', '116T', '117', '117T', '119', 
    '119T', '113', '113T', '131', '105', '104LT', '105ATT', '100MWUPS', '100', '103', '100TE', '104', 
    '104LOOSE', 'FR', '100ATT', '104UPS', '100UPS', '103UPS', '141UPS', '155', '155T', '513T', '106ATT', 
    '104P', '106')
    AND TSCODDEL <> 'D'
    AND SSNAMSTORE = '{store}'
    """
    Flat_Repairs_EACH_STORE_last_year = {}
    Flat_Repairs_EACH_STORE_current_year = {}
    for year in years:
        Months_Numerical = []
        for x in range(1,13):
            Months_Numerical.append(year*100 + x)
    #--------------------------------------------------------------------
        # Each Month values:
        #--------------------------------------------------------------------
        # Each Month values:
        for i, month in enumerate(Months_Numerical):
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            if year == Last_Year:
                Flat_Repairs_EACH_STORE_last_year[f"{Month_Name} {year}"] = float(value)
            else:
                Flat_Repairs_EACH_STORE_current_year[f"{Month_Name} {year}"] = float(value)

        if year == Last_Year:
            Sum_YTD = 0
            for i in Flat_Repairs_EACH_STORE_last_year.values():
                Sum_YTD = Sum_YTD + i
            Flat_Repairs_EACH_STORE_last_year[f'YTD {year}'] = float(Sum_YTD)

        elif year == Current_Year:
            Sum_YTD = 0
            for i in Flat_Repairs_EACH_STORE_current_year.values():
                Sum_YTD = Sum_YTD + i
            Flat_Repairs_EACH_STORE_current_year[f'YTD {year}'] = float(Sum_YTD)

    if year == Current_Year:
        new_line = {"": store, 
        f"YTD {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"YTD {Last_Year}"], 
        f"JAN {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"JAN {Last_Year}"], 
        f"FEB {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"FEB {Last_Year}"], 
        f"MAR {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"MAR {Last_Year}"], 
        f"APR {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"APR {Last_Year}"], 
        f"MAY {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"MAY {Last_Year}"], 
        f"JUN {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"JUN {Last_Year}"], 
        f"JUL {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"JUL {Last_Year}"], 
        f"AUG {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"AUG {Last_Year}"], 
        f"SEPT {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"SEPT {Last_Year}"], 
        f"OCT {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"OCT {Last_Year}"], 
        f"NOV {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"NOV {Last_Year}"], 
        f"DEC {Last_Year}": Flat_Repairs_EACH_STORE_last_year[f"DEC {Last_Year}"],
        f"YTD {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"YTD {Current_Year}"], 
        f"JAN {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"JAN {Current_Year}"], 
        f"FEB {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"FEB {Current_Year}"], 
        f"MAR {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"MAR {Current_Year}"], 
        f"APR {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"APR {Current_Year}"], 
        f"MAY {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"MAY {Current_Year}"], 
        f"JUN {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"JUN {Current_Year}"], 
        f"JUL {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"JUL {Current_Year}"], 
        f"AUG {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"AUG {Current_Year}"], 
        f"SEPT {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"SEPT {Current_Year}"], 
        f"OCT {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"OCT {Current_Year}"], 
        f"NOV {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"NOV {Current_Year}"], 
        f"DEC {Current_Year}": Flat_Repairs_EACH_STORE_current_year[f"DEC {Current_Year}"] 
        }

        Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df = Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df.append(new_line, ignore_index=True)


# Finally, make sure that the current month's values are NOT included in the current year's YTD total in the df

list_to_loop = [['01', "JAN"], ['02', 'FEB'], ['03', 'MAR'], ['04', 'APR'], ['05', 'MAY'], ['06', 'JUN'], ['07', 'JUL'], ['08', 'AUG'], 
['09', 'SEPT'], ['10', 'OCT'], ['11', 'NOV'], ['12', 'DEC']]

for month_number_string, month_name in list_to_loop:
    if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
        Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df[f'YTD {Current_Year}'] = Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df[f'YTD {Current_Year}'] - Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df[f'{month_name} {Current_Year}']


# Sort by the current year's YTD:
Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df = Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df.sort_values(by=f"YTD {Current_Year}", ascending=False)

# Reset Index
Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df = Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df.reset_index(drop=True)

# Finally we will make DASHES for any incomplete month column:
zipped_list_of_current_year_columns_and_yyyymm = list(zip(Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df.columns[15:], Months_Numerical))
# now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
    # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
    if numerical_month >= Current_Month:
        Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df[col] = '-'


# View the df:
pd.set_option('display.max_columns', None)
Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df


-------------------------------------------
year: 2020
-------------------------------------------
year: 2021


,,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,FLAT REPAIRS,1232.16,270.0,37.0,47.94,226.76,0.0,47.94,130.88,0.0,0.0,87.94,35.0,348.70,1681.94,196.44,198.0,148.5,288.5,154.5,119.5,70.0,258.0,49.5,99.0,100.0,0.0
1,MERIDIAN #265,143.82,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,143.82,396.00,0.00,99.0,99.0,148.5,49.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,COLUMBIA #255,47.00,47.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,339.00,0.00,99.0,0.0,70.0,35.0,0.0,35.0,25.0,0.0,0.0,75.0,0.0
3,AUSTIN #465,39.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,39.00,119.50,0.00,0.0,0.0,35.0,0.0,0.0,35.0,0.0,49.5,0.0,0.0,0.0
4,TUSCALOOSA #209,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,99.00,49.50,0.0,0.0,0.0,0.0,0.0,0.0,49.5,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,SULPHUR #315,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,LAFAYETTE RETAIL #311,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113,NEW ORLEANS #307,35.00,0.0,0.0,0.00,0.00,0.0,0.00,35.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,GONZALES #306,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Now that we have 2 Flat Repair tables ready for excel later (one in units and one in dollars), we'll add them to a new dict.
##### When we gather 2 year data for all stores for the other services (Tire Roations, Mounts/Dismounts, etc), we will add them to this dictionary as well.  Again, this dict will ultimatley be used to create the excel workbook for the "Data Downloads" link of the website.

##### Note, we did NOT format the Dollars based df above for excel to ensure it remained numeric-based instead of strings for excel.
##### Instead, we can format to "currency" data type when we use openpyxl later to ensure the data is optimized for excel use.

In [17]:
Dict_for_excel_sheets = {
    "Flat Repairs in Units": Flat_Repairs_ALL_STORES_for_Excel_in_Units_df,
    "Flat Repairs in Dollars": Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df}

#### Now we'll create the dfs for THE WEBSITE display.  In contrast from the dfs we created for excel above, these will:
##### 1. These will display only the top 10 stores (and 'other').
##### 2. For the DOLLARS based df, we'll format it in python for $ and thousands separators to prep it for the website view.
##### 3. These will only display the current year's data (unless it's currently January then it will display last year)
##### 4. ALL the Services in Units will be displayed on ONE table and ALL the Services in $ will be displayed on ONE table also.

In [18]:
# =========================Filter Flat repair df in UNITS to a top 10 df for website ======================
Flat_Repairs_Top_10_Stores_in_Units_for_website_df = Flat_Repairs_ALL_STORES_for_Excel_in_Units_df.iloc[0:11]
# Now group all the NON-top 10 stores into "OTHER":
Other_stores_temp_df = Flat_Repairs_ALL_STORES_for_Excel_in_Units_df.iloc[11:]
Other_stores_temp_df.loc['other'] = Other_stores_temp_df.sum(numeric_only=True)
Other_stores_temp_df = Other_stores_temp_df.iloc[-1:].reset_index(drop=True)
Other_stores_temp_df.iloc[-1, 0] = 'OTHER'
Other_stores_temp_df = Other_stores_temp_df.fillna('-')
# Concat the dfs:
Flat_Repairs_Top_10_Stores_in_Units_for_website_df = pd.concat([Flat_Repairs_Top_10_Stores_in_Units_for_website_df,Other_stores_temp_df],ignore_index=True)
# The above code may change the datatype, but we want UNITS in integers, so ensure all numerical columns are integers:
    # loop through all column names except the first one which is just a blank string name '':
# Flat_Repairs_Top_10_Stores_in_Units_for_website_df = Flat_Repairs_Top_10_Stores_in_Units_for_website_df.astype(str)

for columns_to_convert_to_int in Flat_Repairs_Top_10_Stores_in_Units_for_website_df.loc[:, ~Flat_Repairs_Top_10_Stores_in_Units_for_website_df.columns.isin([''])].columns:
    # AND exclude any column for a month which hasn't completed yet (by excluding columns which have a dash as the first value in that column):
    if Flat_Repairs_Top_10_Stores_in_Units_for_website_df.loc[0, columns_to_convert_to_int] != '-':
        # If the remaining column(s) is/are NOT integer(s) datatype,
        Flat_Repairs_Top_10_Stores_in_Units_for_website_df[columns_to_convert_to_int] = Flat_Repairs_Top_10_Stores_in_Units_for_website_df[columns_to_convert_to_int].astype(int)

#     # If it's January, we'll just use last year's df for the website:
# if f'{Current_Month}' == f'{Current_Year}01':
#     Flat_Repairs_Top_10_Stores_in_Units_for_website_df = Flat_Repairs_Top_10_Stores_in_Units_for_website_df.iloc[:, 0:14]
#     # But if it's after January, we'll have data so we'll use the current year's df on the website instead:
# else:
#     Flat_Repairs_Top_10_Stores_in_Units_for_website_df = Flat_Repairs_Top_10_Stores_in_Units_for_website_df.drop(columns = 
#     [f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', 
#     f'MAY {Last_Year}', f'JUN {Last_Year}', f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', 
#     f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}'])





# =========================Now do the same filter of $ based Flat repair df in $ to a top 10 df for website ======================
# Filter by the top 10 stores:
Flat_Repairs_Top_10_Stores_in_Dollars_for_website_df = Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df.iloc[0:11]
# Now group all the NON-top 10 stores into "OTHER":
Other_stores_temp_df = Flat_Repairs_ALL_STORES_for_Excel_in_Dollars_df.iloc[11:]
Other_stores_temp_df.loc['other'] = Other_stores_temp_df.sum(numeric_only=True)
Other_stores_temp_df = Other_stores_temp_df.iloc[-1:].reset_index(drop=True)
Other_stores_temp_df.iloc[-1, 0] = 'OTHER'
Other_stores_temp_df = Other_stores_temp_df.fillna('-')
# Finally, concat the dfs:
Flat_Repairs_Top_10_Stores_in_Dollars_for_website_df = pd.concat([Flat_Repairs_Top_10_Stores_in_Dollars_for_website_df,Other_stores_temp_df],ignore_index=True)
# Although it should already be so, we want DOLLARS in float format before formatting to string with $ and thousands comma separators, so ensure all numerical columns are float:
    # Our first filter will be to loop through all column names except the first one which is just a blank string name '':
for columns_to_convert_to_flt in Flat_Repairs_Top_10_Stores_in_Dollars_for_website_df.loc[:, ~Flat_Repairs_Top_10_Stores_in_Dollars_for_website_df.columns.isin([''])].columns:
    # AND exclude any column for a month which hasn't completed yet (by excluding columns which have a dash as the first value in that column):
    if Flat_Repairs_Top_10_Stores_in_Dollars_for_website_df.loc[0, columns_to_convert_to_flt] != '-':
        # If the remaining column(s) is/are NOT integer(s) datatype,
        Flat_Repairs_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt] = Flat_Repairs_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt].astype(float)
        Flat_Repairs_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt] = Flat_Repairs_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt].apply(lambda x: '${:,.2f}'.format(x))




# ========================================================
# Uncomment either top 10 dfs (for units or $) to view:
Flat_Repairs_Top_10_Stores_in_Units_for_website_df
Flat_Repairs_Top_10_Stores_in_Dollars_for_website_df

,,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,FLAT REPAIRS,"$1,232.16",$270.00,$37.00,$47.94,$226.76,$0.00,$47.94,$130.88,$0.00,$0.00,$87.94,$35.00,$348.70,"$1,681.94",$196.44,$198.00,$148.50,$288.50,$154.50,$119.50,$70.00,$258.00,$49.50,$99.00,$100.00,$0.00
1,MERIDIAN #265,$143.82,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$143.82,$396.00,$0.00,$99.00,$99.00,$148.50,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
2,COLUMBIA #255,$47.00,$47.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$339.00,$0.00,$99.00,$0.00,$70.00,$35.00,$0.00,$35.00,$25.00,$0.00,$0.00,$75.00,$0.00
3,AUSTIN #465,$39.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$39.00,$119.50,$0.00,$0.00,$0.00,$35.00,$0.00,$0.00,$35.00,$0.00,$49.50,$0.00,$0.00,$0.00
4,TUSCALOOSA #209,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$99.00,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00,$0.00,$0.00
5,SHREVEPORT #324,$47.94,$0.00,$0.00,$0.00,$47.94,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$99.00,$0.00,$0.00,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00
6,LAFAYETTE TTC #310,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$70.00,$0.00,$0.00,$0.00,$0.00,$70.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
7,CORPUS CHRISTI #480,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00
8,TUPELO #270,$47.94,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$47.94,$49.50,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
9,JACKSON TTC #260,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00,$0.00,$0.00


# TIRE ROTATIONS by store

###### We'll do the same process for each service type as we did for flat repairs above
###### Recall we create 2 dfs for excel (containing both years and all stores) and then 2 dfs for website display (only top 10 stores and only current year (unless currently January bc we'll show last year))

### Create both dfs (units and $) to add to dict for excel

In [19]:
# Create a df template for Tire Rotations of all the stores for units df (one row):
Tire_Rotations_ALL_STORES_for_Excel_in_Units_df = pd.DataFrame({'': ['TIRE ROTATIONS']})

# Create a df template for Tire Rotations of all the stores for dollars df (one row):
Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df = pd.DataFrame({'': ['TIRE ROTATIONS']})

years = [Last_Year, Current_Year]


for year in years:
    print('-------------------------------------------')
    print(f'year: {year}')
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)
# ======================================================================================================
# ======================================================================================================
# =======================================UNIT SALES==============================================
# ======================================================================================================
# ======================================================================================================

    # Build first row of df to show totals of all stores for each month of last and current year:
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '60' fo all "Tire Repair Services" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('60', '70')
    -- Select Vendor 084 for "TIRE SERVICE/LABOR
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('110WB', '110IN', '111ATT', '111LT', '156', '110TE', '110', '110UPS', '112ATT', '110ATT', '152')     
    AND TSCODDEL <> 'D'
    """
    Tire_Rotations_ALL_STORES = {}
    #--------------------------------------------------------------------
    # Each Month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        cursor.execute(f"""{Base_Query}
        AND TSYP = {month}""")
        value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Tire_Rotations_ALL_STORES[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Tire_Rotations_ALL_STORES.values():
        Sum_YTD = Sum_YTD + i
    Tire_Rotations_ALL_STORES['YTD'] = Sum_YTD 


    Each_Month_List = ["YTD", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]

    for each_month in Each_Month_List:
        Tire_Rotations_ALL_STORES_for_Excel_in_Units_df[f'{each_month} {year}'] = [
                                        # Flat Repairs
                                        int(Tire_Rotations_ALL_STORES[each_month])
                                        ]

# ====================================================================================================================
# NOW that we have a row for "Tire Rotations" total for all stores at the top of the df, we will add each store's flat repairs to the df:

years = [Last_Year, Current_Year]

    
# Flat repairs for each store
for index, store in enumerate(list_of_store_names): 
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE T
    INNER JOIN SYST S ON S.SSNUMSTORE = T.TSST
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '60' fo all "Tire Repair Services" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('60', '70')
    -- Select Vendor 084 for "TIRE SERVICE/LABOR
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('110WB', '110IN', '111ATT', '111LT', '156', '110TE', '110', '110UPS', '112ATT', '110ATT', '152')     
    AND TSCODDEL <> 'D'
    AND SSNAMSTORE = '{store}'
    """
    Tire_Rotations_EACH_STORE_last_year = {}
    Tire_Rotations_EACH_STORE_current_year = {}
    for year in years:
        Months_Numerical = []
        for x in range(1,13):
            Months_Numerical.append(year*100 + x)
    #--------------------------------------------------------------------
        # Each Month values:
        #--------------------------------------------------------------------
        # Each Month values:
        for i, month in enumerate(Months_Numerical):
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            if year == Last_Year:
                Tire_Rotations_EACH_STORE_last_year[f"{Month_Name} {year}"] = int(value)
            else:
                Tire_Rotations_EACH_STORE_current_year[f"{Month_Name} {year}"] = int(value)

        if year == Last_Year:
            Sum_YTD = 0
            for i in Tire_Rotations_EACH_STORE_last_year.values():
                Sum_YTD = Sum_YTD + i
            Tire_Rotations_EACH_STORE_last_year[f'YTD {year}'] = int(Sum_YTD)

        elif year == Current_Year:
            Sum_YTD = 0
            for i in Tire_Rotations_EACH_STORE_current_year.values():
                Sum_YTD = Sum_YTD + i
            Tire_Rotations_EACH_STORE_current_year[f'YTD {year}'] = int(Sum_YTD)            

    if year == Current_Year:
        new_line = {"": store, 
        f"YTD {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"YTD {Last_Year}"], 
        f"JAN {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"JAN {Last_Year}"], 
        f"FEB {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"FEB {Last_Year}"], 
        f"MAR {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"MAR {Last_Year}"], 
        f"APR {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"APR {Last_Year}"], 
        f"MAY {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"MAY {Last_Year}"], 
        f"JUN {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"JUN {Last_Year}"], 
        f"JUL {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"JUL {Last_Year}"], 
        f"AUG {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"AUG {Last_Year}"], 
        f"SEPT {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"SEPT {Last_Year}"], 
        f"OCT {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"OCT {Last_Year}"], 
        f"NOV {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"NOV {Last_Year}"], 
        f"DEC {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"DEC {Last_Year}"],
        f"YTD {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"YTD {Current_Year}"], 
        f"JAN {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"JAN {Current_Year}"], 
        f"FEB {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"FEB {Current_Year}"], 
        f"MAR {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"MAR {Current_Year}"], 
        f"APR {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"APR {Current_Year}"], 
        f"MAY {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"MAY {Current_Year}"], 
        f"JUN {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"JUN {Current_Year}"], 
        f"JUL {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"JUL {Current_Year}"], 
        f"AUG {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"AUG {Current_Year}"], 
        f"SEPT {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"SEPT {Current_Year}"], 
        f"OCT {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"OCT {Current_Year}"], 
        f"NOV {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"NOV {Current_Year}"], 
        f"DEC {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"DEC {Current_Year}"] 
        }

        Tire_Rotations_ALL_STORES_for_Excel_in_Units_df = Tire_Rotations_ALL_STORES_for_Excel_in_Units_df.append(new_line, ignore_index=True)


# Finally, make sure that the current month's values are NOT included in the current year's YTD total in the df
list_to_loop = [['01', "JAN"], ['02', 'FEB'], ['03', 'MAR'], ['04', 'APR'], ['05', 'MAY'], ['06', 'JUN'], ['07', 'JUL'], ['08', 'AUG'], 
['09', 'SEPT'], ['10', 'OCT'], ['11', 'NOV'], ['12', 'DEC']]

for month_number_string, month_name in list_to_loop:
    if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
        Tire_Rotations_ALL_STORES_for_Excel_in_Units_df[f'YTD {Current_Year}'] = Tire_Rotations_ALL_STORES_for_Excel_in_Units_df[f'YTD {Current_Year}'] - Tire_Rotations_ALL_STORES_for_Excel_in_Units_df[f'{month_name} {Current_Year}']


# Sort by the current year's YTD:
Tire_Rotations_ALL_STORES_for_Excel_in_Units_df = Tire_Rotations_ALL_STORES_for_Excel_in_Units_df.sort_values(by=f"YTD {Current_Year}", ascending=False)

# Reset Index
Tire_Rotations_ALL_STORES_for_Excel_in_Units_df = Tire_Rotations_ALL_STORES_for_Excel_in_Units_df.reset_index(drop=True)

# Finally we will make DASHES for any incomplete month column:
zipped_list_of_current_year_columns_and_yyyymm = list(zip(Tire_Rotations_ALL_STORES_for_Excel_in_Units_df.columns[15:], Months_Numerical))
# now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
    # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
    if numerical_month >= Current_Month:
        Tire_Rotations_ALL_STORES_for_Excel_in_Units_df[col] = '-'



# ======================================================================================================
# ======================================================================================================
# ======================================================================================================
# ======================================================================================================
# =======================================DOLLARS SALES=============================
# ======================================================================================================
# ======================================================================================================
# ======================================================================================================
# ======================================================================================================


years = [Last_Year, Current_Year]


for year in years:
    print('-------------------------------------------')
    print(f'year: {year}')
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)

    # ================================================================================================================================
    # +++++++++++++++++++++++++++++++++++++ TIRE ROTATIONS BY STORE IN DOLLARS ++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ================================================================================================================================
    # Build first row of df to show totals of all stores both years:
    Base_Query = f"""
    SELECT IFNULL(SUM(TSSALES),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '60' fo all "Tire Repair Services" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('60', '70')
    -- Select Vendor 084 for "TIRE SERVICE/LABOR
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('110WB', '110IN', '111ATT', '111LT', '156', '110TE', '110', '110UPS', '112ATT', '110ATT', '152')     
    AND TSCODDEL <> 'D'
    """
    Tire_Rotations_ALL_STORES_dollars = {}
    #--------------------------------------------------------------------
    # Each Month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        cursor.execute(f"""{Base_Query}
        AND TSYP = {month}""")
        value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Tire_Rotations_ALL_STORES_dollars[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Tire_Rotations_ALL_STORES_dollars.values():
        Sum_YTD = Sum_YTD + i
    Tire_Rotations_ALL_STORES_dollars['YTD'] = Sum_YTD 


    Each_Month_List = ["YTD", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]

    for each_month in Each_Month_List:
        Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df[f'{each_month} {year}'] = [
                                        # Flat Repairs
                                        float(Tire_Rotations_ALL_STORES_dollars[each_month])
                                        ]

# ====================================================================================================================
# NOW that we have a row for "FLAT REPAIRS" total for all stores at the top of the df, we will add each store's flat repairs to the df:

years = [Last_Year, Current_Year]

    
# Flat repairs for each store
for index, store in enumerate(list_of_store_names): 
    Base_Query = f"""
    SELECT IFNULL(SUM(TSSALES),0) FROM TMSALE T
    INNER JOIN SYST S ON S.SSNUMSTORE = T.TSST
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '60' fo all "Tire Repair Services" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('60', '70')
    -- Select Vendor 084 for "TIRE SERVICE/LABOR
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('110WB', '110IN', '111ATT', '111LT', '156', '110TE', '110', '110UPS', '112ATT', '110ATT', '152')     
    AND TSCODDEL <> 'D'
    AND SSNAMSTORE = '{store}'
    """
    Tire_Rotations_EACH_STORE_last_year = {}
    Tire_Rotations_EACH_STORE_current_year = {}
    for year in years:
        Months_Numerical = []
        for x in range(1,13):
            Months_Numerical.append(year*100 + x)
    #--------------------------------------------------------------------
        # Each Month values:
        #--------------------------------------------------------------------
        # Each Month values:
        for i, month in enumerate(Months_Numerical):
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            if year == Last_Year:
                Tire_Rotations_EACH_STORE_last_year[f"{Month_Name} {year}"] = float(value)
            else:
                Tire_Rotations_EACH_STORE_current_year[f"{Month_Name} {year}"] = float(value)

        if year == Last_Year:
            Sum_YTD = 0
            for i in Tire_Rotations_EACH_STORE_last_year.values():
                Sum_YTD = Sum_YTD + i
            Tire_Rotations_EACH_STORE_last_year[f'YTD {year}'] = float(Sum_YTD)

        elif year == Current_Year:
            Sum_YTD = 0
            for i in Tire_Rotations_EACH_STORE_current_year.values():
                Sum_YTD = Sum_YTD + i
            Tire_Rotations_EACH_STORE_current_year[f'YTD {year}'] = float(Sum_YTD)            

    if year == Current_Year:
        new_line = {"": store, 
        f"YTD {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"YTD {Last_Year}"], 
        f"JAN {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"JAN {Last_Year}"], 
        f"FEB {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"FEB {Last_Year}"], 
        f"MAR {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"MAR {Last_Year}"], 
        f"APR {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"APR {Last_Year}"], 
        f"MAY {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"MAY {Last_Year}"], 
        f"JUN {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"JUN {Last_Year}"], 
        f"JUL {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"JUL {Last_Year}"], 
        f"AUG {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"AUG {Last_Year}"], 
        f"SEPT {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"SEPT {Last_Year}"], 
        f"OCT {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"OCT {Last_Year}"], 
        f"NOV {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"NOV {Last_Year}"], 
        f"DEC {Last_Year}": Tire_Rotations_EACH_STORE_last_year[f"DEC {Last_Year}"],
        f"YTD {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"YTD {Current_Year}"], 
        f"JAN {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"JAN {Current_Year}"], 
        f"FEB {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"FEB {Current_Year}"], 
        f"MAR {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"MAR {Current_Year}"], 
        f"APR {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"APR {Current_Year}"], 
        f"MAY {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"MAY {Current_Year}"], 
        f"JUN {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"JUN {Current_Year}"], 
        f"JUL {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"JUL {Current_Year}"], 
        f"AUG {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"AUG {Current_Year}"], 
        f"SEPT {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"SEPT {Current_Year}"], 
        f"OCT {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"OCT {Current_Year}"], 
        f"NOV {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"NOV {Current_Year}"], 
        f"DEC {Current_Year}": Tire_Rotations_EACH_STORE_current_year[f"DEC {Current_Year}"] 
        }

        Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df = Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df.append(new_line, ignore_index=True)

# Make sure that the current month's values are NOT included in the current year's YTD total in the df
list_to_loop = [['01', "JAN"], ['02', 'FEB'], ['03', 'MAR'], ['04', 'APR'], ['05', 'MAY'], ['06', 'JUN'], ['07', 'JUL'], ['08', 'AUG'], 
['09', 'SEPT'], ['10', 'OCT'], ['11', 'NOV'], ['12', 'DEC']]

for month_number_string, month_name in list_to_loop:
    if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
        Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df[f'YTD {Current_Year}'] = Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df[f'YTD {Current_Year}'] - Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df[f'{month_name} {Current_Year}']


# Sort by the current year's YTD:
Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df = Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df.sort_values(by=f"YTD {Current_Year}", ascending=False)

# Reset Index
Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df = Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df.reset_index(drop=True)

# Finally we will make DASHES for any incomplete month column:
zipped_list_of_current_year_columns_and_yyyymm = list(zip(Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df.columns[15:], Months_Numerical))
# now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
    # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
    if numerical_month >= Current_Month:
        Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df[col] = '-'



# NOW append both Current and Last year's Unit Sales and Dollar Sales to the dictionary.
# Again, this is the dict that we'll make downloadable on the Data Downloads page of the website.
pd.set_option('display.max_columns', None)
Dict_for_excel_sheets["Tire Rotations in Units"] = Tire_Rotations_ALL_STORES_for_Excel_in_Units_df
Dict_for_excel_sheets["Tire Rotations in Dollars"] = Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df


-------------------------------------------
year: 2020
-------------------------------------------
year: 2021
-------------------------------------------
year: 2020
-------------------------------------------
year: 2021


### Create both Tire Rotations dfs for the website (shows top 10 stores only):

In [20]:
# =========================Filter Tire Rotation df in UNITS to a top 10 df for website ======================
Tire_Rotations_Top_10_Stores_in_Units_for_website_df = Tire_Rotations_ALL_STORES_for_Excel_in_Units_df.iloc[0:11]
# Now group all the NON-top 10 stores into "OTHER":
Other_stores_temp_df = Tire_Rotations_ALL_STORES_for_Excel_in_Units_df.iloc[11:]
Other_stores_temp_df.loc['other'] = Other_stores_temp_df.sum(numeric_only=True)
Other_stores_temp_df = Other_stores_temp_df.iloc[-1:].reset_index(drop=True)
Other_stores_temp_df.iloc[-1, 0] = 'OTHER'
Other_stores_temp_df = Other_stores_temp_df.fillna('-')
# Concat the dfs:
Tire_Rotations_Top_10_Stores_in_Units_for_website_df = pd.concat([Tire_Rotations_Top_10_Stores_in_Units_for_website_df,Other_stores_temp_df],ignore_index=True)
# The above code may change the datatype, but we want UNITS in integers, so ensure all numerical columns are integers:
    # loop through all column names except the first one which is just a blank string name '':
# Tire_Rotations_Top_10_Stores_in_Units_for_website_df = Tire_Rotations_Top_10_Stores_in_Units_for_website_df.astype(str)

for columns_to_convert_to_int in Tire_Rotations_Top_10_Stores_in_Units_for_website_df.loc[:, ~Tire_Rotations_Top_10_Stores_in_Units_for_website_df.columns.isin([''])].columns:
    # AND exclude any column for a month which hasn't completed yet (by excluding columns which have a dash as the first value in that column):
    if Tire_Rotations_Top_10_Stores_in_Units_for_website_df.loc[0, columns_to_convert_to_int] != '-':
        # If the remaining column(s) is/are NOT integer(s) datatype,
        Tire_Rotations_Top_10_Stores_in_Units_for_website_df[columns_to_convert_to_int] = Tire_Rotations_Top_10_Stores_in_Units_for_website_df[columns_to_convert_to_int].astype(int)





# =========================Now do the same filter of $ based Tire Rotations df in $ to a top 10 df for website ======================
# Filter by the top 10 stores:
Tire_Rotations_Top_10_Stores_in_Dollars_for_website_df = Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df.iloc[0:11]
# Now group all the NON-top 10 stores into "OTHER":
Other_stores_temp_df = Tire_Rotations_ALL_STORES_for_Excel_in_Dollars_df.iloc[11:]
Other_stores_temp_df.loc['other'] = Other_stores_temp_df.sum(numeric_only=True)
Other_stores_temp_df = Other_stores_temp_df.iloc[-1:].reset_index(drop=True)
Other_stores_temp_df.iloc[-1, 0] = 'OTHER'
Other_stores_temp_df = Other_stores_temp_df.fillna('-')
# Finally, concat the dfs:
Tire_Rotations_Top_10_Stores_in_Dollars_for_website_df = pd.concat([Tire_Rotations_Top_10_Stores_in_Dollars_for_website_df,Other_stores_temp_df],ignore_index=True)
# Although it should already be so, we want DOLLARS in float format before formatting to string with $ and thousands comma separators, so ensure all numerical columns are float:
    # Our first filter will be to loop through all column names except the first one which is just a blank string name '':
for columns_to_convert_to_flt in Tire_Rotations_Top_10_Stores_in_Dollars_for_website_df.loc[:, ~Tire_Rotations_Top_10_Stores_in_Dollars_for_website_df.columns.isin([''])].columns:
    # AND exclude any column for a month which hasn't completed yet (by excluding columns which have a dash as the first value in that column):
    if Tire_Rotations_Top_10_Stores_in_Dollars_for_website_df.loc[0, columns_to_convert_to_flt] != '-':
        # If the remaining column(s) is/are NOT integer(s) datatype,
        Tire_Rotations_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt] = Tire_Rotations_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt].astype(float)
        Tire_Rotations_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt] = Tire_Rotations_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt].apply(lambda x: '${:,.2f}'.format(x))



# ========================================================
# Uncomment either top 10 dfs (for units or $) to view:
Tire_Rotations_Top_10_Stores_in_Units_for_website_df
Tire_Rotations_Top_10_Stores_in_Dollars_for_website_df

,,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,TIRE ROTATIONS,$277.00,$0.00,$0.00,$15.00,$25.00,$178.50,$0.00,$0.00,$20.00,$0.00,$0.00,$0.00,$38.50,$682.00,$0.00,$0.00,$90.00,$98.50,$0.00,$198.50,$120.00,$80.00,$50.00,$15.00,$0.00,$30.00
1,BIRMINGHAM #211,$200.00,$0.00,$0.00,$0.00,$0.00,$160.00,$0.00,$0.00,$20.00,$0.00,$0.00,$0.00,$20.00,$370.00,$0.00,$0.00,$90.00,$80.00,$0.00,$10.00,$120.00,$20.00,$50.00,$0.00,$0.00,$0.00
2,JACKSON TTC #260,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$90.00,$0.00,$0.00,$0.00,$0.00,$0.00,$90.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
3,FORT SMITH #345,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$80.00,$0.00,$0.00,$0.00,$0.00,$0.00,$80.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
4,TUSCALOOSA #209,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$60.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$60.00,$0.00,$0.00,$0.00,$0.00
5,SHREVEPORT #324,$37.00,$0.00,$0.00,$0.00,$0.00,$18.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$18.50,$37.00,$0.00,$0.00,$0.00,$18.50,$0.00,$18.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
6,BATON ROUGE #305,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$30.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$30.00
7,HOUSTON #450,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$15.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$15.00,$0.00,$0.00
8,EL CAJON #745,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
9,BEAUMONT #458,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00


# MOUNT/MOUNT+DISMOUNTS

In [21]:
# Create a df template for Mounts and Mount + Dismounts of all the stores for units df (one row):
Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df = pd.DataFrame({'': ['MOUNT/MOUNT+DISMOUNTS']})

# Create a df template for Mounts and Mount+Dismounts for all the stores for dollars df (one row):
Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df = pd.DataFrame({'': ['MOUNT/MOUNT+DISMOUNTS']})

years = [Last_Year, Current_Year]


for year in years:
    print('-------------------------------------------')
    print(f'year: {year}')
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)
# ======================================================================================================
# ======================================================================================================
# =======================================UNIT SALES==============================================
# ======================================================================================================
# ======================================================================================================

    # Build first row of df to show totals of all stores for each month of last and current year:
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '60' fo all "Tire Repair Services" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('60', '70')
    -- Select Vendor 084 for "TIRE SERVICE/LABOR
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('182', '213', '179', '120HEAVY', '120VHEAVY', '208', 'PRESS16611', 'PRESS166105', 'PRESS165105', 'PRESS21715', 
    'PRESS ON', 'PRESSON16611', 'PRESSON165105', 'PRESSON166105', 'PRESSON21715', '197A', 'PRESSONOFF',
    '110RT400', '110P', '110RT', '115MT', '115MTT', '1201116', '1201258018', '12012165', '120130024', '12015525', '120160025',
    '120180033', '12019524', '12020525', '120240035', '12026525', '120300051', '120356533', '112', '120IN', '134LT', '134LTUPS', 
    '125LT', '125LTUPS', '121ATT', '121', '124', '124V', '197LT', '157', '134UPS', '132', '125UPS', '125', '110MAT', '120UPSNBAL',
    '120ATT', '120TE', '120WR', '159', '120FFE', '120V', '120UPSBAL', '120FED', '120250', '120250FX', '120', '139UPS', '178', '153', 
    '153T', '134P', '122ATT', '122', '197C', '197O', '197S', '197')
    AND TSCODDEL <> 'D'
    """
    Mounts_Dismounts_ALL_STORES = {}
    #--------------------------------------------------------------------
    # Each Month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        cursor.execute(f"""{Base_Query}
        AND TSYP = {month}""")
        value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Mounts_Dismounts_ALL_STORES[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Mounts_Dismounts_ALL_STORES.values():
        Sum_YTD = Sum_YTD + i
    Mounts_Dismounts_ALL_STORES['YTD'] = Sum_YTD 


    Each_Month_List = ["YTD", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]

    for each_month in Each_Month_List:
        Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df[f'{each_month} {year}'] = [
                                        # Flat Repairs
                                        int(Mounts_Dismounts_ALL_STORES[each_month])
                                        ]

# ====================================================================================================================
# NOW that we have a row for service in question total for all stores at the top of the df, we will add each store's total to the df:

years = [Last_Year, Current_Year]

    
# Flat repairs for each store
for index, store in enumerate(list_of_store_names): 
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE T
    INNER JOIN SYST S ON S.SSNUMSTORE = T.TSST
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '60' fo all "Tire Repair Services" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('60', '70')
    -- Select Vendor 084 for "TIRE SERVICE/LABOR
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('182', '213', '179', '120HEAVY', '120VHEAVY', '208', 'PRESS16611', 'PRESS166105', 'PRESS165105', 'PRESS21715', 
    'PRESS ON', 'PRESSON16611', 'PRESSON165105', 'PRESSON166105', 'PRESSON21715', '197A', 'PRESSONOFF',
    '110RT400', '110P', '110RT', '115MT', '115MTT', '1201116', '1201258018', '12012165', '120130024', '12015525', '120160025',
    '120180033', '12019524', '12020525', '120240035', '12026525', '120300051', '120356533', '112', '120IN', '134LT', '134LTUPS', 
    '125LT', '125LTUPS', '121ATT', '121', '124', '124V', '197LT', '157', '134UPS', '132', '125UPS', '125', '110MAT', '120UPSNBAL',
    '120ATT', '120TE', '120WR', '159', '120FFE', '120V', '120UPSBAL', '120FED', '120250', '120250FX', '120', '139UPS', '178', '153', 
    '153T', '134P', '122ATT', '122', '197C', '197O', '197S', '197')
    AND TSCODDEL <> 'D'
    AND SSNAMSTORE = '{store}'
    """
    Mounts_Dismounts_EACH_STORE_last_year = {}
    Mounts_Dismounts_EACH_STORE_current_year = {}
    for year in years:
        Months_Numerical = []
        for x in range(1,13):
            Months_Numerical.append(year*100 + x)
    #--------------------------------------------------------------------
        # Each Month values:
        #--------------------------------------------------------------------
        # Each Month values:
        for i, month in enumerate(Months_Numerical):
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            if year == Last_Year:
                Mounts_Dismounts_EACH_STORE_last_year[f"{Month_Name} {year}"] = int(value)
            else:
                Mounts_Dismounts_EACH_STORE_current_year[f"{Month_Name} {year}"] = int(value)

        if year == Last_Year:
            Sum_YTD = 0
            for i in Mounts_Dismounts_EACH_STORE_last_year.values():
                Sum_YTD = Sum_YTD + i
            Mounts_Dismounts_EACH_STORE_last_year[f'YTD {year}'] = int(Sum_YTD)

        elif year == Current_Year:
            Sum_YTD = 0
            for i in Mounts_Dismounts_EACH_STORE_current_year.values():
                Sum_YTD = Sum_YTD + i
            Mounts_Dismounts_EACH_STORE_current_year[f'YTD {year}'] = int(Sum_YTD)            

    if year == Current_Year:
        new_line = {"": store, 
        f"YTD {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"YTD {Last_Year}"], 
        f"JAN {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"JAN {Last_Year}"], 
        f"FEB {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"FEB {Last_Year}"], 
        f"MAR {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"MAR {Last_Year}"], 
        f"APR {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"APR {Last_Year}"], 
        f"MAY {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"MAY {Last_Year}"], 
        f"JUN {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"JUN {Last_Year}"], 
        f"JUL {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"JUL {Last_Year}"], 
        f"AUG {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"AUG {Last_Year}"], 
        f"SEPT {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"SEPT {Last_Year}"], 
        f"OCT {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"OCT {Last_Year}"], 
        f"NOV {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"NOV {Last_Year}"], 
        f"DEC {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"DEC {Last_Year}"],
        f"YTD {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"YTD {Current_Year}"], 
        f"JAN {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"JAN {Current_Year}"], 
        f"FEB {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"FEB {Current_Year}"], 
        f"MAR {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"MAR {Current_Year}"], 
        f"APR {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"APR {Current_Year}"], 
        f"MAY {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"MAY {Current_Year}"], 
        f"JUN {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"JUN {Current_Year}"], 
        f"JUL {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"JUL {Current_Year}"], 
        f"AUG {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"AUG {Current_Year}"], 
        f"SEPT {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"SEPT {Current_Year}"], 
        f"OCT {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"OCT {Current_Year}"], 
        f"NOV {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"NOV {Current_Year}"], 
        f"DEC {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"DEC {Current_Year}"] 
        }

        Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df = Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df.append(new_line, ignore_index=True)


# Make sure that the current month's values are NOT included in the current year's YTD total in the df
list_to_loop = [['01', "JAN"], ['02', 'FEB'], ['03', 'MAR'], ['04', 'APR'], ['05', 'MAY'], ['06', 'JUN'], ['07', 'JUL'], ['08', 'AUG'], 
['09', 'SEPT'], ['10', 'OCT'], ['11', 'NOV'], ['12', 'DEC']]

for month_number_string, month_name in list_to_loop:
    if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
        Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df[f'YTD {Current_Year}'] = Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df[f'YTD {Current_Year}'] - Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df[f'{month_name} {Current_Year}']

# Sort by the current year's YTD:
Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df = Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df.sort_values(by=f"YTD {Current_Year}", ascending=False)

# Reset Index
Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df = Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df.reset_index(drop=True)

# Finally we will make DASHES for any incomplete month column:
zipped_list_of_current_year_columns_and_yyyymm = list(zip(Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df.columns[15:], Months_Numerical))
# now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
    # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
    if numerical_month >= Current_Month:
        Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df[col] = '-'



# ======================================================================================================
# ======================================================================================================
# ======================================================================================================
# ======================================================================================================
# =======================================DOLLARS SALES=============================
# ======================================================================================================
# ======================================================================================================
# ======================================================================================================
# ======================================================================================================


years = [Last_Year, Current_Year]


for year in years:
    print('-------------------------------------------')
    print(f'year: {year}')
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)

    # ================================================================================================================================
    # +++++++++++++++++++++++++++++++++++++ Mounts/Dismounts BY STORE IN DOLLARS ++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ================================================================================================================================
    # Build first row of df to show totals of all stores both years:
    Base_Query = f"""
    SELECT IFNULL(SUM(TSSALES),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '60' fo all "Tire Repair Services" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('60', '70')
    -- Select Vendor 084 for "TIRE SERVICE/LABOR
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('182', '213', '179', '120HEAVY', '120VHEAVY', '208', 'PRESS16611', 'PRESS166105', 'PRESS165105', 'PRESS21715', 
    'PRESS ON', 'PRESSON16611', 'PRESSON165105', 'PRESSON166105', 'PRESSON21715', '197A', 'PRESSONOFF',
    '110RT400', '110P', '110RT', '115MT', '115MTT', '1201116', '1201258018', '12012165', '120130024', '12015525', '120160025',
    '120180033', '12019524', '12020525', '120240035', '12026525', '120300051', '120356533', '112', '120IN', '134LT', '134LTUPS', 
    '125LT', '125LTUPS', '121ATT', '121', '124', '124V', '197LT', '157', '134UPS', '132', '125UPS', '125', '110MAT', '120UPSNBAL',
    '120ATT', '120TE', '120WR', '159', '120FFE', '120V', '120UPSBAL', '120FED', '120250', '120250FX', '120', '139UPS', '178', '153', 
    '153T', '134P', '122ATT', '122', '197C', '197O', '197S', '197')  
    AND TSCODDEL <> 'D'
    """
    Mounts_Dismounts_ALL_STORES_dollars = {}
    #--------------------------------------------------------------------
    # Each Month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        cursor.execute(f"""{Base_Query}
        AND TSYP = {month}""")
        value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Mounts_Dismounts_ALL_STORES_dollars[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Mounts_Dismounts_ALL_STORES_dollars.values():
        Sum_YTD = Sum_YTD + i
    Mounts_Dismounts_ALL_STORES_dollars['YTD'] = Sum_YTD 


    Each_Month_List = ["YTD", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]

    for each_month in Each_Month_List:
        Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df[f'{each_month} {year}'] = [
                                        float(Mounts_Dismounts_ALL_STORES_dollars[each_month])
                                        ]

# ====================================================================================================================
# NOW that we have a row for "Mounts/Mount+Dismounts" total for all stores at the top of the df, we will add each store's flat repairs to the df:

years = [Last_Year, Current_Year]

    
# Flat repairs for each store
for index, store in enumerate(list_of_store_names): 
    Base_Query = f"""
    SELECT IFNULL(SUM(TSSALES),0) FROM TMSALE T
    INNER JOIN SYST S ON S.SSNUMSTORE = T.TSST
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '60' fo all "Tire Repair Services" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('60', '70')
    -- Select Vendor 084 for "TIRE SERVICE/LABOR
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('182', '213', '179', '120HEAVY', '120VHEAVY', '208', 'PRESS16611', 'PRESS166105', 'PRESS165105', 'PRESS21715', 
    'PRESS ON', 'PRESSON16611', 'PRESSON165105', 'PRESSON166105', 'PRESSON21715', '197A', 'PRESSONOFF',
    '110RT400', '110P', '110RT', '115MT', '115MTT', '1201116', '1201258018', '12012165', '120130024', '12015525', '120160025',
    '120180033', '12019524', '12020525', '120240035', '12026525', '120300051', '120356533', '112', '120IN', '134LT', '134LTUPS', 
    '125LT', '125LTUPS', '121ATT', '121', '124', '124V', '197LT', '157', '134UPS', '132', '125UPS', '125', '110MAT', '120UPSNBAL',
    '120ATT', '120TE', '120WR', '159', '120FFE', '120V', '120UPSBAL', '120FED', '120250', '120250FX', '120', '139UPS', '178', '153', 
    '153T', '134P', '122ATT', '122', '197C', '197O', '197S', '197')
    AND TSCODDEL <> 'D'
    AND SSNAMSTORE = '{store}'
    """
    Mounts_Dismounts_EACH_STORE_last_year = {}
    Mounts_Dismounts_EACH_STORE_current_year = {}
    for year in years:
        Months_Numerical = []
        for x in range(1,13):
            Months_Numerical.append(year*100 + x)
    #--------------------------------------------------------------------
        # Each Month values:
        #--------------------------------------------------------------------
        # Each Month values:
        for i, month in enumerate(Months_Numerical):
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            if year == Last_Year:
                Mounts_Dismounts_EACH_STORE_last_year[f"{Month_Name} {year}"] = float(value)
            else:
                Mounts_Dismounts_EACH_STORE_current_year[f"{Month_Name} {year}"] = float(value)

        if year == Last_Year:
            Sum_YTD = 0
            for i in Mounts_Dismounts_EACH_STORE_last_year.values():
                Sum_YTD = Sum_YTD + i
            Mounts_Dismounts_EACH_STORE_last_year[f'YTD {year}'] = float(Sum_YTD)

        elif year == Current_Year:
            Sum_YTD = 0
            for i in Mounts_Dismounts_EACH_STORE_current_year.values():
                Sum_YTD = Sum_YTD + i
            Mounts_Dismounts_EACH_STORE_current_year[f'YTD {year}'] = float(Sum_YTD)            

    if year == Current_Year:
        new_line = {"": store, 
        f"YTD {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"YTD {Last_Year}"], 
        f"JAN {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"JAN {Last_Year}"], 
        f"FEB {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"FEB {Last_Year}"], 
        f"MAR {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"MAR {Last_Year}"], 
        f"APR {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"APR {Last_Year}"], 
        f"MAY {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"MAY {Last_Year}"], 
        f"JUN {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"JUN {Last_Year}"], 
        f"JUL {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"JUL {Last_Year}"], 
        f"AUG {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"AUG {Last_Year}"], 
        f"SEPT {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"SEPT {Last_Year}"], 
        f"OCT {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"OCT {Last_Year}"], 
        f"NOV {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"NOV {Last_Year}"], 
        f"DEC {Last_Year}": Mounts_Dismounts_EACH_STORE_last_year[f"DEC {Last_Year}"],
        f"YTD {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"YTD {Current_Year}"], 
        f"JAN {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"JAN {Current_Year}"], 
        f"FEB {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"FEB {Current_Year}"], 
        f"MAR {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"MAR {Current_Year}"], 
        f"APR {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"APR {Current_Year}"], 
        f"MAY {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"MAY {Current_Year}"], 
        f"JUN {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"JUN {Current_Year}"], 
        f"JUL {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"JUL {Current_Year}"], 
        f"AUG {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"AUG {Current_Year}"], 
        f"SEPT {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"SEPT {Current_Year}"], 
        f"OCT {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"OCT {Current_Year}"], 
        f"NOV {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"NOV {Current_Year}"], 
        f"DEC {Current_Year}": Mounts_Dismounts_EACH_STORE_current_year[f"DEC {Current_Year}"] 
        }

        Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df = Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df.append(new_line, ignore_index=True)


# Make sure that the current month's values are NOT included in the current year's YTD total in the df
list_to_loop = [['01', "JAN"], ['02', 'FEB'], ['03', 'MAR'], ['04', 'APR'], ['05', 'MAY'], ['06', 'JUN'], ['07', 'JUL'], ['08', 'AUG'], 
['09', 'SEPT'], ['10', 'OCT'], ['11', 'NOV'], ['12', 'DEC']]

for month_number_string, month_name in list_to_loop:
    if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
        Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df[f'YTD {Current_Year}'] = Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df[f'YTD {Current_Year}'] - Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df[f'{month_name} {Current_Year}']


# Sort by the current year's YTD:
Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df = Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df.sort_values(by=f"YTD {Current_Year}", ascending=False)

# Reset Index
Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df = Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df.reset_index(drop=True)


# Finally we will make DASHES for any incomplete month column:
zipped_list_of_current_year_columns_and_yyyymm = list(zip(Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df.columns[15:], Months_Numerical))
# now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
    # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
    if numerical_month >= Current_Month:
        Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df[col] = '-'


# NOW append both Current and Last year's Unit Sales and Dollar Sales to the dictionary.
# Again, this is the dict that we'll make downloadable on the Data Downloads page of the website.
Dict_for_excel_sheets["Mounts/Mount+Dismounts in Units"] = Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df
Dict_for_excel_sheets["Mounts/Mount+Dismounts in Dollars"] = Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df


# Uncomment either df to view what was added to the dictionary:
pd.set_option('display.max_columns', None)
Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df
Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df

-------------------------------------------
year: 2020
-------------------------------------------
year: 2021
-------------------------------------------
year: 2020
-------------------------------------------
year: 2021


,,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,MOUNT/MOUNT+DISMOUNTS,27198.12,939.18,1269.83,1896.89,2470.47,2234.16,2069.96,2091.79,3442.94,3219.58,2412.68,2617.14,2533.50,34071.18,2305.18,1558.0,2660.0,3874.0,3059.0,3640.0,3734.5,4118.0,3108.0,3508.0,1746.0,760.5
1,SHREVEPORT #324,4410.18,0.00,0.00,42.59,85.18,511.08,553.67,425.90,425.90,851.80,766.62,321.54,425.90,6530.18,481.18,396.0,836.0,803.0,1188.0,264.0,484.0,660.0,484.0,604.0,220.0,110.0
2,MERIDIAN #265,2357.63,15.18,85.18,255.54,468.49,85.18,212.95,170.36,170.36,127.77,255.54,511.08,0.00,4458.00,0.00,220.0,308.0,176.0,704.0,880.0,308.0,528.0,88.0,1044.0,70.0,132.0
3,BIRMINGHAM #211,2122.36,352.00,352.00,154.00,176.00,319.00,42.59,11.00,387.18,0.00,42.59,242.00,44.00,2979.00,0.00,264.0,99.0,77.0,165.0,0.0,929.5,369.0,888.0,105.0,71.0,11.5
4,COLUMBIA #255,1944.39,0.00,85.18,365.18,175.00,0.00,0.00,0.00,35.00,380.36,140.00,438.13,325.54,2651.00,70.00,132.0,308.0,438.0,35.0,228.0,500.0,210.0,335.0,170.0,105.0,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,BRYAN #459,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,KATY #462,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113,ST. GEORGE SC #176,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,FULTON #165,127.77,0.00,0.00,85.18,0.00,0.00,0.00,0.00,0.00,0.00,42.59,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Create both (Units and $) Mount/Mount+Dismounts dfs for the website (shows top 10 stores only):

In [22]:
# =========================Filte Mounts/Mounts+Dismounts df in UNITS to a top 10 df for website ======================
Mount_Dismounts_Top_10_Stores_in_Units_for_website_df = Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df.iloc[0:11]
# Now group all the NON-top 10 stores into "OTHER":
Other_stores_temp_df = Mounts_Dismounts_ALL_STORES_for_Excel_in_Units_df.iloc[11:]
Other_stores_temp_df.loc['other'] = Other_stores_temp_df.sum(numeric_only=True)
Other_stores_temp_df = Other_stores_temp_df.iloc[-1:].reset_index(drop=True)
Other_stores_temp_df.iloc[-1, 0] = 'OTHER'
Other_stores_temp_df = Other_stores_temp_df.fillna('-')
# Concat the dfs:
Mount_Dismounts_Top_10_Stores_in_Units_for_website_df = pd.concat([Mount_Dismounts_Top_10_Stores_in_Units_for_website_df,Other_stores_temp_df],ignore_index=True)
# The above code may change the datatype, but we want UNITS in integers, so ensure all numerical columns are integers:
    # loop through all column names except the first one which is just a blank string name '':
# Mount_Dismounts_Top_10_Stores_in_Units_for_website_df = Mount_Dismounts_Top_10_Stores_in_Units_for_website_df.astype(str)

for columns_to_convert_to_int in Mount_Dismounts_Top_10_Stores_in_Units_for_website_df.loc[:, ~Mount_Dismounts_Top_10_Stores_in_Units_for_website_df.columns.isin([''])].columns:
    # AND exclude any column for a month which hasn't completed yet (by excluding columns which have a dash as the first value in that column):
    if Mount_Dismounts_Top_10_Stores_in_Units_for_website_df.loc[0, columns_to_convert_to_int] != '-':
        # If the remaining column(s) is/are NOT integer(s) datatype,
        Mount_Dismounts_Top_10_Stores_in_Units_for_website_df[columns_to_convert_to_int] = Mount_Dismounts_Top_10_Stores_in_Units_for_website_df[columns_to_convert_to_int].astype(int)





# =========================Now do the same filter of $ based Mounts/Mounts+Dismounts df in $ to a top 10 df for website ======================
# Filter by the top 10 stores:
Mount_Dismounts_Top_10_Stores_in_Dollars_for_website_df = Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df.iloc[0:11]
# Now group all the NON-top 10 stores into "OTHER":
Other_stores_temp_df = Mounts_Dismounts_ALL_STORES_for_Excel_in_Dollars_df.iloc[11:]
Other_stores_temp_df.loc['other'] = Other_stores_temp_df.sum(numeric_only=True)
Other_stores_temp_df = Other_stores_temp_df.iloc[-1:].reset_index(drop=True)
Other_stores_temp_df.iloc[-1, 0] = 'OTHER'
Other_stores_temp_df = Other_stores_temp_df.fillna('-')
# Finally, concat the dfs:
Mount_Dismounts_Top_10_Stores_in_Dollars_for_website_df = pd.concat([Mount_Dismounts_Top_10_Stores_in_Dollars_for_website_df,Other_stores_temp_df],ignore_index=True)
# Although it should already be so, we want DOLLARS in float format before formatting to string with $ and thousands comma separators, so ensure all numerical columns are float:
    # Our first filter will be to loop through all column names except the first one which is just a blank string name '':
for columns_to_convert_to_flt in Mount_Dismounts_Top_10_Stores_in_Dollars_for_website_df.loc[:, ~Mount_Dismounts_Top_10_Stores_in_Dollars_for_website_df.columns.isin([''])].columns:
    # AND exclude any column for a month which hasn't completed yet (by excluding columns which have a dash as the first value in that column):
    if Mount_Dismounts_Top_10_Stores_in_Dollars_for_website_df.loc[0, columns_to_convert_to_flt] != '-':
        # If the remaining column(s) is/are NOT integer(s) datatype,
        Mount_Dismounts_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt] = Mount_Dismounts_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt].astype(float)
        Mount_Dismounts_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt] = Mount_Dismounts_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt].apply(lambda x: '${:,.2f}'.format(x))



# ========================================================
# Uncomment either top 10 dfs (for units or $) to view:
Mount_Dismounts_Top_10_Stores_in_Units_for_website_df
Mount_Dismounts_Top_10_Stores_in_Dollars_for_website_df

,,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,MOUNT/MOUNT+DISMOUNTS,"$27,198.12",$939.18,"$1,269.83","$1,896.89","$2,470.47","$2,234.16","$2,069.96","$2,091.79","$3,442.94","$3,219.58","$2,412.68","$2,617.14","$2,533.50","$34,071.18","$2,305.18","$1,558.00","$2,660.00","$3,874.00","$3,059.00","$3,640.00","$3,734.50","$4,118.00","$3,108.00","$3,508.00","$1,746.00",$760.50
1,SHREVEPORT #324,"$4,410.18",$0.00,$0.00,$42.59,$85.18,$511.08,$553.67,$425.90,$425.90,$851.80,$766.62,$321.54,$425.90,"$6,530.18",$481.18,$396.00,$836.00,$803.00,"$1,188.00",$264.00,$484.00,$660.00,$484.00,$604.00,$220.00,$110.00
2,MERIDIAN #265,"$2,357.63",$15.18,$85.18,$255.54,$468.49,$85.18,$212.95,$170.36,$170.36,$127.77,$255.54,$511.08,$0.00,"$4,458.00",$0.00,$220.00,$308.00,$176.00,$704.00,$880.00,$308.00,$528.00,$88.00,"$1,044.00",$70.00,$132.00
3,BIRMINGHAM #211,"$2,122.36",$352.00,$352.00,$154.00,$176.00,$319.00,$42.59,$11.00,$387.18,$0.00,$42.59,$242.00,$44.00,"$2,979.00",$0.00,$264.00,$99.00,$77.00,$165.00,$0.00,$929.50,$369.00,$888.00,$105.00,$71.00,$11.50
4,COLUMBIA #255,"$1,944.39",$0.00,$85.18,$365.18,$175.00,$0.00,$0.00,$0.00,$35.00,$380.36,$140.00,$438.13,$325.54,"$2,651.00",$70.00,$132.00,$308.00,$438.00,$35.00,$228.00,$500.00,$210.00,$335.00,$170.00,$105.00,$120.00
5,MEMPHIS #249,"$3,998.10",$0.00,$0.00,$42.59,$207.59,$0.00,$85.18,$0.00,"$1,022.16",$681.44,$511.08,$553.67,$894.39,"$1,969.00",$836.00,$264.00,$297.00,$132.00,$88.00,$220.00,$0.00,$44.00,$88.00,$0.00,$0.00,$0.00
6,HOUSTON #450,$343.27,$83.50,$132.00,$0.00,$85.18,$42.59,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,"$1,896.00",$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$620.00,$308.00,$484.00,$484.00,$0.00
7,JACKSON TTC #260,$805.85,$167.00,$0.00,$85.18,$0.00,$0.00,$255.54,$85.18,$0.00,$0.00,$85.18,$0.00,$127.77,"$1,484.00",$0.00,$44.00,$0.00,$484.00,$0.00,$308.00,$88.00,$30.00,$264.00,$222.00,$44.00,$0.00
8,MONROE #320,$511.08,$0.00,$0.00,$42.59,$85.18,$0.00,$85.18,$127.77,$42.59,$42.59,$0.00,$0.00,$85.18,"$1,166.00",$0.00,$0.00,$44.00,$176.00,$88.00,$242.00,$176.00,$220.00,$132.00,$0.00,$88.00,$0.00
9,GREENWOOD #266,$541.04,$285.50,$85.18,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$85.18,$42.59,$42.59,"$1,032.00",$0.00,$0.00,$0.00,$352.00,$88.00,$0.00,$352.00,$30.00,$90.00,$60.00,$60.00,$0.00


# SERVICE CALL HOURS

In [23]:
# Create a df template for Service Call Hours of all the stores for units df (one row):
Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df = pd.DataFrame({'': ['SERVICE CALL HOURS']})

# Create a df template for Service Call Hours for all the stores for dollars df (one row):
Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df = pd.DataFrame({'': ['SERVICE CALL HOURS']})

years = [Last_Year, Current_Year]


for year in years:
    print('-------------------------------------------')
    print(f'year: {year}')
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)
# ======================================================================================================
# ======================================================================================================
# =======================================UNIT SALES==============================================
# ======================================================================================================
# ======================================================================================================

    # Build first row of df to show totals of all stores for each month of last and current year:
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '61' for "SERVICE CALL" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('61', '70')
    -- Select Vendor 084 for TIRE SERVICE/LABOR:
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('160RT', '160SOL', '160P', '161PIKE', '162ATT/LTRK', '161ATT/LTRK', '160ATT/LTRK', '165ML', '165A', 
    '167A', '175A', '176A', '177A', '186A', '170', '166', '175', '175W', '177', '167', '176', '174', '167NFS', '167UPS', 
    '186', '177UPS', '176SPEC', '176UPS', '180', '180RT', '171', '162ATT/MED', '160ATT/MED', '161ATT/MED', '180ATT', 
    '180UPS', '165', '165W', '168', '175UPS', '175NFS', '165UPS', '166UPS', '160', '160TE', '173', '185', '169', '172OTR', 
    '172OTRVG', '172', '172T', '165OTR', '165OTRVG', '165OTRVAL', '165OTRT', '164', '164T', '162PIKE', '162ATT', '161ATT', 
    '160ATT', '160PIKE')
    AND TSCODDEL <> 'D'
    """
    Service_Call_Hours_ALL_STORES = {}
    #--------------------------------------------------------------------
    # Each Month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        cursor.execute(f"""{Base_Query}
        AND TSYP = {month}""")
        value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Service_Call_Hours_ALL_STORES[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Service_Call_Hours_ALL_STORES.values():
        Sum_YTD = Sum_YTD + i
    Service_Call_Hours_ALL_STORES['YTD'] = Sum_YTD 


    Each_Month_List = ["YTD", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]

    for each_month in Each_Month_List:
        Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df[f'{each_month} {year}'] = [
                                        # Flat Repairs
                                        int(Service_Call_Hours_ALL_STORES[each_month])
                                        ]

# ====================================================================================================================
# NOW that we have a row for service in question total for all stores at the top of the df, we will add each store's total to the df:

years = [Last_Year, Current_Year]

    
# Flat repairs for each store
for index, store in enumerate(list_of_store_names): 
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE T
    INNER JOIN SYST S ON S.SSNUMSTORE = T.TSST
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '61' for "SERVICE CALL" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('61', '70')
    -- Select Vendor 084 for TIRE SERVICE/LABOR:
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('160RT', '160SOL', '160P', '161PIKE', '162ATT/LTRK', '161ATT/LTRK', '160ATT/LTRK', '165ML', '165A', 
    '167A', '175A', '176A', '177A', '186A', '170', '166', '175', '175W', '177', '167', '176', '174', '167NFS', '167UPS', 
    '186', '177UPS', '176SPEC', '176UPS', '180', '180RT', '171', '162ATT/MED', '160ATT/MED', '161ATT/MED', '180ATT', 
    '180UPS', '165', '165W', '168', '175UPS', '175NFS', '165UPS', '166UPS', '160', '160TE', '173', '185', '169', '172OTR', 
    '172OTRVG', '172', '172T', '165OTR', '165OTRVG', '165OTRVAL', '165OTRT', '164', '164T', '162PIKE', '162ATT', '161ATT', 
    '160ATT', '160PIKE')
    AND TSCODDEL <> 'D'
    AND SSNAMSTORE = '{store}'
    """
    Service_Call_Hours_EACH_STORE_last_year = {}
    Service_Call_Hours_EACH_STORE_current_year = {}
    for year in years:
        Months_Numerical = []
        for x in range(1,13):
            Months_Numerical.append(year*100 + x)
    #--------------------------------------------------------------------
        # Each Month values:
        #--------------------------------------------------------------------
        # Each Month values:
        for i, month in enumerate(Months_Numerical):
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            if year == Last_Year:
                Service_Call_Hours_EACH_STORE_last_year[f"{Month_Name} {year}"] = int(value)
            else:
                Service_Call_Hours_EACH_STORE_current_year[f"{Month_Name} {year}"] = int(value)

        if year == Last_Year:
            Sum_YTD = 0
            for i in Service_Call_Hours_EACH_STORE_last_year.values():
                Sum_YTD = Sum_YTD + i
            Service_Call_Hours_EACH_STORE_last_year[f'YTD {year}'] = int(Sum_YTD)

        elif year == Current_Year:
            Sum_YTD = 0
            for i in Service_Call_Hours_EACH_STORE_current_year.values():
                Sum_YTD = Sum_YTD + i
            Service_Call_Hours_EACH_STORE_current_year[f'YTD {year}'] = int(Sum_YTD)            

    if year == Current_Year:
        new_line = {"": store, 
        f"YTD {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"YTD {Last_Year}"], 
        f"JAN {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"JAN {Last_Year}"], 
        f"FEB {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"FEB {Last_Year}"], 
        f"MAR {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"MAR {Last_Year}"], 
        f"APR {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"APR {Last_Year}"], 
        f"MAY {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"MAY {Last_Year}"], 
        f"JUN {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"JUN {Last_Year}"], 
        f"JUL {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"JUL {Last_Year}"], 
        f"AUG {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"AUG {Last_Year}"], 
        f"SEPT {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"SEPT {Last_Year}"], 
        f"OCT {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"OCT {Last_Year}"], 
        f"NOV {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"NOV {Last_Year}"], 
        f"DEC {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"DEC {Last_Year}"],
        f"YTD {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"YTD {Current_Year}"], 
        f"JAN {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"JAN {Current_Year}"], 
        f"FEB {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"FEB {Current_Year}"], 
        f"MAR {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"MAR {Current_Year}"], 
        f"APR {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"APR {Current_Year}"], 
        f"MAY {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"MAY {Current_Year}"], 
        f"JUN {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"JUN {Current_Year}"], 
        f"JUL {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"JUL {Current_Year}"], 
        f"AUG {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"AUG {Current_Year}"], 
        f"SEPT {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"SEPT {Current_Year}"], 
        f"OCT {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"OCT {Current_Year}"], 
        f"NOV {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"NOV {Current_Year}"], 
        f"DEC {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"DEC {Current_Year}"] 
        }

        Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df = Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df.append(new_line, ignore_index=True)

# Make sure that the current month's values are NOT included in the current year's YTD total in the df
list_to_loop = [['01', "JAN"], ['02', 'FEB'], ['03', 'MAR'], ['04', 'APR'], ['05', 'MAY'], ['06', 'JUN'], ['07', 'JUL'], ['08', 'AUG'], 
['09', 'SEPT'], ['10', 'OCT'], ['11', 'NOV'], ['12', 'DEC']]

for month_number_string, month_name in list_to_loop:
    if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
        Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df[f'YTD {Current_Year}'] = Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df[f'YTD {Current_Year}'] - Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df[f'{month_name} {Current_Year}']


# Sort by the current year's YTD:
Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df = Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df.sort_values(by=f"YTD {Current_Year}", ascending=False)

# Reset Index
Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df = Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df.reset_index(drop=True)

# Finally we will make DASHES for any incomplete month column:
zipped_list_of_current_year_columns_and_yyyymm = list(zip(Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df.columns[15:], Months_Numerical))
# now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
    # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
    if numerical_month >= Current_Month:
        Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df[col] = '-'


# View the df:
pd.set_option('display.max_columns', None)
Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df



# ======================================================================================================
# ======================================================================================================
# ======================================================================================================
# ======================================================================================================
# =======================================DOLLARS SALES=============================
# ======================================================================================================
# ======================================================================================================
# ======================================================================================================
# ======================================================================================================


years = [Last_Year, Current_Year]


for year in years:
    print('-------------------------------------------')
    print(f'year: {year}')
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)

    # ================================================================================================================================
    # +++++++++++++++++++++++++++++++++++++ TIRE ROTATIONS BY STORE IN DOLLARS ++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ================================================================================================================================
    # Build first row of df to show totals of all stores both years:
    Base_Query = f"""
    SELECT IFNULL(SUM(TSSALES),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '61' for "SERVICE CALL" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('61', '70')
    -- Select Vendor 084 for TIRE SERVICE/LABOR:
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('160RT', '160SOL', '160P', '161PIKE', '162ATT/LTRK', '161ATT/LTRK', '160ATT/LTRK', '165ML', '165A', 
    '167A', '175A', '176A', '177A', '186A', '170', '166', '175', '175W', '177', '167', '176', '174', '167NFS', '167UPS', 
    '186', '177UPS', '176SPEC', '176UPS', '180', '180RT', '171', '162ATT/MED', '160ATT/MED', '161ATT/MED', '180ATT', 
    '180UPS', '165', '165W', '168', '175UPS', '175NFS', '165UPS', '166UPS', '160', '160TE', '173', '185', '169', '172OTR', 
    '172OTRVG', '172', '172T', '165OTR', '165OTRVG', '165OTRVAL', '165OTRT', '164', '164T', '162PIKE', '162ATT', '161ATT', 
    '160ATT', '160PIKE')
    AND TSCODDEL <> 'D'
    """
    Service_Call_Hours_ALL_STORES_dollars = {}
    #--------------------------------------------------------------------
    # Each Month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        cursor.execute(f"""{Base_Query}
        AND TSYP = {month}""")
        value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Service_Call_Hours_ALL_STORES_dollars[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Service_Call_Hours_ALL_STORES_dollars.values():
        Sum_YTD = Sum_YTD + i
    Service_Call_Hours_ALL_STORES_dollars['YTD'] = Sum_YTD 


    Each_Month_List = ["YTD", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]

    for each_month in Each_Month_List:
        Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df[f'{each_month} {year}'] = [
                                        # Flat Repairs
                                        float(Service_Call_Hours_ALL_STORES_dollars[each_month])
                                        ]

# ====================================================================================================================
# NOW that we have a row for "Service Call Hours" total for all stores at the top of the df, we will add each store's flat repairs to the df:

years = [Last_Year, Current_Year]

    
# Flat repairs for each store
for index, store in enumerate(list_of_store_names): 
    Base_Query = f"""
    SELECT IFNULL(SUM(TSSALES),0) FROM TMSALE T
    INNER JOIN SYST S ON S.SSNUMSTORE = T.TSST
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product classes '61' for "SERVICE CALL" and '70' for "PASS/LT - Mechanical":
    AND TSPC IN ('61', '70')
    -- Select Vendor 084 for TIRE SERVICE/LABOR:
    AND TSVN IN ('084')
    -- Select Product Numbers:
    AND TSPD IN ('160RT', '160SOL', '160P', '161PIKE', '162ATT/LTRK', '161ATT/LTRK', '160ATT/LTRK', '165ML', '165A', 
    '167A', '175A', '176A', '177A', '186A', '170', '166', '175', '175W', '177', '167', '176', '174', '167NFS', '167UPS', 
    '186', '177UPS', '176SPEC', '176UPS', '180', '180RT', '171', '162ATT/MED', '160ATT/MED', '161ATT/MED', '180ATT', 
    '180UPS', '165', '165W', '168', '175UPS', '175NFS', '165UPS', '166UPS', '160', '160TE', '173', '185', '169', '172OTR', 
    '172OTRVG', '172', '172T', '165OTR', '165OTRVG', '165OTRVAL', '165OTRT', '164', '164T', '162PIKE', '162ATT', '161ATT', 
    '160ATT', '160PIKE')
    AND TSCODDEL <> 'D'
    AND SSNAMSTORE = '{store}'
    """
    Service_Call_Hours_EACH_STORE_last_year = {}
    Service_Call_Hours_EACH_STORE_current_year = {}
    for year in years:
        Months_Numerical = []
        for x in range(1,13):
            Months_Numerical.append(year*100 + x)
    #--------------------------------------------------------------------
        # Each Month values:
        #--------------------------------------------------------------------
        # Each Month values:
        for i, month in enumerate(Months_Numerical):
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            if year == Last_Year:
                Service_Call_Hours_EACH_STORE_last_year[f"{Month_Name} {year}"] = float(value)
            else:
                Service_Call_Hours_EACH_STORE_current_year[f"{Month_Name} {year}"] = float(value)

        if year == Last_Year:
            Sum_YTD = 0
            for i in Service_Call_Hours_EACH_STORE_last_year.values():
                Sum_YTD = Sum_YTD + i
            Service_Call_Hours_EACH_STORE_last_year[f'YTD {year}'] = float(Sum_YTD)

        elif year == Current_Year:
            Sum_YTD = 0
            for i in Service_Call_Hours_EACH_STORE_current_year.values():
                Sum_YTD = Sum_YTD + i
            Service_Call_Hours_EACH_STORE_current_year[f'YTD {year}'] = float(Sum_YTD)            

    if year == Current_Year:
        new_line = {"": store, 
        f"YTD {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"YTD {Last_Year}"], 
        f"JAN {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"JAN {Last_Year}"], 
        f"FEB {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"FEB {Last_Year}"], 
        f"MAR {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"MAR {Last_Year}"], 
        f"APR {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"APR {Last_Year}"], 
        f"MAY {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"MAY {Last_Year}"], 
        f"JUN {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"JUN {Last_Year}"], 
        f"JUL {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"JUL {Last_Year}"], 
        f"AUG {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"AUG {Last_Year}"], 
        f"SEPT {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"SEPT {Last_Year}"], 
        f"OCT {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"OCT {Last_Year}"], 
        f"NOV {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"NOV {Last_Year}"], 
        f"DEC {Last_Year}": Service_Call_Hours_EACH_STORE_last_year[f"DEC {Last_Year}"],
        f"YTD {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"YTD {Current_Year}"], 
        f"JAN {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"JAN {Current_Year}"], 
        f"FEB {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"FEB {Current_Year}"], 
        f"MAR {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"MAR {Current_Year}"], 
        f"APR {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"APR {Current_Year}"], 
        f"MAY {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"MAY {Current_Year}"], 
        f"JUN {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"JUN {Current_Year}"], 
        f"JUL {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"JUL {Current_Year}"], 
        f"AUG {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"AUG {Current_Year}"], 
        f"SEPT {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"SEPT {Current_Year}"], 
        f"OCT {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"OCT {Current_Year}"], 
        f"NOV {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"NOV {Current_Year}"], 
        f"DEC {Current_Year}": Service_Call_Hours_EACH_STORE_current_year[f"DEC {Current_Year}"] 
        }

        Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df = Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df.append(new_line, ignore_index=True)

# Make sure that the current month's values are NOT included in the current year's YTD total in the df
list_to_loop = [['01', "JAN"], ['02', 'FEB'], ['03', 'MAR'], ['04', 'APR'], ['05', 'MAY'], ['06', 'JUN'], ['07', 'JUL'], ['08', 'AUG'], 
['09', 'SEPT'], ['10', 'OCT'], ['11', 'NOV'], ['12', 'DEC']]

for month_number_string, month_name in list_to_loop:
    if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
        Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df[f'YTD {Current_Year}'] = Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df[f'YTD {Current_Year}'] - Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df[f'{month_name} {Current_Year}']


# Sort by the current year's YTD:
Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df = Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df.sort_values(by=f"YTD {Current_Year}", ascending=False)

# Reset Index
Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df = Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df.reset_index(drop=True)



# Finally we will make DASHES for any incomplete month column:
zipped_list_of_current_year_columns_and_yyyymm = list(zip(Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df.columns[15:], Months_Numerical))
# now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
    # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
    if numerical_month >= Current_Month:
        Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df[col] = '-'


# NOW append both Current and Last year's Unit Sales and Dollar Sales to the dictionary.
# Again, this is the dict that we'll make downloadable on the Data Downloads page of the website.
Dict_for_excel_sheets["Service Call Hours in Units"] = Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df
Dict_for_excel_sheets["Service Call Hours in Dollars"] = Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df


# Uncomment either df to view what was added to the dictionary:
Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df
Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df

-------------------------------------------
year: 2020
-------------------------------------------
year: 2021
-------------------------------------------
year: 2020
-------------------------------------------
year: 2021


,,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,SERVICE CALL HOURS,155098.9,11670.9,14846.1,22697.45,15030.50,6584.88,12136.7,16092.68,12744.6,16196.48,9660.15,8126.38,9312.08,160330.39,7640.03,7048.16,10719.95,19504.38,14643.93,17656.74,14692.72,17239.61,9435.20,15788.75,12944.42,13016.50
1,BIRMINGHAM #211,6841.4,482.4,773.5,266.00,0.00,266.00,1082.5,1113.00,508.5,1357.00,660.00,332.50,0.00,10743.38,704.50,438.00,756.50,716.75,1482.50,616.50,788.38,1324.00,1074.00,0.00,1725.75,1116.50
2,BATON ROUGE #305,7142.5,390.0,1011.0,731.50,1062.50,317.50,0.0,545.00,726.0,931.00,538.50,0.00,889.50,7001.38,683.50,479.50,0.00,0.00,569.38,753.50,1284.25,525.00,780.00,427.50,1198.75,300.00
3,HOUSTON #450,5907.0,374.5,1566.5,1362.50,1193.00,0.00,327.0,0.00,272.5,0.00,266.00,218.00,327.00,6155.50,1039.00,0.00,822.00,834.50,1279.50,0.00,0.00,1256.50,190.00,380.00,190.00,164.00
4,GREENWOOD #266,4719.0,878.6,176.0,0.00,328.16,856.80,192.8,345.94,0.0,907.20,394.20,639.30,0.00,6040.81,0.00,0.00,0.00,1408.18,392.90,0.00,1398.93,1381.80,378.75,509.00,0.00,571.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,ALICE #479,327.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,327.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
112,PHARR #486,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
113,HARLINGEN #488,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
114,FORT SMITH #345,1318.5,260.0,0.0,0.00,508.00,0.00,332.5,0.00,109.0,0.00,0.00,0.00,109.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [24]:
# =========================Filter Service Call Hours df in UNITS to a top 10 df for website ======================
Service_Call_Hours_Top_10_Stores_in_Units_for_website_df = Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df.iloc[0:11]
# Now group all the NON-top 10 stores into "OTHER":
Other_stores_temp_df = Service_Call_Hours_ALL_STORES_for_Excel_in_Units_df.iloc[11:]
Other_stores_temp_df.loc['other'] = Other_stores_temp_df.sum(numeric_only=True)
Other_stores_temp_df = Other_stores_temp_df.iloc[-1:].reset_index(drop=True)
Other_stores_temp_df.iloc[-1, 0] = 'OTHER'
Other_stores_temp_df = Other_stores_temp_df.fillna('-')
# Concat the dfs:
Service_Call_Hours_Top_10_Stores_in_Units_for_website_df = pd.concat([Service_Call_Hours_Top_10_Stores_in_Units_for_website_df, Other_stores_temp_df],ignore_index=True)
# The above code may change the datatype, but we want UNITS in integers, so ensure all numerical columns are integers:
    # loop through all column names except the first one which is just a blank string name '':

for columns_to_convert_to_int in Service_Call_Hours_Top_10_Stores_in_Units_for_website_df.loc[:, ~Service_Call_Hours_Top_10_Stores_in_Units_for_website_df.columns.isin([''])].columns:
    # AND exclude any column for a month which hasn't completed yet (by excluding columns which have a dash as the first value in that column):
    if Service_Call_Hours_Top_10_Stores_in_Units_for_website_df.loc[0, columns_to_convert_to_int] != '-':
        # If the remaining column(s) is/are NOT integer(s) datatype,
        Service_Call_Hours_Top_10_Stores_in_Units_for_website_df[columns_to_convert_to_int] = Service_Call_Hours_Top_10_Stores_in_Units_for_website_df[columns_to_convert_to_int].astype(int)







# =========================Now do the same filter of $ based Service_Call_Hours df in $ to a top 10 df for website ======================
# Filter by the top 10 stores:
Service_Call_Hours_Top_10_Stores_in_Dollars_for_website_df = Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df.iloc[0:11]
# Now group all the NON-top 10 stores into "OTHER":
Other_stores_temp_df = Service_Call_Hours_ALL_STORES_for_Excel_in_Dollars_df.iloc[11:]
Other_stores_temp_df.loc['other'] = Other_stores_temp_df.sum(numeric_only=True)
Other_stores_temp_df = Other_stores_temp_df.iloc[-1:].reset_index(drop=True)
Other_stores_temp_df.iloc[-1, 0] = 'OTHER'
Other_stores_temp_df = Other_stores_temp_df.fillna('-')
# Finally, concat the dfs:
Service_Call_Hours_Top_10_Stores_in_Dollars_for_website_df = pd.concat([Service_Call_Hours_Top_10_Stores_in_Dollars_for_website_df,Other_stores_temp_df],ignore_index=True)
# Although it should already be so, we want DOLLARS in float format before formatting to string with $ and thousands comma separators, so ensure all numerical columns are float:
    # Our first filter will be to loop through all column names except the first one which is just a blank string name '':
for columns_to_convert_to_flt in Service_Call_Hours_Top_10_Stores_in_Dollars_for_website_df.loc[:, ~Service_Call_Hours_Top_10_Stores_in_Dollars_for_website_df.columns.isin([''])].columns:
    # AND exclude any column for a month which hasn't completed yet (by excluding columns which have a dash as the first value in that column):
    if Service_Call_Hours_Top_10_Stores_in_Dollars_for_website_df.loc[0, columns_to_convert_to_flt] != '-':
        # If the remaining column(s) is/are NOT integer(s) datatype,
        Service_Call_Hours_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt] = Service_Call_Hours_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt].astype(float)
        Service_Call_Hours_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt] = Service_Call_Hours_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt].apply(lambda x: '${:,.2f}'.format(x))



# ========================================================
# Uncomment either top 10 dfs (for units or $) to view:
Service_Call_Hours_Top_10_Stores_in_Units_for_website_df
Service_Call_Hours_Top_10_Stores_in_Dollars_for_website_df

,,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,SERVICE CALL HOURS,"$155,098.90","$11,670.90","$14,846.10","$22,697.45","$15,030.50","$6,584.88","$12,136.70","$16,092.68","$12,744.60","$16,196.48","$9,660.15","$8,126.38","$9,312.08","$160,330.39","$7,640.03","$7,048.16","$10,719.95","$19,504.38","$14,643.93","$17,656.74","$14,692.72","$17,239.61","$9,435.20","$15,788.75","$12,944.42","$13,016.50"
1,BIRMINGHAM #211,"$6,841.40",$482.40,$773.50,$266.00,$0.00,$266.00,"$1,082.50","$1,113.00",$508.50,"$1,357.00",$660.00,$332.50,$0.00,"$10,743.38",$704.50,$438.00,$756.50,$716.75,"$1,482.50",$616.50,$788.38,"$1,324.00","$1,074.00",$0.00,"$1,725.75","$1,116.50"
2,BATON ROUGE #305,"$7,142.50",$390.00,"$1,011.00",$731.50,"$1,062.50",$317.50,$0.00,$545.00,$726.00,$931.00,$538.50,$0.00,$889.50,"$7,001.38",$683.50,$479.50,$0.00,$0.00,$569.38,$753.50,"$1,284.25",$525.00,$780.00,$427.50,"$1,198.75",$300.00
3,HOUSTON #450,"$5,907.00",$374.50,"$1,566.50","$1,362.50","$1,193.00",$0.00,$327.00,$0.00,$272.50,$0.00,$266.00,$218.00,$327.00,"$6,155.50","$1,039.00",$0.00,$822.00,$834.50,"$1,279.50",$0.00,$0.00,"$1,256.50",$190.00,$380.00,$190.00,$164.00
4,GREENWOOD #266,"$4,719.00",$878.60,$176.00,$0.00,$328.16,$856.80,$192.80,$345.94,$0.00,$907.20,$394.20,$639.30,$0.00,"$6,040.81",$0.00,$0.00,$0.00,"$1,408.18",$392.90,$0.00,"$1,398.93","$1,381.80",$378.75,$509.00,$0.00,$571.25
5,MERIDIAN #265,"$3,688.50",$0.00,$0.00,$266.00,$518.00,$399.00,"$1,141.00",$0.00,$0.00,$819.50,$327.00,$0.00,$218.00,"$5,529.41",$629.00,$540.75,$592.00,$479.00,$428.25,$545.39,$704.02,$240.00,$748.50,$337.50,$285.00,$0.00
6,MEMPHIS #249,"$3,450.50",$902.00,$0.00,$0.00,$303.00,$272.50,$575.00,$218.00,$332.50,$399.00,$448.50,$0.00,$0.00,"$5,388.50",$0.00,$0.00,$274.00,"$1,084.00",$450.00,$616.50,$548.00,$240.00,$360.00,$651.00,$651.00,$514.00
7,SHREVEPORT #324,"$5,990.50",$539.00,$0.00,$731.50,$399.00,$381.50,"$1,646.00",$926.50,$266.00,$218.00,$218.00,$665.00,$0.00,"$5,302.50",$337.50,$0.00,$225.00,$274.00,$411.00,$910.00,$225.00,$627.50,$190.00,$142.50,$585.00,"$1,375.00"
8,TUSCALOOSA #209,"$1,969.60",$0.00,$532.00,$0.00,$0.00,$218.00,$0.00,$266.00,$0.00,$251.60,$266.00,$218.00,$218.00,"$4,747.10",$0.00,$0.00,$685.00,$0.00,$0.00,$274.00,$0.00,$933.35,$0.00,"$1,763.50",$0.00,"$1,091.25"
9,COLUMBIA #255,"$4,277.62",$918.50,$0.00,$605.00,$988.40,$0.00,$406.90,$243.20,$542.80,$0.00,$329.62,$0.00,$243.20,"$4,646.86",$225.00,$0.00,$0.00,$683.46,$-168.75,$582.20,$874.50,$0.00,"$1,040.70",$618.75,$330.00,$461.00


# SCRAP

In [25]:
# Create a df template for Scraps of all the stores for units df (one row):
Scraps_ALL_STORES_for_Excel_in_Units_df = pd.DataFrame({'': ['SCRAPS']})

# Create a df template for Scraps of all the stores for dollars df (one row):
Scraps_ALL_STORES_for_Excel_in_Dollars_df = pd.DataFrame({'': ['SCRAPS']})

years = [Last_Year, Current_Year]


for year in years:
    print('-------------------------------------------')
    print(f'year: {year}')
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)
# ======================================================================================================
# ======================================================================================================
# =======================================UNIT SALES==============================================
# ======================================================================================================
# ======================================================================================================

    # Build first row of df to show totals of all stores for each month of last and current year:
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product class '69' for "SCRAP TIRE DISPOSAL"
    AND TSPC = '69'
    -- Select vendor of '082' for 'misc parts & items
    AND TSVN = '082'
    AND TSCODDEL <> 'D'
    """
    Scraps_ALL_STORES = {}
    #--------------------------------------------------------------------
    # Each Month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        cursor.execute(f"""{Base_Query}
        AND TSYP = {month}""")
        value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Scraps_ALL_STORES[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Scraps_ALL_STORES.values():
        Sum_YTD = Sum_YTD + i
    Scraps_ALL_STORES['YTD'] = Sum_YTD 


    Each_Month_List = ["YTD", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]

    for each_month in Each_Month_List:
        Scraps_ALL_STORES_for_Excel_in_Units_df[f'{each_month} {year}'] = [
                                        # Flat Repairs
                                        int(Scraps_ALL_STORES[each_month])
                                        ]

# ====================================================================================================================
# NOW that we have a row for "Scraps" total for all stores at the top of the df, we will add each store's flat repairs to the df:

years = [Last_Year, Current_Year]

    
# Flat repairs for each store
for index, store in enumerate(list_of_store_names): 
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE T
    INNER JOIN SYST S ON S.SSNUMSTORE = T.TSST
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product class '69' for "SCRAP TIRE DISPOSAL"
    AND TSPC = '69'
    -- Select vendor of '082' for 'misc parts & items
    AND TSVN = '082'
    AND TSCODDEL <> 'D'
    AND SSNAMSTORE = '{store}'
    """
    Scraps_EACH_STORE_last_year = {}
    Scraps_EACH_STORE_current_year = {}
    for year in years:
        Months_Numerical = []
        for x in range(1,13):
            Months_Numerical.append(year*100 + x)
    #--------------------------------------------------------------------
        # Each Month values:
        #--------------------------------------------------------------------
        # Each Month values:
        for i, month in enumerate(Months_Numerical):
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            if year == Last_Year:
                Scraps_EACH_STORE_last_year[f"{Month_Name} {year}"] = int(value)
            else:
                Scraps_EACH_STORE_current_year[f"{Month_Name} {year}"] = int(value)

        if year == Last_Year:
            Sum_YTD = 0
            for i in Scraps_EACH_STORE_last_year.values():
                Sum_YTD = Sum_YTD + i
            Scraps_EACH_STORE_last_year[f'YTD {year}'] = int(Sum_YTD)

        elif year == Current_Year:
            Sum_YTD = 0
            for i in Scraps_EACH_STORE_current_year.values():
                Sum_YTD = Sum_YTD + i
            Scraps_EACH_STORE_current_year[f'YTD {year}'] = int(Sum_YTD)            

    if year == Current_Year:
        new_line = {"": store, 
        f"YTD {Last_Year}": Scraps_EACH_STORE_last_year[f"YTD {Last_Year}"], 
        f"JAN {Last_Year}": Scraps_EACH_STORE_last_year[f"JAN {Last_Year}"], 
        f"FEB {Last_Year}": Scraps_EACH_STORE_last_year[f"FEB {Last_Year}"], 
        f"MAR {Last_Year}": Scraps_EACH_STORE_last_year[f"MAR {Last_Year}"], 
        f"APR {Last_Year}": Scraps_EACH_STORE_last_year[f"APR {Last_Year}"], 
        f"MAY {Last_Year}": Scraps_EACH_STORE_last_year[f"MAY {Last_Year}"], 
        f"JUN {Last_Year}": Scraps_EACH_STORE_last_year[f"JUN {Last_Year}"], 
        f"JUL {Last_Year}": Scraps_EACH_STORE_last_year[f"JUL {Last_Year}"], 
        f"AUG {Last_Year}": Scraps_EACH_STORE_last_year[f"AUG {Last_Year}"], 
        f"SEPT {Last_Year}": Scraps_EACH_STORE_last_year[f"SEPT {Last_Year}"], 
        f"OCT {Last_Year}": Scraps_EACH_STORE_last_year[f"OCT {Last_Year}"], 
        f"NOV {Last_Year}": Scraps_EACH_STORE_last_year[f"NOV {Last_Year}"], 
        f"DEC {Last_Year}": Scraps_EACH_STORE_last_year[f"DEC {Last_Year}"],
        f"YTD {Current_Year}": Scraps_EACH_STORE_current_year[f"YTD {Current_Year}"], 
        f"JAN {Current_Year}": Scraps_EACH_STORE_current_year[f"JAN {Current_Year}"], 
        f"FEB {Current_Year}": Scraps_EACH_STORE_current_year[f"FEB {Current_Year}"], 
        f"MAR {Current_Year}": Scraps_EACH_STORE_current_year[f"MAR {Current_Year}"], 
        f"APR {Current_Year}": Scraps_EACH_STORE_current_year[f"APR {Current_Year}"], 
        f"MAY {Current_Year}": Scraps_EACH_STORE_current_year[f"MAY {Current_Year}"], 
        f"JUN {Current_Year}": Scraps_EACH_STORE_current_year[f"JUN {Current_Year}"], 
        f"JUL {Current_Year}": Scraps_EACH_STORE_current_year[f"JUL {Current_Year}"], 
        f"AUG {Current_Year}": Scraps_EACH_STORE_current_year[f"AUG {Current_Year}"], 
        f"SEPT {Current_Year}": Scraps_EACH_STORE_current_year[f"SEPT {Current_Year}"], 
        f"OCT {Current_Year}": Scraps_EACH_STORE_current_year[f"OCT {Current_Year}"], 
        f"NOV {Current_Year}": Scraps_EACH_STORE_current_year[f"NOV {Current_Year}"], 
        f"DEC {Current_Year}": Scraps_EACH_STORE_current_year[f"DEC {Current_Year}"] 
        }

        Scraps_ALL_STORES_for_Excel_in_Units_df = Scraps_ALL_STORES_for_Excel_in_Units_df.append(new_line, ignore_index=True)

# Make sure that the current month's values are NOT included in the current year's YTD total in the df
list_to_loop = [['01', "JAN"], ['02', 'FEB'], ['03', 'MAR'], ['04', 'APR'], ['05', 'MAY'], ['06', 'JUN'], ['07', 'JUL'], ['08', 'AUG'], 
['09', 'SEPT'], ['10', 'OCT'], ['11', 'NOV'], ['12', 'DEC']]

for month_number_string, month_name in list_to_loop:
    if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
        Scraps_ALL_STORES_for_Excel_in_Units_df[f'YTD {Current_Year}'] = Scraps_ALL_STORES_for_Excel_in_Units_df[f'YTD {Current_Year}'] - Scraps_ALL_STORES_for_Excel_in_Units_df[f'{month_name} {Current_Year}']



# Sort by the current year's YTD:
Scraps_ALL_STORES_for_Excel_in_Units_df = Scraps_ALL_STORES_for_Excel_in_Units_df.sort_values(by=f"YTD {Current_Year}", ascending=False)

# Reset Index
Scraps_ALL_STORES_for_Excel_in_Units_df = Scraps_ALL_STORES_for_Excel_in_Units_df.reset_index(drop=True)


# Finally we will make DASHES for any incomplete month column:
zipped_list_of_current_year_columns_and_yyyymm = list(zip(Scraps_ALL_STORES_for_Excel_in_Units_df.columns[15:], Months_Numerical))
# now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
    # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
    if numerical_month >= Current_Month:
        Scraps_ALL_STORES_for_Excel_in_Units_df[col] = '-'


# View the df:
pd.set_option('display.max_columns', None)
Scraps_ALL_STORES_for_Excel_in_Units_df



# ======================================================================================================
# ======================================================================================================
# ======================================================================================================
# ======================================================================================================
# =======================================DOLLARS SALES=============================
# ======================================================================================================
# ======================================================================================================
# ======================================================================================================
# ======================================================================================================


years = [Last_Year, Current_Year]


for year in years:
    print('-------------------------------------------')
    print(f'year: {year}')
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)

    # ================================================================================================================================
    # +++++++++++++++++++++++++++++++++++++ TIRE ROTATIONS BY STORE IN DOLLARS ++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ================================================================================================================================
    # Build first row of df to show totals of all stores both years:
    Base_Query = f"""
    SELECT IFNULL(SUM(TSSALES),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product class '69' for "SCRAP TIRE DISPOSAL"
    AND TSPC = '69'
    -- Select vendor of '082' for 'misc parts & items
    AND TSVN = '082'
    AND TSCODDEL <> 'D'
    """
    Scraps_ALL_STORES_dollars = {}
    #--------------------------------------------------------------------
    # Each Month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        cursor.execute(f"""{Base_Query}
        AND TSYP = {month}""")
        value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Scraps_ALL_STORES_dollars[Month_Name] = float(value)
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Scraps_ALL_STORES_dollars.values():
        Sum_YTD = Sum_YTD + i
    Scraps_ALL_STORES_dollars['YTD'] = float(Sum_YTD)


    Each_Month_List = ["YTD", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]

    for each_month in Each_Month_List:
        Scraps_ALL_STORES_for_Excel_in_Dollars_df[f'{each_month} {year}'] = [
                                        # Flat Repairs
                                        float(Scraps_ALL_STORES_dollars[each_month])
                                        ]

# ====================================================================================================================
# NOW that we have a row for "Scraps" total for all stores at the top of the df, we will add each store's flat repairs to the df:

years = [Last_Year, Current_Year]

    
# Flat repairs for each store
for index, store in enumerate(list_of_store_names): 
    Base_Query = f"""
    SELECT IFNULL(SUM(TSSALES),0) FROM TMSALE T
    INNER JOIN SYST S ON S.SSNUMSTORE = T.TSST
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select product class '69' for "SCRAP TIRE DISPOSAL"
    AND TSPC = '69'
    -- Select vendor of '082' for 'misc parts & items
    AND TSVN = '082'
    AND TSCODDEL <> 'D'
    AND SSNAMSTORE = '{store}'
    """
    Scraps_EACH_STORE_last_year = {}
    Scraps_EACH_STORE_current_year = {}
    for year in years:
        Months_Numerical = []
        for x in range(1,13):
            Months_Numerical.append(year*100 + x)
    #--------------------------------------------------------------------
        # Each Month values:
        #--------------------------------------------------------------------
        # Each Month values:
        for i, month in enumerate(Months_Numerical):
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            if year == Last_Year:
                Scraps_EACH_STORE_last_year[f"{Month_Name} {year}"] = float(value)
            else:
                Scraps_EACH_STORE_current_year[f"{Month_Name} {year}"] = float(value)

        if year == Last_Year:
            Sum_YTD = 0
            for i in Scraps_EACH_STORE_last_year.values():
                Sum_YTD = Sum_YTD + i
            Scraps_EACH_STORE_last_year[f'YTD {year}'] = float(Sum_YTD)

        elif year == Current_Year:
            Sum_YTD = 0
            for i in Scraps_EACH_STORE_current_year.values():
                Sum_YTD = Sum_YTD + i
            Scraps_EACH_STORE_current_year[f'YTD {year}'] = float(Sum_YTD)            

    if year == Current_Year:
        new_line = {"": store, 
        f"YTD {Last_Year}": Scraps_EACH_STORE_last_year[f"YTD {Last_Year}"], 
        f"JAN {Last_Year}": Scraps_EACH_STORE_last_year[f"JAN {Last_Year}"], 
        f"FEB {Last_Year}": Scraps_EACH_STORE_last_year[f"FEB {Last_Year}"], 
        f"MAR {Last_Year}": Scraps_EACH_STORE_last_year[f"MAR {Last_Year}"], 
        f"APR {Last_Year}": Scraps_EACH_STORE_last_year[f"APR {Last_Year}"], 
        f"MAY {Last_Year}": Scraps_EACH_STORE_last_year[f"MAY {Last_Year}"], 
        f"JUN {Last_Year}": Scraps_EACH_STORE_last_year[f"JUN {Last_Year}"], 
        f"JUL {Last_Year}": Scraps_EACH_STORE_last_year[f"JUL {Last_Year}"], 
        f"AUG {Last_Year}": Scraps_EACH_STORE_last_year[f"AUG {Last_Year}"], 
        f"SEPT {Last_Year}": Scraps_EACH_STORE_last_year[f"SEPT {Last_Year}"], 
        f"OCT {Last_Year}": Scraps_EACH_STORE_last_year[f"OCT {Last_Year}"], 
        f"NOV {Last_Year}": Scraps_EACH_STORE_last_year[f"NOV {Last_Year}"], 
        f"DEC {Last_Year}": Scraps_EACH_STORE_last_year[f"DEC {Last_Year}"],
        f"YTD {Current_Year}": Scraps_EACH_STORE_current_year[f"YTD {Current_Year}"], 
        f"JAN {Current_Year}": Scraps_EACH_STORE_current_year[f"JAN {Current_Year}"], 
        f"FEB {Current_Year}": Scraps_EACH_STORE_current_year[f"FEB {Current_Year}"], 
        f"MAR {Current_Year}": Scraps_EACH_STORE_current_year[f"MAR {Current_Year}"], 
        f"APR {Current_Year}": Scraps_EACH_STORE_current_year[f"APR {Current_Year}"], 
        f"MAY {Current_Year}": Scraps_EACH_STORE_current_year[f"MAY {Current_Year}"], 
        f"JUN {Current_Year}": Scraps_EACH_STORE_current_year[f"JUN {Current_Year}"], 
        f"JUL {Current_Year}": Scraps_EACH_STORE_current_year[f"JUL {Current_Year}"], 
        f"AUG {Current_Year}": Scraps_EACH_STORE_current_year[f"AUG {Current_Year}"], 
        f"SEPT {Current_Year}": Scraps_EACH_STORE_current_year[f"SEPT {Current_Year}"], 
        f"OCT {Current_Year}": Scraps_EACH_STORE_current_year[f"OCT {Current_Year}"], 
        f"NOV {Current_Year}": Scraps_EACH_STORE_current_year[f"NOV {Current_Year}"], 
        f"DEC {Current_Year}": Scraps_EACH_STORE_current_year[f"DEC {Current_Year}"] 
        }

        Scraps_ALL_STORES_for_Excel_in_Dollars_df = Scraps_ALL_STORES_for_Excel_in_Dollars_df.append(new_line, ignore_index=True)


# Make sure that the current month's values are NOT included in the current year's YTD total in the df
list_to_loop = [['01', "JAN"], ['02', 'FEB'], ['03', 'MAR'], ['04', 'APR'], ['05', 'MAY'], ['06', 'JUN'], ['07', 'JUL'], ['08', 'AUG'], 
['09', 'SEPT'], ['10', 'OCT'], ['11', 'NOV'], ['12', 'DEC']]

for month_number_string, month_name in list_to_loop:
    if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
        Scraps_ALL_STORES_for_Excel_in_Dollars_df[f'YTD {Current_Year}'] = Scraps_ALL_STORES_for_Excel_in_Dollars_df[f'YTD {Current_Year}'] - Scraps_ALL_STORES_for_Excel_in_Dollars_df[f'{month_name} {Current_Year}']


# Sort by the current year's YTD:
Scraps_ALL_STORES_for_Excel_in_Dollars_df = Scraps_ALL_STORES_for_Excel_in_Dollars_df.sort_values(by=f"YTD {Current_Year}", ascending=False)

# Reset Index
Scraps_ALL_STORES_for_Excel_in_Dollars_df = Scraps_ALL_STORES_for_Excel_in_Dollars_df.reset_index(drop=True)

# Finally we will make DASHES for any incomplete month column:
zipped_list_of_current_year_columns_and_yyyymm = list(zip(Scraps_ALL_STORES_for_Excel_in_Dollars_df.columns[15:], Months_Numerical))
# now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
    # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
    if numerical_month >= Current_Month:
        Scraps_ALL_STORES_for_Excel_in_Dollars_df[col] = '-'




# NOW append both Current and Last year's Unit Sales and Dollar Sales to the dictionary.
# Again, this is the dict that we'll make downloadable on the Data Downloads page of the website.
Dict_for_excel_sheets["Scraps in Units"] = Scraps_ALL_STORES_for_Excel_in_Units_df
Dict_for_excel_sheets["Scraps in Dollars"] = Scraps_ALL_STORES_for_Excel_in_Dollars_df

# View either df
pd.set_option('display.max_columns', None)
Scraps_ALL_STORES_for_Excel_in_Units_df
Scraps_ALL_STORES_for_Excel_in_Dollars_df

-------------------------------------------
year: 2020
-------------------------------------------
year: 2021
-------------------------------------------
year: 2020
-------------------------------------------
year: 2021


,,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,SCRAPS,4419.0,289.0,369.0,692.0,509.0,240.0,360.5,464.5,461.0,358.0,241.0,256.0,179.0,5083.5,155.0,263.0,397.0,698.0,534.0,802.5,432.0,501.5,303.0,430.0,335.0,232.5
1,MERIDIAN #265,469.0,0.0,20.0,64.0,114.0,55.0,47.0,25.0,40.0,54.0,25.0,15.0,10.0,613.0,22.0,60.0,52.0,41.0,75.0,138.0,40.0,15.0,25.0,105.0,10.0,30.0
2,HOUSTON #450,324.0,24.0,48.0,84.0,69.0,12.0,15.0,0.0,12.0,0.0,0.0,12.0,48.0,330.0,24.0,0.0,48.0,60.0,96.0,0.0,0.0,35.0,12.0,10.0,40.0,5.0
3,BIRMINGHAM #211,184.0,9.0,30.0,9.0,0.0,0.0,12.0,24.0,25.0,0.0,27.0,48.0,0.0,272.0,24.0,12.0,37.0,24.0,70.0,30.0,10.0,25.0,0.0,0.0,10.0,30.0
4,TYLER #422,378.0,30.0,34.0,30.0,10.0,10.0,80.0,28.0,66.0,10.0,28.0,10.0,42.0,269.0,0.0,26.0,40.0,48.0,0.0,0.0,56.0,54.0,0.0,5.0,30.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,AUSTIN SOUTH #466,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,SAN ANTONIO M/M #475,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113,SCHERTZ #477,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,CARRIZO SPRINGS #478,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# =========================Filter Scraps df in UNITS to a top 10 df for website ======================
Scraps_Top_10_Stores_in_Units_for_website_df = Scraps_ALL_STORES_for_Excel_in_Units_df.iloc[0:11]
# Now group all the NON-top 10 stores into "OTHER":
Other_stores_temp_df = Scraps_ALL_STORES_for_Excel_in_Units_df.iloc[11:]
Other_stores_temp_df.loc['other'] = Other_stores_temp_df.sum(numeric_only=True)
Other_stores_temp_df = Other_stores_temp_df.iloc[-1:].reset_index(drop=True)
Other_stores_temp_df.iloc[-1, 0] = 'OTHER'
Other_stores_temp_df = Other_stores_temp_df.fillna('-')
# Concat the dfs:
Scraps_Top_10_Stores_in_Units_for_website_df = pd.concat([Scraps_Top_10_Stores_in_Units_for_website_df, Other_stores_temp_df],ignore_index=True)
# The above code may change the datatype, but we want UNITS in integers, so ensure all numerical columns are integers:
    # loop through all column names except the first one which is just a blank string name '':

for columns_to_convert_to_int in Scraps_Top_10_Stores_in_Units_for_website_df.loc[:, ~Scraps_Top_10_Stores_in_Units_for_website_df.columns.isin([''])].columns:
    # AND exclude any column for a month which hasn't completed yet (by excluding columns which have a dash as the first value in that column):
    if Scraps_Top_10_Stores_in_Units_for_website_df.loc[0, columns_to_convert_to_int] != '-':
        # If the remaining column(s) is/are NOT integer(s) datatype,
        Scraps_Top_10_Stores_in_Units_for_website_df[columns_to_convert_to_int] = Scraps_Top_10_Stores_in_Units_for_website_df[columns_to_convert_to_int].astype(int)






# =========================Now do the same filter of $ based Scraps df in $ to a top 10 df for website ======================
# Filter by the top 10 stores:
Scraps_Top_10_Stores_in_Dollars_for_website_df = Scraps_ALL_STORES_for_Excel_in_Dollars_df.iloc[0:11]
# Now group all the NON-top 10 stores into "OTHER":
Other_stores_temp_df = Scraps_ALL_STORES_for_Excel_in_Dollars_df.iloc[11:]
Other_stores_temp_df.loc['other'] = Other_stores_temp_df.sum(numeric_only=True)
Other_stores_temp_df = Other_stores_temp_df.iloc[-1:].reset_index(drop=True)
Other_stores_temp_df.iloc[-1, 0] = 'OTHER'
Other_stores_temp_df = Other_stores_temp_df.fillna('-')
# Finally, concat the dfs:
Scraps_Top_10_Stores_in_Dollars_for_website_df = pd.concat([Scraps_Top_10_Stores_in_Dollars_for_website_df,Other_stores_temp_df],ignore_index=True)
# Although it should already be so, we want DOLLARS in float format before formatting to string with $ and thousands comma separators, so ensure all numerical columns are float:
    # Our first filter will be to loop through all column names except the first one which is just a blank string name '':
for columns_to_convert_to_flt in Scraps_Top_10_Stores_in_Dollars_for_website_df.loc[:, ~Scraps_Top_10_Stores_in_Dollars_for_website_df.columns.isin([''])].columns:
    # AND exclude any column for a month which hasn't completed yet (by excluding columns which have a dash as the first value in that column):
    if Scraps_Top_10_Stores_in_Dollars_for_website_df.loc[0, columns_to_convert_to_flt] != '-':
        # If the remaining column(s) is/are NOT integer(s) datatype,
        Scraps_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt] = Scraps_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt].astype(float)
        Scraps_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt] = Scraps_Top_10_Stores_in_Dollars_for_website_df[columns_to_convert_to_flt].apply(lambda x: '${:,.2f}'.format(x))




# ========================================================
# Uncomment either top 10 dfs (for units or $) to view:
Scraps_Top_10_Stores_in_Units_for_website_df
Scraps_Top_10_Stores_in_Dollars_for_website_df

,,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,SCRAPS,"$4,419.00",$289.00,$369.00,$692.00,$509.00,$240.00,$360.50,$464.50,$461.00,$358.00,$241.00,$256.00,$179.00,"$5,083.50",$155.00,$263.00,$397.00,$698.00,$534.00,$802.50,$432.00,$501.50,$303.00,$430.00,$335.00,$232.50
1,MERIDIAN #265,$469.00,$0.00,$20.00,$64.00,$114.00,$55.00,$47.00,$25.00,$40.00,$54.00,$25.00,$15.00,$10.00,$613.00,$22.00,$60.00,$52.00,$41.00,$75.00,$138.00,$40.00,$15.00,$25.00,$105.00,$10.00,$30.00
2,HOUSTON #450,$324.00,$24.00,$48.00,$84.00,$69.00,$12.00,$15.00,$0.00,$12.00,$0.00,$0.00,$12.00,$48.00,$330.00,$24.00,$0.00,$48.00,$60.00,$96.00,$0.00,$0.00,$35.00,$12.00,$10.00,$40.00,$5.00
3,BIRMINGHAM #211,$184.00,$9.00,$30.00,$9.00,$0.00,$0.00,$12.00,$24.00,$25.00,$0.00,$27.00,$48.00,$0.00,$272.00,$24.00,$12.00,$37.00,$24.00,$70.00,$30.00,$10.00,$25.00,$0.00,$0.00,$10.00,$30.00
4,TYLER #422,$378.00,$30.00,$34.00,$30.00,$10.00,$10.00,$80.00,$28.00,$66.00,$10.00,$28.00,$10.00,$42.00,$269.00,$0.00,$26.00,$40.00,$48.00,$0.00,$0.00,$56.00,$54.00,$0.00,$5.00,$30.00,$10.00
5,MEMPHIS #249,$132.00,$24.00,$0.00,$0.00,$12.00,$24.00,$12.00,$0.00,$36.00,$0.00,$24.00,$0.00,$0.00,$269.00,$0.00,$0.00,$21.00,$24.00,$24.00,$70.00,$45.00,$27.00,$5.00,$12.00,$24.00,$17.00
6,ABILENE #492,$48.00,$0.00,$0.00,$0.00,$24.00,$0.00,$24.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$214.00,$0.00,$12.00,$12.00,$0.00,$0.00,$72.00,$0.00,$108.00,$10.00,$0.00,$0.00,$0.00
7,WACO #419,$48.00,$0.00,$12.00,$12.00,$12.00,$0.00,$0.00,$12.00,$0.00,$0.00,$0.00,$0.00,$0.00,$205.00,$0.00,$24.00,$0.00,$80.00,$0.00,$60.00,$24.00,$0.00,$0.00,$12.00,$5.00,$0.00
8,JACKSON TTC #260,$10.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$10.00,$0.00,$0.00,$0.00,$0.00,$0.00,$187.00,$0.00,$10.00,$0.00,$0.00,$0.00,$30.00,$30.00,$30.00,$72.00,$10.00,$5.00,$0.00
9,TUPELO #270,$120.00,$0.00,$16.00,$48.00,$24.00,$0.00,$0.00,$8.00,$0.00,$8.00,$16.00,$0.00,$0.00,$176.00,$8.00,$0.00,$0.00,$32.00,$0.00,$48.00,$0.00,$16.00,$0.00,$47.00,$25.00,$0.00


In [27]:
# Ensure the keys are all present for the dictionary we'll create for excel:

Dict_for_excel_sheets.keys()

dict_keys(['Flat Repairs in Units', 'Flat Repairs in Dollars', 'Tire Rotations in Units', 'Tire Rotations in Dollars', 'Mounts/Mount+Dismounts in Units', 'Mounts/Mount+Dismounts in Dollars', 'Service Call Hours in Units', 'Service Call Hours in Dollars', 'Scraps in Units', 'Scraps in Dollars'])

## Concatenate all the Service dfs in Units into one df AND concatenate all the Service dfs in $ into one df:

In [28]:
# Concat all the Services into one df:


list_of_services_in_units_dataframes = [Flat_Repairs_Top_10_Stores_in_Units_for_website_df, 
                                        Tire_Rotations_Top_10_Stores_in_Units_for_website_df, 
                                        Mount_Dismounts_Top_10_Stores_in_Units_for_website_df, 
                                        Service_Call_Hours_Top_10_Stores_in_Units_for_website_df, 
                                        Scraps_Top_10_Stores_in_Units_for_website_df
                                        ]

All_Services_in_units_top_10_stores_for_website_df = pd.concat(list_of_services_in_units_dataframes)
All_Services_in_units_top_10_stores_for_website_df = All_Services_in_units_top_10_stores_for_website_df.reset_index(drop=True)




list_of_services_in_dollars_dataframes = [Flat_Repairs_Top_10_Stores_in_Dollars_for_website_df, 
                                            Tire_Rotations_Top_10_Stores_in_Dollars_for_website_df, 
                                            Mount_Dismounts_Top_10_Stores_in_Dollars_for_website_df, 
                                            Service_Call_Hours_Top_10_Stores_in_Dollars_for_website_df, 
                                            Scraps_Top_10_Stores_in_Dollars_for_website_df
                                            ]

All_Services_in_dollars_top_10_stores_for_website_df = pd.concat(list_of_services_in_dollars_dataframes)
All_Services_in_dollars_top_10_stores_for_website_df = All_Services_in_dollars_top_10_stores_for_website_df.reset_index(drop=True)



# ====================================================================================================================================================================================================================================
# # For whatever reason, the concatenation for the dollars based df removed the string $ formmating.  So we'll re-apply this formatting below:
# for columns_to_convert_to_flt in All_Services_in_dollars_top_10_stores_for_website_df.loc[:, ~All_Services_in_dollars_top_10_stores_for_website_df.columns.isin([''])].columns:
#     # AND exclude any column for a month which hasn't completed yet (by excluding columns which have a dash as the first value in that column):
#     if All_Services_in_dollars_top_10_stores_for_website_df.loc[0, columns_to_convert_to_flt] != '-':
#         # If the remaining column(s) is/are NOT integer(s) datatype,
#         All_Services_in_dollars_top_10_stores_for_website_df[columns_to_convert_to_flt] = All_Services_in_dollars_top_10_stores_for_website_df[columns_to_convert_to_flt].astype(float)
#         All_Services_in_dollars_top_10_stores_for_website_df[columns_to_convert_to_flt] = All_Services_in_dollars_top_10_stores_for_website_df[columns_to_convert_to_flt].apply(lambda x: '${:,.2f}'.format(x))
# ====================================================================================================================================================================================================================================


# Uncomment to view either:
All_Services_in_units_top_10_stores_for_website_df
All_Services_in_dollars_top_10_stores_for_website_df

,,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,FLAT REPAIRS,"$1,232.16",$270.00,$37.00,$47.94,$226.76,$0.00,$47.94,$130.88,$0.00,$0.00,$87.94,$35.00,$348.70,"$1,681.94",$196.44,$198.00,$148.50,$288.50,$154.50,$119.50,$70.00,$258.00,$49.50,$99.00,$100.00,$0.00
1,MERIDIAN #265,$143.82,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$143.82,$396.00,$0.00,$99.00,$99.00,$148.50,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
2,COLUMBIA #255,$47.00,$47.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$339.00,$0.00,$99.00,$0.00,$70.00,$35.00,$0.00,$35.00,$25.00,$0.00,$0.00,$75.00,$0.00
3,AUSTIN #465,$39.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$39.00,$119.50,$0.00,$0.00,$0.00,$35.00,$0.00,$0.00,$35.00,$0.00,$49.50,$0.00,$0.00,$0.00
4,TUSCALOOSA #209,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$99.00,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00,$0.00,$0.00
5,SHREVEPORT #324,$47.94,$0.00,$0.00,$0.00,$47.94,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$99.00,$0.00,$0.00,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00
6,LAFAYETTE TTC #310,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$70.00,$0.00,$0.00,$0.00,$0.00,$70.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
7,CORPUS CHRISTI #480,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00
8,TUPELO #270,$47.94,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$47.94,$49.50,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
9,JACKSON TTC #260,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00,$0.00,$0.00


## If we only want to see one year on the website, we can default to only displaying the current year's data.  
#### However, if it's January, we won't have any completed months yet so we will want to see only last year.

##### Of course, if you want to see both years on the website instead of just one year, you could comment out the following code.

In [29]:

All_Services_in_units_top_10_stores_for_website_df = All_Services_in_units_top_10_stores_for_website_df.drop(columns = 
                    [f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', 
                    f'MAY {Last_Year}', f'JUN {Last_Year}', f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', 
                    f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}'])
All_Services_in_dollars_top_10_stores_for_website_df = All_Services_in_dollars_top_10_stores_for_website_df.drop(columns =
                    [f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', 
                    f'MAY {Last_Year}', f'JUN {Last_Year}', f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}',
                    f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}'])


# Note:  Again, this is only for the website.  The excel data will have both years.  It will also have all stores, not just the top 10.


# View either Services df:
All_Services_in_units_top_10_stores_for_website_df
All_Services_in_dollars_top_10_stores_for_website_df

,,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,FLAT REPAIRS,"$1,681.94",$196.44,$198.00,$148.50,$288.50,$154.50,$119.50,$70.00,$258.00,$49.50,$99.00,$100.00,$0.00
1,MERIDIAN #265,$396.00,$0.00,$99.00,$99.00,$148.50,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
2,COLUMBIA #255,$339.00,$0.00,$99.00,$0.00,$70.00,$35.00,$0.00,$35.00,$25.00,$0.00,$0.00,$75.00,$0.00
3,AUSTIN #465,$119.50,$0.00,$0.00,$0.00,$35.00,$0.00,$0.00,$35.00,$0.00,$49.50,$0.00,$0.00,$0.00
4,TUSCALOOSA #209,$99.00,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00,$0.00,$0.00
5,SHREVEPORT #324,$99.00,$0.00,$0.00,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00
6,LAFAYETTE TTC #310,$70.00,$0.00,$0.00,$0.00,$0.00,$70.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
7,CORPUS CHRISTI #480,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00
8,TUPELO #270,$49.50,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00
9,JACKSON TTC #260,$49.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$49.50,$0.00,$0.00,$0.00,$0.00


# Build Services sales in units by store table (using the units df we made above) for the website 
### (will include 'Flat Repairs' 'Tire Rotations', 'Mount/Dismounts', 'Service Call Hours', 'Scraps').
#### Note the scraps here are the ones pulled from maddenco that were billed (unlike the scraps table we'll create from webscraping later from basys website)

In [30]:
# Note:  Recall iF you want to use BOTH YEARS in the table, we could scroll up and uncomment the code which filters out last year (or current year if currently January).
# Otherwise, we will be using only the current year's data (unless it's currently January, in which case we'd be using last year) on the website display.
 


# BUILD THE HTML SERVICES TABLE for the WEBSITE:
def create_html_table(func_argument):

    # Create headers for HTML
    row_data = '<!-- Start Services in Units Table --><thead>\n<tr class="header_for_website">'

    # Create columns for HTML:    
    for column in range(func_argument.shape[1]):
        adjust_header_css = "sales_by_store_table_header_for_website"
        row_data += '\n <th class = "'+adjust_header_css+'">'+All_Services_in_units_top_10_stores_for_website_df.columns[column]+'</th>'
    row_data += '\n</tr> \n </head> \n <tbody>'
    
    # Create Body (rows for table) for html
    for row in range(func_argument.shape[0]):
        if row in [0, 12, 24, 36, 48, 60]:
            row_data += '\n<tr class = "ROWS_format_category_headers_for_sales_by_store_table_for_website">'
        else: row_data += '\n<tr>'
        for column in range(func_argument.shape[1]):
            td_class_name = ''
            if column in [0] and row not in [0, 12, 24, 36, 48, 60]:
                td_class_name += "format_SUBcategory_rows_for_sales_by_store_table_for_website"
            if column not in [0] and row not in []:
                td_class_name += " center_for_website"
            row_data += '\n <td class = "'+td_class_name+'">'+str(func_argument.iloc[row,column])+'</td>'
    row_data += '\n</tr> \n</tbody><!-- End Services in Units Table -->'
    # -------------------------------------------------------------------------------------------------

    return row_data

# Call the function:
Services_in_Units_html_table = create_html_table(All_Services_in_units_top_10_stores_for_website_df)




Create_html = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
{string_for_header}
<!-----------------------SERVICE SALES by STORE TABLE PAGE---------------------------------------------->
    <br>
    <br>
    <h1 class=page_title_for_Sales_by_Store_for_website>{Current_Year} SERVICE PURCHASES BY STORE </h1>
    <br>
    <table>
        <!-- Start Services in Units Table -->
        [[DATA_MAIN_TABLE]]
        <!-- End Services in Units Table -->
    </table>
    <br>
     <h5 class=notes_about_data_downloads>*NOTE: This sales table displays the top 10 stores for the current year.  <br> To view all store sales for both last and current year, see "Data Downloads" for an excel workbook</h5>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>

</body>

</html>

"""

# ADD THE SALES TABLE:
html_page_exists = os.path.exists('Service_Sales_in_Units_by_Store.html')
current_index = ''
with open('Service_Sales_in_Units_by_Store.html', 'r') as f:
    current_index = f.read()
with open('Service_Sales_in_Units_by_Store.html', 'w') as f:
    f.write(re.sub(r"<\!-- Start Services in Units Table -->.*<\!-- End Services in Units Table -->", Services_in_Units_html_table, Create_html, flags=re.DOTALL))

print(html_page_exists)

True


## Now, build the "Services $ Sales by Store" page

In [31]:
# Note:  Recall iF you want to use BOTH YEARS in the table, we could scroll up and uncomment the code which filters out last year (or current year if currently January).
# Otherwise, we will be using only the current year's data (unless it's currently January, in which case we'd be using last year) on the website display.
# 



# BUILD THE HTML SERVICES TABLE for the WEBSITE:
def create_html_table(func_argument):

    # Create headers for HTML
    row_data = '<!-- Start Services in Dollars Table --><thead>\n<tr class="header_for_website">'

    # Create columns for HTML:    
    for column in range(func_argument.shape[1]):
        adjust_header_css = "sales_by_store_table_header_for_website"
        row_data += '\n <th class = "'+adjust_header_css+'">'+All_Services_in_dollars_top_10_stores_for_website_df.columns[column]+'</th>'
    row_data += '\n</tr> \n </head> \n <tbody>'
    
    # Create Body (rows for table) for html
    for row in range(func_argument.shape[0]):
        if row in [0, 12, 24, 36, 48, 60]:
            row_data += '\n<tr class = "ROWS_format_category_headers_for_sales_by_store_table_for_website">'
        else: row_data += '\n<tr>'
        for column in range(func_argument.shape[1]):
            td_class_name = ''
            if column in [0] and row not in [0, 12, 24, 36, 48, 60]:
                td_class_name += "format_SUBcategory_rows_for_sales_by_store_table_for_website"
            if column not in [0] and row not in []:
                td_class_name += " center_for_website"
            row_data += '\n <td class = "'+td_class_name+'">'+str(func_argument.iloc[row,column])+'</td>'
    row_data += '\n</tr> \n</tbody><!-- End Services in Dollars Table -->'
    # -------------------------------------------------------------------------------------------------

    return row_data

# Call the function:
Services_in_Dollars_html_table = create_html_table(All_Services_in_dollars_top_10_stores_for_website_df)




Create_html = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
{string_for_header}
<!-----------------------SERVICE SALES by STORE TABLE PAGE---------------------------------------------->
    <br>
    <br>
    <h1 class=page_title_for_Sales_by_Store_for_website>{Current_Year} SERVICE PURCHASES BY STORE</h1>
    <br>
    <table>
        <!-- Start Services in Dollars Table -->
        [[DATA_MAIN_TABLE]]
        <!-- End Services in Dollars Table -->
    </table>
    <br>
     <h5 class=notes_about_data_downloads>*NOTE: This sales table displays the top 10 stores for the current year.  <br> To view all store sales for both last and current year, see "Data Downloads" for an excel workbook</h5>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>

</body>

</html>

"""

# ADD THE SALES TABLE:
# index_exists = os.path.exists('./index_NAV_BAR_files/Sales_Table.html')
# current_index = ''
# with open('./index_NAV_BAR_files/Sales_Table.html', 'r') as f:
#     current_index = f.read()
# with open('./index_NAV_BAR_files/Sales_Table.html', 'w') as f:
#     f.write(re.sub(r"<\!-- Start NRT Table -->.*<\!-- End NRT Table -->",
#             html_table2, current_index, flags=re.DOTALL))

# ADD THE SALES TABLE:
html_page_exists = os.path.exists('Service_Sales_in_Dollars_by_Store.html')
current_index = ''
with open('Service_Sales_in_Dollars_by_Store.html', 'r') as f:
    current_index = f.read()
with open('Service_Sales_in_Dollars_by_Store.html', 'w') as f:
    f.write(re.sub(r"<\!-- Start Services in Dollars Table -->.*<\!-- End Services in Dollars Table -->", Services_in_Dollars_html_table, Create_html, flags=re.DOTALL))

print(html_page_exists)

True


# =======================================
## +++++++++ RETREADS ++++++++++++++
# ========================================

In [32]:
# Create a df template for Retreads of all the stores for units df (one row):
Retreads_ALL_STORES_for_Excel_in_Units_df = pd.DataFrame({'': ['RETREADS']})

years = [Last_Year, Current_Year]


for year in years:
    print('-------------------------------------------')
    print(f'year: {year}')
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)
# ======================================================================================================
# ======================================================================================================
# =======================================UNIT SALES==============================================
# ======================================================================================================
# ======================================================================================================

    # Build first row of df to show totals of all stores for each month of last and current year:
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select Vendor 090 for "CAP ONLY":
    AND TSVN IN ('090')
    -- Select Product Class '50' for "CAP ONLY"
    AND TSPC = '50'
    AND TSCODDEL <> 'D'
    """
    Retreads_ALL_STORES = {}
    #--------------------------------------------------------------------
    # Each Month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        cursor.execute(f"""{Base_Query}
        AND TSYP = {month}""")
        value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Retreads_ALL_STORES[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Retreads_ALL_STORES.values():
        Sum_YTD = Sum_YTD + i
    Retreads_ALL_STORES['YTD'] = Sum_YTD 


    Each_Month_List = ["YTD", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]

    for each_month in Each_Month_List:
        Retreads_ALL_STORES_for_Excel_in_Units_df[f'{each_month} {year}'] = [
                                        # Flat Repairs
                                        int(Retreads_ALL_STORES[each_month])
                                        ]

# ====================================================================================================================
# NOW that we have a row for "Retreads" total for all stores at the top of the df, we will add each store's flat repairs to the df:

years = [Last_Year, Current_Year]

    
# Flat repairs for each store
for index, store in enumerate(list_of_store_names): 
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE T
    INNER JOIN SYST S ON S.SSNUMSTORE = T.TSST
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Select Vendor 090 for "CAP ONLY":
    AND TSVN IN ('090')
    -- Select Product Class '50' for "CAP ONLY"
    AND TSPC = '50'
    AND TSCODDEL <> 'D'
    AND SSNAMSTORE = '{store}'
    """
    Retreads_EACH_STORE_last_year = {}
    Retreads_EACH_STORE_current_year = {}
    for year in years:
        Months_Numerical = []
        for x in range(1,13):
            Months_Numerical.append(year*100 + x)
    #--------------------------------------------------------------------
        # Each Month values:
        #--------------------------------------------------------------------
        # Each Month values:
        for i, month in enumerate(Months_Numerical):
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            if year == Last_Year:
                Retreads_EACH_STORE_last_year[f"{Month_Name} {year}"] = int(value)
            else:
                Retreads_EACH_STORE_current_year[f"{Month_Name} {year}"] = int(value)

        if year == Last_Year:
            Sum_YTD = 0
            for i in Retreads_EACH_STORE_last_year.values():
                Sum_YTD = Sum_YTD + i
            Retreads_EACH_STORE_last_year[f'YTD {year}'] = int(Sum_YTD)

        elif year == Current_Year:
            Sum_YTD = 0
            for i in Retreads_EACH_STORE_current_year.values():
                Sum_YTD = Sum_YTD + i
            Retreads_EACH_STORE_current_year[f'YTD {year}'] = int(Sum_YTD)            

    if year == Current_Year:
        new_line = {"": store, 
        f"YTD {Last_Year}": Retreads_EACH_STORE_last_year[f"YTD {Last_Year}"], 
        f"JAN {Last_Year}": Retreads_EACH_STORE_last_year[f"JAN {Last_Year}"], 
        f"FEB {Last_Year}": Retreads_EACH_STORE_last_year[f"FEB {Last_Year}"], 
        f"MAR {Last_Year}": Retreads_EACH_STORE_last_year[f"MAR {Last_Year}"], 
        f"APR {Last_Year}": Retreads_EACH_STORE_last_year[f"APR {Last_Year}"], 
        f"MAY {Last_Year}": Retreads_EACH_STORE_last_year[f"MAY {Last_Year}"], 
        f"JUN {Last_Year}": Retreads_EACH_STORE_last_year[f"JUN {Last_Year}"], 
        f"JUL {Last_Year}": Retreads_EACH_STORE_last_year[f"JUL {Last_Year}"], 
        f"AUG {Last_Year}": Retreads_EACH_STORE_last_year[f"AUG {Last_Year}"], 
        f"SEPT {Last_Year}": Retreads_EACH_STORE_last_year[f"SEPT {Last_Year}"], 
        f"OCT {Last_Year}": Retreads_EACH_STORE_last_year[f"OCT {Last_Year}"], 
        f"NOV {Last_Year}": Retreads_EACH_STORE_last_year[f"NOV {Last_Year}"], 
        f"DEC {Last_Year}": Retreads_EACH_STORE_last_year[f"DEC {Last_Year}"],
        f"YTD {Current_Year}": Retreads_EACH_STORE_current_year[f"YTD {Current_Year}"], 
        f"JAN {Current_Year}": Retreads_EACH_STORE_current_year[f"JAN {Current_Year}"], 
        f"FEB {Current_Year}": Retreads_EACH_STORE_current_year[f"FEB {Current_Year}"], 
        f"MAR {Current_Year}": Retreads_EACH_STORE_current_year[f"MAR {Current_Year}"], 
        f"APR {Current_Year}": Retreads_EACH_STORE_current_year[f"APR {Current_Year}"], 
        f"MAY {Current_Year}": Retreads_EACH_STORE_current_year[f"MAY {Current_Year}"], 
        f"JUN {Current_Year}": Retreads_EACH_STORE_current_year[f"JUN {Current_Year}"], 
        f"JUL {Current_Year}": Retreads_EACH_STORE_current_year[f"JUL {Current_Year}"], 
        f"AUG {Current_Year}": Retreads_EACH_STORE_current_year[f"AUG {Current_Year}"], 
        f"SEPT {Current_Year}": Retreads_EACH_STORE_current_year[f"SEPT {Current_Year}"], 
        f"OCT {Current_Year}": Retreads_EACH_STORE_current_year[f"OCT {Current_Year}"], 
        f"NOV {Current_Year}": Retreads_EACH_STORE_current_year[f"NOV {Current_Year}"], 
        f"DEC {Current_Year}": Retreads_EACH_STORE_current_year[f"DEC {Current_Year}"] 
        }

        Retreads_ALL_STORES_for_Excel_in_Units_df = Retreads_ALL_STORES_for_Excel_in_Units_df.append(new_line, ignore_index=True)


# Make sure that the current month's values are NOT included in the current year's YTD total in the df
list_to_loop = [['01', "JAN"], ['02', 'FEB'], ['03', 'MAR'], ['04', 'APR'], ['05', 'MAY'], ['06', 'JUN'], ['07', 'JUL'], ['08', 'AUG'], 
['09', 'SEPT'], ['10', 'OCT'], ['11', 'NOV'], ['12', 'DEC']]

for month_number_string, month_name in list_to_loop:
    if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
        Retreads_ALL_STORES_for_Excel_in_Units_df[f'YTD {Current_Year}'] = Retreads_ALL_STORES_for_Excel_in_Units_df[f'YTD {Current_Year}'] - Retreads_ALL_STORES_for_Excel_in_Units_df[f'{month_name} {Current_Year}']


# Remove all rows (stores) which had all zeroes for BOTH YTD years.
# However, we don't want to get rid of the first row as that's the totals row so we'll separate into two temp dfs and only apply the filter to the 2nd temp df.
# Once done, we'll concatenate them into the main df variable.
df_first_row_only = Retreads_ALL_STORES_for_Excel_in_Units_df.iloc[[0]]
df_all_rows_except_first_row = Retreads_ALL_STORES_for_Excel_in_Units_df.iloc[1:]
df_rows_without_zeros_in_both_ytds = df_all_rows_except_first_row[(df_all_rows_except_first_row[[f'YTD {Last_Year}', f'YTD {Current_Year}']] != 0).any(axis=1)]
Retreads_ALL_STORES_for_Excel_in_Units_df = pd.concat([df_first_row_only, df_rows_without_zeros_in_both_ytds])


# Sort by the current year's YTD:
Retreads_ALL_STORES_for_Excel_in_Units_df = Retreads_ALL_STORES_for_Excel_in_Units_df.sort_values(by=f"YTD {Current_Year}", ascending=False)

# Reset Index
Retreads_ALL_STORES_for_Excel_in_Units_df = Retreads_ALL_STORES_for_Excel_in_Units_df.reset_index(drop=True)



# Finally we will make DASHES for any incomplete month column:
zipped_list_of_current_year_columns_and_yyyymm = list(zip(Retreads_ALL_STORES_for_Excel_in_Units_df.columns[15:], Months_Numerical))
# now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
    # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
    if numerical_month >= Current_Month:
        Retreads_ALL_STORES_for_Excel_in_Units_df[col] = '-'


# NOW append both Current and Last year's Unit Sales to the dictionary.
# Again, this is the dict that we'll make downloadable on the Data Downloads page of the website.
Dict_for_excel_sheets["Retread Units"] = Retreads_ALL_STORES_for_Excel_in_Units_df


# View the df:
pd.set_option('display.max_columns', None)
Retreads_ALL_STORES_for_Excel_in_Units_df

-------------------------------------------
year: 2020
-------------------------------------------
year: 2021


,,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,RETREADS,272,32,27,14,19,39,14,20,24,20,26,25,12,626,11,48,30,32,18,15,116,89,185,32,43,7
1,BIRMINGHAM #211,155,32,27,14,16,29,2,3,4,0,2,22,4,421,0,25,10,7,-5,0,101,73,178,20,10,2
2,SHREVEPORT #324,59,0,0,0,0,9,8,8,4,7,16,3,4,117,10,9,17,15,18,6,10,8,5,10,4,5
3,LITTLE ROCK #335,12,0,0,0,0,0,0,0,12,0,0,0,0,20,0,8,0,0,0,0,0,0,0,0,12,0
4,HOUSTON #450,0,0,0,0,0,0,0,0,0,0,0,0,0,17,0,0,0,0,0,0,0,8,2,0,7,0
5,COLUMBIA #255,4,0,0,0,0,0,0,0,0,1,0,0,3,14,0,3,3,1,0,2,5,0,0,0,0,0
6,HOPE #338,15,0,0,0,0,0,0,4,2,7,2,0,0,13,1,3,0,5,4,0,0,0,0,0,0,0
7,JASPER #216,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,10,0
8,MERIDIAN #265,3,0,0,0,0,0,0,1,2,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0
9,NEW ORLEANS #307,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,4,0,0,0,0,0,0,0,0


In [33]:
# =========================Filter Retreads df in UNITS to a top 10 df for website ======================
Retreads_for_website_df = Retreads_ALL_STORES_for_Excel_in_Units_df



# IF YOU WANT TO SEE BOTH YEARS ON THE WEBSITE YOU COULD COMMENT OUT THIS CODE (everything between the 'equal sign' lines below):
# ================================================================================================================================
Retreads_for_website_df = Retreads_for_website_df.drop(columns = 
                        [f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', 
                        f'MAY {Last_Year}', f'JUN {Last_Year}', f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', 
                        f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}'])

# Reminder:  Again, this is only for the website.  The excel data will have both years.
# ================================================================================================================================


# Now filter out the rows (stores) which had zeroes for the year in question.  We'll do this by coding such that if any value is not zero in the YTD column, we'll keep that row:
Retreads_for_website_df = Retreads_for_website_df[(Retreads_for_website_df[[f'YTD {Current_Year}']] != 0).any(axis=1)]


# View table:
Retreads_for_website_df

,,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,RETREADS,626,11,48,30,32,18,15,116,89,185,32,43,7
1,BIRMINGHAM #211,421,0,25,10,7,-5,0,101,73,178,20,10,2
2,SHREVEPORT #324,117,10,9,17,15,18,6,10,8,5,10,4,5
3,LITTLE ROCK #335,20,0,8,0,0,0,0,0,0,0,0,12,0
4,HOUSTON #450,17,0,0,0,0,0,0,0,8,2,0,7,0
5,COLUMBIA #255,14,0,3,3,1,0,2,5,0,0,0,0,0
6,HOPE #338,13,1,3,0,5,4,0,0,0,0,0,0,0
7,JASPER #216,10,0,0,0,0,0,0,0,0,0,0,10,0
8,MERIDIAN #265,5,0,0,0,0,0,5,0,0,0,0,0,0
9,NEW ORLEANS #307,4,0,0,0,4,0,0,0,0,0,0,0,0


## Build Retreads Page for the Website

In [34]:

# We'll also save a variable called "year_depending_on_month" so that the Retreads Sales by Store table will have the correct year according to what year will be displayed in the table.



# BUILD THE HTML TABLE for the MAIN TABLE:

def create_html_table(func_argument):

    # Create headers for HTML
    row_data = '<!-- Start Retreads Table --><thead>\n<tr class="header_for_website">'

    # Create columns for HTML:    
    for column in range(func_argument.shape[1]):
        adjust_header_css = "sales_by_store_table_header_for_website"
        row_data += '\n <th class = "'+adjust_header_css+'">'+Retreads_for_website_df.columns[column]+'</th>'
    row_data += '\n</tr> \n </head> \n <tbody>'
    
    # Create Body (rows for table) for html
    for row in range(func_argument.shape[0]):
        if row in [0]:
            row_data += '\n<tr class = "ROWS_format_category_headers_for_sales_by_store_table_for_website">'
        else: row_data += '\n<tr>'
        for column in range(func_argument.shape[1]):
            td_class_name = ' COLUMNS_for_sales_by_store_for_website'
            if column in [0] and row not in [0]:
                td_class_name += " format_SUBcategory_rows_for_sales_by_store_table_for_website"
            if column not in [0] and row not in []:
                td_class_name += " center_for_website"
            row_data += '\n <td class = "'+td_class_name+'">'+str(func_argument.iloc[row,column])+'</td>'
    row_data += '\n</tr> \n</tbody><!-- End Retreads Table -->'
    # -------------------------------------------------------------------------------------------------

    return row_data

# Call the function:
Retreads_html_table = create_html_table(Retreads_for_website_df)




Create_html = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
{string_for_header}
<!-----------------------RETREAD SALES by STORE TABLE PAGE---------------------------------------------->
    <br>
    <br>
    <h1 class=page_title_for_Sales_by_Store_for_website>{Current_Year} RETREAD PURCHASES BY STORE</h1>
    <br>
    <table class = "Retreads_sales_by_store_for_website">
        <!-- Start Retreads Table -->
        [[DATA_MAIN_TABLE]]
        <!-- End Retreads Table -->
    </table>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>

</body>

</html>

"""

# ADD THE SALES TABLE:
# index_exists = os.path.exists('./index_NAV_BAR_files/Sales_Table.html')
# current_index = ''
# with open('./index_NAV_BAR_files/Sales_Table.html', 'r') as f:
#     current_index = f.read()
# with open('./index_NAV_BAR_files/Sales_Table.html', 'w') as f:
#     f.write(re.sub(r"<\!-- Start NRT Table -->.*<\!-- End NRT Table -->",
#             html_table2, current_index, flags=re.DOTALL))


# ADD THE SALES TABLE:
html_page_exists = os.path.exists('Retread_Sales_by_Store.html')
current_index = ''
with open('Retread_Sales_by_Store.html', 'r') as f:
    current_index = f.read()
with open('Retread_Sales_by_Store.html', 'w') as f:
    f.write(re.sub(r"<\!-- Start Retreads Table -->.*<\!-- End Retreads Table -->", Retreads_html_table, Create_html, flags=re.DOTALL))

print(html_page_exists)

True


## ========================================================================
### ++++++++++++++++++++++++++++ CAP + CASING ++++++++++++++++++++++++
## ========================================================================

## Retrieve data to build table for excel 
### (this table for excel will include last year and current year AND all stores which had data in either year)

In [35]:
# Create a df template for Cap_and_Casings of all the stores for units df (one row):
Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df = pd.DataFrame({'': ['CAP+CASINGS']})

years = [Last_Year, Current_Year]


for year in years:
    print('-------------------------------------------')
    print(f'year: {year}')
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)
# ======================================================================================================
# ======================================================================================================
# =======================================UNIT SALES==============================================
# ======================================================================================================
# ======================================================================================================

    # Build first row of df to show totals of all stores for each month of last and current year:
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    --Select Vendor '091' for Bandag Cap+Casing
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '091')
    --Select Class '51' for "BANDAG RETREAD & CASING":
    AND TSPC = '51'
    AND TSCODDEL <> 'D'
    """
    Cap_and_Casings_ALL_STORES = {}
    #--------------------------------------------------------------------
    # Each Month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        cursor.execute(f"""{Base_Query}
        AND TSYP = {month}""")
        value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Cap_and_Casings_ALL_STORES[Month_Name] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Cap_and_Casings_ALL_STORES.values():
        Sum_YTD = Sum_YTD + i
    Cap_and_Casings_ALL_STORES['YTD'] = Sum_YTD 


    Each_Month_List = ["YTD", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]

    for each_month in Each_Month_List:
        Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df[f'{each_month} {year}'] = [
                                        # Flat Repairs
                                        int(Cap_and_Casings_ALL_STORES[each_month])
                                        ]

# ====================================================================================================================
# NOW that we have a row for "Cap_and_Casings" total for all stores at the top of the df, we will add each store's flat repairs to the df:

years = [Last_Year, Current_Year]

    
# Flat repairs for each store
for index, store in enumerate(list_of_store_names): 
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE T
    INNER JOIN SYST S ON S.SSNUMSTORE = T.TSST
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    --Select Vendor '091' for Bandag Cap+Casing
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDVENDOR = '091')
    --Select Class '51' for "BANDAG RETREAD & CASING":
    AND TSPC = '51'
    AND TSCODDEL <> 'D'
    AND SSNAMSTORE = '{store}'
    """
    Cap_and_Casings_EACH_STORE_last_year = {}
    Cap_and_Casings_EACH_STORE_current_year = {}
    for year in years:
        Months_Numerical = []
        for x in range(1,13):
            Months_Numerical.append(year*100 + x)
    #--------------------------------------------------------------------
        # Each Month values:
        #--------------------------------------------------------------------
        # Each Month values:
        for i, month in enumerate(Months_Numerical):
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            if year == Last_Year:
                Cap_and_Casings_EACH_STORE_last_year[f"{Month_Name} {year}"] = int(value)
            else:
                Cap_and_Casings_EACH_STORE_current_year[f"{Month_Name} {year}"] = int(value)

        if year == Last_Year:
            Sum_YTD = 0
            for i in Cap_and_Casings_EACH_STORE_last_year.values():
                Sum_YTD = Sum_YTD + i
            Cap_and_Casings_EACH_STORE_last_year[f'YTD {year}'] = int(Sum_YTD)

        elif year == Current_Year:
            Sum_YTD = 0
            for i in Cap_and_Casings_EACH_STORE_current_year.values():
                Sum_YTD = Sum_YTD + i
            Cap_and_Casings_EACH_STORE_current_year[f'YTD {year}'] = int(Sum_YTD)            

    if year == Current_Year:
        new_line = {"": store, 
        f"YTD {Last_Year}": Cap_and_Casings_EACH_STORE_last_year[f"YTD {Last_Year}"], 
        f"JAN {Last_Year}": Cap_and_Casings_EACH_STORE_last_year[f"JAN {Last_Year}"], 
        f"FEB {Last_Year}": Cap_and_Casings_EACH_STORE_last_year[f"FEB {Last_Year}"], 
        f"MAR {Last_Year}": Cap_and_Casings_EACH_STORE_last_year[f"MAR {Last_Year}"], 
        f"APR {Last_Year}": Cap_and_Casings_EACH_STORE_last_year[f"APR {Last_Year}"], 
        f"MAY {Last_Year}": Cap_and_Casings_EACH_STORE_last_year[f"MAY {Last_Year}"], 
        f"JUN {Last_Year}": Cap_and_Casings_EACH_STORE_last_year[f"JUN {Last_Year}"], 
        f"JUL {Last_Year}": Cap_and_Casings_EACH_STORE_last_year[f"JUL {Last_Year}"], 
        f"AUG {Last_Year}": Cap_and_Casings_EACH_STORE_last_year[f"AUG {Last_Year}"], 
        f"SEPT {Last_Year}": Cap_and_Casings_EACH_STORE_last_year[f"SEPT {Last_Year}"], 
        f"OCT {Last_Year}": Cap_and_Casings_EACH_STORE_last_year[f"OCT {Last_Year}"], 
        f"NOV {Last_Year}": Cap_and_Casings_EACH_STORE_last_year[f"NOV {Last_Year}"], 
        f"DEC {Last_Year}": Cap_and_Casings_EACH_STORE_last_year[f"DEC {Last_Year}"],
        f"YTD {Current_Year}": Cap_and_Casings_EACH_STORE_current_year[f"YTD {Current_Year}"], 
        f"JAN {Current_Year}": Cap_and_Casings_EACH_STORE_current_year[f"JAN {Current_Year}"], 
        f"FEB {Current_Year}": Cap_and_Casings_EACH_STORE_current_year[f"FEB {Current_Year}"], 
        f"MAR {Current_Year}": Cap_and_Casings_EACH_STORE_current_year[f"MAR {Current_Year}"], 
        f"APR {Current_Year}": Cap_and_Casings_EACH_STORE_current_year[f"APR {Current_Year}"], 
        f"MAY {Current_Year}": Cap_and_Casings_EACH_STORE_current_year[f"MAY {Current_Year}"], 
        f"JUN {Current_Year}": Cap_and_Casings_EACH_STORE_current_year[f"JUN {Current_Year}"], 
        f"JUL {Current_Year}": Cap_and_Casings_EACH_STORE_current_year[f"JUL {Current_Year}"], 
        f"AUG {Current_Year}": Cap_and_Casings_EACH_STORE_current_year[f"AUG {Current_Year}"], 
        f"SEPT {Current_Year}": Cap_and_Casings_EACH_STORE_current_year[f"SEPT {Current_Year}"], 
        f"OCT {Current_Year}": Cap_and_Casings_EACH_STORE_current_year[f"OCT {Current_Year}"], 
        f"NOV {Current_Year}": Cap_and_Casings_EACH_STORE_current_year[f"NOV {Current_Year}"], 
        f"DEC {Current_Year}": Cap_and_Casings_EACH_STORE_current_year[f"DEC {Current_Year}"] 
        }

        Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df = Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df.append(new_line, ignore_index=True)


# Make sure that the current month's values are NOT included in the current year's YTD total in the df
list_to_loop = [['01', "JAN"], ['02', 'FEB'], ['03', 'MAR'], ['04', 'APR'], ['05', 'MAY'], ['06', 'JUN'], ['07', 'JUL'], ['08', 'AUG'], 
['09', 'SEPT'], ['10', 'OCT'], ['11', 'NOV'], ['12', 'DEC']]

for month_number_string, month_name in list_to_loop:
    if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
        Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df[f'YTD {Current_Year}'] = Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df[f'YTD {Current_Year}'] - Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df[f'{month_name} {Current_Year}']


# Remove all rows (stores) which had all zeroes for BOTH YTD years.
# However, we don't want to get rid of the first row as that's the totals row so we'll separate into two temp dfs and only apply the filter to the 2nd temp df.
# Once done, we'll concatenate them into the main df variable.
df_first_row_only = Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df.iloc[[0]]
df_all_rows_except_first_row = Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df.iloc[1:]
df_rows_without_zeros_in_both_ytds = df_all_rows_except_first_row[(df_all_rows_except_first_row[[f'YTD {Last_Year}', f'YTD {Current_Year}']] != 0).any(axis=1)]
Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df = pd.concat([df_first_row_only, df_rows_without_zeros_in_both_ytds])


# Sort by the current year's YTD:
Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df = Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df.sort_values(by=f"YTD {Current_Year}", ascending=False)

# Reset Index
Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df = Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df.reset_index(drop=True)



# Finally we will make DASHES for any incomplete month column:
zipped_list_of_current_year_columns_and_yyyymm = list(zip(Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df.columns[15:], Months_Numerical))
# now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
    # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
    if numerical_month >= Current_Month:
        Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df[col] = '-'



# NOW append both Current and Last year's Unit Sales to the dictionary.
# Again, this is the dict that we'll make downloadable on the Data Downloads page of the website.
Dict_for_excel_sheets["Cap+Casings Units"] = Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df


# View the df:
pd.set_option('display.max_columns', None)
Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df

-------------------------------------------
year: 2020
-------------------------------------------
year: 2021


,,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,CAP+CASINGS,634,25,44,62,75,41,42,66,88,60,41,40,50,708,38,26,53,73,82,80,42,96,30,68,70,50
1,MERIDIAN #265,14,0,0,1,1,0,2,3,0,5,2,0,0,56,0,1,4,4,7,11,1,8,4,12,1,3
2,BIRMINGHAM #211,14,0,1,1,0,0,4,1,1,1,1,4,0,52,2,1,3,3,27,3,2,0,2,2,4,3
3,MEMPHIS #249,110,3,0,1,16,2,4,0,27,13,10,13,21,49,15,6,7,5,4,4,1,2,1,2,0,2
4,HOUSTON #450,15,2,5,3,2,1,0,0,2,0,0,0,0,44,1,0,4,4,9,0,0,22,0,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,SAN ANGELO #493,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
85,ODESSA #495,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
86,BRYAN #459,4,0,0,1,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
87,FORT SMITH #345,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Prep Cap+Casing data for webpage
### (to show only current year unless it's january then show only last year))
### Also, we'll get rid of any rows which have 0 for the ytd column

In [36]:
# =========================Filter df in UNITS to a top 10 df for website ======================
Cap_and_Casings_for_website_df = Cap_and_Casings_ALL_STORES_for_Excel_in_Units_df


# IF YOU WANT TO SEE BOTH YEARS ON THE WEBSITE YOU COULD COMMENT OUT THIS CODE (everything between the 'equal sign' lines below):
# ================================================================================================================================
Cap_and_Casings_for_website_df = Cap_and_Casings_for_website_df.drop(columns = 
                        [f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', 
                        f'MAY {Last_Year}', f'JUN {Last_Year}', f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', 
                        f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}'])

# Reminder:  Again, this is only for the website.  The excel data will have both years.
# ================================================================================================================================

# Now filter out the rows (stores) which had zeroes for the year in question.  We'll do this by coding such that if any value is not zero in the YTD column, we'll keep that row:
Cap_and_Casings_for_website_df = Cap_and_Casings_for_website_df[(Cap_and_Casings_for_website_df[[f'YTD {Current_Year}']] != 0).any(axis=1)]


# View table:
Cap_and_Casings_for_website_df

,,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,CAP+CASINGS,708,38,26,53,73,82,80,42,96,30,68,70,50
1,MERIDIAN #265,56,0,1,4,4,7,11,1,8,4,12,1,3
2,BIRMINGHAM #211,52,2,1,3,3,27,3,2,0,2,2,4,3
3,MEMPHIS #249,49,15,6,7,5,4,4,1,2,1,2,0,2
4,HOUSTON #450,44,1,0,4,4,9,0,0,22,0,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,AUSTIN SOUTH #466,1,0,0,0,0,0,1,0,0,0,0,0,0
76,HOT SPRINGS #337,1,0,0,0,0,0,0,0,0,0,1,0,0
77,PENSACOLA #142,1,0,0,0,0,0,0,1,0,0,0,0,0
78,GAINESVILLE #413,1,0,0,0,0,0,1,0,0,0,0,0,0


## Build Cap+Casings page for the Website

In [37]:

# We'll also save a variable called "year_depending_on_month" so that the Retreads Sales by Store table will have the correct year according to what year will be displayed in the table.



# BUILD THE HTML TABLE for the MAIN TABLE:

def create_html_table(func_argument):

    # Create headers for HTML
    row_data = '<!-- Start Cap_and_Casing Table --><thead>\n<tr class="header_for_website">'

    # Create columns for HTML:    
    for column in range(func_argument.shape[1]):
        adjust_header_css = "sales_by_store_table_header_for_website"
        row_data += '\n <th class = "'+adjust_header_css+'">'+Cap_and_Casings_for_website_df.columns[column]+'</th>'
    row_data += '\n</tr> \n </head> \n <tbody>'
    
    # Create Body (rows for table) for html
    for row in range(func_argument.shape[0]):
        if row in [0]:
            row_data += '\n<tr class = "ROWS_format_category_headers_for_sales_by_store_table_for_website">'
        else: row_data += '\n<tr>'
        for column in range(func_argument.shape[1]):
            td_class_name = ''
            if column in [0] and row not in [0]:
                td_class_name = "format_SUBcategory_rows_for_sales_by_store_table_for_website"
            if column not in [0] and row not in []:
                td_class_name += " center_for_website"
            row_data += '\n <td class = "'+td_class_name+'">'+str(func_argument.iloc[row,column])+'</td>'
    row_data += '\n</tr> \n</tbody><!-- End Cap_and_Casing Table -->'
    # -------------------------------------------------------------------------------------------------

    return row_data

# Call the function:
Cap_and_Casing_html_table = create_html_table(Cap_and_Casings_for_website_df)




Create_html = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
{string_for_header}
<!-----------------------CAP+CASING SALES by STORE TABLE PAGE---------------------------------------------->
    <br>
    <br>
    <h1 class=page_title_smaller_for_Sales_by_Store_for_website>{Current_Year} CAP + CASING PURCHASES BY STORE</h1>
    <br>
    <table class = "Cap_and_Casing_sales_by_store_for_website">
        <!-- Start Cap_and_Casing Table -->
        [[DATA_MAIN_TABLE]]
        <!-- End Cap_and_Casing Table -->
    </table>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>

</body>

</html>

"""


# ADD THE TABLE to the HTML page where it says "DATA_MAIN_TABLE":
html_page_exists = os.path.exists('Cap_and_Casing_Sales_by_Store.html')
current_index = ''
with open('Cap_and_Casing_Sales_by_Store.html', 'r') as f:
    current_index = f.read()
with open('Cap_and_Casing_Sales_by_Store.html', 'w') as f:
    f.write(re.sub(r"<\!-- Start Cap_and_Casing Table -->.*<\!-- End Cap_and_Casing Table -->", Cap_and_Casing_html_table, Create_html, flags=re.DOTALL))

print(html_page_exists)

True


## ==================================================
### ++++++++++++++++ NEW TIRES by store +++++++++++++
## ==================================================

### Get new tires by Store for each vendor requested.
### We'll loop through each one and create a webpage for each vender with all the store sales in order

In [38]:
zipped_vendor_names_with_numbers = list(zip(vendor_names_list, vendor_numbers))

for vendor_name, vendor_number in zipped_vendor_names_with_numbers:
    print(vendor_name, vendor_number)


Bridgestone 007
Continental 021
Yokohama 005


In [39]:
# Dict_Vendor_Sales_by_Store = {}

Dict_for_vendor_by_store_for_website = {}

for vendor_name, vendor_number in zipped_vendor_names_with_numbers:


    # First, start with the first ("Total") row:
    Vendor_df = pd.DataFrame({'': [f'{vendor_name.upper()} NEW TIRES']})

    years = [Last_Year, Current_Year]


    for year in years:
        print('-------------------------------------------')
        print(f'year: {year}')
        Months_Numerical = []
        for x in range(1,13):
            Months_Numerical.append(year*100 + x)
    # ======================================================================================================
    # ======================================================================================================
    # =======================================UNIT SALES==============================================
    # ======================================================================================================
    # ======================================================================================================

        # Build first row of df to show totals of all stores for each month of last and current year:
        Base_Query = f"""
        SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
        WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
        -- Get rid of Bill Hold, get rid of anything that is commission lines or FET lines (For govt transactions)
        AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE TRIM(PDNUMBER) NOT LIKE '%BH' AND PDFLDUSR4 <> 'BH' AND PDNUMBER NOT LIKE 'FET%' AND PDDESCRIP NOT LIKE '%COMM%' AND PDFLDUSR1 = '')
        -- Select all relevant product classes (01-09 for Passenger/Light Truck, 10,11,12,14 for Medium Truck, 16 (ATV), 17(Lawn & Garden), 20 (INDUSTRIAL), 21 (FARM) and 25-26 for OTRs)
        AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDCLASS IN ('01','02','03','04','05','06','07','08','09','10','11','12','14','16','17','20','21', '25','26'))
        AND TSVN = '{vendor_number}'
        AND TSCODDEL <> 'D'
        """
        Vendor_New_Tires_ALL_STORES = {}
        #--------------------------------------------------------------------
        # Each Month values:
        for i, month in enumerate(Months_Numerical):
            value = 0
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            Vendor_New_Tires_ALL_STORES[Month_Name] = value
        # Get SUM of YTD Values by adding up each month value:
        Sum_YTD = 0
        for i in Vendor_New_Tires_ALL_STORES.values():
            Sum_YTD = Sum_YTD + i
        Vendor_New_Tires_ALL_STORES['YTD'] = Sum_YTD 


        Each_Month_List = ["YTD", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEPT", "OCT", "NOV", "DEC"]

        for each_month in Each_Month_List:
            Vendor_df[f'{each_month} {year}'] = [
                                            # Flat Repairs
                                            int(Vendor_New_Tires_ALL_STORES[each_month])
                                            ]

    # ====================================================================================================================
    # NOW that we have a row for "Vendor_New_Tires" total for all stores at the top of the df, we will add each store's flat repairs to the df:

    years = [Last_Year, Current_Year]

        
    # Flat repairs for each store
    for index, store in enumerate(list_of_store_names): 
        Base_Query = f"""
        SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE T
        INNER JOIN SYST S ON S.SSNUMSTORE = T.TSST
        WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
        -- Get rid of Bill Hold, get rid of anything that is commission lines or FET lines (For govt transactions)
        AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE TRIM(PDNUMBER) NOT LIKE '%BH' AND PDFLDUSR4 <> 'BH' AND PDNUMBER NOT LIKE 'FET%' AND PDDESCRIP NOT LIKE '%COMM%' AND PDFLDUSR1 = '')
        -- Select all relevant product classes (01-09 for Passenger/Light Truck, 10,11,12,14 for Medium Truck, 16 (ATV), 17(Lawn & Garden), 20 (INDUSTRIAL), 21 (FARM) and 25-26 for OTRs)
        AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDCLASS IN ('01','02','03','04','05','06','07','08','09','10','11','12','14','16','17','20','21', '25','26'))
        AND TSVN = '{vendor_number}'
        AND TSCODDEL <> 'D'
        AND SSNAMSTORE = '{store}'
        """
        Vendor_New_Tires_EACH_STORE_last_year = {}
        Vendor_New_Tires_EACH_STORE_current_year = {}
        for year in years:
            Months_Numerical = []
            for x in range(1,13):
                Months_Numerical.append(year*100 + x)
        #--------------------------------------------------------------------
            # Each Month values:
            #--------------------------------------------------------------------
            # Each Month values:
            for i, month in enumerate(Months_Numerical):
                cursor.execute(f"""{Base_Query}
                AND TSYP = {month}""")
                value = cursor.fetchone()[0]
                Month_Name = Months_Strings[i]
                if year == Last_Year:
                    Vendor_New_Tires_EACH_STORE_last_year[f"{Month_Name} {year}"] = int(value)
                else:
                    Vendor_New_Tires_EACH_STORE_current_year[f"{Month_Name} {year}"] = int(value)

            if year == Last_Year:
                Sum_YTD = 0
                for i in Vendor_New_Tires_EACH_STORE_last_year.values():
                    Sum_YTD = Sum_YTD + i
                Vendor_New_Tires_EACH_STORE_last_year[f'YTD {year}'] = int(Sum_YTD)

            elif year == Current_Year:
                Sum_YTD = 0
                for i in Vendor_New_Tires_EACH_STORE_current_year.values():
                    Sum_YTD = Sum_YTD + i
                Vendor_New_Tires_EACH_STORE_current_year[f'YTD {year}'] = int(Sum_YTD)            

        if year == Current_Year:
            new_line = {"": store, 
            f"YTD {Last_Year}": Vendor_New_Tires_EACH_STORE_last_year[f"YTD {Last_Year}"], 
            f"JAN {Last_Year}": Vendor_New_Tires_EACH_STORE_last_year[f"JAN {Last_Year}"], 
            f"FEB {Last_Year}": Vendor_New_Tires_EACH_STORE_last_year[f"FEB {Last_Year}"], 
            f"MAR {Last_Year}": Vendor_New_Tires_EACH_STORE_last_year[f"MAR {Last_Year}"], 
            f"APR {Last_Year}": Vendor_New_Tires_EACH_STORE_last_year[f"APR {Last_Year}"], 
            f"MAY {Last_Year}": Vendor_New_Tires_EACH_STORE_last_year[f"MAY {Last_Year}"], 
            f"JUN {Last_Year}": Vendor_New_Tires_EACH_STORE_last_year[f"JUN {Last_Year}"], 
            f"JUL {Last_Year}": Vendor_New_Tires_EACH_STORE_last_year[f"JUL {Last_Year}"], 
            f"AUG {Last_Year}": Vendor_New_Tires_EACH_STORE_last_year[f"AUG {Last_Year}"], 
            f"SEPT {Last_Year}": Vendor_New_Tires_EACH_STORE_last_year[f"SEPT {Last_Year}"], 
            f"OCT {Last_Year}": Vendor_New_Tires_EACH_STORE_last_year[f"OCT {Last_Year}"], 
            f"NOV {Last_Year}": Vendor_New_Tires_EACH_STORE_last_year[f"NOV {Last_Year}"], 
            f"DEC {Last_Year}": Vendor_New_Tires_EACH_STORE_last_year[f"DEC {Last_Year}"],
            f"YTD {Current_Year}": Vendor_New_Tires_EACH_STORE_current_year[f"YTD {Current_Year}"], 
            f"JAN {Current_Year}": Vendor_New_Tires_EACH_STORE_current_year[f"JAN {Current_Year}"], 
            f"FEB {Current_Year}": Vendor_New_Tires_EACH_STORE_current_year[f"FEB {Current_Year}"], 
            f"MAR {Current_Year}": Vendor_New_Tires_EACH_STORE_current_year[f"MAR {Current_Year}"], 
            f"APR {Current_Year}": Vendor_New_Tires_EACH_STORE_current_year[f"APR {Current_Year}"], 
            f"MAY {Current_Year}": Vendor_New_Tires_EACH_STORE_current_year[f"MAY {Current_Year}"], 
            f"JUN {Current_Year}": Vendor_New_Tires_EACH_STORE_current_year[f"JUN {Current_Year}"], 
            f"JUL {Current_Year}": Vendor_New_Tires_EACH_STORE_current_year[f"JUL {Current_Year}"], 
            f"AUG {Current_Year}": Vendor_New_Tires_EACH_STORE_current_year[f"AUG {Current_Year}"], 
            f"SEPT {Current_Year}": Vendor_New_Tires_EACH_STORE_current_year[f"SEPT {Current_Year}"], 
            f"OCT {Current_Year}": Vendor_New_Tires_EACH_STORE_current_year[f"OCT {Current_Year}"], 
            f"NOV {Current_Year}": Vendor_New_Tires_EACH_STORE_current_year[f"NOV {Current_Year}"], 
            f"DEC {Current_Year}": Vendor_New_Tires_EACH_STORE_current_year[f"DEC {Current_Year}"] 
            }

      
            Vendor_df = Vendor_df.append(new_line, ignore_index=True)


            # Make sure that the current month's values are NOT included in the current year's YTD total in the df
            list_to_loop = [['01', "JAN"], ['02', 'FEB'], ['03', 'MAR'], ['04', 'APR'], ['05', 'MAY'], ['06', 'JUN'], ['07', 'JUL'], ['08', 'AUG'], 
            ['09', 'SEPT'], ['10', 'OCT'], ['11', 'NOV'], ['12', 'DEC']]

            for month_number_string, month_name in list_to_loop:
                if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
                    Vendor_df[f'YTD {Current_Year}'] = Vendor_df[f'YTD {Current_Year}'] - Vendor_df[f'{month_name} {Current_Year}']



            # Remove all rows (stores) which had all zeroes for BOTH YTD years. 
            # # However, we don't want to get rid of the first row as that's the totals row so we'll separate into two temp dfs and only apply the filter to the 2nd temp df.
            # Once done, we'll concatenate them into the main df variable.
            df_first_row_only = Vendor_df.iloc[[0]]
            df_all_rows_except_first_row = Vendor_df.iloc[1:]
            df_rows_without_zeros_in_both_ytds = df_all_rows_except_first_row[(df_all_rows_except_first_row[[f'YTD {Last_Year}', f'YTD {Current_Year}']] != 0).any(axis=1)]
            Vendor_df = pd.concat([df_first_row_only, df_rows_without_zeros_in_both_ytds])



            # Sort by the current year's YTD:
            Vendor_df = Vendor_df.sort_values(by=f"YTD {Current_Year}", ascending=False)

            # Reset Index
            Vendor_df = Vendor_df.reset_index(drop=True)


            # Finally we will make DASHES for any incomplete month column:
            zipped_list_of_current_year_columns_and_yyyymm = list(zip(Vendor_df.columns[15:], Months_Numerical))
            # now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
            for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
                # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
                if numerical_month >= Current_Month:
                    Vendor_df[col] = '-'



            # NOW append both Current and Last year's Unit Sales to the dictionary.
            # Again, this is the dict that we'll make downloadable on the Data Downloads page of the website.
            Dict_for_excel_sheets[f"{vendor_name} New Tires (Units)"] = Vendor_df


            # ==================================================================== Now format for the website ===============================================================

            # Dict_for_vendor_by_store_for_website[f'{vendor_name}'] = Vendor_df


            # IF YOU WANT TO SEE BOTH YEARS ON THE WEBSITE YOU COULD COMMENT OUT THIS CODE (everything between the 'equal sign' lines below):
            # ================================================================================================================================

            Vendor_df = Vendor_df.drop(columns = 
                    [f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', 
                    f'MAY {Last_Year}', f'JUN {Last_Year}', f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', 
                    f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}'])

            # Reminder:  Again, this is only for the website.  The excel data will have both years.
            # ================================================================================================================================

            # Now filter out the rows (stores) which had zeroes for the year in question.
            Vendor_df = Vendor_df[(Vendor_df[[f'YTD {Current_Year}']] != 0).any(axis=1)]



            # ==================================================BUILD HTML for each vendor's Sales by Store's page:

            def create_html_table(func_argument):

                # Create headers for HTML
                row_data = f'<!-- Start Vendor_New_Tires Table --><thead>\n<tr class="header_for_website">'

                # Create columns for HTML:    
                for column in range(func_argument.shape[1]):
                    adjust_header_css = "sales_by_store_table_header_for_website"
                    row_data += '\n <th class = "'+adjust_header_css+'">'+Vendor_df.columns[column]+'</th>'
                row_data += '\n</tr> \n </head> \n <tbody>'
                
                # Create Body (rows for table) for html
                for row in range(func_argument.shape[0]):
                    if row in [0]:
                        row_data += '\n<tr class = "ROWS_format_category_headers_for_sales_by_store_table_for_website">'
                    else: row_data += '\n<tr>'
                    for column in range(func_argument.shape[1]):
                        td_class_name = ' COLUMNS_for_sales_by_store_for_website'
                        if column in [0] and row not in [0]:
                            td_class_name += " format_SUBcategory_rows_for_sales_by_store_table_for_website"
                        if column not in [0] and row not in []:
                            td_class_name += " center_for_website"
                        row_data += '\n <td class = "'+td_class_name+'">'+str(func_argument.iloc[row,column])+'</td>'
                row_data += '\n</tr> \n</tbody><!-- End Vendor_New_Tires Table -->'
                # -------------------------------------------------------------------------------------------------

                return row_data

            # Call the function:
            Vendor_New_Tires_html_table = create_html_table(Vendor_df)




            Create_html = f"""

            <!DOCTYPE html>
            <html lang="en">

            <head>
                <meta charset="UTF-8" />
                <meta name="viewport" content="width=device-width, initial-scale=1.0" />
                <meta http-equiv="X-UA-Compatible" content="ie=edge" />
                <!--Bootstrap CSS using CDN-->
                <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
                <link rel="stylesheet" href="css/main.css">

                <title>Report</title>
            </head>

            <body class = 'container'>
            {string_for_header}
            <!-----------------------VENDOR NEW TIRE SALES by STORE TABLE PAGE---------------------------------------------->
                <br>
                <br>
                <h1 class=page_title_smaller_for_Sales_by_Store_for_website>{Current_Year} {vendor_name.upper()} PURCHASES BY STORE</h1>
                <br>
                <table class = "Cap_and_Casing_sales_by_store_for_website">
                    <!-- Start Vendor_New_Tires Table -->
                    [[DATA_MAIN_TABLE]]
                    <!-- End Vendor_New_Tires Table -->
                </table>
                <br>
                <br>
                <br>
                <br>
                <br>
                <br>
                <br>

            </body>

            </html>

            """



            # ADD THE Bridgestone new tire sales by store TABLE:
            html_page_exists = os.path.exists(f'{vendor_name}_New_Tires_Sales_by_Store.html')
            current_index = ''
            with open(f'{vendor_name}_New_Tires_Sales_by_Store.html', 'r') as f:
                current_index = f.read()
            with open(f'{vendor_name}_New_Tires_Sales_by_Store.html', 'w') as f:
                f.write(re.sub(r"<\!-- Start Vendor_New_Tires Table -->.*<\!-- End Vendor_New_Tires Table -->", Vendor_New_Tires_html_table, Create_html, flags=re.DOTALL))

            print(html_page_exists)

    

-------------------------------------------
year: 2020
-------------------------------------------
year: 2021
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
-------------------------------------------
year: 2020
-------------------------------------------
year: 2021
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


### ===========================================================================
### ===========================================================================
### =====================Build "SALES BY TIRE SIZE - ALL VENDORS" table========
#### ==========================================================================
### ===========================================================================
### ===========================================================================

In [40]:
# DYNAMICALLLY BUILD A DATAFRAME TEMPLATE WITH ONE COLUMN BASES ON WHICH TIRE SIZES WERE SPECIFIED.

# START BY CREATING A LIST SO THAT WE CAN USE IT AS THE FIRST COLUMN FOR A DATAFRAME TEMPLATE:


list_of_tire_sizes_for_first_template = ['RETREADS']

for each_tire_size in list_of_tire_sizes:
    # Note!  We'll put "RETREAD" TEMPORARILY at the end of each tire size (we'll remove this later).
    # This is to ensure each tire size row will be unique when we add data to that column:
    list_of_tire_sizes_for_first_template.append(f'{each_tire_size} RTEMP')
    
list_of_tire_sizes_for_first_template.append('OTHER RETREADS RTEMP')
list_of_tire_sizes_for_first_template.append('CAP+CASINGS')

for each_tire_size in list_of_tire_sizes:
    list_of_tire_sizes_for_first_template.append(f'{each_tire_size} CCTEMP')

list_of_tire_sizes_for_first_template.append('OTHER CAP+CASINGS CCTEMP')
list_of_tire_sizes_for_first_template.append('REPAIR ONLY')
list_of_tire_sizes_for_first_template.append('NEW TIRES (All Vendors)')

for each_tire_size in list_of_tire_sizes:
    list_of_tire_sizes_for_first_template.append(f'{each_tire_size} NEWTEMP')

list_of_tire_sizes_for_first_template.append('OTHER NEW TIRES (All Vendors) NEWTEMP')

dash_list = []
for each in range(len(list_of_tire_sizes_for_first_template)):
        dash_list.append('-')

template = {
    
'Category': list_of_tire_sizes_for_first_template, 
f'YTD {Last_Year}': dash_list,
f'JAN {Last_Year}': dash_list,
f'FEB {Last_Year}': dash_list,
f'MAR {Last_Year}': dash_list,
f'APR {Last_Year}': dash_list,
f'MAY {Last_Year}': dash_list,
f'JUN {Last_Year}': dash_list,
f'JUL {Last_Year}': dash_list,
f'AUG {Last_Year}': dash_list,
f'SEPT {Last_Year}': dash_list,
f'OCT {Last_Year}': dash_list,
f'NOV {Last_Year}': dash_list,
f'DEC {Last_Year}': dash_list,
f'YTD {Last_Year}': dash_list,
f'JAN {Current_Year}': dash_list,
f'FEB {Current_Year}': dash_list,
f'MAR {Current_Year}': dash_list,
f'APR {Current_Year}': dash_list,
f'MAY {Current_Year}': dash_list,
f'JUN {Current_Year}': dash_list,
f'JUL {Current_Year}': dash_list,
f'AUG {Current_Year}': dash_list,
f'SEPT {Current_Year}': dash_list,
f'OCT {Current_Year}': dash_list,
f'NOV {Current_Year}': dash_list,
f'DEC {Current_Year}': dash_list
}

Tire_Sales_Summary_Table_for_Excel_in_Units_df = pd.DataFrame(template)

test = Tire_Sales_Summary_Table_for_Excel_in_Units_df['Category'].to_list()
# # Make sure the columns don't have any 'white space' on either end:
Tire_Sales_Summary_Table_for_Excel_in_Units_df['Category'] = Tire_Sales_Summary_Table_for_Excel_in_Units_df['Category'].str.rstrip()

# Make "Category" the index instead of the first column.  This is so we can use "at" method easily later.
Tire_Sales_Summary_Table_for_Excel_in_Units_df = Tire_Sales_Summary_Table_for_Excel_in_Units_df.set_index('Category')


Tire_Sales_Summary_Table_for_Excel_in_Units_df


,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
Category,,,,,,,,,,,,,,,,,,,,,,,,,
RETREADS,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
295/75R22.5 RTEMP,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
11R22.5 RTEMP,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
OTHER RETREADS RTEMP,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
CAP+CASINGS,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
295/75R22.5 CCTEMP,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
11R22.5 CCTEMP,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
OTHER CAP+CASINGS CCTEMP,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
REPAIR ONLY,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


In [41]:

zipped_tire_sizes_and_pdsize_codes = list(zip(list_of_tire_sizes, list_of_pdsize_codes))

years = [Last_Year, Current_Year]

# Note 10 is Steer, 11 is Drive, 12 is Trailer
product_class_numbers_for_wheel_position = ['10', '11', '12'] 

for year in years:
    print('-------------------------------------------')
    print(f'year: {year}')
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)
    print("Months_Numerical: ", Months_Numerical)



    # ============================================================================================================================================================
    # +++++++++++++++++++++++++ BANDAG RETREADS ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ============================================================================================================================================================
    
    # Retreads (all sizes total row)
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = {Enterprise_Number})
    --select only retreads:
    AND TSVN = '090'
    AND TSPC = '50'
    AND TSCODDEL <> 'D'
    """
    Retreads_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Retreads_Columns[f'{Month_Name} {year}'] = int(value)
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Retreads_Columns.values():
        Sum_YTD = Sum_YTD + i
    Retreads_Columns[f'YTD {year}'] = Sum_YTD


    # Update the first row for the dataframe for the "RETREADS" (total row)
    for key, value in Retreads_Columns.items():
        Tire_Sales_Summary_Table_for_Excel_in_Units_df.at['RETREADS', key] = int(value)

    if year == Current_Year:
    # Reorder columns:
        Tire_Sales_Summary_Table_for_Excel_in_Units_df = Tire_Sales_Summary_Table_for_Excel_in_Units_df[[f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', 
        f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}', 
        f'YTD {Current_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', 
        f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', f'NOV {Current_Year}', f'DEC {Current_Year}']]

    # ==========================================================================
    #-------------------RETREADS PER TIRE SIZE----------------------------------
    # For each tire size we'll loop through them dynamically as each customer may have different sizes and number of sizes:
    for tire_size, pdsize_code in zipped_tire_sizes_and_pdsize_codes:
        # LOOP THROUGH TIRE SIZES for retreads:
        Base_Query = f"""
        SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
        WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
        AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDSIZE LIKE '{pdsize_code}' AND PDNUMBER NOT LIKE '%RC%')
        --select only retreads:
        AND TSVN = '090'
        AND TSPC = '50'
        AND TSCODDEL <> 'D'
        """

        Retreads_FOR_EACH_TIRE_SIZE_Columns = {}
        # Each month values:
        for i, month in enumerate(Months_Numerical):
            value = 0
            if month <= Last_Month:
                cursor.execute(f"""{Base_Query}
                AND TSYP = {month}""")
                value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            Retreads_FOR_EACH_TIRE_SIZE_Columns[f'{Month_Name} {year}'] = int(value)
        # Get SUM of YTD Values by adding up each month value:
        Sum_YTD = 0
        for i in Retreads_FOR_EACH_TIRE_SIZE_Columns.values():
            Sum_YTD = Sum_YTD + i
        Retreads_FOR_EACH_TIRE_SIZE_Columns[f'YTD {year}'] = int(Sum_YTD)

        # Update the first row for the dataframe for the "RETREADS" (total row)
        for key, value in Retreads_FOR_EACH_TIRE_SIZE_Columns.items():
            Tire_Sales_Summary_Table_for_Excel_in_Units_df.at[f'{tire_size} RTEMP', key] = int(value)

        if year == Current_Year:
        # Reorder columns:
            Tire_Sales_Summary_Table_for_Excel_in_Units_df = Tire_Sales_Summary_Table_for_Excel_in_Units_df[[f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', 
            f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}', 
            f'YTD {Current_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', 
            f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', f'NOV {Current_Year}', f'DEC {Current_Year}']]

    # ============================================================================================================================================================
    # +++++++++++++++++++++++++ CAP+CASINGS ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ============================================================================================================================================================
    
    # CAP+CASING (all sizes total row)
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = {Enterprise_Number})
    --select only Cap+Casing:
    AND TSVN = '091'
    AND TSPC = '51'
    AND TSCODDEL <> 'D'
    """
    Cap_plus_Casing_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Cap_plus_Casing_Columns[f'{Month_Name} {year}'] = int(value)
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Cap_plus_Casing_Columns.values():
        Sum_YTD = Sum_YTD + i
    Cap_plus_Casing_Columns[f'YTD {year}'] = Sum_YTD


    # Update the first row for the dataframe for the "RETREADS" (total row)
    for key, value in Cap_plus_Casing_Columns.items():
        Tire_Sales_Summary_Table_for_Excel_in_Units_df.at['CAP+CASINGS', key] = int(value)

    if year == Current_Year:
    # Reorder columns:
        Tire_Sales_Summary_Table_for_Excel_in_Units_df = Tire_Sales_Summary_Table_for_Excel_in_Units_df[[f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', 
        f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}', 
        f'YTD {Current_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', 
        f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', f'NOV {Current_Year}', f'DEC {Current_Year}']]

    # ==========================================================================
    #---------------CAP+CASINGS PER TIRE SIZE--------------------------------------
    # For each tire size, we'll loop through them dynamically as each customer may have different sizes and number of sizes:

    for tire_size, pdsize_code in zipped_tire_sizes_and_pdsize_codes:
        # LOOP THROUGH TIRE SIZES for retreads:
        Base_Query = f"""
        SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
        WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
        AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDSIZE LIKE '{pdsize_code}' AND PDNUMBER NOT LIKE '%RC%')
        --select only Cap+Casing:
        AND TSVN = '091'
        AND TSPC = '51'
        AND TSCODDEL <> 'D'
        """

        Cap_plus_Casing_FOR_EACH_TIRE_SIZE_Columns = {}
        # Each month values:
        for i, month in enumerate(Months_Numerical):
            value = 0
            if month <= Last_Month:
                cursor.execute(f"""{Base_Query}
                AND TSYP = {month}""")
                value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            Cap_plus_Casing_FOR_EACH_TIRE_SIZE_Columns[f'{Month_Name} {year}'] = int(value)
        # Get SUM of YTD Values by adding up each month value:
        Sum_YTD = 0
        for i in Cap_plus_Casing_FOR_EACH_TIRE_SIZE_Columns.values():
            Sum_YTD = Sum_YTD + i
        Cap_plus_Casing_FOR_EACH_TIRE_SIZE_Columns[f'YTD {year}'] = int(Sum_YTD)

        # Update the first row for the dataframe for the "RETREADS" (total row)
        for key, value in Cap_plus_Casing_FOR_EACH_TIRE_SIZE_Columns.items():
            Tire_Sales_Summary_Table_for_Excel_in_Units_df.at[f'{tire_size} CCTEMP', key] = int(value)

        if year == Current_Year:
        # Reorder columns:
            Tire_Sales_Summary_Table_for_Excel_in_Units_df = Tire_Sales_Summary_Table_for_Excel_in_Units_df[[f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', 
            f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}', 
            f'YTD {Current_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', 
            f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', f'NOV {Current_Year}', f'DEC {Current_Year}']]

    # =====================================================================================================================================
    # +++++++++++++++++++++++++ REPAIR ONLY ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # =====================================================================================================================================
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDNUMBER LIKE '2R0%' OR PDNUMBER LIKE '2B0%')
    AND TSCODDEL <> 'D'
    """
    Repair_Only_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        Repair_Only_Columns[f'{Month_Name} {year}'] = value
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in Repair_Only_Columns.values():
        Sum_YTD = Sum_YTD + i
    Repair_Only_Columns[f'YTD {year}'] = Sum_YTD

    
    # Update the first row for the dataframe for the "RETREADS" (total row)
    for key, value in Repair_Only_Columns.items():
        Tire_Sales_Summary_Table_for_Excel_in_Units_df.at['REPAIR ONLY', key] = int(value)

    if year == Current_Year:
    # Reorder columns:
        Tire_Sales_Summary_Table_for_Excel_in_Units_df = Tire_Sales_Summary_Table_for_Excel_in_Units_df[[f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', 
        f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}', 
        f'YTD {Current_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', 
        f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', f'NOV {Current_Year}', f'DEC {Current_Year}']]



    # =====================================================================================================================================================
    # +++++++++++++++++++++++++++ NEW TIRES (ALL VENDORS)+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # =====================================================================================================================================================

    # ALL New Tires (all vendors and sizes, one row total):
    Base_Query = f"""
    SELECT IFNULL(SUM(TSUNITS),0) AS TOTAL FROM TMSALE
    -- Select enterprise number:
    WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
    -- Get rid of Bill Hold, get rid of anything that is commission lines or FET lines (For govt transactions)
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE TRIM(PDNUMBER) NOT LIKE '%BH' AND PDFLDUSR4 <> 'BH' AND PDNUMBER NOT LIKE 'FET%' AND PDDESCRIP NOT LIKE '%COMM%' AND PDFLDUSR1 = '')
    -- Select all relevant product classes (01-09 for Passenger/Light Truck, 10,11,12,14 for Medium Truck, 16 (ATV), 17(Lawn & Garden), 20 (INDUSTRIAL), 21 (FARM) and 25-26 for OTRs)
    AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDCLASS IN ('01','02','03','04','05','06','07','08','09','10','11','12','14','16','17','20','21','25','26'))
    AND TSCODDEL <> 'D'
    """
    New_Tires_Columns = {}
    # Each month values:
    for i, month in enumerate(Months_Numerical):
        value = 0
        if month <= Last_Month:
            cursor.execute(f"""{Base_Query}
            AND TSYP = {month}""")
            value = cursor.fetchone()[0]
        Month_Name = Months_Strings[i]
        New_Tires_Columns[f'{Month_Name} {year}'] = int(value)
    # Get SUM of YTD Values by adding up each month value:
    Sum_YTD = 0
    for i in New_Tires_Columns.values():
        Sum_YTD = Sum_YTD + i
    New_Tires_Columns[f'YTD {year}'] = Sum_YTD
    
    # Update the first row for the dataframe for the "RETREADS" (total row)
    for key, value in New_Tires_Columns.items():
        Tire_Sales_Summary_Table_for_Excel_in_Units_df.at['NEW TIRES (All Vendors)', key] = int(value)

    if year == Current_Year:
    # Reorder columns:
        Tire_Sales_Summary_Table_for_Excel_in_Units_df = Tire_Sales_Summary_Table_for_Excel_in_Units_df[[f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', 
        f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}', 
        f'YTD {Current_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', 
        f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', f'NOV {Current_Year}', f'DEC {Current_Year}']]

    # ==========================================================================
    #---------------NEW TIRES - PER TIRE SIZE--------------------------------------
    # For each tire size, we'll loop through them dynamically as each customer may have different sizes and number of sizes:
    for tire_size, pdsize_code in zipped_tire_sizes_and_pdsize_codes:
        Base_Query = f"""
        SELECT IFNULL(SUM(TSUNITS),0) AS TOTAL FROM TMSALE
        -- Select enterprise number:
        WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
        -- Get rid of Bill Hold, get rid of anything that is commission lines or FET lines (For govt transactions)
        AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE TRIM(PDNUMBER) NOT LIKE '%BH' AND PDFLDUSR4 <> 'BH' AND PDNUMBER NOT LIKE 'FET%' AND PDDESCRIP NOT LIKE '%COMM%' AND PDFLDUSR1 = '')
        -- Select all relevant product classes (01-09 for Passenger/Light Truck, 10,11,12,14 for Medium Truck, 16 (ATV), 17(Lawn & Garden), 20 (INDUSTRIAL), 21 (FARM) and 25-26 for OTRs)
        AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDCLASS IN ('01','02','03','04','05','06','07','08','09','10','11','12','14','16','17','20','21','25','26'))
        AND TSCODDEL <> 'D'
        -- Choose tire size in form of pdsize column of TMPROD
        AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDSIZE LIKE '{pdsize_code}')
        """
        New_Tires_EACH_TIRE_SIZE_Columns = {}
        # Each month values:
        for i, month in enumerate(Months_Numerical):
            value = 0
            if month <= Last_Month:
                cursor.execute(f"""{Base_Query}
                AND TSYP = {month}""")
                value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            New_Tires_EACH_TIRE_SIZE_Columns[f'{Month_Name} {year}'] = int(value)
        # Get SUM of YTD Values by adding up each month value:
        Sum_YTD = 0
        for i in New_Tires_EACH_TIRE_SIZE_Columns.values():
            Sum_YTD = Sum_YTD + i
        New_Tires_EACH_TIRE_SIZE_Columns[f'YTD {year}'] = Sum_YTD
        
        # Update the first row for the dataframe for the "RETREADS" (total row)
        for key, value in New_Tires_EACH_TIRE_SIZE_Columns.items():
            Tire_Sales_Summary_Table_for_Excel_in_Units_df.at[f'{tire_size} NEWTEMP', key] = int(value)

        if year == Current_Year:
        # Reorder columns:
            Tire_Sales_Summary_Table_for_Excel_in_Units_df = Tire_Sales_Summary_Table_for_Excel_in_Units_df[[f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', 
            f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}', 
            f'YTD {Current_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', 
            f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', f'NOV {Current_Year}', f'DEC {Current_Year}']]


    # =====================================================================================================================================================
    # +++++++++++++++++++++++++++ CALCULATE "OTHER" rows ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # ====================================================================================================================================================

# Make "Category" index the first column:
Tire_Sales_Summary_Table_for_Excel_in_Units_df = Tire_Sales_Summary_Table_for_Excel_in_Units_df.reset_index()

# Create a list of all the main "Totals" based rows that are in the "Category" column:
List_of_totals_categories = ['RETREADS', 'CAP+CASINGS', 'NEW TIRES (All Vendors)']

for each_total_category in List_of_totals_categories:
    if each_total_category == 'RETREADS':
        temp_suffix_abbrev = 'RTEMP'
    if each_total_category == 'CAP+CASINGS':
        temp_suffix_abbrev = 'CCTEMP'
    if each_total_category == 'NEW TIRES (All Vendors)':
        temp_suffix_abbrev = 'NEWTEMP'


    # For each Category Total row (in the loop), create a new copy of that single row df and call it temp_row_for_other_df:
    temp_row_for_other_df = Tire_Sales_Summary_Table_for_Excel_in_Units_df.loc[Tire_Sales_Summary_Table_for_Excel_in_Units_df['Category'] == f'{each_total_category}']

    # Use this new temp_row_for_other_df as the name implies.  In other words, we'll replace "RETREADS" (or whichever Category it is in the loop) 
    # with "OTHER RETREADS R_TEMP" (again, if it's RETREADS and R_TEMP in the loop)
    temp_row_for_other_df.loc[temp_row_for_other_df['Category'] == f'{each_total_category}', 'Category'] = f'OTHER {each_total_category} {temp_suffix_abbrev}'

    # Create a list of temp suffix abbrev and tire sizes:
    list_of_tire_sizes_plus_suffix_abbrevs = []
    for size in list_of_tire_sizes:
        size = f'{size} {temp_suffix_abbrev}'
        list_of_tire_sizes_plus_suffix_abbrevs.append(size)


    # Using that list, make a filtered df of only the rows of tire sizes for that category.  (We'll use this to subtract from the temp_row_for_other_df to get the "OTHER" row we need ultimately):
    temp_df_rows_to_subtract = Tire_Sales_Summary_Table_for_Excel_in_Units_df[Tire_Sales_Summary_Table_for_Excel_in_Units_df['Category'].isin(list_of_tire_sizes_plus_suffix_abbrevs)]

    # Ensure the df we just made has its numerical values in integer form:
    for each_col in temp_df_rows_to_subtract.columns[1:]:
        temp_df_rows_to_subtract[each_col] = temp_df_rows_to_subtract[each_col].astype(int)

    # Condense (aka sum) all of those tire size rows into a one-row df:
    # First, add a row called "'OTHER {each_total_category} {temp_suffix_abbrev}" which will be the sum of all the rows
    temp_df_rows_to_subtract.loc[f'OTHER {each_total_category} {temp_suffix_abbrev}'] = temp_df_rows_to_subtract.sum(numeric_only=True)
    # reset the index so that "Category" is now a column instead of the index header
    temp_df_rows_to_subtract = temp_df_rows_to_subtract.reset_index(drop=True)
    # We'll just replace this will "OTHER {temp_suffix_abbrev}" even though in that df it would not be other it would be the sum of each tire size row into one.
    # We're just making it "OTHER {temp_suffix_abbrev}" so that it's the same as the temp_row_for_other_df.  This way it will be easy to add them together.
    temp_df_rows_to_subtract.iloc[-1, 0] = f'OTHER {each_total_category} {temp_suffix_abbrev}'
    temp_df_rows_to_subtract = temp_df_rows_to_subtract.iloc[[-1]]


    # Make both temp one-row dfs ("OTHER" df and tire sizes one row df) have category as the index 
    temp_row_for_other_df = temp_row_for_other_df.set_index('Category')
    temp_df_rows_to_subtract = temp_df_rows_to_subtract.set_index('Category')
    # Now subtract tire sizes from the totals to get "OTHER":
    temp_row_for_other_df = temp_row_for_other_df.subtract(temp_df_rows_to_subtract, axis=1)
    temp_row_for_other_df = temp_row_for_other_df.reset_index()



    # Finally, use this temp "OTHER" row to replace the ACTUAL row for the actual df:
    list_of_values_for_other = temp_row_for_other_df.values.tolist()
    Tire_Sales_Summary_Table_for_Excel_in_Units_df.loc[Tire_Sales_Summary_Table_for_Excel_in_Units_df['Category'] == f'OTHER {each_total_category} {temp_suffix_abbrev}'] = list_of_values_for_other

    # if we're in the last part of the loop, we'll rename the last Category value before removing the temporary abbreviations from all the tire sizes and "other" rows:
    if temp_suffix_abbrev == 'NEWTEMP':
        Tire_Sales_Summary_Table_for_Excel_in_Units_df['Category'] = Tire_Sales_Summary_Table_for_Excel_in_Units_df['Category'].replace('OTHER NEW TIRES (All Vendors) NEWTEMP', 'OTHER NEW NEWTEMP')

# Ensure all numerical columns are integers
for each_col in Tire_Sales_Summary_Table_for_Excel_in_Units_df.columns[1:]:
    Tire_Sales_Summary_Table_for_Excel_in_Units_df[each_col] = Tire_Sales_Summary_Table_for_Excel_in_Units_df[each_col].astype(int)


# ============================================================
# NOTE: We will save a copy of JUST retread sizes and "OTHER" retreads data so that we can use it for graphs later.
# This is because currently all the rows are UNIQUE.  So before we format to change that, we can use this to our advantage later.
df_retreads_of_sizes_and_other = Tire_Sales_Summary_Table_for_Excel_in_Units_df[Tire_Sales_Summary_Table_for_Excel_in_Units_df['Category'].str.contains('RTEMP')]
# ============================================================


Tire_Sales_Summary_Table_for_Excel_in_Units_df



-------------------------------------------
year: 2020
Months_Numerical:  [202001, 202002, 202003, 202004, 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012]
-------------------------------------------
year: 2021
Months_Numerical:  [202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108, 202109, 202110, 202111, 202112]


,Category,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,RETREADS,272,32,27,14,19,39,14,20,24,20,26,25,12,626,11,48,30,32,18,15,116,89,185,32,43,7
1,295/75R22.5 RTEMP,271,32,27,14,19,39,14,20,24,19,26,25,12,614,11,47,30,32,18,14,116,89,185,32,33,7
2,11R22.5 RTEMP,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,1,0,0,0,0,10,0
3,OTHER RETREADS RTEMP,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
4,CAP+CASINGS,634,25,44,62,75,41,42,66,88,60,41,40,50,708,38,26,53,73,82,80,42,96,30,68,70,50
5,295/75R22.5 CCTEMP,613,25,43,60,75,37,40,66,85,60,32,40,50,696,38,26,52,71,80,76,42,96,30,67,69,49
6,11R22.5 CCTEMP,11,0,0,0,0,0,2,0,3,0,6,0,0,8,0,0,1,1,2,4,0,0,0,0,0,0
7,OTHER CAP+CASINGS CCTEMP,10,0,1,2,0,4,0,0,0,0,3,0,0,4,0,0,0,1,0,0,0,0,0,1,1,1
8,REPAIR ONLY,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,1,1,0,0,0,0
9,NEW TIRES (All Vendors),587,41,36,68,65,47,57,48,55,68,32,41,29,688,50,32,61,81,46,75,84,55,51,69,48,36


# Now back to cleaning up the dfs:

In [42]:



# Finally, use regex to remove the vendor abbreviations from the Tire Size and "OTHER" rows in the "Category" column:
list_of_temp_suffixes = ['RTEMP', 'CCTEMP', 'NEWTEMP']
for each_suffix in list_of_temp_suffixes:
    Tire_Sales_Summary_Table_for_Excel_in_Units_df['Category'] = Tire_Sales_Summary_Table_for_Excel_in_Units_df['Category'].str.replace(f'{each_suffix}', '')


# Get rid of the blank space on the right of those tire sizes now that we've removed the vendor abbreviation:
Tire_Sales_Summary_Table_for_Excel_in_Units_df['Category'] = Tire_Sales_Summary_Table_for_Excel_in_Units_df['Category'].str.rstrip()



# Ensure all number columns are integers
for each_col in Tire_Sales_Summary_Table_for_Excel_in_Units_df.columns[1:]:
    Tire_Sales_Summary_Table_for_Excel_in_Units_df[each_col] = Tire_Sales_Summary_Table_for_Excel_in_Units_df[each_col].astype(int)


# ================================
# Make sure that the current month's values are NOT included in the current year's YTD total in the df
list_to_loop = [['01', "JAN"], ['02', 'FEB'], ['03', 'MAR'], ['04', 'APR'], ['05', 'MAY'], ['06', 'JUN'], ['07', 'JUL'], ['08', 'AUG'], 
['09', 'SEPT'], ['10', 'OCT'], ['11', 'NOV'], ['12', 'DEC']]

for month_number_string, month_name in list_to_loop:
    if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
        Tire_Sales_Summary_Table_for_Excel_in_Units_df[f'YTD {Current_Year}'] = Tire_Sales_Summary_Table_for_Excel_in_Units_df[f'YTD {Current_Year}'] - Tire_Sales_Summary_Table_for_Excel_in_Units_df[f'{month_name} {Current_Year}']


# Make DASHES for any incomplete month column:
zipped_list_of_current_year_columns_and_yyyymm = list(zip(Tire_Sales_Summary_Table_for_Excel_in_Units_df.columns[15:], Months_Numerical))
# now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
    # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
    if numerical_month >= Current_Month:
        Tire_Sales_Summary_Table_for_Excel_in_Units_df[col] = '-'


# Add the Tire Sales Summary Table of Both current and Last Year to the dictionary we will use to produce the excel workbook.
# Like all the ones done prior, this table will be its own sheet in the workbook when we create it at the end of this python script/notebook. 
Dict_for_excel_sheets['Tire Sales Summary Table'] = Tire_Sales_Summary_Table_for_Excel_in_Units_df



Tire_Sales_Summary_Table_for_Excel_in_Units_df


,Category,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,RETREADS,272,32,27,14,19,39,14,20,24,20,26,25,12,626,11,48,30,32,18,15,116,89,185,32,43,7
1,295/75R22.5,271,32,27,14,19,39,14,20,24,19,26,25,12,614,11,47,30,32,18,14,116,89,185,32,33,7
2,11R22.5,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,1,0,0,0,0,10,0
3,OTHER RETREADS,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
4,CAP+CASINGS,634,25,44,62,75,41,42,66,88,60,41,40,50,708,38,26,53,73,82,80,42,96,30,68,70,50
5,295/75R22.5,613,25,43,60,75,37,40,66,85,60,32,40,50,696,38,26,52,71,80,76,42,96,30,67,69,49
6,11R22.5,11,0,0,0,0,0,2,0,3,0,6,0,0,8,0,0,1,1,2,4,0,0,0,0,0,0
7,OTHER CAP+CASINGS,10,0,1,2,0,4,0,0,0,0,3,0,0,4,0,0,0,1,0,0,0,0,0,1,1,1
8,REPAIR ONLY,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,1,1,0,0,0,0
9,NEW TIRES (All Vendors),587,41,36,68,65,47,57,48,55,68,32,41,29,688,50,32,61,81,46,75,84,55,51,69,48,36


In [43]:
# =========================Filter df in units to only include one year for website ======================
Tire_Sales_Summary_Table_for_website_df = Tire_Sales_Summary_Table_for_Excel_in_Units_df


# IF YOU WANT TO SEE BOTH YEARS ON THE WEBSITE YOU COULD COMMENT OUT THIS CODE (everything between the 'equal sign' lines below):
# ================================================================================================================================
# ONLY if the current month is January, we'll drop this year from the Table for the website:
Tire_Sales_Summary_Table_for_website_df = Tire_Sales_Summary_Table_for_website_df.drop(columns = 
                        [f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', 
                        f'MAY {Last_Year}', f'JUN {Last_Year}', f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', 
                        f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}'])

# Reminder:  Again, this is only for the website.  The excel data will have both years.
# ================================================================================================================================


# View table:
Tire_Sales_Summary_Table_for_website_df

,Category,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,RETREADS,626,11,48,30,32,18,15,116,89,185,32,43,7
1,295/75R22.5,614,11,47,30,32,18,14,116,89,185,32,33,7
2,11R22.5,11,0,0,0,0,0,1,0,0,0,0,10,0
3,OTHER RETREADS,1,0,1,0,0,0,0,0,0,0,0,0,0
4,CAP+CASINGS,708,38,26,53,73,82,80,42,96,30,68,70,50
5,295/75R22.5,696,38,26,52,71,80,76,42,96,30,67,69,49
6,11R22.5,8,0,0,1,1,2,4,0,0,0,0,0,0
7,OTHER CAP+CASINGS,4,0,0,0,1,0,0,0,0,0,1,1,1
8,REPAIR ONLY,3,0,1,0,0,0,0,1,1,0,0,0,0
9,NEW TIRES (All Vendors),688,50,32,61,81,46,75,84,55,51,69,48,36


In [44]:
Dict_for_excel_sheets.keys()

dict_keys(['Flat Repairs in Units', 'Flat Repairs in Dollars', 'Tire Rotations in Units', 'Tire Rotations in Dollars', 'Mounts/Mount+Dismounts in Units', 'Mounts/Mount+Dismounts in Dollars', 'Service Call Hours in Units', 'Service Call Hours in Dollars', 'Scraps in Units', 'Scraps in Dollars', 'Retread Units', 'Cap+Casings Units', 'Bridgestone New Tires (Units)', 'Continental New Tires (Units)', 'Yokohama New Tires (Units)', 'Tire Sales Summary Table'])

# Now complete the Sales by Tire Size BY EACH SPECIFIED VENDOR!

In [45]:
# Create Dataframe of just categories.  Then we'll loop through and add each Location column to the dataframe:

# DYNAMICALLLY BUILD A DATAFRAME TEMPLATE WITH ONE COLUMN BASES ON WHICH TIRE SIZES WERE SPECIFIED.

# START BY CREATING A LIST SO THAT WE CAN USE IT AS THE FIRST COLUMN FOR A DATAFRAME TEMPLATE:


list_for_column_template = []

for vendor in vendor_names_list:
    print(vendor)

    if vendor == 'Bridgestone':
        vendor_abbrev = 'BS'
    elif vendor == 'Yokohama':
        vendor_abbrev = 'YK'
    elif vendor == 'Continental':
        vendor_abbrev = 'CONT'
    elif vendor == 'Michelin':
        vendor_abbrev = 'MICH'
    elif vendor == 'Firestone':
        vendor_abbrev = 'FS'

    list_for_column_template.append(f'{vendor.upper()} NEW')

    for each_tire_size in list_of_tire_sizes:
        list_for_column_template.append(f'{each_tire_size} {vendor_abbrev}')
        
    list_for_column_template.append(f'OTHER SIZES {vendor_abbrev}')
    list_for_column_template.append(f'{vendor_abbrev} STEER-NEW')
    list_for_column_template.append(f'{vendor_abbrev} DRIVE-NEW')
    list_for_column_template.append(f'{vendor_abbrev} TRAILER-NEW')


dash_list = []
for each in range(len(list_for_column_template)):
        dash_list.append('-')

template = {
    
'Category': list_for_column_template, 
f'YTD {Last_Year}': dash_list,
f'JAN {Last_Year}': dash_list,
f'FEB {Last_Year}': dash_list,
f'MAR {Last_Year}': dash_list,
f'APR {Last_Year}': dash_list,
f'MAY {Last_Year}': dash_list,
f'JUN {Last_Year}': dash_list,
f'JUL {Last_Year}': dash_list,
f'AUG {Last_Year}': dash_list,
f'SEPT {Last_Year}': dash_list,
f'OCT {Last_Year}': dash_list,
f'NOV {Last_Year}': dash_list,
f'DEC {Last_Year}': dash_list,
f'YTD {Last_Year}': dash_list,
f'JAN {Current_Year}': dash_list,
f'FEB {Current_Year}': dash_list,
f'MAR {Current_Year}': dash_list,
f'APR {Current_Year}': dash_list,
f'MAY {Current_Year}': dash_list,
f'JUN {Current_Year}': dash_list,
f'JUL {Current_Year}': dash_list,
f'AUG {Current_Year}': dash_list,
f'SEPT {Current_Year}': dash_list,
f'OCT {Current_Year}': dash_list,
f'NOV {Current_Year}': dash_list,
f'DEC {Current_Year}': dash_list
}

# We'll create a starting Dataframe with one column based on the above code:
Tire_Sales_by_Vendor_for_Excel_in_Units_df = pd.DataFrame(template)

Tire_Sales_by_Vendor_for_Excel_in_Units_df = Tire_Sales_by_Vendor_for_Excel_in_Units_df.set_index('Category')

Tire_Sales_by_Vendor_for_Excel_in_Units_df

Bridgestone
Continental
Yokohama


,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
Category,,,,,,,,,,,,,,,,,,,,,,,,,
BRIDGESTONE NEW,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
295/75R22.5 BS,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
11R22.5 BS,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
OTHER SIZES BS,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
BS STEER-NEW,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
BS DRIVE-NEW,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
BS TRAILER-NEW,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
CONTINENTAL NEW,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
295/75R22.5 CONT,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


In [46]:
# vendor_names_list = ['Bridgestone', 'Continental']
# vendor_numbers = ['007', '021']


zipped_tire_sizes_and_pdsize_codes = list(zip(list_of_tire_sizes, list_of_pdsize_codes))

years = [Last_Year, Current_Year]

# Note 10 is Steer, 11 is Drive, 12 is Trailer
product_class_numbers_for_wheel_position = ['10', '11', '12'] 


for year in years:
    Months_Numerical = []
    for x in range(1,13):
        Months_Numerical.append(year*100 + x)
    # =====================================================================================================================================================
    # +++++++++++++++++++++++++++ BUILD SALES BY TIRE SIZE PER VENDOR TABLE +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # =====================================================================================================================================================
    

    for vendor_name, vendor_number in zipped_vendor_names_with_numbers:
        if vendor_name == 'Bridgestone':
            vendor_abbrev = 'BS'
        elif vendor_name == 'Yokohama':
            vendor_abbrev = 'YK'
        elif vendor_name == 'Continental':
            vendor_abbrev = 'CONT'
        elif vendor_name == 'Michelin':
            vendor_abbrev = 'MICH'
        elif vendor_name == 'Firestone':
            vendor_abbrev = 'FS'

        # UPDATE ROW(S) for the TOTAL of Sales for EACH VENDOR specified:
        Base_Query = f"""
        SELECT IFNULL(SUM(TSUNITS),0) AS TOTAL FROM TMSALE
        -- Select enterprise number:
        WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
        -- Get rid of Bill Hold, get rid of anything that is commission lines or FET lines (For govt transactions)
        AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE TRIM(PDNUMBER) NOT LIKE '%BH' AND PDFLDUSR4 <> 'BH' AND PDNUMBER NOT LIKE 'FET%' AND PDDESCRIP NOT LIKE '%COMM%' AND PDFLDUSR1 = '')
        -- Select all relevant product classes (01-09 for Passenger/Light Truck, 10,11,12,14 for Medium Truck, 16 (ATV), 17 (Lawn & Garden), 20 (INDUSTRIAL), 21 (FARM) and 25-26 for OTRs)
        AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDCLASS IN ('01','02','03','04','05','06','07','08','09','10','11','12','14','16','17','20','21','25','26'))
        AND TSCODDEL <> 'D'
        AND TSVN = '{vendor_number}'
        """ 
        Vendor_All_Sizes_Columns = {}
        # Each month values:
        for i, month in enumerate(Months_Numerical):
            value = 0
            if month <= Last_Month:
                cursor.execute(f"""{Base_Query}
                AND TSYP = {month}""")
                value = cursor.fetchone()[0]
            Month_Name = Months_Strings[i]
            Vendor_All_Sizes_Columns[f'{Month_Name} {year}'] = value
        # Get SUM of YTD Values by adding up each month value:
        Sum_YTD = 0
        for i in Vendor_All_Sizes_Columns.values():
            Sum_YTD = Sum_YTD + i
        Vendor_All_Sizes_Columns[f'YTD {year}'] = Sum_YTD

        # Update the row for the dataframe (this may update more than one row depending on how many vendors specified.  If 2 vendors, it will update two rows for example):
        for key, value in Vendor_All_Sizes_Columns.items():
            Tire_Sales_by_Vendor_for_Excel_in_Units_df.at[f'{vendor_name.upper()} NEW', key] = int(value)

        if year == Current_Year:
        # Reorder columns:
            Tire_Sales_by_Vendor_for_Excel_in_Units_df = Tire_Sales_by_Vendor_for_Excel_in_Units_df[[f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', 
            f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}', 
            f'YTD {Current_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', 
            f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', f'NOV {Current_Year}', f'DEC {Current_Year}']]


        # UPDATE THE ROWS FOR EACH TIRE SIZE FOR EACH VENDOR:
        for size, pdsize in zipped_tire_sizes_and_pdsize_codes:
            Base_Query = f"""
            SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
            --select customer:
            WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
            -- Get rid of Bill Hold, get rid of anything that is commission lines or FET lines (For govt transactions)
            AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE TRIM(PDNUMBER) NOT LIKE '%BH' AND PDFLDUSR4 <> 'BH' AND PDNUMBER NOT LIKE 'FET%' AND PDDESCRIP NOT LIKE '%COMM%' AND PDFLDUSR1 = '')
            -- Select all relevant product classes (01-09 for Passenger/Light Truck, 10,11,12,14 for Medium Truck and 25-26 for OTRs)
            AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDCLASS IN ('01','02','03','04','05','06','07','08','09','10','11','12','14','25','26'))
            --select vendor (bridgestone)
            AND TSVN = '{vendor_number}'
            -- Select Tire Size:
            AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE PDSIZE LIKE '{pdsize}')
            AND TSCODDEL <> 'D'
            """
            Tire_Size_Columns = {}
            # Each month values:
            for i, month in enumerate(Months_Numerical):
                value = 0
                if month <= Last_Month:
                    cursor.execute(f"""{Base_Query}
                    AND TSYP = {month}""")
                    value = cursor.fetchone()[0]
                Month_Name = Months_Strings[i]
                Tire_Size_Columns[f'{Month_Name} {year}'] = int(value)
            # Get SUM of YTD Values by adding up each month value:
            Sum_YTD = 0
            for i in Tire_Size_Columns.values():
                Sum_YTD = Sum_YTD + i
            Tire_Size_Columns[f'YTD {year}'] = int(Sum_YTD)

            # Update the row for the dataframe (this may update more than one row depending on how many vendors specified.  If 2 vendors, it will update to rows for example):
            for key, value in Tire_Size_Columns.items():
                Tire_Sales_by_Vendor_for_Excel_in_Units_df.at[f'{size} {vendor_abbrev}', key] = int(value)

            if year == Current_Year:
            # Reorder columns:
                Tire_Sales_by_Vendor_for_Excel_in_Units_df = Tire_Sales_by_Vendor_for_Excel_in_Units_df[[f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', 
                f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}', 
                f'YTD {Current_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', 
                f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', f'NOV {Current_Year}', f'DEC {Current_Year}']]

        
        # Update each vendor's STEER, DRIVE, TRAILER rows:
        for product_class_number in product_class_numbers_for_wheel_position:
            Base_Query = f"""
            SELECT IFNULL(SUM(TSUNITS),0) FROM TMSALE
            WHERE TSCUS IN (SELECT CUNUMBER FROM TMCUST WHERE CUNUMENT = '{Enterprise_Number}')
            -- Get rid of Bill Hold, get rid of anything that is commission lines or FET lines (For govt transactions)
            AND TSPD IN (SELECT PDNUMBER FROM TMPROD WHERE TRIM(PDNUMBER) NOT LIKE '%BH' AND PDFLDUSR4 <> 'BH' AND PDNUMBER NOT LIKE 'FET%' AND PDDESCRIP NOT LIKE '%COMM%' AND PDFLDUSR1 = '')
            --Select Vendor using Vendor Number:
            AND TSVN = '{vendor_number}'
            --Specify Product Class 10 for STEER, 11 for DRIVE, or 12 for TRAILER:
            AND TSPC = '{product_class_number}'
            AND TSCODDEL <> 'D'
            """
            New_Tires_ST_DR_OR_TR_Columns = {}
            # Each month values:
            for i, month in enumerate(Months_Numerical):
                value = 0
                if month <= Last_Month:
                    cursor.execute(f"""{Base_Query}
                    AND TSYP = {month}""")
                    value = cursor.fetchone()[0]
                Month_Name = Months_Strings[i]
                New_Tires_ST_DR_OR_TR_Columns[f'{Month_Name} {year}'] = value
            # Get SUM of YTD Values by adding up each month value:
            Sum_YTD = 0
            for i in New_Tires_ST_DR_OR_TR_Columns.values():
                Sum_YTD = Sum_YTD + i
            New_Tires_ST_DR_OR_TR_Columns[f'YTD {year}'] = Sum_YTD

          # Update the row for the dataframe (this may update more than one row depending on how many vendors specified.  If 2 vendors, it will update to rows for example):
            for key, value in New_Tires_ST_DR_OR_TR_Columns.items():
                
                if product_class_number == '10':
                    Tire_Sales_by_Vendor_for_Excel_in_Units_df.at[f'{vendor_abbrev} STEER-NEW', key] = int(value)
                elif product_class_number == '11':
                    Tire_Sales_by_Vendor_for_Excel_in_Units_df.at[f'{vendor_abbrev} DRIVE-NEW', key] = int(value)
                elif product_class_number == '12':
                    Tire_Sales_by_Vendor_for_Excel_in_Units_df.at[f'{vendor_abbrev} TRAILER-NEW', key] = int(value)


            if year == Current_Year:
            # Reorder columns:
                Tire_Sales_by_Vendor_for_Excel_in_Units_df = Tire_Sales_by_Vendor_for_Excel_in_Units_df[[f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', 
                f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}', 
                f'YTD {Current_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', 
                f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', f'NOV {Current_Year}', f'DEC {Current_Year}']]


# Make "Category" index the first column:
Tire_Sales_by_Vendor_for_Excel_in_Units_df = Tire_Sales_by_Vendor_for_Excel_in_Units_df.reset_index()

# ====================================================================================================
# Update the "OTHER" rows with a calculation:
# ====================================================================================================
for vendor_name in vendor_names_list:
    if vendor_name == 'Bridgestone':
        vendor_abbrev = 'BS'
    elif vendor_name == 'Yokohama':
        vendor_abbrev = 'YK'
    elif vendor_name == 'Continental':
        vendor_abbrev = 'CONT'
    elif vendor_name == 'Michelin':
        vendor_abbrev = 'MICH'
    elif vendor_name == 'Firestone':
        vendor_abbrev = 'FS'

    # Locate the row for the Vendor (in the loop)'s row is the total row and set that equal to a temp_row_for_other_df
    temp_row_for_other_df = Tire_Sales_by_Vendor_for_Excel_in_Units_df.loc[Tire_Sales_by_Vendor_for_Excel_in_Units_df['Category'] == f'{vendor_name.upper()} NEW']

    # Use this new temp_row_for_other_df as the name implies.  In other words, we'll replace "BRIDGESTONE NEW" (or whichever vendor it is) with "OTHER BS" (again, if it's Bridgestone in the loop)
    temp_row_for_other_df.loc[temp_row_for_other_df['Category'] == f'{vendor_name.upper()} NEW', 'Category'] = f'OTHER SIZES {vendor_abbrev}'

    # Now, we have a df of one row that contains the TOTALS row but is called "OTHER".  Although the numbers are same with the totals row for now, we'll update this
    # by subtracting all the sizes rows.

    # Create a list of tire sizes and vendor abbreviation suffixes:
    list_of_vendor_abbrev_plus_tire_sizes = []
    for size in list_of_tire_sizes:
        size = f'{size} {vendor_abbrev}'
        list_of_vendor_abbrev_plus_tire_sizes.append(size)


    # Using that list, make a filtered df of only the rows of tire sizes for that vendor.  (We'll use this to subtract from the temp_row_for_other_df to get the "OTHER" row we need ultimately):
    temp_df_rows_to_subtract = Tire_Sales_by_Vendor_for_Excel_in_Units_df[Tire_Sales_by_Vendor_for_Excel_in_Units_df['Category'].isin(list_of_vendor_abbrev_plus_tire_sizes)]

    # Ensure the df we just made has its numerical values in integer form:
    for each_col in temp_df_rows_to_subtract.columns[1:]:
        temp_df_rows_to_subtract[each_col] = temp_df_rows_to_subtract[each_col].astype(int)

    # Condense (aka sum) all of those tire size rows into a one-row df:
    temp_df_rows_to_subtract.loc[f'OTHER SIZES {vendor_abbrev}'] = temp_df_rows_to_subtract.sum(numeric_only=True)
    temp_df_rows_to_subtract = temp_df_rows_to_subtract.reset_index(drop=True)
    # We'll just replace this will "OTHER {vendor_abbrev}" even though in that df it would not be other it would be the sum of each tire size row into one.
    # We're just making it "OTHER {vendor_abbrev}" so that it's the same as the temp_row_for_other_df.  This way it will be easy to add them together.
    temp_df_rows_to_subtract.iloc[-1, 0] = f'OTHER SIZES {vendor_abbrev}'
    temp_df_rows_to_subtract = temp_df_rows_to_subtract.iloc[[-1]]


    # Make both temp one-row dfs ("OTHER" df and tire sizes one row df) have category as the index 
    temp_row_for_other_df = temp_row_for_other_df.set_index('Category')
    temp_df_rows_to_subtract = temp_df_rows_to_subtract.set_index('Category')
    # Now subtract tire sizes from the totals to get "OTHER":
    temp_row_for_other_df = temp_row_for_other_df.subtract(temp_df_rows_to_subtract, axis=1)
    temp_row_for_other_df = temp_row_for_other_df.reset_index()



    # Finally, us this temp "OTHER" row to replace the ACTUALY row for the actual df:
    list_of_values_for_other = temp_row_for_other_df.values.tolist()
    Tire_Sales_by_Vendor_for_Excel_in_Units_df.loc[Tire_Sales_by_Vendor_for_Excel_in_Units_df['Category'] == f'OTHER SIZES {vendor_abbrev}'] = list_of_values_for_other

    # Done.

    # Finally, use regex to remove the vendor abbreviations from the Tire Size and "OTHER" rows in the "Category" column:
    Tire_Sales_by_Vendor_for_Excel_in_Units_df['Category'] = Tire_Sales_by_Vendor_for_Excel_in_Units_df['Category'].map(lambda x: x.rstrip(f'{vendor_abbrev}'))
    # Get rid of the blank space on the right of those tire sizes now that we've removed the vendor abbreviation:
    Tire_Sales_by_Vendor_for_Excel_in_Units_df['Category'] = Tire_Sales_by_Vendor_for_Excel_in_Units_df['Category'].str.rstrip()
# Now just ensure all number columns are integers
for each_col in Tire_Sales_by_Vendor_for_Excel_in_Units_df.columns[1:]:
    Tire_Sales_by_Vendor_for_Excel_in_Units_df[each_col] = Tire_Sales_by_Vendor_for_Excel_in_Units_df[each_col].astype(int)


# ================================
# Make sure that the current month's values are NOT included in the current year's YTD total in the df
list_to_loop = [['01', "JAN"], ['02', 'FEB'], ['03', 'MAR'], ['04', 'APR'], ['05', 'MAY'], ['06', 'JUN'], ['07', 'JUL'], ['08', 'AUG'], 
['09', 'SEPT'], ['10', 'OCT'], ['11', 'NOV'], ['12', 'DEC']]

for month_number_string, month_name in list_to_loop:
    if f'{Current_Month}' == f'{Current_Year}{month_number_string}':
        Tire_Sales_by_Vendor_for_Excel_in_Units_df[f'YTD {Current_Year}'] = Tire_Sales_by_Vendor_for_Excel_in_Units_df[f'YTD {Current_Year}'] - Tire_Sales_by_Vendor_for_Excel_in_Units_df[f'{month_name} {Current_Year}']


# Make DASHES for any incomplete month column:
zipped_list_of_current_year_columns_and_yyyymm = list(zip(Tire_Sales_by_Vendor_for_Excel_in_Units_df.columns[15:], Months_Numerical))
# now loop through the current year's columns and current year's yyyymms (Months_Numerical) and only add DASHES to columns of incomplete months:
for col, numerical_month in zipped_list_of_current_year_columns_and_yyyymm:
    # if the current column's numerical yyyymm is at least as big as the actual Current_Month yyyymm, we'll create a dash for that month column.
    if numerical_month >= Current_Month:
        Tire_Sales_by_Vendor_for_Excel_in_Units_df[col] = '-'


# Add the Tire Sales Summary Table of Both current and Last Year to the dictionary we will use to produce the excel workbook.
# Like all the ones done prior, this table will be its own sheet in the workbook when we create it at the end of this python script/notebook. 
Dict_for_excel_sheets['Tire Sales by Vendor'] = Tire_Sales_by_Vendor_for_Excel_in_Units_df


Tire_Sales_by_Vendor_for_Excel_in_Units_df


,Category,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,BRIDGESTONE NEW,410,30,22,56,48,26,44,28,37,46,23,29,21,463,28,17,22,53,38,52,64,40,31,57,38,23
1,295/75R22.5,340,26,17,49,41,24,37,22,33,40,18,17,16,349,17,15,19,42,36,39,44,30,19,42,28,18
2,11R22.5,24,0,1,1,0,1,4,3,0,0,2,12,0,46,2,0,0,7,1,3,9,5,12,7,0,0
3,OTHER SIZES,46,4,4,6,7,1,3,3,4,6,3,0,5,68,9,2,3,4,1,10,11,5,0,8,10,5
4,BS STEER-NEW,178,12,10,26,13,12,22,15,17,18,7,16,10,175,8,6,12,29,14,11,16,18,6,22,17,16
5,BS DRIVE-NEW,108,8,9,15,14,8,8,7,10,14,7,1,7,201,13,6,6,19,21,34,23,13,20,30,16,0
6,BS TRAILER-NEW,124,10,3,15,21,6,14,6,10,14,9,12,4,87,7,5,4,5,3,7,25,9,5,5,5,7
7,CONTINENTAL NEW,79,8,5,0,8,14,11,17,4,8,2,1,1,43,0,12,27,4,0,0,0,0,0,0,0,0
8,295/75R22.5,66,4,1,0,7,14,11,15,4,6,2,1,1,43,0,12,27,4,0,0,0,0,0,0,0,0
9,11R22.5,12,4,4,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [47]:
# =========================Filter df in units to only include one year for website ======================
Tire_Sales_by_Vendor_for_website_df = Tire_Sales_by_Vendor_for_Excel_in_Units_df


# IF YOU WANT TO SEE BOTH YEARS ON THE WEBSITE YOU COULD COMMENT OUT THIS CODE (everything between the 'equal sign' lines below):
# ================================================================================================================================
# ONLY if the current month is January, we'll drop this year from the Table for the website:
if Current_Month == Jan:
    Tire_Sales_by_Vendor_for_website_df = Tire_Sales_by_Vendor_for_website_df.iloc[:, 0:14]

# OTHERWISE, if it's any other month currently, we'll drop last year from the Table for the website
else:
    Tire_Sales_by_Vendor_for_website_df = Tire_Sales_by_Vendor_for_website_df.drop(columns = 
                        [f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', 
                        f'MAY {Last_Year}', f'JUN {Last_Year}', f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', 
                        f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}'])

# Reminder:  Again, this is only for the website.  The excel data will have both years.
# ================================================================================================================================


# View table:
Tire_Sales_by_Vendor_for_website_df

,Category,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,BRIDGESTONE NEW,463,28,17,22,53,38,52,64,40,31,57,38,23
1,295/75R22.5,349,17,15,19,42,36,39,44,30,19,42,28,18
2,11R22.5,46,2,0,0,7,1,3,9,5,12,7,0,0
3,OTHER SIZES,68,9,2,3,4,1,10,11,5,0,8,10,5
4,BS STEER-NEW,175,8,6,12,29,14,11,16,18,6,22,17,16
5,BS DRIVE-NEW,201,13,6,6,19,21,34,23,13,20,30,16,0
6,BS TRAILER-NEW,87,7,5,4,5,3,7,25,9,5,5,5,7
7,CONTINENTAL NEW,43,0,12,27,4,0,0,0,0,0,0,0,0
8,295/75R22.5,43,0,12,27,4,0,0,0,0,0,0,0,0
9,11R22.5,0,0,0,0,0,0,0,0,0,0,0,0,0


## Create Percentage Breakdown Bar Graphs:

In [48]:
len(Tire_Sales_Summary_Table_for_website_df)

13

In [49]:

# If the current month is NOT January, we'll use the Current Year's YTD Data for the graph:

Retreads_YTD_TOTAL = Tire_Sales_Summary_Table_for_website_df.loc[Tire_Sales_Summary_Table_for_website_df['Category'] == 'RETREADS', f'YTD {Current_Year}'].values[0]
Retreads_plus_Casings_YTD_TOTAL = Tire_Sales_Summary_Table_for_website_df.loc[Tire_Sales_Summary_Table_for_website_df['Category'] == 'CAP+CASINGS', f'YTD {Current_Year}'].values[0]
Repair_Only_YTD_TOTAL = Tire_Sales_Summary_Table_for_website_df.loc[Tire_Sales_Summary_Table_for_website_df['Category'] == 'REPAIR ONLY', f'YTD {Current_Year}'].values[0]
New_Tires_YTD_TOTAL = Tire_Sales_Summary_Table_for_website_df.loc[Tire_Sales_Summary_Table_for_website_df['Category'] == 'NEW TIRES (All Vendors)',  f'YTD {Current_Year}'].values[0]

print(Retreads_YTD_TOTAL)
print(Retreads_plus_Casings_YTD_TOTAL)
print(Repair_Only_YTD_TOTAL)
print(New_Tires_YTD_TOTAL)


626
708
3
688


In [50]:
Total_of_Categories = Retreads_YTD_TOTAL + Retreads_plus_Casings_YTD_TOTAL + Repair_Only_YTD_TOTAL + New_Tires_YTD_TOTAL

if Retreads_YTD_TOTAL > 0:
    Retreads_percentage = round(100 * (Retreads_YTD_TOTAL/Total_of_Categories), 2)
    Retreads_plus_Casings_percentage = round(100 * (Retreads_plus_Casings_YTD_TOTAL/Total_of_Categories), 2)
    Repair_Only_percentage = round(100 * (Repair_Only_YTD_TOTAL/Total_of_Categories), 2)
    New_Tires_percentage = round(100 * (New_Tires_YTD_TOTAL/Total_of_Categories), 2)

else:
    Retreads_percentage = 0
    Retreads_plus_Casings_percentage = 0
    Repair_Only_percentage = 0
    New_Tires_percentage = 0

In [51]:
import plotly.graph_objects as go

Categories = ['RETREADS', 'RETREAD + CASING', 'REPAIR ONLY', 'NEW TIRES']

# Create blank figure
fig1 = go.Figure()

# Add bar for RETREADS:
if Retreads_percentage >= 1:
    fig1.add_trace(go.Bar(
        x=['RETREADS'],
        y=[round(Retreads_percentage, 0)],
        name='Retreads', 
        marker_color = customer_color_1))

else: 
    fig1.add_trace(go.Bar(
        x=['RETREADS'],
        y=[round(Retreads_percentage, 1)],
        name='Retreads', 
        marker_color = customer_color_1))

# Add Bar for RETREAD + CASINGS
if Retreads_plus_Casings_percentage >= 1:
    fig1.add_trace(go.Bar(
        x=['RETREAD + CASING'],
        y=[round(Retreads_plus_Casings_percentage, 0)],
        name='Retread + Casing',
        marker_color=customer_color_2))
else:
    fig1.add_trace(go.Bar(
    x=['RETREAD + CASING'],
    y=[round(Retreads_plus_Casings_percentage, 1)],
    name='Retread + Casing',
    marker_color=customer_color_2))

# Add Bar for REPAIR ONLY
if Repair_Only_percentage >= 1:
    fig1.add_trace(go.Bar(
        x=['REPAIR ONLY'],
        y=[round(Repair_Only_percentage, 0)],
        name='Repair Only',
        marker_color='black'))
else:
    fig1.add_trace(go.Bar(
        x=['REPAIR ONLY'],
        y=[round(Repair_Only_percentage, 1)],
        name='Repair Only',
        marker_color='black'))

# Add Bar for NEW TIRES
if New_Tires_percentage >= 1 and New_Tires_percentage <= 99:
    fig1.add_trace(go.Bar(
        x=['NEW TIRES'],
        y=[round(New_Tires_percentage, 0)],
        name='New Tires',
        marker_color='dimgray'))
else:
    fig1.add_trace(go.Bar(
        x=['NEW TIRES'],
        y=[round(New_Tires_percentage, 1)],
        name='New Tires',
        marker_color='dimgray'))



fig1.update_traces(texttemplate='%{y:.s}%', textposition= 'auto', textfont = {'size': [27]} )

# Here let's setup fig2 = fig1 so that we can use fig2 as a different size JUST for the website.  We'll use fig1 for the FIRST index.html to generate the pdf.
# Since the website needs smaller graphs, we'll need to do this last part twice JUST to change the size amounts.


# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig1.update_layout(
    barmode='group', 
    title_text=f"<b>YTD {Current_Year} - Purchases Percentage Breakdown<b>", 
    title_font_size=38,
    title_font_color="#351C15",
    title_x = 0.5,
    xaxis_title="Purchase Categories",
    yaxis_title="Percentage",
    autosize=False, 
    width=1445, 
    height=900,
    # xaxis_tickangle=-45,
    showlegend=False, 
    font_size=25)
    # font=dict(
    #         size=25,
    #         color="black"
    #     )
    # )



# Write the image that we'll use for index.html to generate the pdf
# We'll save this image as a 'png' rather than an svg because svgs give unwanted dark lines over the gridlines:
fig1.write_image(f"Images_graphs/{Current_Year}_Sales_Percentage_Breakdown.png")
fig1.show()

# NOW write fig2's size to be used for plotly interactive in the website afterwards
# PLOTLY INTERACTIVE:
fig2 = fig1
fig2.update_traces(texttemplate='%{y:.s}%', textposition= 'auto', textfont = {'size': [20]} )
# NOW do the same thing but with different image sizes for fig2 (for the website)!!!
fig2.update_layout(
    barmode='group', 
    title_text=f"<b>YTD {Current_Year} - Purchases Percentage Breakdown<b>", 
    title_font_size=25,
    title_font_color="#351C15",
    title_x = 0.5,
    xaxis_title="Purchase Categories",
    yaxis_title="Percentage",
    autosize=False, width=980, height=600,
    # xaxis_tickangle=-45,
    showlegend=False, 
    font_size=16)


# We don't need this bar chart to be interactive on the website, so we'll just save it as an svg:
fig2.write_image(f"Images_graphs/{Current_Year}_Sales_Percentage_Breakdown.svg")


fig2.show()

# Now make a percentage breakdown bar chart:

In [52]:
# RECALL the df we saved earler "for later" called df_for_just_current_year_retread_sizes.  We can use that now.
df_retreads_of_sizes_and_other

,Category,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
1,295/75R22.5 RTEMP,271,32,27,14,19,39,14,20,24,19,26,25,12,614,11,47,30,32,18,14,116,89,185,32,33,7
2,11R22.5 RTEMP,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,1,0,0,0,0,10,0
3,OTHER RETREADS RTEMP,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0


In [53]:
# Filter to only rows which do NOT contain "OTHER" in the "Category" column:
df_for_just_current_year_retread_sizes = df_retreads_of_sizes_and_other[~df_retreads_of_sizes_and_other['Category'].str.contains('OTHER')]
# Remove the "RTEMP":
df_for_just_current_year_retread_sizes['Category'] = df_for_just_current_year_retread_sizes['Category'].str.replace('RTEMP', '')
# Ensure no spaces at the end:
df_for_just_current_year_retread_sizes['Category'] = df_for_just_current_year_retread_sizes['Category'].str.rstrip()

# Finally, all we need is the df for YTD of the current year
df_for_just_current_year_retread_sizes = df_for_just_current_year_retread_sizes[['Category', f'YTD {Current_Year}']]

df_for_just_current_year_retread_sizes

,Category,YTD 2021
1,295/75R22.5,614
2,11R22.5,11


In [54]:
# Get a list of the YTD values from each tire size:
list_of_YTD_Current_Year_Retread_values_per_tire_size = df_for_just_current_year_retread_sizes[f'YTD {Current_Year}'].to_list()
# Also, get the TOTAL of all the YTD retread values:
Total_Retreads_Current_Year = Tire_Sales_Summary_Table_for_Excel_in_Units_df.at[0, f'YTD {Current_Year}']

# Make a list zipped of tire size, retread for that size:
zipped_list_of_Tire_Sizes_with_retread_numbers_from_current_year = list(zip(list_of_tire_sizes, list_of_YTD_Current_Year_Retread_values_per_tire_size))

# Create a dictionary such that the keys will be the tire sizes and the values will be the percentages of their retread counts compared to the total retreads
dict_of_size_percentages = {}
for tire_size, retread_count in zipped_list_of_Tire_Sizes_with_retread_numbers_from_current_year:
    key = tire_size
    value = round(100 * (retread_count/Total_Retreads_Current_Year), 2)
    dict_of_size_percentages[key] = value

dict_of_size_percentages.items()

dict_items([('295/75R22.5', 98.08), ('11R22.5', 1.76)])

In [132]:
list_of_tire_sizes

['295/75R22.5', '11R22.5']

In [133]:
import plotly.graph_objects as go

Categories = list_of_tire_sizes

# Create blank figure
fig1 = go.Figure()

for key, percentage in dict_of_size_percentages.items():
    if key == list_of_tire_sizes[0]:
        color = customer_color_1
    elif key == list_of_tire_sizes[1]:
        color = customer_color_2
    else:
        color = None

    if percentage >=1 and percentage < 99:

        fig1.add_trace(go.Bar(
            x=[key],
            y=[round(percentage, 0)],
            name=key, 
            marker_color = color
            ))
    else:
        print('false')
        fig1.add_trace(go.Bar(
                x=[key],
                y=[round(percentage, 3)],
                name=key, 
                marker_color = color
        ))


fig1.update_traces(texttemplate='%{y:.s}%', textposition= 'auto', textfont = {'size': [32]} )

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig1.update_layout(
    barmode='group', 
    title_text=f"<b>YTD {Current_Year} - Retreads Percentage Breakdown<b>", 
    title_font_size=38,
    title_font_color="#351C15",
    title_x = 0.5,
    xaxis_title="Tire Size",
    yaxis_title="Percentage",
    autosize=False, width=1445, height=900,
    # xaxis_tickangle=-45,
    showlegend=False, 
    font_size=22)

# Write the image that we'll use for index.html to generate the pdf
# We'll save this image as a 'png' rather than an svg because svgs give unwanted dark lines over the gridlines:
fig1.write_image(f"Images_graphs/{Current_Year}_Retreads_Percentage_Breakdown.png")
fig1.show()


fig2 = fig1
fig2.update_traces(texttemplate='%{y:.s}%', textposition= 'auto', textfont = {'size': [20]} )
fig2.update_layout(
    barmode='group', 
    title_text=f"<b>YTD {Current_Year} - Retreads Percentage Breakdown<b>", 
    title_font_size=25,
    title_font_color="#351C15",
    title_x = 0.5,
    xaxis_title="Tire Size",
    yaxis_title="Percentage",
    autosize=False, width=980, height=600,
    # xaxis_tickangle=-45,
    showlegend=False, 
    font_size=16)



# We don't need this bar chart to be interactive on the website, so we'll just save it as an svg:
fig2.write_image(f"Images_graphs/{Current_Year}_Retreads_Percentage_Breakdown.svg")


fig2.show()

## 1. Service Trend Line Graph

In [56]:
Flat_Repairs_Both_Years = Dict_for_excel_sheets['Flat Repairs in Units'].iloc[[0]]
Tire_Rotations_Both_Years = Dict_for_excel_sheets['Tire Rotations in Units'].iloc[[0]]
Mount_Dismounts_Both_Years = Dict_for_excel_sheets['Mounts/Mount+Dismounts in Units'].iloc[[0]]
Service_Call_Hours_Both_Years = Dict_for_excel_sheets['Service Call Hours in Units'].iloc[[0]]
Scraps_Both_Years = Dict_for_excel_sheets['Scraps in Units'].iloc[[0]]

list_of_one_row_service_dfs_to_concatenate = [Flat_Repairs_Both_Years, Tire_Rotations_Both_Years, Mount_Dismounts_Both_Years, Service_Call_Hours_Both_Years, Scraps_Both_Years]
Services_df_for_graphing = pd.concat(list_of_one_row_service_dfs_to_concatenate).reset_index(drop=True)

# Drop the YTD Columns as we won't need them for the line graph:
Services_df_for_graphing = Services_df_for_graphing.drop(columns = [f'YTD {Last_Year}', f'YTD {Current_Year}'])

# For the Current Year, drop columns with dashes:
Services_df_for_graphing = Services_df_for_graphing.loc[:, ~(Services_df_for_graphing.astype(str) == '-').all()]

# View Services df prepped for line graph:
Services_df_for_graphing

,,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,FLAT REPAIRS,6,1,1,5,0,1,3,0,0,2,1,8,4,4,3,7,4,3,2,6,1,2,4,0
1,TIRE ROTATIONS,0,0,1,2,15,0,0,2,0,0,0,3,0,24,10,9,0,16,12,7,5,1,0,2
2,MOUNT/MOUNT+DISMOUNTS,49,56,59,95,77,54,56,87,82,59,80,63,53,55,78,102,81,87,168,163,228,106,50,23
3,SERVICE CALL HOURS,701,288,467,459,136,285,362,720,655,345,298,518,102,256,190,757,381,506,1137,1021,655,496,337,1016
4,SCRAPS,28,39,76,51,23,36,50,46,40,28,25,16,18,24,40,65,54,83,50,53,40,63,54,39


In [57]:
# Transpose the Merged_Service_df to make it easier for graphing:
Service_df_TRANSPOSED = Services_df_for_graphing.T

Description_list_from_first_row = list(Service_df_TRANSPOSED.iloc[0])
Service_df_TRANSPOSED = Service_df_TRANSPOSED.iloc[1:, :]
Service_df_TRANSPOSED.columns = Description_list_from_first_row

Service_df_TRANSPOSED = Service_df_TRANSPOSED.reset_index()
Service_df_TRANSPOSED = Service_df_TRANSPOSED.rename(columns={'index': 'month-yr'})

Service_df_TRANSPOSED

,month-yr,FLAT REPAIRS,TIRE ROTATIONS,MOUNT/MOUNT+DISMOUNTS,SERVICE CALL HOURS,SCRAPS
0,JAN 2020,6,0,49,701,28
1,FEB 2020,1,0,56,288,39
2,MAR 2020,1,1,59,467,76
3,APR 2020,5,2,95,459,51
4,MAY 2020,0,15,77,136,23
5,JUN 2020,1,0,54,285,36
6,JUL 2020,3,0,56,362,50
7,AUG 2020,0,2,87,720,46
8,SEPT 2020,0,0,82,655,40
9,OCT 2020,2,0,59,345,28


In [58]:
# TO GRAPH, we'll setup x and y variables:

# Setup our x (independent) variable
Month_LIST = [f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', 
                f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', f'JUL {Last_Year}', 
                f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', 
                f'DEC {Last_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', 
                f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', 
                f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', 
                f'NOV {Current_Year}', f'DEC {Current_Year}'
            ]

# Setup our dependent Y Variables using the data columns:
Flat_Repair_Values = Service_df_TRANSPOSED['FLAT REPAIRS'].to_list()
Tire_Rotation_Values = Service_df_TRANSPOSED["TIRE ROTATIONS"].tolist()
Mount_and_Dismount_Values = Service_df_TRANSPOSED["MOUNT/MOUNT+DISMOUNTS"].to_list()
Service_Call_Hours_Values = Service_df_TRANSPOSED["SERVICE CALL HOURS"].to_list()
Scrap_Values = Service_df_TRANSPOSED["SCRAPS"].to_list()

fig = go.Figure()
# Add a light blue filter over last year's graph to distinguish it from this year:
fig.add_vrect(x0=0, x1=12,
                annotation_text=" Last Year ", annotation_position="top left",
                annotation=dict(
    font_size=27, font_family="Times New Roman"),
    fillcolor=None, opacity=0.25, line_width=0)
# Optionally add a filter color for this yr too, lightgreen works (or choose "white" or comment out this block for none)
fig.add_vrect(x0=12, x1=12,
                annotation_text=" Current Year ", annotation_position="top left",
                annotation=dict(
    font_size=27, font_family="Times New Roman"),
    fillcolor=None, opacity=0.15, line_width=0)

# Add a green vertical line the graph to separate last year from the current one:
fig.add_vline(x=11.5, line_width=3.5,
                line_dash="dash", line_color="green")

# Create and Style Traces:
fig.add_trace(go.Scatter(x=Month_LIST, y=Flat_Repair_Values, name='Flat Repairs',
                            line=dict(color='purple', width=2)))
fig.add_trace(go.Scatter(x=Month_LIST, y=Tire_Rotation_Values, name='Tire Rotations',
                            line=dict(color='#FFB500', width=2)))
fig.add_trace(go.Scatter(x=Month_LIST, y=Mount_and_Dismount_Values, name='Mounts/Mount+Dismounts',
                            line=dict(color='royalblue', width=2)))
fig.add_trace(go.Scatter(x=Month_LIST, y=Service_Call_Hours_Values, name='Service Call Hours',
                            line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=Month_LIST, y=Scrap_Values, name='Scraps',
                            line=dict(color='#351C15', width=2)))
# HERE great a hack TO MAKE SURE ALL THE MONTHS SHOW UP.  ADD A WHITE LINE THAT HAS all zeros and name it as a blank string so the legend doesn't show it
# fig.add_trace(go.Scatter(x=Month_LIST2, y=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], name='',
#                          line=dict(color='white', width=1)))


# Now using this graph, we create 2 different customized graph sizes; one scaled to be used for the pdf "fig1" (larger) and one for the website size "fig2" (smaller):
# for pdf
fig1 = fig
# for website
fig2 = fig


# Edit the Layout for pdf size:
fig1.update_layout(
    title_text=f"<b>Service Purchases Trend {Last_Year} - {Current_Year}</b>",
    title_x=0.5,
    title_font_size=24,
    title_font_color="#351C15",
    xaxis_title="Month",
    yaxis_title='Units Sold',
    autosize=False, width=1350, height=675,
    showlegend=True,
    legend=dict(
        x=1,
        y=1,
        title_font_family="Times New Roman",
        font=dict(
            family=None,
            size=15,
            color="black"
        ),
        bgcolor="white",
        bordercolor="dimgray",
        borderwidth=.05
    ),
    # General axes and axes labels fonts:
    font_size=16)
# fig1.update_xaxes(autorange=False, type='category', range=list(f'YTD {Last_Year}', f'DEC {Current_Year}'))
# fig1.update_xaxes(autorange=True, type='category', fixedrange=True)
# fig1.update_xaxes(autorange=True, categoryorder='array', categoryarray=Month_LIST)


# Add a green vertical line the graph to separate last year from the current one:
fig1.add_vline(x=11.5, line_width=3.5,
                line_dash="dash", line_color="green")

# Write the image that we'll use for index.html to generate the pdf
# We'll save this image as a 'png' rather than an svg because svgs give unwanted dark lines over the gridlines for the pdf:
fig1.write_image(f"Images_graphs/{Current_Year}_Service_Purchase_Analysis.png")
fig1.show()



# NOW, create the size to be used for the interactive webpage version:

# Edit the Layout:
fig2.update_layout(
    title_text=f"<b>Service Purchases Trend {Last_Year} - {Current_Year}</b>",
    title_x=0.5,
    title_font_size=20,
    title_font_color="#351C15",
    xaxis_title="Month",
    yaxis_title='Units Sold',
    autosize=False, width=1050, height=525,
    showlegend=True,
    legend=dict(
        x=1,
        y=1,
        title_font_family="Times New Roman",
        font_size=12.5,
        bgcolor="white",
        bordercolor="dimgray",
        borderwidth=.05
    ),
    # General axes and axes labels fonts:
    font_size=12.5)


# Add a green vertical line the graph to separate last year from the current one:
fig2.add_vline(x=11.5, line_width=3.5,
                line_dash="dash", line_color="green")



# PLOTLY INTERACTIVE:
Service_Purchases_Trend_graph = fig2
Service_Purchases_Trend_graph_for_iframe = py.plot(Service_Purchases_Trend_graph, filename = f'{Customer_Name_Underscores}_Service_Purchases_Trend_graph', auto_open=False)
Service_Purchases_Trend_graph_for_iframe
fig2.show()

## 2. Bandag Retread Purchase Analysis Line Chart:

In [59]:
# RECALL, AGAIN, the df we saved earlier called df_for_just_current_year_retread_sizes.  We can use that now.
df_retreads_of_sizes_and_other_copy = df_retreads_of_sizes_and_other.copy()

# Remove the "RTEMP":
df_retreads_of_sizes_and_other_copy['Category'] = df_retreads_of_sizes_and_other_copy['Category'].str.replace('RTEMP', '')
# Ensure no spaces at the end:
df_retreads_of_sizes_and_other_copy['Category'] = df_retreads_of_sizes_and_other_copy['Category'].str.rstrip()

df_retreads_of_sizes_and_other_copy

,Category,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
1,295/75R22.5,271,32,27,14,19,39,14,20,24,19,26,25,12,614,11,47,30,32,18,14,116,89,185,32,33,7
2,11R22.5,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,1,0,0,0,0,10,0
3,OTHER RETREADS,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0


In [60]:
Retreads_by_Size_df_for_graphing = df_retreads_of_sizes_and_other_copy
Retreads_by_Size_df_for_graphing = Retreads_by_Size_df_for_graphing.drop(columns = [f'YTD {Last_Year}', f'YTD {Current_Year}'])
# For the Current Year, drop columns with dashes:
Retreads_by_Size_df_for_graphing = Retreads_by_Size_df_for_graphing.loc[:, ~(Retreads_by_Size_df_for_graphing.astype(str) == '-').all()]
Retreads_by_Size_df_for_graphing

# Transpose the Merged_Bandag_Retread_df to make it easier for graphing:
Retreads_by_Size_df_for_graphing_TRANSPOSED = Retreads_by_Size_df_for_graphing.T

Description_list_from_first_row = list(Retreads_by_Size_df_for_graphing_TRANSPOSED.iloc[0])
Retreads_by_Size_df_for_graphing_TRANSPOSED = Retreads_by_Size_df_for_graphing_TRANSPOSED.iloc[1:, :]
Retreads_by_Size_df_for_graphing_TRANSPOSED.columns = Description_list_from_first_row

Retreads_by_Size_df_for_graphing_TRANSPOSED = Retreads_by_Size_df_for_graphing_TRANSPOSED.reset_index()
Retreads_by_Size_df_for_graphing_TRANSPOSED = Retreads_by_Size_df_for_graphing_TRANSPOSED.rename(columns={'index': 'month-yr'})

Retreads_by_Size_df_for_graphing_TRANSPOSED

,month-yr,295/75R22.5,11R22.5,OTHER RETREADS
0,JAN 2020,32,0,0
1,FEB 2020,27,0,0
2,MAR 2020,14,0,0
3,APR 2020,19,0,0
4,MAY 2020,39,0,0
5,JUN 2020,14,0,0
6,JUL 2020,20,0,0
7,AUG 2020,24,0,0
8,SEPT 2020,19,0,1
9,OCT 2020,26,0,0


In [61]:
# Create Bandag Purchase Analysis Graph:

# Setup independent X Variables (which is just the month and save a list):
# Setup our x (independent) variable
Month_LIST2 = [f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', 
                f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', f'JUL {Last_Year}', 
                f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', 
                f'DEC {Last_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', 
                f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', 
                f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', 
                f'NOV {Current_Year}', f'DEC {Current_Year}'
            ]


fig = go.Figure()
# Add a filter over last year's graph to distinguish it from this year:
fig.add_vrect(x0=0, x1=12,
                annotation_text=" Last Year ", annotation_position="top left",
                annotation=dict(
    font_size=27, font_family="Times New Roman"),
    fillcolor=None, opacity=0.25, line_width=0)

# Optionally add a filter color for this yr too, lightgreen works (or choose "white" or comment out this block for none)
fig.add_vrect(x0=12, x1=12,
                annotation_text=" Current Year ", annotation_position="top left",
                annotation=dict(
    font_size=27, font_family="Times New Roman"),
    fillcolor=None, opacity=0.15, line_width=0)




# Setup a dependent Y Variables (By Creating separate lists for each type of Bandag Retread):
for i in range(len(Retreads_by_Size_df_for_graphing)):
    # Here, we will save a list for each column.  We'll set up to 10, assuming we won't have more than 9 tire sizes + 'Other sizes' to deal with.
    # Note, we will start with 1 instead of zero because the first column is month-yr (NOT a tire size column):
    if i == 1:
        one = Retreads_by_Size_df_for_graphing_TRANSPOSED.iloc[:, 1].to_list()
        # Create and Style Traces:
        fig.add_trace(go.Scatter(x=Month_LIST2, y=one, name=Retreads_by_Size_df_for_graphing_TRANSPOSED.columns[1], line=dict(color='black', width=2.5)))
    elif i == 2:
        two = Retreads_by_Size_df_for_graphing_TRANSPOSED.iloc[:, 2].to_list()
        fig.add_trace(go.Scatter(x=Month_LIST2, y=two, name=Retreads_by_Size_df_for_graphing_TRANSPOSED.columns[2], line=dict(color='royalblue', width=2.5)))
    elif i == 3:
        three = Retreads_by_Size_df_for_graphing_TRANSPOSED.iloc[:, 3].to_list()
        fig.add_trace(go.Scatter(x=Month_LIST2, y=three, name=Retreads_by_Size_df_for_graphing_TRANSPOSED.columns[3], line=dict(color='darkgreen', width=2.5)))
    elif i == 4:
        four = Retreads_by_Size_df_for_graphing_TRANSPOSED.iloc[:, 4].to_list()
        fig.add_trace(go.Scatter(x=Month_LIST2, y=four, name=Retreads_by_Size_df_for_graphing_TRANSPOSED.columns[4], line=dict(color='red', width=2.5)))
    elif i == 5:
        five = Retreads_by_Size_df_for_graphing_TRANSPOSED.iloc[:, 5].to_list()
        fig.add_trace(go.Scatter(x=Month_LIST2, y=five, name=Retreads_by_Size_df_for_graphing_TRANSPOSED.columns[5], line=dict(color='dimgray', width=2.5)))
    elif i == 6:
        six = Retreads_by_Size_df_for_graphing_TRANSPOSED.iloc[:, 6].to_list()
        fig.add_trace(go.Scatter(x=Month_LIST2, y=six, name=Retreads_by_Size_df_for_graphing_TRANSPOSED.columns[6], line=dict(color='turquoise', width=2.5)))
    elif i == 7:
        seven = Retreads_by_Size_df_for_graphing_TRANSPOSED.iloc[:, 7].to_list()
        fig.add_trace(go.Scatter(x=Month_LIST2, y=seven, name=Retreads_by_Size_df_for_graphing_TRANSPOSED.columns[7], line=dict(color='purple', width=2.5)))
    elif i == 8:
        eight = Retreads_by_Size_df_for_graphing_TRANSPOSED.iloc[:, 8].to_list()
        fig.add_trace(go.Scatter(x=Month_LIST2, y=eight, name=Retreads_by_Size_df_for_graphing_TRANSPOSED.columns[8], line=dict(color='#caa74d', width=2.5)))

# HERE IS A GREAT Hack TO MAKE SURE ALL THE MONTHS SHOW UP!!!!  ADD A WHITE LINE THAT HAS all zeros and name it as a blank string so the legend doesn't show it!
# fig.add_trace(go.Scatter(x=Month_LIST2, y=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], name='',
#                          line=dict(color='white', width=1)))


fig1 = fig
fig2 = fig

fig1.update_layout(
    title_text=f"<b>Bandag Retreads by Size Trend {Last_Year} - {Current_Year}</b>",
    title_x=.5,
    title_font_size=24,
    title_font_color="#351C15",
    xaxis_title='Month',
    yaxis_title='Units Sold',
    autosize=False, width=1350, height=675,
    showlegend=True,
    legend=dict(
        x=1,
        y=1,
        title_font_family="Times New Roman",
        font=dict(
            family=None,
            size=15,
            color="black"
        ),
        bgcolor="white",
        bordercolor="dimgray",
        borderwidth=.05
    ),
    # General axes and axes labels fonts:
    font=dict(
        family=None,
        size=16,
        color="black"
    ))

# Add a green vertical line the graph to separate last year from the current one:
fig1.add_vline(x=11.5, line_width=3.5,
                line_dash="dash", line_color="green")


# Write the image that we'll use for index.html to generate the pdf
# We'll save this image as a 'png' rather than an svg because svgs give unwanted dark lines over the gridlines:
fig1.write_image(f"Images_graphs/{Current_Year}_Bandag_Purchase_Analysis_by_Tire_Size.png")
fig1.show()





# ============================================================================================================================================
# AGAIN, we'll need to create a graph for the webpage version as well that will be different sizes and interactive with plotly:
# ============================================================================================================================================

# Edit the Layout:
fig2.update_layout(
    title_text=f"<b>Bandag Retreads by Size Trend {Last_Year} - {Current_Year}</b>",
    title_x=.5,
    title_font_size=20,
    title_font_color="#351C15",
    xaxis_title='Month',
    yaxis_title='Units Sold',
    autosize=False, width=1050, height=525,
    showlegend=True,
    legend=dict(
        x=1,
        y=1,
        title_font_family="Times New Roman",
        font=dict(
            family=None,
            size=13,
            color="black"
        ),
        bgcolor="white",
        bordercolor="dimgray",
        borderwidth=.05
    ),
    # General axes and axes labels fonts:
    font=dict(
        family=None,
        size=13,
        color="black"
    ))

# Add a green vertical line the graph to separate last year from the current one:
fig2.add_vline(x=11.5, line_width=3.5,
                line_dash="dash", line_color="green")

# Save this smaller version for the website:
# fig2.write_image(f"Images_graphs/{Current_Year}_Bandag_Purchase_Analysis_by_Tire_Size.svg")
fig2.show()





# PLOTLY INTERACTIVE:
Bandag_Purchases_Trend_graph = fig2
Bandag_Purchases_Trend_graph_by_Tire_Size_for_iframe = py.plot(Bandag_Purchases_Trend_graph, filename = f'{Customer_Name_Underscores}_Bandag_Purchases_Trend_graph_by_Tire_Size', auto_open=False)
Bandag_Purchases_Trend_graph_by_Tire_Size_for_iframe


'https://plotly.com/~alexanderconerly/122/'

## 3. Build a bar graph (not interactive) for all retreads

In [62]:
ALL_Retreads_df_for_graphing = Dict_for_excel_sheets['Tire Sales Summary Table'].iloc[[0]].reset_index(drop=True)
ALL_Retreads_df_for_graphing = ALL_Retreads_df_for_graphing.drop(columns = [f'YTD {Last_Year}', f'YTD {Current_Year}'])
# For the Current Year, drop columns with dashes:
ALL_Retreads_df_for_graphing = ALL_Retreads_df_for_graphing.loc[:, ~(ALL_Retreads_df_for_graphing.astype(str) == '-').all()]


# Transpose the Merged_Bandag_Retread_df to make it easier for graphing:
ALL_Retreads_df_for_graphing_TRANSPOSED = ALL_Retreads_df_for_graphing.T

Description_list_from_first_row = list(ALL_Retreads_df_for_graphing_TRANSPOSED.iloc[0])
ALL_Retreads_df_for_graphing_TRANSPOSED = ALL_Retreads_df_for_graphing_TRANSPOSED.iloc[1:, :]
ALL_Retreads_df_for_graphing_TRANSPOSED.columns = Description_list_from_first_row

ALL_Retreads_df_for_graphing_TRANSPOSED = ALL_Retreads_df_for_graphing_TRANSPOSED.reset_index()
ALL_Retreads_df_for_graphing_TRANSPOSED = ALL_Retreads_df_for_graphing_TRANSPOSED.rename(columns={'index': 'month-yr'})

ALL_Retreads_df_for_graphing_TRANSPOSED

,month-yr,RETREADS
0,JAN 2020,32
1,FEB 2020,27
2,MAR 2020,14
3,APR 2020,19
4,MAY 2020,39
5,JUN 2020,14
6,JUL 2020,20
7,AUG 2020,24
8,SEPT 2020,20
9,OCT 2020,26


In [63]:
# We can also create an ALL Retreads graph:

# Setup our x (independent) variable
month_yr_list = [f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', 
                f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', f'JUL {Last_Year}', 
                f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', 
                f'DEC {Last_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', 
                f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', 
                f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', 
                f'NOV {Current_Year}', f'DEC {Current_Year}'
            ]

# Get y values for each year:
year1 = ALL_Retreads_df_for_graphing_TRANSPOSED.iloc[0:12, 1].to_list()
year2 = ALL_Retreads_df_for_graphing_TRANSPOSED.iloc[12:, 1].to_list()

# Create the bar graphs for each year:
fig = go.Figure(data=[
    go.Bar(name=f'{Last_Year}', x=['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV', 'DEC'], y=year1, text=year1, textposition='outside', marker_color='#A6ACAF'), 
    go.Bar(name=f'{Current_Year}', x=['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV', 'DEC'], y=year2, text=year2, textposition='outside', marker_color=customer_color_1)
])


# Create two graph figures (one sized for website (smaller) another sized for pdf (needs larger)).  Note, in this case, we don't need to use Plotly interactive because
# These bar graphs can just be static.
fig1 = fig
fig2 = fig


# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig1.update_layout(
    barmode='group', 
    title_text=f"<b>Bandag Retread Purchases Trend {Last_Year} - {Current_Year}</b>",
    title_font_size=38,
    title_font_color="#351C15",
    title_x = 0.5,
    xaxis_title="Month",
    yaxis_title='Units Sold',
    autosize=False, 
    width=1445, 
    height=650,
    # xaxis_tickangle=-45,
    showlegend=True, 
    font_size=25)
    # font=dict(
    #         size=25,
    #         color="black"
    #     )
    # )

# Save the file to use for Website.
fig2.write_image(f"Images_graphs/{Current_Year}_Bandag_Purchase_Analysis_ALL_SIZES_pdf_size_BAR_GRAPH.png")
fig1.show()



# Edit the Layout for the Webpage (smaller)... for bar chart we will NOT need to use plotly chart studio:
fig2.update_layout(
    title_text=f"<b>Bandag Retreads Trend {Last_Year} - {Current_Year}</b>",
    title_x=.5,
    title_font_size=20,
    title_font_color="#351C15",
    xaxis_title='Month',
    yaxis_title='Units Sold',
    autosize=False, width=1050, height=525,
    showlegend=True,
    legend=dict(
        x=1,
        y=1,
        title_font_family="Times New Roman",
        font=dict(
            family=None,
            size=13,
            color="black"
        ),
        bgcolor="white",
        bordercolor="dimgray",
        borderwidth=.05
    ),
    # General axes and axes labels fonts:
    font=dict(
        family=None,
        size=13,
        color="black"
    ))

# Save the file to use in website (we won't use plotly chart studio iframe because this is just a bar graph and doesn't need to be interactive).
fig2.write_image(f"Images_graphs/{Current_Year}_Bandag_Purchase_Analysis_ALL_SIZES_website_size_BAR_GRAPH.svg")
fig2.show()


## 4. Create ALL SIZES Purchases Bar Graphs for each specified vendor:

In [64]:
vendor_names_list

['Bridgestone', 'Continental', 'Yokohama']

In [65]:

for i, each_vendor in enumerate(vendor_names_list):

    vendor_df_for_graphing = Dict_for_excel_sheets['Tire Sales by Vendor'].loc[Dict_for_excel_sheets['Tire Sales by Vendor']['Category'] == f'{vendor_names_list[i].upper()} NEW'].reset_index(drop=True)
    vendor_df_for_graphing = vendor_df_for_graphing.drop(columns = [f'YTD {Last_Year}', f'YTD {Current_Year}'])
    # For the Current Year, drop columns with dashes:
    vendor_df_for_graphing = vendor_df_for_graphing.loc[:, ~(vendor_df_for_graphing.astype(str) == '-').all()]


    # Transpose the Merged_Bandag_Retread_df to make it easier for graphing:
    vendor_df_for_graphing_TRANSPOSED = vendor_df_for_graphing.T

    Description_list_from_first_row = list(vendor_df_for_graphing_TRANSPOSED.iloc[0])
    vendor_df_for_graphing_TRANSPOSED = vendor_df_for_graphing_TRANSPOSED.iloc[1:, :]
    vendor_df_for_graphing_TRANSPOSED.columns = Description_list_from_first_row

    vendor_df_for_graphing_TRANSPOSED = vendor_df_for_graphing_TRANSPOSED.reset_index()
    vendor_df_for_graphing_TRANSPOSED = vendor_df_for_graphing_TRANSPOSED.rename(columns={'index': 'month-yr'})

        
    # Setup our x (independent) variables
    month_yr_list = [f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', 
                    f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', f'JUL {Last_Year}', 
                    f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', 
                    f'DEC {Last_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', 
                    f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', 
                    f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', 
                    f'NOV {Current_Year}', f'DEC {Current_Year}'
                ]

    # Get y values for each year:
    year1 = vendor_df_for_graphing_TRANSPOSED.iloc[0:12, 1].to_list()
    year2 = vendor_df_for_graphing_TRANSPOSED.iloc[12:, 1].to_list()


    # Create the bar graphs:
    fig = go.Figure(data=[
        go.Bar(name=f'{Last_Year}', x=['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV', 'DEC'], y=year1, text=year1, textposition='outside', marker_color='#A6ACAF'), 
        go.Bar(name=f'{Current_Year}', x=['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV', 'DEC'], y=year2, text=year2, textposition='outside', marker_color='royalblue')
    ])

    # Create two graph figures (one sized for website (smaller) another sized for pdf (needs larger)).  Note, in this case, we don't need to use Plotly interactive because
    # These bar graphs can just be static.
    fig1 = fig
    fig2 = fig


    # Bigger version FOR THE PDF (if desired):
    fig1.update_layout(
        barmode='group', 
        title_text=f"<b>{vendor_names_list[i]} New Tire Purchases Trend {Last_Year} - {Current_Year}</b>",
        title_font_size=26,
        title_font_color="#351C15",
        title_x = 0.5,
        xaxis_title="Month",
        yaxis_title='Units Sold',
        autosize=False, 
        width=1445, 
        height=650,
        # xaxis_tickangle=-45,
        showlegend=True, 
        font_size=25)
        # font=dict(
        #         size=25,
        #         color="black"
        #     )
        # )

    # Save the file to use for pdf if desired.
    fig1.write_image(f"Images_graphs/{Current_Year}_{vendor_names_list[i]}_Purchase_Analysis_ALL_SIZES_pdf_size_BAR_GRAPH.png")
    fig1.show()



    # Edit the Layout for the Webpage (smaller)... for bar chart we will NOT need to use plotly chart studio:
    fig2.update_layout(
        title_text=f"<b>{vendor_names_list[i]} New Tire Purchases Trend {Last_Year} - {Current_Year}</b>",
        title_x=.5,
        title_font_size=20,
        title_font_color="#351C15",
        xaxis_title='Month',
        yaxis_title='Units Sold',
        autosize=False, width=1050, height=525,
        showlegend=True,
        legend=dict(
            x=1,
            y=1,
            title_font_family="Times New Roman",
            font=dict(
                family=None,
                size=13,
                color="black"
            ),
            bgcolor="white",
            bordercolor="dimgray",
            borderwidth=.05
        ),
        # General axes and axes labels fonts:
        font=dict(
            family=None,
            size=13,
            color="black"
        ))

    # Save this smaller file to use in website:
    fig2.write_image(f"Images_graphs/{Current_Year}_{vendor_names_list[i]}_Purchase_Analysis_ALL_SIZES_website_size_BAR_GRAPH.svg")
    fig2.show()


## 5. Create each Vendor's Purchases by Size Trend Analysis Line Chart:

In [66]:
Dict_for_excel_sheets['Tire Sales by Vendor']

,Category,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,BRIDGESTONE NEW,410,30,22,56,48,26,44,28,37,46,23,29,21,463,28,17,22,53,38,52,64,40,31,57,38,23
1,295/75R22.5,340,26,17,49,41,24,37,22,33,40,18,17,16,349,17,15,19,42,36,39,44,30,19,42,28,18
2,11R22.5,24,0,1,1,0,1,4,3,0,0,2,12,0,46,2,0,0,7,1,3,9,5,12,7,0,0
3,OTHER SIZES,46,4,4,6,7,1,3,3,4,6,3,0,5,68,9,2,3,4,1,10,11,5,0,8,10,5
4,BS STEER-NEW,178,12,10,26,13,12,22,15,17,18,7,16,10,175,8,6,12,29,14,11,16,18,6,22,17,16
5,BS DRIVE-NEW,108,8,9,15,14,8,8,7,10,14,7,1,7,201,13,6,6,19,21,34,23,13,20,30,16,0
6,BS TRAILER-NEW,124,10,3,15,21,6,14,6,10,14,9,12,4,87,7,5,4,5,3,7,25,9,5,5,5,7
7,CONTINENTAL NEW,79,8,5,0,8,14,11,17,4,8,2,1,1,43,0,12,27,4,0,0,0,0,0,0,0,0
8,295/75R22.5,66,4,1,0,7,14,11,15,4,6,2,1,1,43,0,12,27,4,0,0,0,0,0,0,0,0
9,11R22.5,12,4,4,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [67]:
df_temp = Dict_for_excel_sheets['Tire Sales by Vendor']

# We'll create a new dictionary to house the names for each vendor's graph so we can add it to the html page called "Trend_Analysis_Charts.html"
Dict_for_plotly_iframes_of_Line_Charts_by_Tire_Sizes = {}


# Loop through each vendor name list:
for i, each_value in enumerate(vendor_names_list):

    # To graph each vendor's tire size data, we'll need to use iloc to filter the data to just those tire sizes for each vendor.
    # However, we'll need to dynamically get the index position of both the first and last tire size for each vendor
    # so we can use iloc.

    # Get the index position of the first tire size row for that vendor:
    index_position_integer_of_vendor_first_tire_size = df_temp.index[df_temp['Category'] == f'{vendor_names_list[i].upper()} NEW'][0] + 1

    # Recall we saved a variable at the top of the notebook called number_of_tire_sizes.  We'll use this to add to the first tire size.
    # This will give us the index position of the "OTHER" tire size row for that vendor.  This is what we'll want for the right side of the 
    # iloc filter because iloc is exclusive on the right.
    ending_number = index_position_integer_of_vendor_first_tire_size + number_of_tire_sizes


    Vendor_by_Size_df_for_graphing = df_temp.iloc[index_position_integer_of_vendor_first_tire_size:ending_number, :].reset_index(drop=True)
    Vendor_by_Size_df_for_graphing = Vendor_by_Size_df_for_graphing.drop(columns = [f'YTD {Last_Year}', f'YTD {Current_Year}'])
    # For the Current Year, drop columns with dashes:
    Vendor_by_Size_df_for_graphing = Vendor_by_Size_df_for_graphing.loc[:, ~(Vendor_by_Size_df_for_graphing.astype(str) == '-').all()]


    # Transpose the _df to make it easier for graphing:
    Vendor_by_Size_df_for_graphing_TRANSPOSED = Vendor_by_Size_df_for_graphing.T

    Description_list_from_first_row = list(Vendor_by_Size_df_for_graphing_TRANSPOSED.iloc[0])
    Vendor_by_Size_df_for_graphing_TRANSPOSED = Vendor_by_Size_df_for_graphing_TRANSPOSED.iloc[1:, :]
    Vendor_by_Size_df_for_graphing_TRANSPOSED.columns = Description_list_from_first_row

    Vendor_by_Size_df_for_graphing_TRANSPOSED = Vendor_by_Size_df_for_graphing_TRANSPOSED.reset_index()
    Vendor_by_Size_df_for_graphing_TRANSPOSED = Vendor_by_Size_df_for_graphing_TRANSPOSED.rename(columns={'index': 'month-yr'})


    # Now that we have the df in the loop filtered to only the tire sizes rows, we'll create the graph for that vendor in the loop:

    Month_LIST2 = Vendor_by_Size_df_for_graphing_TRANSPOSED['month-yr'].to_list()

    fig = go.Figure()
    # Add a filter over last year's graph to distinguish it from this year:
    fig.add_vrect(x0=0, x1=12,
                    annotation_text=" Last Year ", annotation_position="top left",
                    annotation=dict(
        font_size=27, font_family="Times New Roman"),
        fillcolor=None, opacity=0.25, line_width=0)

    # Optionally add a filter color for this yr too, lightgreen works (or choose "white" or comment out this block for none)
    fig.add_vrect(x0=12, x1=12,
                    annotation_text=" Current Year ", annotation_position="top left",
                    annotation=dict(
        font_size=27, font_family="Times New Roman"),
        fillcolor=None, opacity=0.15, line_width=0)


    # Create a list of colors:
    list_of_colors = ['black', 'royalblue', 'darkgreen', 'red', 'dimgray', 'turquoise', 'purple', 'orange', 'pink']

    # Loop through every column in the transformed df for the vendor in the loop and create a trace of its data (the y values):
    for loop_number, column in enumerate(Vendor_by_Size_df_for_graphing_TRANSPOSED):
        y_values = Vendor_by_Size_df_for_graphing_TRANSPOSED.iloc[:, loop_number].to_list()
        # Create and Style Traces:
        # Note:  We have to skip the first column because it's "month-yr" which is our x values.  So we'll use loop_number > 0 to use only the other columns for y values.
        if loop_number > 0:
            fig.add_trace(go.Scatter(x=Month_LIST2, y=y_values, name=Vendor_by_Size_df_for_graphing_TRANSPOSED.columns[loop_number], line=dict(color=list_of_colors[loop_number], width=2.5)))

    # HERE IS A GREAT Hack TO MAKE SURE ALL THE MONTHS SHOW UP!!!!  ADD A WHITE LINE THAT HAS all zeros and name it as a blank string so the legend doesn't show it!
    # fig.add_trace(go.Scatter(x=Month_LIST2, y=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], name='',
    #                          line=dict(color='white', width=1)))
    # Edit the Layout:


    fig1 = fig
    fig2 = fig

    fig1.update_layout(
        title_text=f"<b>{vendor_names_list[i]} Purchases by Size Trend {Last_Year} - {Current_Year}</b>",
        title_x=.5,
        title_font_size=24,
        title_font_color="#351C15",
        xaxis_title='Month',
        yaxis_title='Units Sold',
        autosize=False, width=1350, height=675,
        showlegend=True,
        legend=dict(
            x=1,
            y=1,
            title_font_family="Times New Roman",
            font=dict(
                family=None,
                size=15,
                color="black"
            ),
            bgcolor="white",
            bordercolor="dimgray",
            borderwidth=.05
        ),
        # General axes and axes labels fonts:
        font=dict(
            family=None,
            size=16,
            color="black"
        ))

    # Add a green vertical line the graph to separate last year from the current one:
    fig1.add_vline(x=11.5, line_width=3.5,
                    line_dash="dash", line_color="green")



    # Save this larger image for the PDF:
    # We'll save this image as a 'png' rather than an svg because svgs give unwanted dark lines over the gridlines:
    fig1.write_image(f"Images_graphs/{Current_Year}_{vendor_names_list[i]}_Purchase_Analysis.png")
    fig1.show()




    # Now generate a smaller graph more suitable for the website size
    fig2.update_layout(
        title_text=f"<b>{vendor_names_list[i]} Purchases by Size Trend {Last_Year} - {Current_Year}</b>",
        title_x=.5,
        title_font_size=20,
        title_font_color="#351C15",
        xaxis_title='Month',
        yaxis_title='Units Sold',
        autosize=False, width=1050, height=525,
        showlegend=True,
        legend=dict(
            x=1,
            y=1,
            title_font_family="Times New Roman",
            font=dict(
                family=None,
                size=13,
                color="black"
            ),
            bgcolor="white",
            bordercolor="dimgray",
            borderwidth=.05
        ),
        # General axes and axes labels fonts:
        font=dict(
            family=None,
            size=13,
            color="black"
        ))

    # Add a green vertical line the graph to separate last year from the current one:
    fig2.add_vline(x=11.5, line_width=3.5,
                    line_dash="dash", line_color="green")

    # Save the PLOTLY INTERACTIVE graph to plotly chart studio and 
    # save it into a dictionary to use that iframe in the "Trend_Analysis_Line_Charts.html" generation near the end of this notebook:

    Dict_for_plotly_iframes_of_Line_Charts_by_Tire_Sizes[f'{Customer_Name_Underscores}_{vendor_names_list[i]}_Purchases_Trend_graph_for_iframe'] = py.plot(fig2, filename = f'{Customer_Name_Underscores}_{vendor_names_list[i]}_Purchases_Trend_graph', auto_open=False)

    fig2.show()



# Get NRT DATA by scraping BFI site.  

### NOTE:  We will scrape both last year and the current YTD bc we want to compare last year with current YTD in graphs:

## Since the website appears to do better the second attempt of selecting the NRT template, we'll 
## do a mock selection here prior to doing it for real in the loop afterwards:

In [68]:
# ---------Set the executable path and initialize the chrome browser in splinter -------
executable_path = {'executable_path': 'C:/webdrivers/chromedriver'}
browser = Browser('chrome', **executable_path)

# --------------------------- LOGIN ------------------------------------------
# ---Set Variables for login:
Page_usrname = 'stmreport'
Page_pword = 'stmreport'

attempts = 0

# Go to BFI login page:
browser.visit('https://fleetinventory.com')

#Find the username and password boxes and fill them in:
browser.find_by_id('username').fill(Page_usrname)
browser.find_by_id('password').fill(Page_pword)
# Click on login button:
browser.find_by_name('submit').first.click()

# ---------------- Navigate to the NRT Summary page ---------------------------
# Click on "Reporting" tab and then click "NRT Summary":
time.sleep(3)
browser.find_by_text('Reporting', wait_time=4).first.click()
browser.find_by_text('NRT Summary', wait_time=4).first.click()
time.sleep(10)

# Click on the "Report Profile" drop down box.  This will give you options from saved profiles you can choose from (for example, RR-UPS SCRAP SUMMARY)
browser.find_by_id('id_report_profile', wait_time=6).first.click()

# ----------------- To select the region on the BFI site, we'll SELECT ONE OF THE SAVED TEMPLATES FROM THE DROP-DOWN WE JUST CLICKED ON -----------------
time.sleep(2)
browser.find_by_text(f'{bfi_website_saved_NRT_Report_Name_for_Webscrape}', wait_time=10).first.click()
time.sleep(15)
# Once done, logout and quit the browser:
browser.find_by_id('logout').first.click()
browser.quit()

In [69]:
# Loop through last year and YTD of current year:
years = [Last_Year, Current_Year]

for year in years:

    Jan = year * 100 + 1
    Feb = year * 100 + 2
    Mar = year * 100 + 3
    Apr = year * 100 + 4
    May = year * 100 + 5
    Jun = year * 100 + 6
    Jul = year * 100 + 7
    Aug = year * 100 + 8
    Sept = year * 100 + 9
    Oct = year * 100 + 10
    Nov = year * 100 + 11
    Dec = year * 100 + 12

    # We can also define a list of these numerical Months:
    # Again, these will be in format like Jan = 202101.
    Months = [Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sept, Oct, Nov, Dec]

    # ---------Set the executable path and initialize the chrome browser in splinter -------
    executable_path = {'executable_path': 'C:/webdrivers/chromedriver'}
    browser = Browser('chrome', **executable_path)

    # --------------------------- LOGIN ------------------------------------------
    # ---Set Variables for login:
    Page_usrname = 'stmreport'
    Page_pword = 'stmreport'

    attempts = 0

    # Go to BFI login page:
    browser.visit('https://fleetinventory.com')

    #Find the username and password boxes and fill them in:
    browser.find_by_id('username').fill(Page_usrname)
    browser.find_by_id('password').fill(Page_pword)
    # Click on login button:
    browser.find_by_name('submit').first.click()

    # ---------------- Navigate to the NRT Summary page ---------------------------
    # Click on "Reporting" tab and then click "NRT Summary":
    time.sleep(3)
    browser.find_by_text('Reporting', wait_time=4).first.click()
    browser.find_by_text('NRT Summary', wait_time=4).first.click()
    time.sleep(10)

    # Click on the "Report Profile" drop down box.  This will give you options from saved profiles you can choose from (for example, RR-UPS SCRAP SUMMARY)
    browser.find_by_id('id_report_profile', wait_time=6).first.click()

    # ----------------- To select the region on the BFI site, we'll SELECT ONE OF THE SAVED TEMPLATES FROM THE DROP-DOWN WE JUST CLICKED ON -----------------
    time.sleep(2)
    browser.find_by_text(f'{bfi_website_saved_NRT_Report_Name_for_Webscrape}', wait_time=10).first.click()
    time.sleep(15)

    while attempts < 4:
        attempts += 1

        browser.find_by_id('id_date_range', wait_time=15).first.click()
        browser.find_by_text('Range', wait_time=1).first.click()


        # ------------- To fill in the date range on the BFI site, we'll first need to define Start and End Date variables for each month --------------
        January_Start = f'01-01-{year}'
        January_End = f'01-31-{year}'

        February_Start = f'02-01-{year}'
        # ----------------------
        #Adjust for Leap year:
        if (year % 400 == 0) and (year % 100 != 0) and (year % 4 != 0): 
            # (If all the above are true, it's a leap year)
            February_End = f'02-29-{year}'
        else:
            February_End = f'02-28-{year}'
        # ----------------------

        March_Start = f'03-01-{year}'
        March_End = f'03-31-{year}'

        April_Start = f'04-01-{year}'
        April_End = f'04-30-{year}'

        May_Start = f'05-01-{year}'
        May_End = f'05-31-{year}'

        June_Start = f'06-01-{year}'
        June_End = f'06-30-{year}'

        July_Start = f'07-01-{year}'
        July_End = f'07-31-{year}'

        August_Start = f'08-01-{year}'
        August_End = f'08-31-{year}'

        September_Start = f'09-01-{year}'
        September_End = f'09-30-{year}'

        October_Start = f'10-01-{year}'
        October_End = f'10-31-{year}'

        November_Start = f'11-01-{year}'
        November_End = f'11-30-{year}'

        December_Start = f'12-01-{year}'
        December_End = f'12-31-{year}'

        # ------ Create a List of lists.  The inner lists will each be for each month's start, end, and number (for instacn Jan's number could be 202101 if 2021 is year).
        Months_Start_End_Numerical = [[January_Start, January_End, Jan],
                                    [February_Start, February_End, Feb],
                                    [March_Start, March_End, Mar],
                                    [April_Start, April_End, Apr],
                                    [May_Start, May_End, May],
                                    [June_Start, June_End, Jun], 
                                    [July_Start, July_End, Jul],
                                    [August_Start, August_End, Aug],
                                    [September_Start,September_End, Sept],
                                    [October_Start, October_End, Oct], 
                                    [November_Start, November_End, Nov], 
                                    [December_Start, December_End, Dec]]

        #Give the webpage a moment to continue loading:
        time.sleep(2)
        prompt = browser.get_alert()
        # Now loop through the list of lists to download the CSVs for only the months that have been completed (Current_Month > month_abbr)
        Was_Successful = True
        for start, end, month_as_YYYYMM in Months_Start_End_Numerical:
            if Current_Month > month_as_YYYYMM:
                try:
                    # Fill in the date range for the NRT CSV:
                    time.sleep(2)
                    browser.find_by_id('id_start_date', wait_time=4).fill(start)
                    browser.find_by_id('id_end_date', wait_time=4).fill(end)
                    # Click the "run report" button to download the NRT CSV
                    browser.find_by_id('form_submit').first.click()
                    # Give the dowload 2 seconds to complete:
                    time.sleep(1)
                    # Move the file from "downloads" folder 
                    # and rename it from nrt_analysis.csv to specify the region and date
                    original = r'C:/Users/alex.conerly/Downloads/nrt_analysis.csv'
                    target = f'{os.getcwd()}/NRT_CSVs_BFI_SCRAPE/CUSTOMER_NRT_{month_as_YYYYMM}.csv'

                    # Attempt to move the file
                    shutil.move(original,target)
                    time.sleep(1)

                except:
                    print('FileNotFoundError, likely because page did not fully load')
                    Was_Successful = False
                    break

        if Was_Successful == True:
            break

    # Once done, logout and quit the browser:
    browser.find_by_id('logout').first.click()
    browser.quit()

## Now that we've scraped and imported the NRT data for two years and saved them as CSVs, we'll loop through both years again
## and format the data to prepare it for html and also graphing.  Of course, html will only show the current year.  The graph will show both.

In [70]:

# HERE, we will loop through the current year FIRST!!!!
# This way we can dynamically save the list of top NRT codes for this year.
# That way, once we have those codes, we can filter LAST YEAR's nrt df manually to only include that list of codes.
# For example, if NRT code 2001 (which is for "Crown Cut(s)/Puncture(s)") is one of the top causes a tire could not be retreaded in the current year,
# we will find that out in the first loop (current year) and then manually ensure code 2001 is included for last year's data, EVEN if 
# code 2001 was not one of the top reasons last year.  This just allows us to easily merge both years on the same codes and description later into one df.

years = [Current_Year, Last_Year]

for year in years:

    Jan = year * 100 + 1
    Feb = year * 100 + 2
    Mar = year * 100 + 3
    Apr = year * 100 + 4
    May = year * 100 + 5
    Jun = year * 100 + 6
    Jul = year * 100 + 7
    Aug = year * 100 + 8
    Sept = year * 100 + 9
    Oct = year * 100 + 10
    Nov = year * 100 + 11
    Dec = year * 100 + 12



    # JAN
    if Jan < Current_Month:
        Nrt_JAN = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/CUSTOMER_NRT_{Jan}.csv', skiprows=24)
        # Get rid of the Top 25 unecessary rows and the bottom 2 rows for "Total" and "Grand Total"
        Nrt_JAN = Nrt_JAN.iloc[:-2, :]
        # Get rid of the "% of Total" column.  We will calculate percentages manually later after merging all month dfs etc.
        Nrt_JAN = Nrt_JAN.drop(columns=[r'% of Total'])
        # Rename the "Total" column to the month from which this NRT data was derived:
        Nrt_JAN = Nrt_JAN.rename(columns = {'Total': 'JAN'})

    # For all the rest of the months, do the same as above BUT drop the Description column to 
    # prepare for merging with the January df (which already has the Description column):

    # FEB
    if Feb < Current_Month:
        Nrt_FEB = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/CUSTOMER_NRT_{Feb}.csv', skiprows=24)
        Nrt_FEB = Nrt_FEB.iloc[:-2, :]
        Nrt_FEB = Nrt_FEB.drop(columns=[r'% of Total'])
        Nrt_FEB = Nrt_FEB.rename(columns = {'Total': 'FEB'})
        Nrt_FEB = Nrt_FEB.drop(columns = ['Description'])

    # MAR
    if Mar < Current_Month:
        Nrt_MAR = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/CUSTOMER_NRT_{Mar}.csv', skiprows=24)
        Nrt_MAR = Nrt_MAR.iloc[:-2, :]
        Nrt_MAR = Nrt_MAR.drop(columns=[r'% of Total'])
        Nrt_MAR = Nrt_MAR.rename(columns = {'Total': 'MAR'})
        Nrt_MAR = Nrt_MAR.drop(columns = ['Description'])

    # APR
    if Apr < Current_Month:
        Nrt_APR = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/CUSTOMER_NRT_{Apr}.csv', skiprows=24)
        Nrt_APR = Nrt_APR.iloc[:-2, :]
        Nrt_APR = Nrt_APR.drop(columns=[r'% of Total'])
        Nrt_APR = Nrt_APR.rename(columns = {'Total': 'APR'})
        Nrt_APR = Nrt_APR.drop(columns = ['Description'])

    # MAY
    if May < Current_Month:
        Nrt_MAY = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/CUSTOMER_NRT_{May}.csv', skiprows=24)
        Nrt_MAY = Nrt_MAY.iloc[:-2, :]
        Nrt_MAY = Nrt_MAY.drop(columns=[r'% of Total'])
        Nrt_MAY = Nrt_MAY.rename(columns = {'Total': 'MAY'})
        Nrt_MAY = Nrt_MAY.drop(columns = ['Description'])

    # JUN
    if Jun < Current_Month:
        Nrt_JUN = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/CUSTOMER_NRT_{Jun}.csv', skiprows=24)
        Nrt_JUN = Nrt_JUN.iloc[:-2, :]
        Nrt_JUN = Nrt_JUN.drop(columns=[r'% of Total'])
        Nrt_JUN = Nrt_JUN.rename(columns = {'Total': 'JUN'})
        Nrt_JUN = Nrt_JUN.drop(columns = ['Description'])

    # JUL
    if Jul < Current_Month:
        Nrt_JUL = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/CUSTOMER_NRT_{Jul}.csv', skiprows=24)
        Nrt_JUL = Nrt_JUL.iloc[:-2, :]
        Nrt_JUL = Nrt_JUL.drop(columns=[r'% of Total'])
        Nrt_JUL = Nrt_JUL.rename(columns = {'Total': 'JUL'})
        Nrt_JUL = Nrt_JUL.drop(columns = ['Description'])

    # AUG
    if Aug < Current_Month:
        Nrt_AUG = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/CUSTOMER_NRT_{Aug}.csv', skiprows=24)
        Nrt_AUG = Nrt_AUG.iloc[:-2, :]
        Nrt_AUG = Nrt_AUG.drop(columns=[r'% of Total'])
        Nrt_AUG = Nrt_AUG.rename(columns = {'Total': 'AUG'})
        Nrt_AUG = Nrt_AUG.drop(columns = ['Description'])

    # SEPT
    if Sept < Current_Month:
        Nrt_SEPT = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/CUSTOMER_NRT_{Sept}.csv', skiprows=24)
        Nrt_SEPT = Nrt_SEPT.iloc[:-2, :]
        Nrt_SEPT = Nrt_SEPT.drop(columns=[r'% of Total'])
        Nrt_SEPT = Nrt_SEPT.rename(columns = {'Total': 'SEPT'})
        Nrt_SEPT = Nrt_SEPT.drop(columns = ['Description'])

    # OCT
    if Oct < Current_Month:
        Nrt_OCT = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/CUSTOMER_NRT_{Oct}.csv', skiprows=24)
        Nrt_OCT = Nrt_OCT.iloc[:-2, :]
        Nrt_OCT = Nrt_OCT.drop(columns=[r'% of Total'])
        Nrt_OCT = Nrt_OCT.rename(columns = {'Total': 'OCT'})
        Nrt_OCT = Nrt_OCT.drop(columns = ['Description'])

    # NOV
    if Nov < Current_Month:
        Nrt_NOV = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/CUSTOMER_NRT_{Nov}.csv', skiprows=24)
        Nrt_NOV = Nrt_NOV.iloc[:-2, :]
        Nrt_NOV = Nrt_NOV.drop(columns=[r'% of Total'])
        Nrt_NOV = Nrt_NOV.rename(columns = {'Total': 'NOV'})
        Nrt_NOV = Nrt_NOV.drop(columns = ['Description'])

    # DEC
    if Dec < Current_Month:
        Nrt_DEC = pd.read_csv(f'NRT_CSVs_BFI_SCRAPE/CUSTOMER_NRT_{Dec}.csv', skiprows=24)
        Nrt_DEC = Nrt_DEC.iloc[:-2, :]
        Nrt_DEC = Nrt_DEC.drop(columns=[r'% of Total'])
        Nrt_DEC = Nrt_DEC.rename(columns = {'Total': 'DEC'})
        Nrt_DEC = Nrt_DEC.drop(columns = ['Description'])


    # Merge each month if it exists:
    if Current_Month > Jan:
        Customer_Nrt_Merged = Nrt_JAN
    if Current_Month > Feb:
        Customer_Nrt_Merged = pd.merge(Customer_Nrt_Merged, Nrt_FEB, how='outer', on="NRT Code") # Merged JAN - FEB
    if Current_Month > Mar:
        Customer_Nrt_Merged = pd.merge(Customer_Nrt_Merged, Nrt_MAR, how='outer', on="NRT Code") # Merged JAN - MAR
    if Current_Month > Apr:
        Customer_Nrt_Merged = pd.merge(Customer_Nrt_Merged, Nrt_APR, how='outer', on="NRT Code") # Merged JAN - APR
    if Current_Month > May:
        Customer_Nrt_Merged = pd.merge(Customer_Nrt_Merged, Nrt_MAY, how='outer', on="NRT Code") # Merged JAN - MAY
    if Current_Month > Jun:
        Customer_Nrt_Merged = pd.merge(Customer_Nrt_Merged, Nrt_JUN, how='outer', on="NRT Code") # Merged JAN - JUN
    if Current_Month > Jul:
        Customer_Nrt_Merged = pd.merge(Customer_Nrt_Merged, Nrt_JUL, how='outer', on="NRT Code") # Merged JAN - JUL
    if Current_Month > Aug:
        Customer_Nrt_Merged = pd.merge(Customer_Nrt_Merged, Nrt_AUG, how='outer', on="NRT Code") # Merged JAN - AUG
    if Current_Month > Sept:
        Customer_Nrt_Merged = pd.merge(Customer_Nrt_Merged, Nrt_SEPT, how='outer', on="NRT Code") # Merged JAN - SEPT
    if Current_Month > Oct:
        Customer_Nrt_Merged = pd.merge(Customer_Nrt_Merged, Nrt_OCT, how='outer', on="NRT Code") # Merged JAN - OCT
    if Current_Month > Nov:
        Customer_Nrt_Merged = pd.merge(Customer_Nrt_Merged, Nrt_NOV, how='outer', on="NRT Code") # Merged JAN - NOV
    if Current_Month > Dec:
        Customer_Nrt_Merged = pd.merge(Customer_Nrt_Merged, Nrt_DEC, how='outer', on="NRT Code") # Merged JAN - DEC

    Customer_Nrt_Merged = Customer_Nrt_Merged.fillna(0)



    # Create the YTD COlUMN by adding all the months (if they have) completed:
    if Current_Month > Dec:
        Customer_Nrt_Merged[f'YTD'] = Customer_Nrt_Merged['JAN'] + \
                            Customer_Nrt_Merged['FEB'] + \
                            Customer_Nrt_Merged['MAR'] + \
                            Customer_Nrt_Merged['APR'] + \
                            Customer_Nrt_Merged['MAY'] + \
                            Customer_Nrt_Merged['JUN'] + \
                            Customer_Nrt_Merged['JUL'] + \
                            Customer_Nrt_Merged['AUG'] + \
                            Customer_Nrt_Merged['SEPT'] + \
                            Customer_Nrt_Merged['OCT'] + \
                            Customer_Nrt_Merged['NOV'] + \
                            Customer_Nrt_Merged['DEC']
    elif Current_Month > Nov:
        Customer_Nrt_Merged['YTD'] = Customer_Nrt_Merged['JAN'] + \
                            Customer_Nrt_Merged['FEB'] + \
                            Customer_Nrt_Merged['MAR'] + \
                            Customer_Nrt_Merged['APR'] + \
                            Customer_Nrt_Merged['MAY'] + \
                            Customer_Nrt_Merged['JUN'] + \
                            Customer_Nrt_Merged['JUL'] + \
                            Customer_Nrt_Merged['AUG'] + \
                            Customer_Nrt_Merged['SEPT'] + \
                            Customer_Nrt_Merged['OCT'] + \
                            Customer_Nrt_Merged['NOV']
    elif Current_Month > Oct:
        Customer_Nrt_Merged['YTD'] = Customer_Nrt_Merged['JAN'] + \
                            Customer_Nrt_Merged['FEB'] + \
                            Customer_Nrt_Merged['MAR'] + \
                            Customer_Nrt_Merged['APR'] + \
                            Customer_Nrt_Merged['MAY'] + \
                            Customer_Nrt_Merged['JUN'] + \
                            Customer_Nrt_Merged['JUL'] + \
                            Customer_Nrt_Merged['AUG'] + \
                            Customer_Nrt_Merged['SEPT'] + \
                            Customer_Nrt_Merged['OCT']
    elif Current_Month > Sept:
        Customer_Nrt_Merged['YTD'] = Customer_Nrt_Merged['JAN'] + \
                            Customer_Nrt_Merged['FEB'] + \
                            Customer_Nrt_Merged['MAR'] + \
                            Customer_Nrt_Merged['APR'] + \
                            Customer_Nrt_Merged['MAY'] + \
                            Customer_Nrt_Merged['JUN'] + \
                            Customer_Nrt_Merged['JUL'] + \
                            Customer_Nrt_Merged['AUG'] + \
                            Customer_Nrt_Merged['SEPT']
    elif Current_Month > Aug:
        Customer_Nrt_Merged['YTD'] = Customer_Nrt_Merged['JAN'] + \
                            Customer_Nrt_Merged['FEB'] + \
                            Customer_Nrt_Merged['MAR'] + \
                            Customer_Nrt_Merged['APR'] + \
                            Customer_Nrt_Merged['MAY'] + \
                            Customer_Nrt_Merged['JUN'] + \
                            Customer_Nrt_Merged['JUL'] + \
                            Customer_Nrt_Merged['AUG']
    elif Current_Month > Jul:
        Customer_Nrt_Merged['YTD'] = Customer_Nrt_Merged['JAN'] + \
                            Customer_Nrt_Merged['FEB'] + \
                            Customer_Nrt_Merged['MAR'] + \
                            Customer_Nrt_Merged['APR'] + \
                            Customer_Nrt_Merged['MAY'] + \
                            Customer_Nrt_Merged['JUN'] + \
                            Customer_Nrt_Merged['JUL']
    elif Current_Month > Jun:
        Customer_Nrt_Merged['YTD'] = Customer_Nrt_Merged['JAN'] + \
                            Customer_Nrt_Merged['FEB'] + \
                            Customer_Nrt_Merged['MAR'] + \
                            Customer_Nrt_Merged['APR'] + \
                            Customer_Nrt_Merged['MAY'] + \
                            Customer_Nrt_Merged['JUN']
    elif Current_Month > May:
        Customer_Nrt_Merged['YTD'] = Customer_Nrt_Merged['JAN'] + \
                            Customer_Nrt_Merged['FEB'] + \
                            Customer_Nrt_Merged['MAR'] + \
                            Customer_Nrt_Merged['APR'] + \
                            Customer_Nrt_Merged['MAY']
    elif Current_Month > Apr:
        # Customer_Nrt_Merged['YTD'] = Customer_Nrt_Merged.sum(numeric_only=True)
        Customer_Nrt_Merged['YTD'] = Customer_Nrt_Merged['JAN'] + \
                            Customer_Nrt_Merged['FEB'] + \
                            Customer_Nrt_Merged['MAR'] + \
                            Customer_Nrt_Merged['APR']
    elif Current_Month > Mar:
        Customer_Nrt_Merged['YTD'] = Customer_Nrt_Merged['JAN'] + \
                            Customer_Nrt_Merged['FEB'] + \
                            Customer_Nrt_Merged['MAR']
    elif Current_Month > Feb:
        Customer_Nrt_Merged['YTD'] = Customer_Nrt_Merged['JAN'] + \
                            Customer_Nrt_Merged['FEB']
    elif Current_Month >= Jan:
        Customer_Nrt_Merged['YTD'] = Customer_Nrt_Merged['JAN']

    # Reorder YTD to be right after description:
    if Current_Month > Dec:
        Customer_Nrt_Merged = Customer_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV', 'DEC']]
    elif Current_Month > Nov:
        Customer_Nrt_Merged = Customer_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV']]
    elif Current_Month > Oct:
        Customer_Nrt_Merged = Customer_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT']]
    elif Current_Month > Sept:
        Customer_Nrt_Merged = Customer_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT']]
    elif Current_Month > Aug:
        Customer_Nrt_Merged = Customer_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG']]
    elif Current_Month > Jul:
        Customer_Nrt_Merged = Customer_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL']]
    elif Current_Month > Jun:
        Customer_Nrt_Merged = Customer_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']]
    elif Current_Month > May:
        Customer_Nrt_Merged = Customer_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY']]
    elif Current_Month > Apr:
        Customer_Nrt_Merged = Customer_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR']]
    elif Current_Month > Mar:
        Customer_Nrt_Merged = Customer_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR']]
    elif Current_Month > Feb:
        Customer_Nrt_Merged = Customer_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN', 'FEB']]
    elif Current_Month >= Jan:
        Customer_Nrt_Merged = Customer_Nrt_Merged[['NRT Code', "Description", 'YTD', 'JAN']]


    # Sort Nrt Codes in order.  Then Rename Missing Descriptions:
    Customer_Nrt_Merged = Customer_Nrt_Merged.sort_values(['NRT Code'])
    Customer_Nrt_Merged = Customer_Nrt_Merged.reset_index().drop(columns=['index'])



    # Here we will create an expansive list of all possible NRT Codes along with their corresponding descriptions as shown in Basys:
    list_of_all_possible_nrt_codes_and_descriptions  = [
                                        ['2001', 'Crown Cut(s)/Puncture(s)'],
                                        ['2002', 'Sidewall Cut(s)/Snag(s)'],
                                        ['2003', 'Combination Of Crown Cut(s)/Puncture(s)'],
                                        ['2004', 'Run Under-Inflated'],
                                        ['2005', 'Run To Destruction'],
                                        ['2006', 'Impact Split(s)'],
                                        ['2007', 'Excessive Sidewall Abrasion'],
                                        ['2008', 'Brake Skid(s)'],
                                        ['2009', 'Pinch Shock'],
                                        ['2010', 'Rock Drilling'],
                                        ['2011', 'Bead Distortion'],
                                        ['2012', 'Overloaded/Over-Inflated'],
                                        ['2013', 'Shock/Curbing Induced Belt Sep'],
                                        ['2014', 'Shock/Curbing Induced Bead Sep'],
                                        ['2015', 'Scrub Induced Tread Edge Loose'],
                                        ['2016', 'Scrub Induced Belt Separation'],
                                        ['2017', 'Contact With Vehicle Component'],
                                        ['2018', 'Tire/Retread Design Incorrect'],
                                        ['2019', 'Petrochemical Exposure'],
                                        ['2020', 'Damage From Tire Spinning'],
                                        ['2021', 'Chain Or Mechanical Drive Damage'],
                                        ['2022', 'Drive Or Brake Torque'],
                                        ['2023', 'Bead Damage From Brake Heat'],
                                        ['2024', 'Lightning/Electrical Discharge'],
                                        ['2025', 'Vandalism'],
                                        ['2026', 'Damage From Low Ambient Temper'],
                                        ['2101', 'Bead Damage - Mount/Dismount R'],
                                        ['2102', 'Worn Too Far'],
                                        ['2103', 'Improper Flat Repair - Road Or'],
                                        ['2104', 'Regrooved Too Deeply'],
                                        ['2105', 'Bead Seating/Rim Or Wheel Impr'],
                                        ['2106', 'Bead Damage - Wheel Related'],
                                        ['2107', 'Improper Branding'],
                                        ['2108', 'Object Left In Tire When Mount'],
                                        ['2109', 'Forklift Damage'],
                                        ['2201', 'Ozone/Weather Cracking'],
                                        ['2202', 'Belt Separation (Radial Only)'],
                                        ['2203', 'Bead Separation/Splitting'],
                                        ['2204', 'Liner Defect(s)'],
                                        ['2205', 'Shoulder Separation'],
                                        ['2206', 'Sidewall Blisters/Bumps'],
                                        ['2207', 'Diagonal Cracking'],
                                        ['2208', 'Improper Body Ply Spacing (Ste'],
                                        ['2209', 'Flow Cracking'],
                                        ['2301', 'Times Retreaded Exceeds Fleet'],
                                        ['2302', 'Casing Age Exceeds Fleet Specs'],
                                        ['2303', 'Repairs Exceed Fleet Specs'],
                                        ['2304', 'Casing Grade Below Fleet Specs'],
                                        ['2305', 'Tire Size/Type Not Retreaded'],
                                        ['2306', 'Tire Brand Outside Fleet Specs'],
                                        ['2307', 'Tire Does Not Meet Steer Specs'],
                                        ['2308', 'Tread Design Outside Fleet Spe'],
                                        ['2309', 'Tire Size Outside Fleet Specs'],
                                        ['2401', 'Repair Failure - Bandag Sectio'],
                                        ['2402', 'Repair Failure - Bandag Nailho'],
                                        ['2403', 'Repair Failure - Non Bandag'],
                                        ['2404', 'Repair Failure - Bandag Spot R'],
                                        ['2405', 'Repair Failure - Bandag Bead R'],
                                        ['2406', 'Injuries Too Large'],
                                        ['2407', 'Injuries Too Close'],
                                        ['2408', 'Too Many Repairs'],
                                        ['2501', 'Missed Puncture'],
                                        ['2502', 'Improper Cure'],
                                        ['2503', 'Missed Separation During Retread'],
                                        ['2504', 'Tread Splice Failure'],
                                        ['2505', 'Improperly Texturized Area(s)'],
                                        ['2506', 'Cushion Starvation/Missing'],
                                        ['2507', 'Plies Damaged During Buffing'],
                                        ['2508', 'Improper Rasp Buff Texture'],
                                        ['2509', 'Contamination'],
                                        ['2510', 'Tread Size Mismatch With Casing'],
                                        ['2511', 'Improper Trimmed Width/Imprope'],
                                        ['2512', 'Tread Design Mismatch With Casing'],
                                        ['2513', 'Lateral/Radial Runout'],
                                        ['2514', 'Excessive Splice Overlap'],
                                        ['2601', 'No Significant Problem'],
                                        ['2602', 'Defective Retread Material'],
                                        ['2603', 'Improper Aftermarket Siping'],
                                        ['2604', 'Overall Tire Condition Not Acc'],
                                        ['3202', 'Tub Leak']
                                        ]

    # In some cases, where a particular NRT Code had all zeros, the "Description" column value will have a '0'
    # Therefore, we will manually ensure each NRT Code has the correct description
    # by replacing whatever was there prior with the correct Description for that NRT Code. 
    # Since this code will be created for flexibility of viewing any nrt code, we'll include all of them.  We'll filter which codes we want in a list of codes later.

    for each_code, each_description in list_of_all_possible_nrt_codes_and_descriptions:
        Customer_Nrt_Merged.loc[Customer_Nrt_Merged['NRT Code'] == each_code, 'Description'] = each_description

    for each_code, each_description in list_of_all_possible_nrt_codes_and_descriptions:
        if each_code not in Customer_Nrt_Merged.values:
            line = pd.DataFrame({"NRT Code": each_code, "Description": each_description}, index=[1.1])
            Customer_Nrt_Merged = Customer_Nrt_Merged.append(line, ignore_index=False)
            Customer_Nrt_Merged = Customer_Nrt_Merged.sort_index().reset_index(drop=True)

    
    Customer_Nrt_Merged = Customer_Nrt_Merged.fillna(0)

    # -------------------------------------------------
    # AT THIS POINT, we have an nrt df with ALL codes and descriptions!  (or 2 if we're on the current year since we're looping)
    # We can narrow down the list downstream in our code.  For now, we can group all 230(digit) based codes together and just call it 2300.  
    # We can later change this to 230* for clarity in the table but not yet as we need it to remain a number (even though it's a string)
    
    # ALSO we will group 2406 and 2407 together as Injuries too Close/Large (respectively).  We will (at least temporarily) call this one 2400.
    # ---------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # Group all codes that start with 23** to 2300 and change description to "Outside of Fleet Specifications" AND
    # Group all codes that start with 2406 or 2407 and change description to "Injuries too Close/Large"

    List_of_OUTSIDE_FLEET_SPECS_codes = ['2300', '2301', '2302', '2303', '2304', '2305', '2306', '2307', '2308', '2309']
    List_of_INJURIES_TOO_CLOSE_OR_LARGE_codes = ['2406', '2407']

    # Create two TEMPORARY dfs, one for 2300 based codes and the other for 2400 based codes:
    Outside_fleet_specs_df = Customer_Nrt_Merged.loc[Customer_Nrt_Merged['NRT Code'].isin(List_of_OUTSIDE_FLEET_SPECS_codes)]
    Injuries_too_close_or_large_df = Customer_Nrt_Merged.loc[Customer_Nrt_Merged['NRT Code'].isin(List_of_INJURIES_TOO_CLOSE_OR_LARGE_codes)]

    # Drop all ungrouped 2300 and 2400 based codes from main Nrt df:
    Customer_Nrt_Merged = Customer_Nrt_Merged.loc[~Customer_Nrt_Merged["NRT Code"].isin(List_of_OUTSIDE_FLEET_SPECS_codes)]
    Customer_Nrt_Merged = Customer_Nrt_Merged.loc[~Customer_Nrt_Merged["NRT Code"].isin(List_of_INJURIES_TOO_CLOSE_OR_LARGE_codes)]

    # Set ALL values for the first TEMPORARY df to 2300 and all values for the second TEMPORARY df to 2400:
    Outside_fleet_specs_df["NRT Code"] = '2300'
    Injuries_too_close_or_large_df["NRT Code"] = '2406'

    # Group all the NRT Codes that are 2300 into ONE (Summed) row... each column will be summed (YTD, JAN, etc):
    Outside_fleet_specs_df = Outside_fleet_specs_df.groupby(Outside_fleet_specs_df["NRT Code"]).sum()
    Outside_fleet_specs_df = Outside_fleet_specs_df.reset_index()
    Outside_fleet_specs_df['Description'] = "Outside of Fleet Specifications"

    # Group all the NRT Codes that are 2400 into ONE (Summed) row... each column will be summed (YTD, Jan, etc):
    Injuries_too_close_or_large_df = Injuries_too_close_or_large_df.groupby(Injuries_too_close_or_large_df["NRT Code"]).sum()
    Injuries_too_close_or_large_df = Injuries_too_close_or_large_df.reset_index()
    Injuries_too_close_or_large_df['Description'] = "Injuries too Close/Large"

    #---------------------------- REORDER the columns depending on whether the month exists yet:
    if Current_Month > Dec:
        Outside_fleet_specs_df = Outside_fleet_specs_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV', 'DEC']]
        Injuries_too_close_or_large_df = Injuries_too_close_or_large_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV', 'DEC']]

    elif Current_Month > Nov:
        Outside_fleet_specs_df = Outside_fleet_specs_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV']]
        Injuries_too_close_or_large_df = Injuries_too_close_or_large_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT', 'NOV']]

    elif Current_Month > Oct:
        Outside_fleet_specs_df = Outside_fleet_specs_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT']]
        Injuries_too_close_or_large_df = Injuries_too_close_or_large_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT', 'OCT']]

    elif Current_Month > Sept:
        Outside_fleet_specs_df = Outside_fleet_specs_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT']]
        Injuries_too_close_or_large_df = Injuries_too_close_or_large_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEPT']]

    elif Current_Month > Aug:
        Outside_fleet_specs_df = Outside_fleet_specs_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG']]
        Injuries_too_close_or_large_df = Injuries_too_close_or_large_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG']]

    elif Current_Month > Jul:
        Outside_fleet_specs_df = Outside_fleet_specs_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL']]
        Injuries_too_close_or_large_df = Injuries_too_close_or_large_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL']]

    elif Current_Month > Jun:
        Outside_fleet_specs_df = Outside_fleet_specs_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']]
        Injuries_too_close_or_large_df = Injuries_too_close_or_large_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']]

    elif Current_Month > May:
        Outside_fleet_specs_df = Outside_fleet_specs_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY']]
        Injuries_too_close_or_large_df = Injuries_too_close_or_large_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR', 'MAY']]

    elif Current_Month > Apr:
        Outside_fleet_specs_df = Outside_fleet_specs_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR']]
        Injuries_too_close_or_large_df = Injuries_too_close_or_large_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR', 'APR']]

    elif Current_Month > Mar:
        Outside_fleet_specs_df = Outside_fleet_specs_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR']]
        Injuries_too_close_or_large_df = Injuries_too_close_or_large_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB', 'MAR']]

    elif Current_Month > Feb:
        Outside_fleet_specs_df = Outside_fleet_specs_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB']]
        Injuries_too_close_or_large_df = Injuries_too_close_or_large_df[['NRT Code', "Description", 'YTD', 'JAN', 'FEB']]

    elif Current_Month > Jan:
        Outside_fleet_specs_df = Outside_fleet_specs_df[['NRT Code', "Description", 'YTD', 'JAN']]
        Injuries_too_close_or_large_df = Injuries_too_close_or_large_df[['NRT Code', "Description", 'YTD', 'JAN']]

    # NOW, Append the "2300" and "2406" (which is actually 2406 or 2407) one-row dataframes to the main NRT df:
    Customer_Nrt_Merged = Customer_Nrt_Merged.append(Outside_fleet_specs_df)
    Customer_Nrt_Merged = Customer_Nrt_Merged.reset_index().drop(columns=['index'])
 
    Customer_Nrt_Merged = Customer_Nrt_Merged.append(Injuries_too_close_or_large_df)
    Customer_Nrt_Merged = Customer_Nrt_Merged.reset_index().drop(columns=['index'])

    #Convert YTD and Each Month from object to int:
    Customer_Nrt_Merged = Customer_Nrt_Merged.fillna(0)
    Customer_Nrt_Merged.iloc[:, 2:] = Customer_Nrt_Merged.iloc[:, 2:].astype(int)
    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # NOW... when dealing with all these codes, we have 2 options for choosing which codes to narrow down to for our table/graph.
        # 1. Dynamically sort by the top causes and keep only, say, the top 10 codes/descriptions etc
        # 2. Filter to only include a manual list of causes.

    # For our purposes we will do option 1 for the current year (if we're on current year in the loop).
    # Then, as we loop through last year, we'll manually force last year's df to conform to those codes.

    # Again, this will help us ensure we merge the two years based on the top causes of the current year.


    # =====================================================================================================================================================================================
    # ==================== Current Year ===========================================
    # FOR THE CURRENT YEAR, we will keep only the top causes dynamically (rather than through a manual list of codes):
    if year == Current_Year:
        # To summarize where we are at this point, we have a df that has EVERY Code in it.
        # To get the top causes, let's sort them by YTD first.
        Customer_Nrt_Merged = Customer_Nrt_Merged.sort_values('YTD', ascending=False).reset_index(drop=True)
        # Now create a df which ONLY contains the desired number of top causes:
        Current_Year_Customer_Nrt_Codes_df = Customer_Nrt_Merged.iloc[0:11, :]

        # # -------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # # NOW we need to create a new row with a Description called  "OTHER" which will be a sum of ALL the other nrt codes not in the rows from our most recent code line above
        # We'll begin by just filtering the original Customer_Nrt_Merged df again, but this time for anything ABOVE the desired number
        Nrt_df_NOT_in_Customer_CODES = Customer_Nrt_Merged.iloc[11:, :]

        # Make all "NRT Code" values in this new df for OTHER NRT Codes to ALL be OTHER:
        Nrt_df_NOT_in_Customer_CODES["NRT Code"] = 'OTHER'

        # Group by ALL of the OTHER codes to sum into ONE ROW:
        Nrt_df_NOT_in_Customer_CODES = Nrt_df_NOT_in_Customer_CODES.groupby(Nrt_df_NOT_in_Customer_CODES['NRT Code']).sum()
        Nrt_df_NOT_in_Customer_CODES = Nrt_df_NOT_in_Customer_CODES.reset_index()
        Nrt_df_NOT_in_Customer_CODES['Description'] = 'OTHER'

        # NOW, Append the "OTHER" df (the one-row df we just created) to the main NRT df:
        Current_Year_Customer_Nrt_Codes_df = Current_Year_Customer_Nrt_Codes_df.append(Nrt_df_NOT_in_Customer_CODES)
        Current_Year_Customer_Nrt_Codes_df = Current_Year_Customer_Nrt_Codes_df.reset_index().drop(columns=['index'])
    
                        # ------------------------------------------------- Now this will set it up for HTML aesthetically---------------------------------------
        # SUM COLUMNS FOR NUMERIC ONLY!
        Current_Year_Customer_Nrt_Codes_df.loc['this_value_will_be_in_the_index_and_thus_removed_in_the_next_step'] = Current_Year_Customer_Nrt_Codes_df.sum(numeric_only=True)

        # Reset the index and then drop the extra index column to make it clean:
        Current_Year_Customer_Nrt_Codes_df = Current_Year_Customer_Nrt_Codes_df.reset_index().drop(columns=['index'])

        # Replace NaNs with empty strings:
        Current_Year_Customer_Nrt_Codes_df = Current_Year_Customer_Nrt_Codes_df.fillna('')

        # Replace the blank Description with the word "TOTAL", then replace OTHER in row 10 with blank and "OTHER NRT CODES"
        Current_Year_Customer_Nrt_Codes_df['Description'] = Current_Year_Customer_Nrt_Codes_df["Description"].replace('',"TOTAL")
        Current_Year_Customer_Nrt_Codes_df['NRT Code'] = Current_Year_Customer_Nrt_Codes_df["NRT Code"].replace('OTHER',"")
        Current_Year_Customer_Nrt_Codes_df['Description'] = Current_Year_Customer_Nrt_Codes_df["Description"].replace('OTHER',"OTHER NRT REASONS")

        # Ensure the YTD TOTAL at the very bottom of the df is a sum of each of the months total:
        Current_Year_Customer_Nrt_Codes_df.iloc[-1,2] = Current_Year_Customer_Nrt_Codes_df.iloc[-1, 3:].sum()
            # Convert all YTD and months columns to Integers:
        for column in Current_Year_Customer_Nrt_Codes_df.columns:
            if column != 'Description' and column != 'NRT Code':
                Current_Year_Customer_Nrt_Codes_df[f'{column}'] = Current_Year_Customer_Nrt_Codes_df[f'{column}'].astype(int)

        # Add a dash instead of blank for the Nrt code of "Other".  We'll remove this later.
        # This is just so we can have every value under the NRT code column be unique.
        # This way we'll be able to sort last year's df on this year's df later.
        Current_Year_Customer_Nrt_Codes_df.iloc[11, 0] = '-'

        # Save the NRT table of the current year.
        Current_Year_Customer_Nrt_Codes_df.to_csv(f'./CSVs/Customer_NRT_TABLE_{year}.csv', index=False)

    # ======================================================================================================================================
    # ==================== Last Year ===========================================
    # At this point, we need to get the list of NRT codes that were the top causes THIS YEAR, so we can manually use that list of codes to filter last year's df:
        Customer_Codes_List = Current_Year_Customer_Nrt_Codes_df.iloc[0:-2, 0].to_list()

    elif year == Last_Year:
        # ---------------------------------------------------------------------------------------------------------------
        # Assign to variable a filtered merged df that only includes Codes we want according to Customer:
        Last_Year_Customer_Nrt_Codes_df = Customer_Nrt_Merged[Customer_Nrt_Merged['NRT Code'].isin(Customer_Codes_List)]
        Last_Year_Customer_Nrt_Codes_df = Last_Year_Customer_Nrt_Codes_df.reset_index(drop=True)

        # -------------------------------------------------------------------------------------------------------------------------------------------------------------------
        # NOW we need to create a new row with a Description called  "OTHER" which will be a total of all other codes that were not part of the list above.

        # Assign a TEMPORARY df of all OTHER codes NOT in relevant list:
        Nrt_df_NOT_in_Customer_CODES = Customer_Nrt_Merged[~Customer_Nrt_Merged['NRT Code'].isin(Customer_Codes_List)]

        # Make all "NRT Code" values in this new df for OTHER NRT Codes to ALL be OTHER:
        Nrt_df_NOT_in_Customer_CODES["NRT Code"] = 'OTHER'

        # Group by ALL of the OTHER codes to sum into ONE ROW:
        Nrt_df_NOT_in_Customer_CODES = Nrt_df_NOT_in_Customer_CODES.groupby(Nrt_df_NOT_in_Customer_CODES['NRT Code']).sum()
        Nrt_df_NOT_in_Customer_CODES = Nrt_df_NOT_in_Customer_CODES.reset_index()
        Nrt_df_NOT_in_Customer_CODES['Description'] = 'OTHER'

        # NOW, Append the "OTHER" df (the one-row df we just created) to the main NRT df:
        Last_Year_Customer_Nrt_Codes_df = Last_Year_Customer_Nrt_Codes_df.append(Nrt_df_NOT_in_Customer_CODES)
        Last_Year_Customer_Nrt_Codes_df = Last_Year_Customer_Nrt_Codes_df.reset_index().drop(columns=['index'])
    
                        # ------------------------------------------------- Now this will set it up for HTML aesthetically---------------------------------------
        # SUM COLUMNS FOR NUMERIC ONLY!
        Last_Year_Customer_Nrt_Codes_df.loc['this_value_will_be_in_the_index_and_thus_removed_in_the_next_step'] = Last_Year_Customer_Nrt_Codes_df.sum(numeric_only=True)

        # Reset the index and then drop the extra index column to make it clean:
        Last_Year_Customer_Nrt_Codes_df = Last_Year_Customer_Nrt_Codes_df.reset_index().drop(columns=['index'])

        # Replace NaNs with empty strings:
        Last_Year_Customer_Nrt_Codes_df = Last_Year_Customer_Nrt_Codes_df.fillna('')
        Last_Year_Customer_Nrt_Codes_df

        # Replace the blank Description with the word "TOTAL", then replace OTHER in row 10 with blank and "OTHER NRT CODES"
        Last_Year_Customer_Nrt_Codes_df['Description'] = Last_Year_Customer_Nrt_Codes_df["Description"].replace('',"TOTAL")
        Last_Year_Customer_Nrt_Codes_df['NRT Code'] = Last_Year_Customer_Nrt_Codes_df["NRT Code"].replace('OTHER',"")
        Last_Year_Customer_Nrt_Codes_df['Description'] = Last_Year_Customer_Nrt_Codes_df["Description"].replace('OTHER',"OTHER NRT REASONS")

        # Ensure the YTD TOTAL at the very bottom of the df is a sum of each of the months total:
        Last_Year_Customer_Nrt_Codes_df.iloc[-1,2] = Last_Year_Customer_Nrt_Codes_df.iloc[-1, 3:].sum()
            # Convert all YTD and months columns to Integers:
        for column in Last_Year_Customer_Nrt_Codes_df.columns:
            if column != 'Description' and column != 'NRT Code':
                Last_Year_Customer_Nrt_Codes_df[f'{column}'] = Last_Year_Customer_Nrt_Codes_df[f'{column}'].astype(int)

        # Again...
        # Add a dash instead of blank for the Nrt code of "Other".  We'll remove this later.
        # This is just so we can have every value under the NRT code column be unique.
        # This way we'll be able to sort last year's df on this year's df later.
        Last_Year_Customer_Nrt_Codes_df.iloc[11, 0] = '-'


        # Save the NRT table for last year.
        Last_Year_Customer_Nrt_Codes_df.to_csv(f'./CSVs/Customer_NRT_TABLE_{year}.csv', index=False)


# Note... Since we looped through LAST YEAR last, we'll reset the "current month" variables for later uses:
Jan = Current_Year * 100 + 1
Feb = Current_Year * 100 + 2
Mar = Current_Year * 100 + 3
Apr = Current_Year * 100 + 4
May = Current_Year * 100 + 5
Jun = Current_Year * 100 + 6
Jul = Current_Year * 100 + 7
Aug = Current_Year * 100 + 8
Sept = Current_Year * 100 + 9
Oct = Current_Year * 100 + 10
Nov = Current_Year * 100 + 11
Dec = Current_Year * 100 + 12


Current_Year_Customer_Nrt_Codes_df

,NRT Code,Description,YTD,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEPT,OCT,NOV,DEC
0,2300,Outside of Fleet Specifications,13,2,1,0,2,1,3,0,0,4,0,0,0
1,2202,Belt Separation (Radial Only),9,1,1,0,0,1,0,1,2,2,1,0,0
2,2001,Crown Cut(s)/Puncture(s),8,0,0,0,1,1,0,1,1,1,2,1,0
3,2205,Shoulder Separation,8,0,1,0,0,0,1,1,2,1,1,0,1
4,2002,Sidewall Cut(s)/Snag(s),5,0,0,0,0,1,0,0,2,1,1,0,0
5,2206,Sidewall Blisters/Bumps,3,0,0,0,0,1,0,2,0,0,0,0,0
6,2507,Plies Damaged During Buffing,2,0,0,0,0,0,1,0,0,0,1,0,0
7,2604,Overall Tire Condition Not Acc,2,0,0,0,0,0,0,0,0,0,2,0,0
8,2102,Worn Too Far,2,0,0,0,0,1,0,1,0,0,0,0,0
9,2007,Excessive Sidewall Abrasion,2,0,0,0,0,0,0,1,0,1,0,0,0


## SORT last year's NRT df based on this year's top NRT codes:

In [71]:
# At this point, the current year's nrt df is orderd by YTD, which is what we want.
# However, last year's df is ordered by nrt code.
# Instead of sorting by ytd for last year (like we did for the current year), we will 
# Sort last year's Nrt df to be same order as current year's df for merging compatibility later.


Last_Year_Customer_Nrt_Codes_df['ordered'] = pd.Categorical(
    Last_Year_Customer_Nrt_Codes_df['NRT Code'], 
    categories=list(Current_Year_Customer_Nrt_Codes_df['NRT Code']), 
    ordered=True)

Last_Year_Customer_Nrt_Codes_df = Last_Year_Customer_Nrt_Codes_df.sort_values('ordered')
Last_Year_Customer_Nrt_Codes_df = Last_Year_Customer_Nrt_Codes_df.drop(columns = ['ordered'])

Last_Year_Customer_Nrt_Codes_df

,NRT Code,Description,YTD,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEPT,OCT,NOV,DEC
10,2300,Outside of Fleet Specifications,4,0,0,2,1,1,0,0,0,0,0,0,0
8,2202,Belt Separation (Radial Only),8,1,3,0,2,1,0,0,0,0,0,0,1
0,2001,Crown Cut(s)/Puncture(s),2,0,0,0,0,0,0,0,0,0,2,0,0
9,2205,Shoulder Separation,2,0,0,0,0,1,0,0,0,0,1,0,0
5,2002,Sidewall Cut(s)/Snag(s),0,0,0,0,0,0,0,0,0,0,0,0,0
3,2206,Sidewall Blisters/Bumps,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2507,Plies Damaged During Buffing,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2604,Overall Tire Condition Not Acc,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2102,Worn Too Far,1,0,0,0,0,0,0,0,0,0,0,0,1
4,2007,Excessive Sidewall Abrasion,0,0,0,0,0,0,0,0,0,0,0,0,0


In [72]:
## Now that we have both years nrt dfs sorted by the current year's nrt codes (based on current year's ytd), 
## We can get rid of that '-' next to OTHER NRT REASONS on both dfs.
Last_Year_Customer_Nrt_Codes_df['NRT Code'] = Last_Year_Customer_Nrt_Codes_df['NRT Code'].replace("-", "")
Current_Year_Customer_Nrt_Codes_df['NRT Code'] = Current_Year_Customer_Nrt_Codes_df["NRT Code"].replace("-", "")

Last_Year_Customer_Nrt_Codes_df = Last_Year_Customer_Nrt_Codes_df.reset_index(drop=True)
Last_Year_Customer_Nrt_Codes_df

,NRT Code,Description,YTD,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEPT,OCT,NOV,DEC
0,2300,Outside of Fleet Specifications,4,0,0,2,1,1,0,0,0,0,0,0,0
1,2202,Belt Separation (Radial Only),8,1,3,0,2,1,0,0,0,0,0,0,1
2,2001,Crown Cut(s)/Puncture(s),2,0,0,0,0,0,0,0,0,0,2,0,0
3,2205,Shoulder Separation,2,0,0,0,0,1,0,0,0,0,1,0,0
4,2002,Sidewall Cut(s)/Snag(s),0,0,0,0,0,0,0,0,0,0,0,0,0
5,2206,Sidewall Blisters/Bumps,0,0,0,0,0,0,0,0,0,0,0,0,0
6,2507,Plies Damaged During Buffing,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2604,Overall Tire Condition Not Acc,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2102,Worn Too Far,1,0,0,0,0,0,0,0,0,0,0,0,1
9,2007,Excessive Sidewall Abrasion,0,0,0,0,0,0,0,0,0,0,0,0,0


## Now, back to the current year, we'll create dashes for months without data so it looks nice in on website/pdf table

In [73]:
# Now, ensure each numerical column in the NRT TABLE has commas as separators:
# Customer_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML = Current_Year_Customer_Nrt_Codes_df.copy()
Customer_Nrt_Codes_df_with_DASHES = Current_Year_Customer_Nrt_Codes_df.copy()

# columns_in_df = Customer_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML.iloc[:,2:].columns
# for columns in columns_in_df:
#     Customer_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML[columns] = Customer_Nrt_Codes_df_with_COMMAS_and_DASHES_for_HTML.apply(lambda x: "{:,}".format(x[columns]), axis=1)

# Finally, we'll add dashes for months that don't exist for the html table:

if Current_Month < Jan:
    Customer_Nrt_Codes_df_with_DASHES['JAN'] = '-'
if Current_Month <= Feb:
    Customer_Nrt_Codes_df_with_DASHES['FEB'] = '-'
if Current_Month <= Mar:
    Customer_Nrt_Codes_df_with_DASHES['MAR'] = '-'
if Current_Month <= Apr:
    Customer_Nrt_Codes_df_with_DASHES['APR'] = '-'
if Current_Month <= May:
    Customer_Nrt_Codes_df_with_DASHES['MAY'] = '-'
if Current_Month <= Jun:
    Customer_Nrt_Codes_df_with_DASHES['JUN'] = '-'
if Current_Month <= Jul:
    Customer_Nrt_Codes_df_with_DASHES['JUL'] = '-'
if Current_Month <= Aug:
    Customer_Nrt_Codes_df_with_DASHES['AUG'] = '-'
if Current_Month <= Sept:
    Customer_Nrt_Codes_df_with_DASHES['SEPT'] = '-'
if Current_Month <= Oct:
    Customer_Nrt_Codes_df_with_DASHES['OCT'] = '-'
if Current_Month <= Nov:
    Customer_Nrt_Codes_df_with_DASHES['NOV'] = '-'
if Current_Month <= Dec:
    Customer_Nrt_Codes_df_with_DASHES['DEC'] = '-'


# Check the df that only has dashes to be used for graphing (we'll also use the dashes to our advantage to remove incompleted month columns for prepping a table for graphing):
Customer_Nrt_Codes_df_with_DASHES

,NRT Code,Description,YTD,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEPT,OCT,NOV,DEC
0,2300,Outside of Fleet Specifications,13,2,1,0,2,1,3,0,0,4,0,0,0
1,2202,Belt Separation (Radial Only),9,1,1,0,0,1,0,1,2,2,1,0,0
2,2001,Crown Cut(s)/Puncture(s),8,0,0,0,1,1,0,1,1,1,2,1,0
3,2205,Shoulder Separation,8,0,1,0,0,0,1,1,2,1,1,0,1
4,2002,Sidewall Cut(s)/Snag(s),5,0,0,0,0,1,0,0,2,1,1,0,0
5,2206,Sidewall Blisters/Bumps,3,0,0,0,0,1,0,2,0,0,0,0,0
6,2507,Plies Damaged During Buffing,2,0,0,0,0,0,1,0,0,0,1,0,0
7,2604,Overall Tire Condition Not Acc,2,0,0,0,0,0,0,0,0,0,2,0,0
8,2102,Worn Too Far,2,0,0,0,0,1,0,1,0,0,0,0,0
9,2007,Excessive Sidewall Abrasion,2,0,0,0,0,0,0,1,0,1,0,0,0


# Sort the NRT Table for the html

In [74]:
Months_Numerical

[202101,
 202102,
 202103,
 202104,
 202105,
 202106,
 202107,
 202108,
 202109,
 202110,
 202111,
 202112]

In [75]:
list_of_months_and_columns = [
                            [Jan, f'YTD'], [Feb, f'JAN'], [Mar, f'FEB'], [Apr, f'MAR'], [May, f'APR'], 
                            [Jun, f'MAY'], [Jul, f'JUN'], [Aug, f'JUL'], [Sept, f'AUG'], [Oct, f'SEPT'], 
                            [Nov, f'OCT'], [Dec, f'NOV']
                            ]

for yyyymm, column in list_of_months_and_columns:
    if Current_Month >= yyyymm:
        Customer_Nrt_Codes_df_with_DASHES[column] = Customer_Nrt_Codes_df_with_DASHES[column].astype(int)

# Customer_NRT_df_for_website_or_pdf_table['YTD'] = int(Customer_NRT_df_for_website_or_pdf_table['YTD'])

Customer_Nrt_Codes_df_with_DASHES.dtypes

NRT Code       object
Description    object
YTD             int32
JAN             int32
FEB             int32
MAR             int32
APR             int32
MAY             int32
JUN             int32
JUL             int32
AUG             int32
SEPT            int32
OCT             int32
NOV             int32
DEC             int32
dtype: object

In [76]:
# To ensure we only add commas to the months which have completed, we'll loop through "Months_Numerical" list and only add the commas to those columns:
Customer_NRT_df_for_website_or_pdf_table = Customer_Nrt_Codes_df_with_DASHES.copy()

for column in Customer_NRT_df_for_website_or_pdf_table:
    if type(column) == int:
        Customer_NRT_df_for_website_or_pdf_table[column] = Customer_NRT_df_for_website_or_pdf_table.apply(lambda x: "{:,}".format(x[column]), axis=1)

Customer_NRT_df_for_website_or_pdf_table

,NRT Code,Description,YTD,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEPT,OCT,NOV,DEC
0,2300,Outside of Fleet Specifications,13,2,1,0,2,1,3,0,0,4,0,0,0
1,2202,Belt Separation (Radial Only),9,1,1,0,0,1,0,1,2,2,1,0,0
2,2001,Crown Cut(s)/Puncture(s),8,0,0,0,1,1,0,1,1,1,2,1,0
3,2205,Shoulder Separation,8,0,1,0,0,0,1,1,2,1,1,0,1
4,2002,Sidewall Cut(s)/Snag(s),5,0,0,0,0,1,0,0,2,1,1,0,0
5,2206,Sidewall Blisters/Bumps,3,0,0,0,0,1,0,2,0,0,0,0,0
6,2507,Plies Damaged During Buffing,2,0,0,0,0,0,1,0,0,0,1,0,0
7,2604,Overall Tire Condition Not Acc,2,0,0,0,0,0,0,0,0,0,2,0,0
8,2102,Worn Too Far,2,0,0,0,0,1,0,1,0,0,0,0,0
9,2007,Excessive Sidewall Abrasion,2,0,0,0,0,0,0,1,0,1,0,0,0


## NOW, we have a table prepped and complete for the current year called "Customer_NRT_df_for_website_or_pdf_table".
## We had to make this separately from the tables for graphing because we added thousands commas and dashes for incomplete months.

## Now, to prep tables for the year-year graph, we'll go back to using "Last_Year_Customer_Nrt_Codes_df" and Current_Year_Customer_Nrt_Codes_df

# Create NRT Year to Year line Graph

In [77]:
Last_Year_NRT_df_for_graphing = Last_Year_Customer_Nrt_Codes_df
Current_Year_NRT_df_for_graphing = Customer_Nrt_Codes_df_with_DASHES


Last_Year_NRT_df_for_graphing

,NRT Code,Description,YTD,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEPT,OCT,NOV,DEC
0,2300,Outside of Fleet Specifications,4,0,0,2,1,1,0,0,0,0,0,0,0
1,2202,Belt Separation (Radial Only),8,1,3,0,2,1,0,0,0,0,0,0,1
2,2001,Crown Cut(s)/Puncture(s),2,0,0,0,0,0,0,0,0,0,2,0,0
3,2205,Shoulder Separation,2,0,0,0,0,1,0,0,0,0,1,0,0
4,2002,Sidewall Cut(s)/Snag(s),0,0,0,0,0,0,0,0,0,0,0,0,0
5,2206,Sidewall Blisters/Bumps,0,0,0,0,0,0,0,0,0,0,0,0,0
6,2507,Plies Damaged During Buffing,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2604,Overall Tire Condition Not Acc,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2102,Worn Too Far,1,0,0,0,0,0,0,0,0,0,0,0,1
9,2007,Excessive Sidewall Abrasion,0,0,0,0,0,0,0,0,0,0,0,0,0


In [78]:
Current_Year_NRT_df_for_graphing

,NRT Code,Description,YTD,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEPT,OCT,NOV,DEC
0,2300,Outside of Fleet Specifications,13,2,1,0,2,1,3,0,0,4,0,0,0
1,2202,Belt Separation (Radial Only),9,1,1,0,0,1,0,1,2,2,1,0,0
2,2001,Crown Cut(s)/Puncture(s),8,0,0,0,1,1,0,1,1,1,2,1,0
3,2205,Shoulder Separation,8,0,1,0,0,0,1,1,2,1,1,0,1
4,2002,Sidewall Cut(s)/Snag(s),5,0,0,0,0,1,0,0,2,1,1,0,0
5,2206,Sidewall Blisters/Bumps,3,0,0,0,0,1,0,2,0,0,0,0,0
6,2507,Plies Damaged During Buffing,2,0,0,0,0,0,1,0,0,0,1,0,0
7,2604,Overall Tire Condition Not Acc,2,0,0,0,0,0,0,0,0,0,2,0,0
8,2102,Worn Too Far,2,0,0,0,0,1,0,1,0,0,0,0,0
9,2007,Excessive Sidewall Abrasion,2,0,0,0,0,0,0,1,0,1,0,0,0


In [79]:
# Rename the columns on Each Year's data so it shows not only the month but also the year:
    # Last Year:
Last_Year_NRT_df_for_graphing.columns = ['NRT Code', 'Description', f'YTD {Last_Year}', f'JAN {Last_Year}', f'FEB {Last_Year}', f'MAR {Last_Year}', f'APR {Last_Year}', f'MAY {Last_Year}', f'JUN {Last_Year}', f'JUL {Last_Year}', f'AUG {Last_Year}', f'SEPT {Last_Year}', f'OCT {Last_Year}', f'NOV {Last_Year}', f'DEC {Last_Year}']
    # Current Year:
Current_Year_NRT_df_for_graphing.columns = ['NRT Code', 'Description', f'YTD {Current_Year}', f'JAN {Current_Year}', f'FEB {Current_Year}', f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', f'JUN {Current_Year}', f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', f'OCT {Current_Year}', f'NOV {Current_Year}', f'DEC {Current_Year}']

# For the Current Year, drop columns with dashes:
Current_Year_NRT_df_for_graphing = Current_Year_NRT_df_for_graphing.loc[:, ~(Current_Year_NRT_df_for_graphing.astype(str) == '-').all()]

Current_Year_NRT_df_for_graphing

,NRT Code,Description,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,2300,Outside of Fleet Specifications,13,2,1,0,2,1,3,0,0,4,0,0,0
1,2202,Belt Separation (Radial Only),9,1,1,0,0,1,0,1,2,2,1,0,0
2,2001,Crown Cut(s)/Puncture(s),8,0,0,0,1,1,0,1,1,1,2,1,0
3,2205,Shoulder Separation,8,0,1,0,0,0,1,1,2,1,1,0,1
4,2002,Sidewall Cut(s)/Snag(s),5,0,0,0,0,1,0,0,2,1,1,0,0
5,2206,Sidewall Blisters/Bumps,3,0,0,0,0,1,0,2,0,0,0,0,0
6,2507,Plies Damaged During Buffing,2,0,0,0,0,0,1,0,0,0,1,0,0
7,2604,Overall Tire Condition Not Acc,2,0,0,0,0,0,0,0,0,0,2,0,0
8,2102,Worn Too Far,2,0,0,0,0,1,0,1,0,0,0,0,0
9,2007,Excessive Sidewall Abrasion,2,0,0,0,0,0,0,1,0,1,0,0,0


In [80]:
# To merge, we ONLY want "Category" on ONE df.  We'll drop "NRT Code" and "Description" on ALL dfs that are not the oldest year we're testing.
Current_Year_NRT_df_for_graphing = Current_Year_NRT_df_for_graphing.drop(columns = ['NRT Code', 'Description'])

# Now add a new TEMPORARY column for ALL dfs we're testing which will be ALL unique values but identical for each df, so we can merge "on" that temporary column
Last_Year_NRT_df_for_graphing['Temp Col'] = range(len(Last_Year_NRT_df_for_graphing))
Current_Year_NRT_df_for_graphing['Temp Col'] = range(len(Current_Year_NRT_df_for_graphing))

# Merge each year together into one DF:
Merged_NRT_df_for_graphing = pd.merge(Last_Year_NRT_df_for_graphing, Current_Year_NRT_df_for_graphing, how='outer', on='Temp Col')

# Now delete the "Temp Col" from the merged df (and each year since we can use those later too):
Last_Year_NRT_df_for_graphing = Last_Year_NRT_df_for_graphing.drop(columns = ['Temp Col'])
Current_Year_NRT_df_for_for_graphing = Current_Year_NRT_df_for_graphing.drop(columns = ['Temp Col'])
Merged_NRT_df_for_graphing = Merged_NRT_df_for_graphing.drop(columns = ['Temp Col'])

pd.set_option('display.max_columns', None)
Merged_NRT_df_for_graphing

,NRT Code,Description,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,2300,Outside of Fleet Specifications,4,0,0,2,1,1,0,0,0,0,0,0,0,13,2,1,0,2,1,3,0,0,4,0,0,0
1,2202,Belt Separation (Radial Only),8,1,3,0,2,1,0,0,0,0,0,0,1,9,1,1,0,0,1,0,1,2,2,1,0,0
2,2001,Crown Cut(s)/Puncture(s),2,0,0,0,0,0,0,0,0,0,2,0,0,8,0,0,0,1,1,0,1,1,1,2,1,0
3,2205,Shoulder Separation,2,0,0,0,0,1,0,0,0,0,1,0,0,8,0,1,0,0,0,1,1,2,1,1,0,1
4,2002,Sidewall Cut(s)/Snag(s),0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,1,0,0,2,1,1,0,0
5,2206,Sidewall Blisters/Bumps,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,0,2,0,0,0,0,0
6,2507,Plies Damaged During Buffing,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,1,0,0
7,2604,Overall Tire Condition Not Acc,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,0,0
8,2102,Worn Too Far,1,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,1,0,1,0,0,0,0,0
9,2007,Excessive Sidewall Abrasion,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,1,0,0,0


In [81]:
# DROP THE YTD COLUMNS and the TOTALS row:

# Drop last year's YTD column:
Merged_NRT_df_for_graphing = Merged_NRT_df_for_graphing.drop(columns = [f'YTD {Last_Year}'])
# Now drop this year's YTD column
Merged_NRT_df_for_graphing = Merged_NRT_df_for_graphing.drop(columns = [f'YTD {Current_Year}'])

# Drop the last "TOTAL" row:
Merged_NRT_df_for_graphing = Merged_NRT_df_for_graphing.iloc[:-1, :]
Merged_NRT_df_for_graphing = Merged_NRT_df_for_graphing.reset_index(drop=True)

Merged_NRT_df_for_graphing

,NRT Code,Description,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,2300,Outside of Fleet Specifications,0,0,2,1,1,0,0,0,0,0,0,0,2,1,0,2,1,3,0,0,4,0,0,0
1,2202,Belt Separation (Radial Only),1,3,0,2,1,0,0,0,0,0,0,1,1,1,0,0,1,0,1,2,2,1,0,0
2,2001,Crown Cut(s)/Puncture(s),0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,1,0,1,1,1,2,1,0
3,2205,Shoulder Separation,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,2,1,1,0,1
4,2002,Sidewall Cut(s)/Snag(s),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,1,1,0,0
5,2206,Sidewall Blisters/Bumps,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0
6,2507,Plies Damaged During Buffing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
7,2604,Overall Tire Condition Not Acc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0
8,2102,Worn Too Far,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0
9,2007,Excessive Sidewall Abrasion,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0


In [82]:
# Transpose the Merged_Service_df to make it easier for graphing:
Merged_NRT_df_TRANSPOSED_for_graphing = Merged_NRT_df_for_graphing.T

Description_list_from_second_row = list(Merged_NRT_df_TRANSPOSED_for_graphing.iloc[1])
Merged_NRT_df_TRANSPOSED_for_graphing = Merged_NRT_df_TRANSPOSED_for_graphing.iloc[2:, :]
Merged_NRT_df_TRANSPOSED_for_graphing.columns = Description_list_from_second_row

Merged_NRT_df_TRANSPOSED_for_graphing = Merged_NRT_df_TRANSPOSED_for_graphing.reset_index()
Merged_NRT_df_TRANSPOSED_for_graphing = Merged_NRT_df_TRANSPOSED_for_graphing.rename(columns = {'index': 'month-yr'})

# Also drop "OTHER NRT CODES" because we don't care to graph that one:
Merged_NRT_df_TRANSPOSED_for_graphing = Merged_NRT_df_TRANSPOSED_for_graphing.drop(columns=['OTHER NRT REASONS'])


Merged_NRT_df_TRANSPOSED_for_graphing

,month-yr,Outside of Fleet Specifications,Belt Separation (Radial Only),Crown Cut(s)/Puncture(s),Shoulder Separation,Sidewall Cut(s)/Snag(s),Sidewall Blisters/Bumps,Plies Damaged During Buffing,Overall Tire Condition Not Acc,Worn Too Far,Excessive Sidewall Abrasion,Scrub Induced Belt Separation
0,JAN 2020,0,1,0,0,0,0,0,0,0,0,0
1,FEB 2020,0,3,0,0,0,0,0,0,0,0,0
2,MAR 2020,2,0,0,0,0,0,0,0,0,0,0
3,APR 2020,1,2,0,0,0,0,0,0,0,0,0
4,MAY 2020,1,1,0,1,0,0,0,0,0,0,1
5,JUN 2020,0,0,0,0,0,0,0,0,0,0,0
6,JUL 2020,0,0,0,0,0,0,0,0,0,0,0
7,AUG 2020,0,0,0,0,0,0,0,0,0,0,0
8,SEPT 2020,0,0,0,0,0,0,0,0,0,0,0
9,OCT 2020,0,0,2,1,0,0,0,0,0,0,0


In [83]:
 Column_Names_List = list(Merged_NRT_df_TRANSPOSED_for_graphing.columns)[1:]
 Column_Names_List

['Outside of Fleet Specifications',
 'Belt Separation (Radial Only)',
 'Crown Cut(s)/Puncture(s)',
 'Shoulder Separation',
 'Sidewall Cut(s)/Snag(s)',
 'Sidewall Blisters/Bumps',
 'Plies Damaged During Buffing',
 'Overall Tire Condition Not Acc',
 'Worn Too Far',
 'Excessive Sidewall Abrasion',
 'Scrub Induced Belt Separation']

# Graph for the PDF: 
### (Create the NRT line graph that will be sized (larger) for the PDF) and will only include the top 3 causes of NRT (non-retreadable)):

In [84]:
# Setup independent X Variables (which is just the month and save a list):
Month_LIST = Merged_NRT_df_TRANSPOSED_for_graphing['month-yr'].to_list()


#Setup a dependent Y Variables (By Creating separate lists for each type of service):

first = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[0]}'].to_list()
second = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[1]}'].to_list()
third = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[2]}'].to_list()



fig = go.Figure()


# Note... TO SEE THE TOP 3 WE WILL COMMENT OUT THE OTHER 7.  IF YOU WANT TO SEE A DIFFERENT NUMBER OF CAUSES, HAVE THAT MANY UNCOMMENTED INSTEAD..
#  BUT OF COURSE, YOU WOULD NEED TO ALSO CHANGE THE TITLE TO "TOP _____ CAUSES!!!!!!"

# Create and Style Traces:
fig.add_trace(go.Scatter(x=Month_LIST, y=first, name = f'{Column_Names_List[0]} ',
                         line=dict(color='royalblue', width=2.5),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=second, name = f'{Column_Names_List[1]} ',
                         line=dict(color='orange', width=2.6),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=third, name = f'{Column_Names_List[2]} ',
                         line=dict(color='turquoise', width=2.5),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))



# Edit the Layout:
fig.update_layout(
    title_text=f"<b>TOP 3 NRT CAUSES TREND  {Last_Year} - {Current_Year}</b>", 
    title_x= 0.5,
    title_font_size=30,
    title_font_color="black",
    xaxis_title= "Month-Year",
    yaxis_title= 'NUMBER OF NRT',
    autosize=False, width=1445, height=1100,
    showlegend=True, 
     legend=dict(
        x=0,
        y=-.32,
        title_font_family="Times New Roman",
        font=dict(
            family=None,
            size=29,
            color="black"
        ),
        bgcolor="white",
        bordercolor="black",
        borderwidth=.03
    ),
    # General axes and axes labels fonts:
    font=dict(
        family='Arial',
        size=21,
        color="black"
    ))

fig.add_vline(x=11.5, line_width=6, line_dash="dash", line_color="green")



fig.add_vrect(x0=0, x1=12, 
              annotation_text="<b>LAST YEAR<b>", annotation_position="top left",
              annotation=dict(font_size=26, font_family="Helvetica"),
              fillcolor=None, opacity=0.25, line_width=0)

fig.add_vrect(x0=12, x1=12, 
              annotation_text="<b>CURRENT YEAR<b>", annotation_position="top left",
              annotation=dict(font_size=26, font_family="Helvetica"),
              fillcolor=None, opacity=0.15, line_width=0)


# SAVE this relatively larger size for the PDF.  Also will save it as a png rather than an svg for this graph for the pdf to avoid any weird lines that could occur on the pdf's graph lines.
fig.write_image(f"Images_graphs/NRT_Graph_{Last_Year}-{Current_Year}.png")
fig.show()

# NOW THAT WE HAVE THE NRT GRAPH FOR THE PDF, we'll make a smaller one for the website
# This one will INCLUDE ALL causes for NRT/SCRAP because the website allows us to interact and filter as we please!!

In [85]:
# Setup independent X Variables (which is just the month and save a list):
Month_LIST = Merged_NRT_df_TRANSPOSED_for_graphing['month-yr'].to_list()


#Setup a dependent Y Variables (By Creating separate lists for each type of service):

first = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[0]}'].to_list()
second = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[1]}'].to_list()
third = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[2]}'].to_list()
fourth = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[3]}'].to_list()
fifth = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[4]}'].to_list()
sixth = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[5]}'].to_list()
seventh = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[6]}'].to_list()
eighth = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[7]}'].to_list()
ninth = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[8]}'].to_list()
tenth = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[9]}'].to_list()
eleventh = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[10]}'].to_list()
# twelfth = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[11]}'].to_list()
# thirteenth = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[12]}'].to_list()
# fourteenth = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[13]}'].to_list()
# fifteenth = Merged_NRT_df_TRANSPOSED_for_graphing[f'{Column_Names_List[14]}'].to_list()


fig = go.Figure()


# Note... TO SEE THE TOP 3 WE WILL COMMENT OUT THE OTHER 7.  IF YOU WANT TO SEE A DIFFERENT NUMBER OF CAUSES, HAVE THAT MANY UNCOMMENTED INSTEAD..
#  BUT OF COURSE, YOU WOULD NEED TO ALSO CHANGE THE TITLE TO "TOP _____ CAUSES!!!!!!"

# Create and Style Traces:
fig.add_trace(go.Scatter(x=Month_LIST, y=first, name = f'{Column_Names_List[0]} ',
                         line=dict(color='royalblue', width=2.5),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=second, name = f'{Column_Names_List[1]} ',
                         line=dict(color='orange', width=2.6),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=third, name = f'{Column_Names_List[2]} ',
                         line=dict(color='turquoise', width=2.5),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=fourth, name = f'{Column_Names_List[3]} ',
                         line=dict(color='red', width=2.5),
                         # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=fifth, name = f'{Column_Names_List[4]} ',
                         line=dict(color='black', width=2),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='triangle-up', size=8, line=dict(width=.8))))
fig.add_trace(go.Scatter(x=Month_LIST, y=sixth, name = f'{Column_Names_List[5]} ',
                         line=dict(color='black', width=2.2),
                        # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=seventh, name = f'{Column_Names_List[6]}  ',
                         line=dict(color='#72ced8', width=2.2, dash='dash'),
                        # change the marker style to be so small it's unseen
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=eighth, name = f'{Column_Names_List[7]} ',
                         line=dict(color='brown', width=2.2, dash='dot'),
                         # change the marker style for this one to "triangle-up"
                        marker=dict(symbol='diamond', size=2, line=dict(width=0))))
fig.add_trace(go.Scatter(x=Month_LIST, y=ninth, name = f'{Column_Names_List[8]} ',
                         line=dict(color='darkgreen', width=2.2, dash='dot')))
fig.add_trace(go.Scatter(x=Month_LIST, y=tenth, name = f'{Column_Names_List[9]} ',
                         line=dict(color='royalblue', width=2.2, dash='dot')))
fig.add_trace(go.Scatter(x=Month_LIST, y=eleventh, name = f'{Column_Names_List[10]} ',
                         line=dict(color='lightgreen', width=2),
                         # change the marker style for this one to "star"
                        marker=dict(symbol='triangle-up', size=6, line=dict(width=0))))
# fig.add_trace(go.Scatter(x=Month_LIST, y=twelfth, name = f'{Column_Names_List[11]} ',
#                          line=dict(color='black', width=2),
#                          # change the marker style for this one to "triangle-up"
#                         marker=dict(symbol='triangle-up', size=8, line=dict(width=.8))))
# fig.add_trace(go.Scatter(x=Month_LIST, y=thirteenth, name = f'{Column_Names_List[12]} ',
#                          line=dict(color='purple', width=3.2),
#                          # change the marker style for this one to "triangle-up"
#                         marker=dict(symbol='diamond', size=7, line=dict(width=0))))



# Edit the Layout:
fig.update_layout(
    title_text=f"<b>SCRAP TREND  {Last_Year} - {Current_Year}</b>", 
    title_x= 0.5,
    title_font_size=20,
    title_font_color="black",
    xaxis_title= "Month-Year",
    yaxis_title= 'NUMBER OF NRT',
    autosize=False, width=1100, height=1020,
    showlegend=True, 
     legend=dict(
        x=0,
        y=-.9,
        title_font_family="Times New Roman",
        font=dict(
            family=None,
            size=19,
            color="black"
        ),
        bgcolor="white",
        bordercolor="black",
        borderwidth=.03
    ),
    # General axes and axes labels fonts:
    font=dict(
        family='Arial',
        size=13,
        color="black"
    ))

fig.add_vline(x=11.5, line_width=6, line_dash="dash", line_color="green")

# For aesthetics, force the graph to display the zero point near the bottom IF THERE ARE NO VALUES.  Otherwise, plotly will automatically put the zero point in the middle.
# if (sum(Crown_Cut_List) + sum(Sidewall_Cut_List) + sum(Run_Under_Inflated_List) + sum(Impact_Split_List) + sum(Bead_Damage_Mount_Dismount_List) + sum(Worn_Too_Far_List) + sum(Belt_Separation_Radial_Only_List) + sum(Bead_Separation_Splitting_List) + sum(Shoulder_Separation_List) + sum(Sidewall_Blisters_Or_Bumps_List) + sum(Outside_Of_Fleet_Specifications_List) + sum(Other_NRT_Codes_List)) == 0:
#     fig.layout['yaxis1'] = {'range': [-.5, 5]}


fig.add_vrect(x0=0, x1=12, 
              annotation_text="<b>LAST YEAR<b>", annotation_position="top left",
              annotation=dict(font_size=20, font_family="Helvetica"),
              fillcolor=None, opacity=0.25, line_width=0)

fig.add_vrect(x0=12, x1=12, 
              annotation_text="<b>CURRENT YEAR<b>", annotation_position="top left",
              annotation=dict(font_size=20, font_family="Helvetica"),
              fillcolor=None, opacity=0.15, line_width=0)


# PLOTLY INTERACTIVE:
Scrap_Trend_graph = fig
Scrap_Trend_graph_for_iframe = py.plot(Scrap_Trend_graph, filename = f'{Customer_Name_Underscores}_Scrap_Trend_graph', auto_open=False)


fig.show()




## Build the Tire Sales Summary Table (Shows by each Tire Size) Table page for the website

In [86]:
length_of_list_of_tire_sizes = len(list_of_tire_sizes)
length_of_list_of_tire_sizes

2

In [87]:
list_of_numbers_for_main_categories_for_Sales_by_Size_ALL_VENDORS_table = [
    # Retreads index position:
    0, 
    # Cap+Casing index position (will depend on number of tire sizes)
    2+length_of_list_of_tire_sizes, 
    # Repair Only index position (will depend on number of tire sizes)
    (4+(length_of_list_of_tire_sizes*2)), 
    # New Tires (All Vendors) (Will be directly after the repair only index position)
    5+(length_of_list_of_tire_sizes*2), 
]


list_of_numbers_for_main_categories_for_Sales_by_Size_ALL_VENDORS_table

[0, 4, 8, 9]

In [88]:
# For the html table only, we'll get rid of the name "Category" as the first column since we want that part blank in the final pdf table:
Tire_Sales_Summary_Table_for_website_df = Tire_Sales_Summary_Table_for_website_df.rename(
    columns={'Category': ''})
# If we don't want the "SERVICE" category to have totals, we can replace all the totals with empty strings:
# Tire_Sales_Summary_Table_for_website_df.iloc[0, 1:] = ''



# BUILD THE HTML TABLE for the MAIN TABLE:

def create_html_table(Primary_table):

    # Create headers for HTML
    row_data = '<!-- Start Tire Sales by Size Summary Table --><thead>\n<tr class="header_for_website">'

    # Create columns for HTML:    
    for column in range(Primary_table.shape[1]):
        adjust_header_css = "header_for_website"
        if column in [0,1]:
            adjust_header_css += " make_white_for_website"
        if column not in []:
            adjust_header_css += " sales_by_store_table_header_for_website"
        if column not in []:
            adjust_header_css += " center_for_website"    
        row_data += '\n <th class = "'+adjust_header_css+'">'+Tire_Sales_Summary_Table_for_website_df.columns[column]+'</th>'
    row_data += '\n</tr> \n </head> \n <tbody>'
    
    # Create Body (rows for table) for html
    for row in range(Primary_table.shape[0]):
        if row in list_of_numbers_for_main_categories_for_Sales_by_Size_ALL_VENDORS_table:
            row_data += '\n<tr class = "ROWS_format_category_headers_for_sales_by_store_table_for_website make_bold_for_website">'
        else: row_data += '\n<tr>'
        for column in range(Primary_table.shape[1]):
            td_class_name = 'text_column_for_website'
            if column in [0] and row not in list_of_numbers_for_main_categories_for_Sales_by_Size_ALL_VENDORS_table:
                td_class_name += " right_align_for_website"
            if column not in [0]:
                td_class_name += " center_for_website"   
            row_data += '\n <td class = "'+td_class_name+'">'+str(Primary_table.iloc[row,column])+'</td>'
    row_data += '\n</tr> \n</tbody><!-- End Tire Sales by Size Summary Table -->'
    # -------------------------------------------------------------------------------------------------

    return row_data

# Call the function:
Tire_Sales_Summary_Table_for_website_HTML = create_html_table(Tire_Sales_Summary_Table_for_website_df)

Create_html = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
{string_for_header}
<!-------------------------SALES SUMMARY TABLE PAGE---------------------------------------------->
    <br>
    <br>
    <h1 class=page_title_for_website>{Current_Year} YTD PURCHASES</h1>
    <br>
    <table class = "make_table_smaller_for_website">
        <!-- Start Tire Sales by Size Summary Table -->
        [[DATA_MAIN_TABLE]]
        <!-- End Tire Sales by Size Summary Table -->
    </table>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>

</body>

</html>

"""

# ADD THE SALES TABLE:
# index_exists = os.path.exists('./index_NAV_BAR_files/Sales_Table.html')
# current_index = ''
# with open('./index_NAV_BAR_files/Sales_Table.html', 'r') as f:
#     current_index = f.read()
# with open('./index_NAV_BAR_files/Sales_Table.html', 'w') as f:
#     f.write(re.sub(r"<\!-- Start NRT Table -->.*<\!-- End NRT Table -->",
#             html_table2, current_index, flags=re.DOTALL))

# ADD THE SALES BY TIRE SIZE TABLE:
html_page_exists = os.path.exists('Sales_Table_by_Tire_Size_All_Vendors.html')
current_index = ''
with open('Sales_Table_by_Tire_Size_All_Vendors.html', 'r') as f:
    current_index = f.read()
with open('Sales_Table_by_Tire_Size_All_Vendors.html', 'w') as f:
    f.write(re.sub(r"<\!-- Start Tire Sales by Size Summary Table -->.*<\!-- End Tire Sales by Size Summary Table -->", Tire_Sales_Summary_Table_for_website_HTML, Create_html, flags=re.DOTALL))

print(html_page_exists)

True


# Build Sales by Tire Size's dropdown to "Specified Vendors" html page:

In [89]:
# Create a dynamic list of numbers to be used to generate the table rows formatting.
# The dynamical element will be based on how many tire sizes and vendors were specified at the
# beginning of this python script or notebook.


list_of_numbers_for_main_categories_for_Sales_by_Size_SPECIFIED_VENDORS_table = [
    # First Vendor index position:
    0, 
    # Second Vendor index position (if it exists)
    (5+length_of_list_of_tire_sizes), 
    # Third Vendor index position (if it exists)
    (10+(length_of_list_of_tire_sizes*2)), 
    # fourth Vendor index position (if it exists)
    (15+(length_of_list_of_tire_sizes*3))
    ]


list_of_numbers_for_steer_drive_trailer_for_Sales_by_Size_SPECIFIED_VENDORS_table = [
    # First Vendor (if exists) steer, drive, and trailer rows:
    2+length_of_list_of_tire_sizes, 3+length_of_list_of_tire_sizes, 4+length_of_list_of_tire_sizes, 
    # Second Vendor (if exists) steer, drive, and trailer rows:
    7+(length_of_list_of_tire_sizes*2), 8+(length_of_list_of_tire_sizes*2), 9+(length_of_list_of_tire_sizes*2), 
    # Third Vendor (if exists) steer, drive, and trailer rows:
    12+(length_of_list_of_tire_sizes*3), 13+(length_of_list_of_tire_sizes*3), 14+(length_of_list_of_tire_sizes*3), 
    # Fourth Vendor (if exists) steer, drive, and trailer rows:
    15+(length_of_list_of_tire_sizes*4), 16+(length_of_list_of_tire_sizes*4), 17+(length_of_list_of_tire_sizes*4), 
    ]

list_of_numbers_for_main_categories_for_Sales_by_Size_SPECIFIED_VENDORS_table

[0, 7, 14, 21]

In [90]:
# For the html table only, we'll get rid of the name "Category" as the first column since we want that part blank in the final pdf table:
Tire_Sales_by_Vendor_for_website_df = Tire_Sales_by_Vendor_for_website_df.rename(
    columns={'Category': ''})
# If we don't want the "SERVICE" category to have totals, we can replace all the totals with empty strings:
# Tire_Sales_Summary_Table_for_website_df.iloc[0, 1:] = ''



# BUILD THE HTML TABLE for the MAIN TABLE:

def create_html_table(Primary_table):

    # Create headers for HTML
    row_data = '<!-- Start Tire Sales by Size Summary Table --><thead>\n<tr class="header_for_website">'

    # Create columns for HTML:    
    for column in range(Primary_table.shape[1]):
        adjust_header_css = "header_for_website"
        if column in [0,1]:
            adjust_header_css += " make_white_for_website"
        if column not in []:
            adjust_header_css += " sales_by_store_table_header_for_website"
        if column not in []:
            adjust_header_css += " center_for_website"    
        row_data += '\n <th class = "'+adjust_header_css+'">'+Tire_Sales_by_Vendor_for_website_df.columns[column]+'</th>'
    row_data += '\n</tr> \n </head> \n <tbody>'
    
    # Create Body (rows for table) for html
    for row in range(Primary_table.shape[0]):
        if row in list_of_numbers_for_main_categories_for_Sales_by_Size_SPECIFIED_VENDORS_table:
            row_data += '\n<tr class = "ROWS_format_category_headers_for_sales_by_store_table_for_website make_bold_for_website">'
        else: row_data += '\n<tr>'
        for column in range(Primary_table.shape[1]):
            td_class_name = 'text_column_for_website'
            if column in [0] and row not in list_of_numbers_for_main_categories_for_Sales_by_Size_SPECIFIED_VENDORS_table:
                td_class_name += " right_align_for_website"
            if column not in [0]:
                td_class_name += " center_for_website"
            if column in [0] and row in list_of_numbers_for_steer_drive_trailer_for_Sales_by_Size_SPECIFIED_VENDORS_table:
                td_class_name += " make_bold_for_website left_align_for_website customer_color_2_background"         
            row_data += '\n <td class = "'+td_class_name+'">'+str(Primary_table.iloc[row,column])+'</td>'
    row_data += '\n</tr> \n</tbody><!-- End Tire Sales by Size Summary Table -->'
    # -------------------------------------------------------------------------------------------------

    return row_data

# Call the function:
Sales_by_Size_Specified_Vendors_for_website_HTML = create_html_table(Tire_Sales_by_Vendor_for_website_df)

Create_html = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
{string_for_header}
<!-------------------------SALES SUMMARY TABLE PAGE---------------------------------------------->
    <br>
    <br>
    <h1 class=page_title_for_website>{Current_Year} YTD PURCHASES</h1>
    <br>
    <table class = "make_table_smaller_for_website">
        <!-- Start Tire Sales by Size Summary Table -->
        [[DATA_MAIN_TABLE]]
        <!-- End Tire Sales by Size Summary Table -->
    </table>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>

</body>

</html>

"""

# ADD THE SALES TABLE:
# index_exists = os.path.exists('./index_NAV_BAR_files/Sales_Table.html')
# current_index = ''
# with open('./index_NAV_BAR_files/Sales_Table.html', 'r') as f:
#     current_index = f.read()
# with open('./index_NAV_BAR_files/Sales_Table.html', 'w') as f:
#     f.write(re.sub(r"<\!-- Start NRT Table -->.*<\!-- End NRT Table -->",
#             html_table2, current_index, flags=re.DOTALL))

# ADD THE SALES BY TIRE SIZE TABLE:
html_page_exists = os.path.exists('Sales_Table_by_Tire_Size_by_Vendor.html')
current_index = ''
with open('Sales_Table_by_Tire_Size_by_Vendor.html', 'r') as f:
    current_index = f.read()
with open('Sales_Table_by_Tire_Size_by_Vendor.html', 'w') as f:
    f.write(re.sub(r"<\!-- Start Tire Sales by Size Summary Table -->.*<\!-- End Tire Sales by Size Summary Table -->", Sales_by_Size_Specified_Vendors_for_website_HTML, Create_html, flags=re.DOTALL))

print(html_page_exists)

True


# Build SALES BREAKDOWN percentage graphs page for website:

In [91]:
Create_html_for_sales_breakdown = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
{string_for_header}
   <!----------------------------------------- SALES BREAKDOWN PERCENTAGE COLUMN GRAPHS for web page --------------------------------------->
    <br>
    <h1 class = "page_title_smaller_for_website">PURCHASES BREAKDOWN BY CATEGORY</h1>
    <div class = "sales_percentage_breakdown_bar_for_website">
        <img src="Images_graphs/{Current_Year}_Sales_Percentage_Breakdown.svg">
        <img src="Images_graphs/{Current_Year}_Retreads_Percentage_Breakdown.svg">
    </div>
</body>

</html>

"""

# html_page_exists = os.path.exists('Sales_Table.html')
# current_index = ''
# with open('Sales_Table.html', 'r') as f:
#     current_index = f.read()

with open('Sales_Breakdown_Column_Charts.html', 'w') as new_file:
    new_file.write(Create_html_for_sales_breakdown)

# Build webpage for TREND LINE GRAPHS:

In [92]:
# Here we'll dynamically ensure that we add each vendor's graphs (bar and line graphs) for this website.
# If we didn't do this, every time we copied this into a new notebook to create a new customer report,
# we'd have to manually type in the iframes and img names for each vendor.
# For example, <img src="Images_graphs/{Current_Year}_Bridgestone_Purchase_Analysis_ALL_SIZES_website_size_BAR_GRAPH.svg">
# Then <img src="Images_graphs/{Current_Year}_Yokohama_Purchase_Analysis_ALL_SIZES_website_size_BAR_GRAPH.svg">
# Then whatever next vendor and so on... this just lessens the chance of error and reduces this work if we do this dymanic string.
# After we add to the string, we'll then add it to the "create_html_for_trend_lines" below for the website.

string_for_however_many_vendors_there_are_for_bar_and_plotly_graphs = ''

for index, each in enumerate(vendor_names_list):
        string_for_however_many_vendors_there_are_for_bar_and_plotly_graphs = string_for_however_many_vendors_there_are_for_bar_and_plotly_graphs + f"""<img src="Images_graphs/{Current_Year}_{vendor_names_list[index]}_Purchase_Analysis_ALL_SIZES_website_size_BAR_GRAPH.svg">
        <iframe width="1050px" height="525px" frameborder="0" scrolling="no" src="{Dict_for_plotly_iframes_of_Line_Charts_by_Tire_Sizes[f'{Customer_Name_Underscores}_{vendor_names_list[index]}_Purchases_Trend_graph_for_iframe']}"></iframe>"""

string_for_however_many_vendors_there_are_for_bar_and_plotly_graphs

'<img src="Images_graphs/2021_Bridgestone_Purchase_Analysis_ALL_SIZES_website_size_BAR_GRAPH.svg">\n        <iframe width="1050px" height="525px" frameborder="0" scrolling="no" src="https://plotly.com/~alexanderconerly/124/"></iframe><img src="Images_graphs/2021_Continental_Purchase_Analysis_ALL_SIZES_website_size_BAR_GRAPH.svg">\n        <iframe width="1050px" height="525px" frameborder="0" scrolling="no" src="https://plotly.com/~alexanderconerly/126/"></iframe><img src="Images_graphs/2021_Yokohama_Purchase_Analysis_ALL_SIZES_website_size_BAR_GRAPH.svg">\n        <iframe width="1050px" height="525px" frameborder="0" scrolling="no" src="https://plotly.com/~alexanderconerly/128/"></iframe>'

In [93]:



Create_html_for_trend_lines = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
{string_for_header}
   <!----------------------------------------- SALES TREND LINE GRAPHS for web page --------------------------------------->
    <br>
    <h1 class = "page_title_smaller_for_website">SALES TREND GRAPHS</h1>
    <br>
    <br>
        
        <p class = "instruction_notes_for_graph_interaction">Note:  The line graphs on this page are interactive.  A few common examples include:
            <br>Single-click an item in the legend to exclude that category from view.
            <br> Double-click an item in the legend to view ONLY that category.
            <br>As you hover over the graph, additional functions will appear above the graph (feel free to experiment)
            <br>To reset a graph, double-click the legend (or graph) or just refresh the webpage
        </p>
    <div class = "fit_three_graphs_on_pdf_page_for_website">
        <iframe width="1050px" height="525px" frameborder="0" scrolling="no" src="{Service_Purchases_Trend_graph_for_iframe}"></iframe>
        <img src="Images_graphs/{Current_Year}_Bandag_Purchase_Analysis_ALL_SIZES_pdf_size_BAR_GRAPH.svg">
        <iframe width="1050px" height="525px" frameborder="0" scrolling="no" src="{Bandag_Purchases_Trend_graph_by_Tire_Size_for_iframe}"></iframe>
        {string_for_however_many_vendors_there_are_for_bar_and_plotly_graphs}
    </div>
    <br>
    <br>
    <br>
    <br>
</body>

</html>

"""

# html_page_exists = os.path.exists('Sales_Table.html')
# current_index = ''
# with open('Sales_Table.html', 'r') as f:
#     current_index = f.read()

with open('Trend_Analysis_Line_Charts.html', 'w') as new_file:
    new_file.write(Create_html_for_trend_lines)

# BUILD THE SCRAPS ANALYSIS Web Page:

In [94]:
# For BOTH the website and the PDF, we want to display the current year's NRT/SCRAPS table UNLESS it's currently January...
# If it's January we want to display LAST YEAR's NRT basys data.  So we'll set that code up now:


# Before saving this for display of website or pdf... we'll go ahead and ensure the columns include the year so there's no ambiguity when it's displayed:
Customer_NRT_df_for_website_or_pdf_table.columns = ['NRT Code', 'Description', f'YTD {Current_Year}', f'JAN {Current_Year}', 
                                                    f'FEB {Current_Year}', f'MAR {Current_Year}', f'APR {Current_Year}', f'MAY {Current_Year}', 
                                                    f'JUN {Current_Year}', f'JUL {Current_Year}', f'AUG {Current_Year}', f'SEPT {Current_Year}', 
                                                    f'OCT {Current_Year}', f'NOV {Current_Year}', f'DEC {Current_Year}']



Customer_NRT_df_for_website_or_pdf_table

,NRT Code,Description,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,2300,Outside of Fleet Specifications,13,2,1,0,2,1,3,0,0,4,0,0,0
1,2202,Belt Separation (Radial Only),9,1,1,0,0,1,0,1,2,2,1,0,0
2,2001,Crown Cut(s)/Puncture(s),8,0,0,0,1,1,0,1,1,1,2,1,0
3,2205,Shoulder Separation,8,0,1,0,0,0,1,1,2,1,1,0,1
4,2002,Sidewall Cut(s)/Snag(s),5,0,0,0,0,1,0,0,2,1,1,0,0
5,2206,Sidewall Blisters/Bumps,3,0,0,0,0,1,0,2,0,0,0,0,0
6,2507,Plies Damaged During Buffing,2,0,0,0,0,0,1,0,0,0,1,0,0
7,2604,Overall Tire Condition Not Acc,2,0,0,0,0,0,0,0,0,0,2,0,0
8,2102,Worn Too Far,2,0,0,0,0,1,0,1,0,0,0,0,0
9,2007,Excessive Sidewall Abrasion,2,0,0,0,0,0,0,1,0,1,0,0,0


In [95]:
# BUILD THE HTML TABLE for the NRT TABLE:

def create_html_table(Nrt_table):
    
    # Create headers for HTML
    row_data = '<!-- Start NRT Table --><thead>\n<tr class="NRT_header_for_website">'

    # Create columns for HTML:    
    for column in range(Nrt_table.shape[1]):
        adjust_header_css = "NRT_header_for_website"
        if column not in [2]:
            adjust_header_css += " customer_color_1_background_for_website make_bold_for_website make_text_white_for_website"
        if column in [2]:
            adjust_header_css += " customer_color_2_background_for_website make_bold_for_website make_text_white_for_website"
        if column not in []:
            adjust_header_css += " center_for_website"    
        row_data += '\n <th class = "'+adjust_header_css+'">'+Customer_NRT_df_for_website_or_pdf_table.columns[column]+'</th>'
    row_data += '\n</tr> \n </head> \n <tbody>'
    
    # Create Body (rows for table) for html
    for row in range(Nrt_table.shape[0]):
        if row in [0,1,2,3,4,5,6,7,8,9,10,11,12]:
            row_data += '\n<tr class = "make_bold_for_website">'
        else: row_data += '\n<tr>'
        for column in range(Nrt_table.shape[1]):
            td_class_name = 'NRT_text_column_for_website'
            if column not in []:
                td_class_name += " center_for_website"
            if row in [12] and column not in [2]:
                td_class_name += " customer_color_1_background_for_website make_bold_for_website make_text_white_for_website"
            if column in [2]:
                td_class_name += " customer_color_2_background_for_website make_bold_for_website make_text_white_for_website"
            if column in [2] and row in [12]:
                td_class_name += " customer_color_2_background_for_website make_text_white_for_website"
            row_data += '\n <td class = "'+td_class_name+'">'+str(Nrt_table.iloc[row,column])+'</td>'
    row_data += '\n</tr> \n</tbody><!-- End NRT Table -->'
                # -------------------------------------------------------------------------------------------------

    return row_data

# Call the function:
html_table2 = create_html_table(Customer_NRT_df_for_website_or_pdf_table)

Create_html_for_the_scraps_analysis = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
{string_for_header}
   <!----------------------------------------- SCRAPS TREND TABLE AND GRAPH for web page --------------------------------------->
    <br>
        <h1 class = "page_title_smaller_for_website">BFI SCRAP TREND ANALYSIS</h1>
    <br>
    <br>
        <p class = "instruction_notes_for_graph_interaction">Note:  The line graph on this page is interactive.  A few common examples include:
            <br>Single-click an item in the legend to exclude that category from view.
            <br> Double-click an item in the legend to view ONLY that category.
            <br>As you hover over the graph, additional functions will appear above the graph (feel free to experiment)
            <br>To reset a graph, double-click the legend (or graph) or just refresh the webpage
        </p>
    <br>
    <h2 class=scrap_summary_table_heading_for_website>SCRAP SUMMARY TABLE:<h2>
    <table>
        <!-- Start NRT Table -->
        [[DATA_MAIN_TABLE]]
        <!-- End NRT Table -->
    </table>
    <br>
    <br>
    <br>
    <iframe width="1100px" height="1020px" frameborder="0" scrolling="no" src="{Scrap_Trend_graph_for_iframe}"></iframe>
    <br>
    <br>
    <br>
</body>

</html>

"""

# 
with open('BFI_Scraps_Analysis.html', 'w') as new_file:
    new_file.write(Create_html_for_the_scraps_analysis)

index_exists = os.path.exists('BFI_Scraps_Analysis.html')
current_index = ''
with open('BFI_Scraps_Analysis.html', 'r') as f:
    current_index = f.read()
with open('BFI_Scraps_Analysis.html', 'w') as f:
    f.write(re.sub(r"<\!-- Start NRT Table -->.*<\!-- End NRT Table -->", html_table2, current_index, flags=re.DOTALL))


# CREATE A PAGE THAT WILL HOUSE A LINK TO DOWNLOADS:

### First, we'll need to create the excel workbook from the Dictionary of data we created in the notebook.

#### Recall, each sheet name will come from the keys from the dictionary.  The data will be from the values.

In [96]:
Dict_for_excel_sheets.keys()

dict_keys(['Flat Repairs in Units', 'Flat Repairs in Dollars', 'Tire Rotations in Units', 'Tire Rotations in Dollars', 'Mounts/Mount+Dismounts in Units', 'Mounts/Mount+Dismounts in Dollars', 'Service Call Hours in Units', 'Service Call Hours in Dollars', 'Scraps in Units', 'Scraps in Dollars', 'Retread Units', 'Cap+Casings Units', 'Bridgestone New Tires (Units)', 'Continental New Tires (Units)', 'Yokohama New Tires (Units)', 'Tire Sales Summary Table', 'Tire Sales by Vendor'])

In [97]:
Dict_for_excel_sheets["Mount_Dismounts Units"] = Dict_for_excel_sheets.pop('Mounts/Mount+Dismounts in Units')
Dict_for_excel_sheets["Mount_Dismounts Dollars"] = Dict_for_excel_sheets.pop('Mounts/Mount+Dismounts in Dollars')

In [98]:

Dict_for_excel_sheets["Mount_Dismounts Dollars"]

,,YTD 2020,JAN 2020,FEB 2020,MAR 2020,APR 2020,MAY 2020,JUN 2020,JUL 2020,AUG 2020,SEPT 2020,OCT 2020,NOV 2020,DEC 2020,YTD 2021,JAN 2021,FEB 2021,MAR 2021,APR 2021,MAY 2021,JUN 2021,JUL 2021,AUG 2021,SEPT 2021,OCT 2021,NOV 2021,DEC 2021
0,MOUNT/MOUNT+DISMOUNTS,27198.12,939.18,1269.83,1896.89,2470.47,2234.16,2069.96,2091.79,3442.94,3219.58,2412.68,2617.14,2533.50,34071.18,2305.18,1558.0,2660.0,3874.0,3059.0,3640.0,3734.5,4118.0,3108.0,3508.0,1746.0,760.5
1,SHREVEPORT #324,4410.18,0.00,0.00,42.59,85.18,511.08,553.67,425.90,425.90,851.80,766.62,321.54,425.90,6530.18,481.18,396.0,836.0,803.0,1188.0,264.0,484.0,660.0,484.0,604.0,220.0,110.0
2,MERIDIAN #265,2357.63,15.18,85.18,255.54,468.49,85.18,212.95,170.36,170.36,127.77,255.54,511.08,0.00,4458.00,0.00,220.0,308.0,176.0,704.0,880.0,308.0,528.0,88.0,1044.0,70.0,132.0
3,BIRMINGHAM #211,2122.36,352.00,352.00,154.00,176.00,319.00,42.59,11.00,387.18,0.00,42.59,242.00,44.00,2979.00,0.00,264.0,99.0,77.0,165.0,0.0,929.5,369.0,888.0,105.0,71.0,11.5
4,COLUMBIA #255,1944.39,0.00,85.18,365.18,175.00,0.00,0.00,0.00,35.00,380.36,140.00,438.13,325.54,2651.00,70.00,132.0,308.0,438.0,35.0,228.0,500.0,210.0,335.0,170.0,105.0,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,BRYAN #459,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,KATY #462,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113,ST. GEORGE SC #176,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,FULTON #165,127.77,0.00,0.00,85.18,0.00,0.00,0.00,0.00,0.00,0.00,42.59,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
Dict_for_excel_sheets.keys()

dict_keys(['Flat Repairs in Units', 'Flat Repairs in Dollars', 'Tire Rotations in Units', 'Tire Rotations in Dollars', 'Service Call Hours in Units', 'Service Call Hours in Dollars', 'Scraps in Units', 'Scraps in Dollars', 'Retread Units', 'Cap+Casings Units', 'Bridgestone New Tires (Units)', 'Continental New Tires (Units)', 'Yokohama New Tires (Units)', 'Tire Sales Summary Table', 'Tire Sales by Vendor', 'Mount_Dismounts Units', 'Mount_Dismounts Dollars'])

In [100]:
Dict_for_excel_sheets['Tire Sales Summary Table'] = Dict_for_excel_sheets['Tire Sales Summary Table'].rename(columns = {'Category': ''})
Dict_for_excel_sheets['Tire Sales by Vendor'] = Dict_for_excel_sheets['Tire Sales by Vendor'].rename(columns = {'Category': ''})

## Before creating excel workbook, we'll reorder the dictioanary to make the Tire Sales Summary table first in the dictionary list

In [101]:
desired_order_list = ['Tire Sales Summary Table', 'Tire Sales by Vendor', 
                        'Flat Repairs in Units', 'Flat Repairs in Dollars', 
                        'Tire Rotations in Units', 'Tire Rotations in Dollars', 
                        'Mount_Dismounts Units', 'Mount_Dismounts Dollars',
                        'Service Call Hours in Units', 'Service Call Hours in Dollars', 
                        'Scraps in Units', 'Scraps in Dollars', 
                        'Retread Units', 'Cap+Casings Units']

# Depending on how many vendors we're using, we'll add those at the end dynammically:
for each_vendor in vendor_names_list:
    desired_order_list.append(f'{each_vendor} New Tires (Units)')

Dict_for_excel_sheets_REORDERED = {k: Dict_for_excel_sheets[k] for k in desired_order_list}
Dict_for_excel_sheets_REORDERED.keys()

dict_keys(['Tire Sales Summary Table', 'Tire Sales by Vendor', 'Flat Repairs in Units', 'Flat Repairs in Dollars', 'Tire Rotations in Units', 'Tire Rotations in Dollars', 'Mount_Dismounts Units', 'Mount_Dismounts Dollars', 'Service Call Hours in Units', 'Service Call Hours in Dollars', 'Scraps in Units', 'Scraps in Dollars', 'Retread Units', 'Cap+Casings Units', 'Bridgestone New Tires (Units)', 'Continental New Tires (Units)', 'Yokohama New Tires (Units)'])

## Now, using the dictionary which is now ordered as desired, we will create an excel workbook 
## and then make each key of the dict a sheet name and the corresponding value of the dict the data that will go into that sheet.

In [102]:

# Get the "Current_Date" in the format you want so you can use it in the file name when you export to excel
# Current_Date = datetime.now().strftime("%m-%d-%Y")
# Current_Date

# We'll define an Excel writer object and the target file
Excelwriter = pd.ExcelWriter(f"{Customer_Name_Underscores}_YTD_{Last_Month_Descriptive}_{Current_Year}.xlsx",engine="xlsxwriter")

# We now loop through the dictionary to create sheets where each Key is the sheet name and the Value is the data from the dataframe:
for df_name, df in Dict_for_excel_sheets_REORDERED.items():
    df.to_excel(Excelwriter, sheet_name=df_name, index=False)
    
    # Freeze pane of first row for every sheet
    # Excelwriter.sheets[df_name].freeze_panes(1,0)
    
    # Auto-adjust columns' width for each df in the dictionary
    for column in df:
        column_width = max(df[column].astype(str).map(len).max() + 2, len(column) + 2)
        col_idx = df.columns.get_loc(column)
        Excelwriter.sheets[df_name].set_column(col_idx, col_idx, column_width)

# And finally save the file
Excelwriter.save()
    

### Now that the notebook exists, we can style the sheets of the workbook using openpyxl:

In [103]:

# import openpyxl
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side, numbers

workbook_name = f"{Customer_Name_Underscores}_YTD_{Last_Month_Descriptive}_{Current_Year}.xlsx"

# Specify which excel workbook to a variable "wb"
wb = load_workbook(workbook_name)


# NOTE, for the excel sheets, we can auto-format certain rows using openpyxl.
### This formatting will be similar what we did for the css of the html pages tables.
### However, we'll since python for the html/css pages was zero indexed AND the header index was separated from the
### table rows indexing, we'll need to add 2 to every number in each list variable that we used to create the html/css formatting.

## First, we'll style just the first two sheet's data ('Tire Sales Summary Table' and 'Tire Sales by Vendor')

In [104]:
# So, to format the rows for the main categories, recall we already created a list of numbers as a variable saved
# which was used for the html/css formatting for the main summary table for all vendors earlier.
# So, we can use this list to create a new variable for the list of rows to format in EXCEL.
# We'll just need to add 2 to every number in that list.  We'll save the variable name with "FOR_EXCEL" in the end.
list_of_numbers_for_main_categories_for_Sales_by_Size_ALL_VENDORS_table_FOR_EXCEL = []

for each_number in list_of_numbers_for_main_categories_for_Sales_by_Size_ALL_VENDORS_table:
    each_number = each_number + 2
    list_of_numbers_for_main_categories_for_Sales_by_Size_ALL_VENDORS_table_FOR_EXCEL.append(each_number)

# View the new list variable contents:
list_of_numbers_for_main_categories_for_Sales_by_Size_ALL_VENDORS_table_FOR_EXCEL

[2, 6, 10, 11]

In [105]:

first_sheet = wb['Tire Sales Summary Table']
# Center align the data:
# To apply a style to all the worksheets, let's create a list of all of them first:
# list_of_each_worksheet_name = list(Dict_for_excel_sheets_REORDERED.keys())
# now, we can loop through that list and apply styles:


# Create a variable that will find the number of rows that have data.  

length_of_one_of_the_columns = len(first_sheet['A'])

# Center align the data:
for row in range(2, length_of_one_of_the_columns + 1):
    for col in range(2,50):
        char = get_column_letter(col)
        first_sheet[char + str(row)].alignment = Alignment(horizontal = 'center')

# Right align first column except for rows for categories like "SERVICE" or "BANDAG RETREADS", etc:"
# First get a list of row numbers to right align
list_of_row_numbers_to_right_align = []
for numbers in range(2, length_of_one_of_the_columns+1):
    if numbers not in list_of_numbers_for_main_categories_for_Sales_by_Size_ALL_VENDORS_table_FOR_EXCEL:
        list_of_row_numbers_to_right_align.append(numbers)
# Now right align those rows:
for row in list_of_row_numbers_to_right_align:
    for col in range(1, 2):
        char = get_column_letter(col)
        first_sheet[char + str(row)].alignment = Alignment(horizontal='right')



# For a specific list of rows (for the category headers "RETREADS", "CAP+CASING", etc):
for row in list_of_numbers_for_main_categories_for_Sales_by_Size_ALL_VENDORS_table_FOR_EXCEL:
    # Add "bold" on the first column:
    for col in range(1, 2):
        char = get_column_letter(col)
        first_sheet[char + str(row)].font = Font(bold=True)
    # Add a light blue highlight to all the columns:
    light_blue_fill = PatternFill(start_color='CCE5FF', end_color='CCE5FF', fill_type='solid')
    for col in range(1, 28):
        char = get_column_letter(col)
        # If there is a value in the cell, OR if that value is ZERO, color it light blue!
        if first_sheet[char + str(row)].value or first_sheet[char + str(row)].value == 0:
            first_sheet[char + str(row)].fill = light_blue_fill


# Put a red border around the YTD column of last year
for row in range(2, length_of_one_of_the_columns + 1):
        for col in range(2, 3):
            char = get_column_letter(col)
            side = openpyxl.styles.Side(border_style='medium', color='FF0000')
            border = openpyxl.styles.Border(left=side, right=side)
            first_sheet[char + str(row)].border = border

for row in range(length_of_one_of_the_columns + 1, length_of_one_of_the_columns + 2):
        for col in range(2, 3):
            char = get_column_letter(col)
            side = openpyxl.styles.Side(border_style='medium', color='FF0000')
            top_border = openpyxl.styles.Border(top=side)
            first_sheet[char + str(row)].border = top_border

# Put a red border around the YTD column of current year
for row in range(2, length_of_one_of_the_columns + 1):
        for col in range(15, 16):
            char = get_column_letter(col)
            side = openpyxl.styles.Side(border_style='medium', color='FF0000')
            border = openpyxl.styles.Border(left=side, right=side)
            first_sheet[char + str(row)].border = border

for row in range(length_of_one_of_the_columns + 1, length_of_one_of_the_columns + 2):
        for col in range(15, 16):
            char = get_column_letter(col)
            side = openpyxl.styles.Side(border_style='medium', color='FF0000')
            top_border = openpyxl.styles.Border(top=side)
            first_sheet[char + str(row)].border = top_border




wb.save(workbook_name)
    

## Style JUST the second sheet for Tire Sales by Vendor:

In [106]:
list_of_numbers_for_vendor_names_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL2 = [
    # First Vendor index position:
    2, 
    # Second Vendor index position (if it exists)
    (7+length_of_list_of_tire_sizes), 
    # Third Vendor index position (if it exists)
    (12+(length_of_list_of_tire_sizes*2)), 
    # fourth Vendor index position (if it exists)
    (17+(length_of_list_of_tire_sizes*3))
    ]


list_of_numbers_for_steer_drive_trailer_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL2 = [
    # First Vendor (if exists) steer, drive, and trailer rows:
    4+length_of_list_of_tire_sizes, 5+length_of_list_of_tire_sizes, 6+length_of_list_of_tire_sizes, 
    # Second Vendor (if exists) steer, drive, and trailer rows:
    9+(length_of_list_of_tire_sizes*2), 10+(length_of_list_of_tire_sizes*2), 11+(length_of_list_of_tire_sizes*2), 
    # Third Vendor (if exists) steer, drive, and trailer rows:
    14+(length_of_list_of_tire_sizes*3), 15+(length_of_list_of_tire_sizes*3), 16+(length_of_list_of_tire_sizes*3), 
    # Fourth Vendor (if exists) steer, drive, and trailer rows:
    18+(length_of_list_of_tire_sizes*4), 19+(length_of_list_of_tire_sizes*4), 20+(length_of_list_of_tire_sizes*4), 
    ]


# View either:
print(f'Vendor rows: {list_of_numbers_for_vendor_names_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL2}')
print(f'Steer, Drive, Trailer rows: {list_of_numbers_for_steer_drive_trailer_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL2}')

Vendor rows: [2, 9, 16, 23]
Steer, Drive, Trailer rows: [6, 7, 8, 13, 14, 15, 20, 21, 22, 26, 27, 28]


In [107]:
# Again, to format the excel table here, we can use the previously saved list variables
# to add 2 to every number in that list to create new variables which will be indexed
# according to excel.  Recall, this is because python is 0 indexed AND because we have to 
# count the header as a number for excel, whereas when we generated the html tables earlier
# we were indexing from below the header to format the table data.


# SO, we'll create a new list of numbers for row formatting the vendor totals rows in excel:
list_of_numbers_for_main_categories_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL = []
for each_number in list_of_numbers_for_main_categories_for_Sales_by_Size_SPECIFIED_VENDORS_table:
    each_number = each_number + 2
    list_of_numbers_for_main_categories_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL.append(each_number)


# Do it again for formatting steer, drive, and trailer rows in Excel:
list_of_numbers_for_steer_drive_trailer_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL = []
for each_number in list_of_numbers_for_steer_drive_trailer_for_Sales_by_Size_SPECIFIED_VENDORS_table:
    each_number = each_number + 2
    list_of_numbers_for_steer_drive_trailer_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL.append(each_number)
    # First Vendor (if exists) steer, drive, and trailer rows:


# View either of the new lists we'll use to format excel rows for the "Tire Sales by Vendor" excel sheet:
print(f'Vendor rows: {list_of_numbers_for_main_categories_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL}')
print(f'Steer, Drive, Trailer rows: {list_of_numbers_for_steer_drive_trailer_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL}')

Vendor rows: [2, 9, 16, 23]
Steer, Drive, Trailer rows: [6, 7, 8, 13, 14, 15, 20, 21, 22, 25, 26, 27]


In [108]:
list_of_numbers_for_main_categories_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL

[2, 9, 16, 23]

In [109]:
list_of_row_numbers_to_right_align

[3, 4, 5, 7, 8, 9, 12, 13, 14]

In [110]:
second_sheet = wb['Tire Sales by Vendor']

# Create a variable that will find the number of rows that have data.  
length_of_one_of_the_columns = len(second_sheet['A'])

# Center align the data:
for row in range(2, length_of_one_of_the_columns + 1):
    for col in range(2,28):
        char = get_column_letter(col)
        second_sheet[char + str(row)].alignment = Alignment(horizontal = 'center')

# Right align first column except for rows for specific rows:
list_of_row_numbers_to_right_align = []
for numbers in range(2, length_of_one_of_the_columns+1):
    if numbers not in list_of_numbers_for_main_categories_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL:
        if numbers not in list_of_numbers_for_steer_drive_trailer_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL:
            list_of_row_numbers_to_right_align.append(numbers)
# Now right align those rows:
for row in list_of_row_numbers_to_right_align:
    for col in range(1, 2):
        char = get_column_letter(col)
        second_sheet[char + str(row)].alignment = Alignment(horizontal='right')



# MAKE BOLD AND COLOR BLUE LINE for vendor totals rows:
for row in list_of_numbers_for_main_categories_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL:
    # Add "bold" on the first column:
    for col in range(1, 2):
        char = get_column_letter(col)
        second_sheet[char + str(row)].font = Font(bold=True)
    # Add a light blue highlight to all the columns:
    light_blue_fill = PatternFill(start_color='CCE5FF', end_color='CCE5FF', fill_type='solid')
    for col in range(1, 28):
        char = get_column_letter(col)
        # If there is a value in the cell, OR if that value is ZERO, color it light blue!
        if second_sheet[char + str(row)].value or first_sheet[char + str(row)].value == 0:
            second_sheet[char + str(row)].fill = light_blue_fill



# For rows of the STEER, DRIVE, and TRAILER rows, make them bold:
for row in list_of_numbers_for_steer_drive_trailer_for_Sales_by_Size_SPECIFIED_VENDORS_table_FOR_EXCEL:
    # Make them "bold" on the first column:
    for col in range(1, 2):
        char = get_column_letter(col)
        second_sheet[char + str(row)].font = Font(bold=True)



# Put a red border around the YTD column of last year
for row in range(2, length_of_one_of_the_columns + 1):
        for col in range(2, 3):
            char = get_column_letter(col)
            side = openpyxl.styles.Side(border_style='medium', color='FF0000')
            border = openpyxl.styles.Border(left=side, right=side)
            second_sheet[char + str(row)].border = border

for row in range(length_of_one_of_the_columns + 1, length_of_one_of_the_columns + 2):
        for col in range(2, 3):
            char = get_column_letter(col)
            side = openpyxl.styles.Side(border_style='medium', color='FF0000')
            top_border = openpyxl.styles.Border(top=side)
            second_sheet[char + str(row)].border = top_border

# Put a red border around the YTD column of current year
for row in range(2, length_of_one_of_the_columns + 1):
        for col in range(15, 16):
            char = get_column_letter(col)
            side = openpyxl.styles.Side(border_style='medium', color='FF0000')
            border = openpyxl.styles.Border(left=side, right=side)
            second_sheet[char + str(row)].border = border

for row in range(length_of_one_of_the_columns + 1, length_of_one_of_the_columns + 2):
        for col in range(15, 16):
            char = get_column_letter(col)
            side = openpyxl.styles.Side(border_style='medium', color='FF0000')
            top_border = openpyxl.styles.Border(top=side)
            second_sheet[char + str(row)].border = top_border



wb.save(workbook_name)

## Next, format all the Sales by Stores excel sheets 
### (everything else besides the first 2 sheets)

In [111]:
test = desired_order_list[2:]
test

['Flat Repairs in Units',
 'Flat Repairs in Dollars',
 'Tire Rotations in Units',
 'Tire Rotations in Dollars',
 'Mount_Dismounts Units',
 'Mount_Dismounts Dollars',
 'Service Call Hours in Units',
 'Service Call Hours in Dollars',
 'Scraps in Units',
 'Scraps in Dollars',
 'Retread Units',
 'Cap+Casings Units',
 'Bridgestone New Tires (Units)',
 'Continental New Tires (Units)',
 'Yokohama New Tires (Units)']

In [112]:

list_of_worksheets_for_sales_by_stores = desired_order_list[2:]

for each_sheet in list_of_worksheets_for_sales_by_stores:
    sheet = wb[each_sheet]

    length_of_column_B = len(sheet['B']) + 1

    # Center align the data:
    for row in range(2, length_of_column_B):
        for col in range(2,50):
            char = get_column_letter(col)
            sheet[char + str(row)].alignment = Alignment(horizontal = 'center')
    # -----------------------------------------------------------------
    # Right align first column store names but not the type of category on top"
    for row in range(3, length_of_column_B):
        for col in range(1, 2):
            char = get_column_letter(col)
            sheet[char + str(row)].alignment = Alignment(horizontal='right')
    # -----------------------------------------------------------------
    # Make category header in first column bold:
    for row in [2]:
        for col in range(1, 2):
            char = get_column_letter(col)
            sheet[char + str(row)].font = Font(bold=True)

    # -----------------------------------------------------------------
    # Put a red border around the YTD column of last year
    for row in range(1, length_of_column_B):
            for col in range(2, 3):
                char = get_column_letter(col)
                side = openpyxl.styles.Side(border_style='medium', color='FF0000')
                border = openpyxl.styles.Border(left=side, right=side)
                sheet[char + str(row)].border = border


    for row in range(length_of_column_B, length_of_column_B + 1):
            for col in range(2, 3):
                char = get_column_letter(col)
                side = openpyxl.styles.Side(border_style='medium', color='FF0000')
                top_border = openpyxl.styles.Border(top=side)
                sheet[char + str(row)].border = top_border

    # -------------------------------------------------------------
    # Put a red border around the YTD column of current year
    for row in range(2, length_of_column_B):
            for col in range(15, 16):
                char = get_column_letter(col)
                side = openpyxl.styles.Side(border_style='medium', color='FF0000')
                border = openpyxl.styles.Border(left=side, right=side)
                sheet[char + str(row)].border = border

    for row in range(length_of_column_B, length_of_column_B + 1):
            for col in range(15, 16):
                char = get_column_letter(col)
                side = openpyxl.styles.Side(border_style='medium', color='FF0000')
                top_border = openpyxl.styles.Border(top=side)
                sheet[char + str(row)].border = top_border



    # Now, let's color blue for each category row (Services, Retreads, etc)

    # Save blue line pattern fill variable:
    light_blue_fill = PatternFill(start_color='CCE5FF', end_color='CCE5FF', fill_type='solid')

    # Choose which cells (rows and columns) you want to add formatting unto:
    for row in [2]:
        for col in range(1, 28):
            char = get_column_letter(col)
            # If there is a value in the cell, OR if that value is ZERO, color it light blue!
            if sheet[char + str(row)].value or sheet[char + str(row)].value == 0:
                sheet[char + str(row)].fill = light_blue_fill




wb.save(workbook_name)

## Finally, format all the dollars based sheets to have dollars data types:

In [113]:
list_of_worksheets_needing_currency_data_type_for_excel = ['Flat Repairs in Dollars', 'Tire Rotations in Dollars', 
'Mount_Dismounts Dollars', 'Service Call Hours in Dollars', 'Scraps in Dollars']



for each_sheet in list_of_worksheets_needing_currency_data_type_for_excel:
    sheet = wb[each_sheet]

    length_of_column_B = len(sheet['B']) + 1

    # Center align the data:
    for row in range(2, length_of_column_B):
        for col in range(2,50):
            char = get_column_letter(col)
            sheet[char + str(row)].number_format = '"$"#,##0.00_-'

wb.save(workbook_name)

# Create the Data Downloads Page

In [114]:
Create_html_for_the_data_downloads_page = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">

    <title>Report</title>
</head>

<body class = 'container'>
{string_for_header}
   <!----------------------------------------- Downloads data for web page --------------------------------------->
    <br>
        <h1 class = "page_title_smaller_for_website">DOWNLOADS</h1>
    <br>
    <br>
    <br>
    <form>
        <input class = "download_button_for_website" type="button" value="Download PDF Report" onClick="window.location.href='{Last_Month}_{Customer_Name_Underscores}_YTD_Purchase_and_Scrap_Report.pdf'">
    </form>
    <br>
    <form>
    <input class = "download_button_for_website" type="button" value="Download Excel Data" onClick="window.location.href='{Customer_Name_Underscores}_YTD_{Last_Month_Descriptive}_{Current_Year}.xlsx'">
    </form>
    <br>
    <br>
    <br>
</body>

</html>

"""

# 
with open('Data_Downloads.html', 'w') as file:
    file.write(Create_html_for_the_data_downloads_page)


# Finally, build the PDF by adding to the blank Pdf.html file
### We will then use pdfkit to read the added html code to generate the pdf
### Note: The main.scss file includes a section at the bottom just for pdf class names for formatting tables and graphs to be sized right for pdf aesthetics:

In [115]:
string_for_however_many_vendors_there_are_for_bar_and_plotly_graphs_for_pdf = ''

for index, each in enumerate(vendor_names_list):
        if index < 2:
                string_for_however_many_vendors_there_are_for_bar_and_plotly_graphs_for_pdf = string_for_however_many_vendors_there_are_for_bar_and_plotly_graphs_for_pdf + f"""
                <img src="Images_graphs/{Current_Year}_{vendor_names_list[index]}_Purchase_Analysis_ALL_SIZES_pdf_size_BAR_GRAPH.png">
                <img src="Images_graphs/{Current_Year}_{vendor_names_list[index]}_Purchase_Analysis.png">
                """
        else:
                string_for_however_many_vendors_there_are_for_bar_and_plotly_graphs_for_pdf = string_for_however_many_vendors_there_are_for_bar_and_plotly_graphs_for_pdf + f"""
                <br>
                <br>
                <br>
                <br>
                <br>
                <br>
                <br>
                <br>
                <br>
                <br>
                <br>
                <img src="Images_graphs/{Current_Year}_{vendor_names_list[index]}_Purchase_Analysis_ALL_SIZES_pdf_size_BAR_GRAPH.png">
                <br>
                <br>
                <br>
                <br>
                <br>
                <br>
                <br>
                <br>
                <br>
                <img src="Images_graphs/{Current_Year}_{vendor_names_list[index]}_Purchase_Analysis.png">
                """

string_for_however_many_vendors_there_are_for_bar_and_plotly_graphs_for_pdf

'\n                <img src="Images_graphs/2021_Bridgestone_Purchase_Analysis_ALL_SIZES_pdf_size_BAR_GRAPH.png">\n                <img src="Images_graphs/2021_Bridgestone_Purchase_Analysis.png">\n                \n                <img src="Images_graphs/2021_Continental_Purchase_Analysis_ALL_SIZES_pdf_size_BAR_GRAPH.png">\n                <img src="Images_graphs/2021_Continental_Purchase_Analysis.png">\n                \n                <br>\n                <br>\n                <br>\n                <br>\n                <br>\n                <br>\n                <br>\n                <br>\n                <br>\n                <br>\n                <br>\n                <img src="Images_graphs/2021_Yokohama_Purchase_Analysis_ALL_SIZES_pdf_size_BAR_GRAPH.png">\n                <br>\n                <br>\n                <br>\n                <br>\n                <br>\n                <br>\n                <br>\n                <br>\n                <br>\n                <img src="I

In [116]:
list_of_numbers_for_main_categories_for_Sales_by_Size_ALL_VENDORS_table = [
    # Retreads index position:
    0, 
    # Cap+Casing index position (will depend on number of tire sizes)
    2+length_of_list_of_tire_sizes, 
    # Repair Only index position (will depend on number of tire sizes)
    (4+(length_of_list_of_tire_sizes*2)), 
    # New Tires (All Vendors) (Will be directly after the repair only index position)
    5+(length_of_list_of_tire_sizes*2)
]


# Create a dynamic list of numbers to be used to generate the table rows formatting.
# The dynamical element will be based on how many tire sizes and vendors were specified at the
# beginning of this python script or notebook.
list_of_numbers_for_main_categories_for_Sales_by_Size_SPECIFIED_VENDORS_table = [
    # First Vendor index position:
    0, 
    # Second Vendor index position (if it exists)
    (5+length_of_list_of_tire_sizes), 
    # Third Vendor index position (if it exists)
    (10+(length_of_list_of_tire_sizes*2)), 
    # fourth Vendor index position (if it exists)
    (15+(length_of_list_of_tire_sizes*3))
    ]


list_of_numbers_for_steer_drive_trailer_for_Sales_by_Size_SPECIFIED_VENDORS_table = [
    # First Vendor (if exists) steer, drive, and trailer rows:
    2+length_of_list_of_tire_sizes, 3+length_of_list_of_tire_sizes, 4+length_of_list_of_tire_sizes, 
    # Second Vendor (if exists) steer, drive, and trailer rows:
    7+(length_of_list_of_tire_sizes*2), 8+(length_of_list_of_tire_sizes*2), 9+(length_of_list_of_tire_sizes*2), 
    # Third Vendor (if exists) steer, drive, and trailer rows:
    12+(length_of_list_of_tire_sizes*3), 13+(length_of_list_of_tire_sizes*3), 14+(length_of_list_of_tire_sizes*3), 
    # Fourth Vendor (if exists) steer, drive, and trailer rows:
    15+(length_of_list_of_tire_sizes*4), 16+(length_of_list_of_tire_sizes*4), 17+(length_of_list_of_tire_sizes*4)
]

In [117]:
string_for_vendors_and_bandag_logo_display_on_home_page_of_PDF = ''

if len(vendor_names_list) <= 1:

    string_for_vendors_and_bandag_logo_display_on_home_page_of_PDF = string_for_vendors_and_bandag_logo_display_on_home_page_of_PDF + f"""
            <div class = "vendors_and_bandag_logo_container_for_pdf">
                <img class= "{vendor_names_list[0]}_logo_for_pdf" src= "./Images_logos/{vendor_names_list[0]}_logo.png">
                <img class= "bandag_logo_for_pdf_IF-ODD-NUMBER-OF-VENDORS" src= "./Images_logos/Bandag_logo.svg">
            </div>
            """
elif len(vendor_names_list) == 2:
        string_for_vendors_and_bandag_logo_display_on_home_page_of_PDF = string_for_vendors_and_bandag_logo_display_on_home_page_of_PDF + f"""
    <div class = "vendors_and_bandag_logo_container_for_pdf">
    <img class= "{vendor_names_list[0]}_logo_for_pdf" src= "./Images_logos/{vendor_names_list[0]}_logo.png">
    <img class= "{vendor_names_list[1]}_logo_for_pdf" src= "./Images_logos/{vendor_names_list[1]}_logo.png">
    </div>
    <div class = "vendors_and_bandag_logo_container_for_pdf">
    <img class= "bandag_logo_for_pdf" src= "./Images_logos/Bandag_logo.svg">
    </div>
    """
elif len(vendor_names_list) == 3:
        string_for_vendors_and_bandag_logo_display_on_home_page_of_PDF = string_for_vendors_and_bandag_logo_display_on_home_page_of_PDF + f"""
    <div class = "vendors_and_bandag_logo_container_for_pdf">
    <img class= "{vendor_names_list[0]}_logo_for_pdf" src= "./Images_logos/{vendor_names_list[0]}_logo.png">
    <img class= "{vendor_names_list[1]}_logo_for_pdf" src= "./Images_logos/{vendor_names_list[1]}_logo.png">
    </div>
    <div class = "vendors_and_bandag_logo_container_for_pdf">
    <img class= "{vendor_names_list[2]}_logo_for_pdf" src= "./Images_logos/{vendor_names_list[2]}_logo.png">
    <img class= "bandag_logo_for_pdf" src= "./Images_logos/Bandag_logo.svg">
    </div>
    """





In [134]:
# Build the Summary Table formatting for the PDF:
# BUILD THE HTML TABLE for the MAIN TABLE:

def create_html_table(Primary_table):

    # Create headers for HTML
    row_data = '<!-- Start Sales Table --><thead>\n<tr class="header_for_pdf">'

    # Create columns for HTML:    
    for column in range(Primary_table.shape[1]):
        adjust_header_css = "header_for_pdf"
        if column in [0]:
            adjust_header_css += " make_white_for_pdf"
        if column not in []:
            adjust_header_css += " sales_by_store_table_header_for_pdf"
        if column not in []:
            adjust_header_css += " center_for_pdf"    
        row_data += '\n <th class = "'+adjust_header_css+'">'+Tire_Sales_Summary_Table_for_website_df.columns[column]+'</th>'
    row_data += '\n</tr> \n </head> \n <tbody>'
    
    # Create Body (rows for table) for html
    for row in range(Primary_table.shape[0]):
        if row in list_of_numbers_for_main_categories_for_Sales_by_Size_ALL_VENDORS_table:
            row_data += '\n<tr class = "ROWS_format_category_headers_for_sales_by_store_table_for_pdf make_bold_for_pdf">'
        else: row_data += '\n<tr>'
        for column in range(Primary_table.shape[1]):
            td_class_name = 'text_column_for_pdf'
            if column in [0] and row not in list_of_numbers_for_main_categories_for_Sales_by_Size_ALL_VENDORS_table:
                td_class_name += " right_align_for_pdf"
            if column not in [0]:
                td_class_name += " center_for_pdf"
            if column in [0] and row in list_of_numbers_for_main_categories_for_Sales_by_Size_ALL_VENDORS_table:
                td_class_name += " left_align_for_pdf customer_color_2_background_for_pdf"        
            row_data += '\n <td class = "'+td_class_name+'">'+str(Primary_table.iloc[row,column])+'</td>'
    row_data += '\n</tr> \n</tbody><!-- End Sales Table -->'
    # -------------------------------------------------------------------------------------------------

    return row_data

# Call the function:
pdf_summary_table = create_html_table(Tire_Sales_Summary_Table_for_website_df)



#----------Sales by TIre SIze BY VENDOR----------------------------
def create_html_table(table):

    # Create headers for HTML
    row_data = '<!-- Start Sales_by_Size_by_Vendor Table --><thead>\n<tr class="header_for_website">'

    # Create columns for HTML:    
    for column in range(table.shape[1]):
        adjust_header_css = "header_for_pdf"
        if column in [0,1]:
            adjust_header_css += " make_white_for_pdf"
        if column not in []:
            adjust_header_css += " sales_by_store_table_header_for_pdf"
        if column not in []:
            adjust_header_css += " center_for_pdf"    
        row_data += '\n <th class = "'+adjust_header_css+'">'+Tire_Sales_by_Vendor_for_website_df.columns[column]+'</th>'
    row_data += '\n</tr> \n </head> \n <tbody>'
    
    # Create Body (rows for table) for html
    for row in range(table.shape[0]):
        if row in list_of_numbers_for_main_categories_for_Sales_by_Size_SPECIFIED_VENDORS_table:
            row_data += '\n<tr class = "ROWS_format_category_headers_for_sales_by_store_table_for_pdf make_bold_for_pdf">'
        else: row_data += '\n<tr>'
        for column in range(table.shape[1]):
            td_class_name = 'text_column_for_pdf'
            if column in [0] and row not in list_of_numbers_for_main_categories_for_Sales_by_Size_SPECIFIED_VENDORS_table:
                td_class_name += " right_align_for_pdf"
            if column not in [0]:
                td_class_name += " center_for_pdf"
            # Format Steer, Drive, and Trailer
            if column in [0] and row in list_of_numbers_for_steer_drive_trailer_for_Sales_by_Size_SPECIFIED_VENDORS_table:
                td_class_name += " make_bold_for_pdf left_align_for_pdf customer_color_2_background"         
            row_data += '\n <td class = "'+td_class_name+'">'+str(table.iloc[row,column])+'</td>'
    row_data += '\n</tr> \n</tbody><!-- End Sales_by_Size_by_Vendor Table -->'
    # -------------------------------------------------------------------------------------------------

    return row_data

# Call the function:
Sales_by_Size_Specified_Vendors_for_pdf_HTML = create_html_table(Tire_Sales_by_Vendor_for_website_df)



# -----------------NRT TABLE for pdf -----------------------------------------------------------------------
# NOW build the NRT Table formatting for the PDF
def create_html_table(Nrt_table):
    
    # Create headers for HTML
    row_data = '<!-- Start NRT Table --><thead>\n<tr class="NRT_header_for_pdf">'

    # Create columns for HTML:    
    for column in range(Nrt_table.shape[1]):
        adjust_header_css = ""
        if column in [2]:
            adjust_header_css += " customer_color_2_background_for_pdf make_text_white_for_pdf make_bold_for_pdf"
        if column not in [2]:
            adjust_header_css += " make_bold_for_pdf make_text_white_for_pdf customer_color_1_background_for_pdf"
        if column not in []:
            adjust_header_css += " size_of_nrt_header center_for_pdf"
        row_data += '\n <th class = "'+adjust_header_css+'">'+Customer_NRT_df_for_website_or_pdf_table.columns[column]+'</th>'
    row_data += '\n</tr> \n </head> \n <tbody>'
    
    # Create Body (rows for table) for html
    for row in range(Nrt_table.shape[0]):
        if row in [0,1,2,3,4,5,6,7,8,9,10,11,12]:
            row_data += '\n<tr class = "make_bold_for_pdf">'
        else: row_data += '\n<tr>'
        for column in range(Nrt_table.shape[1]):
            td_class_name = 'NRT_text_column_for_pdf'
            if column not in []:
                td_class_name += " center_for_pdf"
            if column in [2]:
                td_class_name += " customer_color_2_background_for_pdf make_bold_for_pdf make_text_white_for_pdf"
            if column in [2] and row in [12]:
                td_class_name += " customer_color_2_background_for_pdf make_text_white_for_pdf make_text_white_for_pdf"
            if column not in [2] and row in [12]:
                td_class_name += " make_bold_for_pdf customer_color_1_background_for_pdf make_text_white_for_pdf"
            row_data += '\n <td class = "'+td_class_name+'">'+str(Nrt_table.iloc[row,column])+'</td>'
    row_data += '\n</tr> \n</tbody><!-- End NRT Table -->'
                # -------------------------------------------------------------------------------------------------

    return row_data

# Call the function:
pdf_nrt_table = create_html_table(Customer_NRT_df_for_website_or_pdf_table)


Create_html = f"""

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="X-UA-Compatible" content="ie=edge" />
    <!--Bootstrap CSS using CDN-->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" />
    <link rel="stylesheet" href="css/main.css">
    <title>Report</title>
</head>

<body>
    <!--------------------------------------INTRO PDF PAGE----------------------------------------------------------------->
    <!-- STM LOGO -->
    <br>
    <br>
    <div class="STM_logo_for_pdf">
        <img src=".\Images_logos\STM_logo.svg">
    </div>
    <br>
    <br>
    <br>
    <!--YEAR TO DATE BANNER text-->
    <h1 class="first_banner_text_for_pdf">YEAR TO DATE - {Last_Month_Descriptive_Short} {Current_Year}</h1>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <!-- Customer LOGO -->
    <div class="Customer_logo_image_for_pdf">
        <img src='./Images_logos/{Customer_Logo_Image_Name_for_pdf}'>
    </div>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <!--  Second Banner Text -->
    <h1 class="second_banner_text_for_pdf">PURCHASE AND SCRAP PROFILE</h1>
    <br>
{string_for_vendors_and_bandag_logo_display_on_home_page_of_PDF}
    </div> 
    <!-----------------------------PURCHASES TABLES pdf pages--------------------------------------------------------------->
    <br>
    <br>
    <br>
    <br>
    <br>
    <h1 class=page_title_smaller_for_pdf>{Current_Year} YTD SALES BY TIRE SIZE</h1>
    <br>
    <table class = "make_table_smaller_for_pdf">
        <!-- Start Tire Sales by Size Summary Table -->
        [[DATA_TABLE]]
        <!-- End Tire Sales by Size Summary Table -->
    </table>
    <br>
    <br>
    <h1 class=page_title_smaller_for_pdf>{Current_Year} YTD SALES BY TIRE SIZE PER VENDOR</h1>
    <br>
        <table class = "make_table_smaller_for_website">
        <!-- Start Sales_by_Size_by_Vendor Table -->
        [[DATA_TABLE]]
        <!-- End Sales_by_Size_by_Vendor Table -->
        </table>
    <br>
    <!----------------------------------------- SALES BREAKDOWN PERCENTAGE COLUMN GRAPHS pdf page --------------------------------------->
    <br>
    <div class = "sales_percentage_breakdown_bar_for_pdf">
        <img src="Images_graphs/{Current_Year}_Sales_Percentage_Breakdown.png">
        <br>
        <br>
        <br>
        <br>
        <br>
        <br>
        <img src="Images_graphs/{Current_Year}_Retreads_Percentage_Breakdown.png">
    </div>
    <br>
    <!----------------------------------------- TREND LINE GRAPHS pdf page --------------------------------------------------------------------->
    <div class = "fit_three_graphs_on_pdf_page">
        <img src="Images_graphs/{Current_Year}_Service_Purchase_Analysis.png">
        <img src="Images_graphs/{Current_Year}_Bandag_Purchase_Analysis_by_Tire_Size.png">
        {string_for_however_many_vendors_there_are_for_bar_and_plotly_graphs_for_pdf}
    </div>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <!--------------------------- SCRAPS ANALYSIS pdf page ---------------------------------------------------------------------------------------->
    <br>
    <br>
    <h1 class=page_title_smaller_for_pdf>{Current_Year} YTD BFI SCRAP ANALYSIS</h1>
    <br>
    <br>
    <table>
        <!-- Start NRT Table -->
        [[DATA_TABLE]]
        <!-- End NRT Table -->
    </table>
    <br>
    <br>
    <br>
    <div class = "NRT_year_to_year_line_graph_for_pdf">
        <img src='Images_graphs/NRT_Graph_{Last_Year}-{Current_Year}.png'>
    </div>

</body>

</html>

"""
# ------------------------------------------------------------------------------------------------------------------------------
# ADD THE SALES by Size All Vendors TABLE:
with open('pdf.html', 'w') as f:
    f.write(re.sub(r"<\!-- Start Tire Sales by Size Summary Table -->.*<\!-- End Tire Sales by Size Summary Table -->", pdf_summary_table, Create_html, flags=re.DOTALL))


# ADD SALES by Size per Specified Vendor
if os.path.exists('pdf.html') == True:
    current_index = ''
    with open('pdf.html', 'r') as f:
        current_index = f.read()
    with open('pdf.html', 'w') as f:
        f.write(re.sub(r"<\!-- Start Sales_by_Size_by_Vendor Table -->.*<\!-- End Sales_by_Size_by_Vendor Table -->", Sales_by_Size_Specified_Vendors_for_pdf_HTML, current_index, flags=re.DOTALL))



# with open('pdf.html', 'w') as f:
#     f.write(re.sub(r"<\!-- Start Sales_by_Size_by_Vendor Table -->.*<\!-- End Sales_by_Size_by_Vendor Table -->", Sales_by_Size_Specified_Vendors_for_pdf_HTML, Create_html, flags=re.DOTALL))

# Then add NRT TABLE:
if os.path.exists('pdf.html') == True:
    current_index2 = ' '
    with open('pdf.html', 'r') as f:
        current_index2 = f.read()
    with open('pdf.html', 'w') as f:
        f.write(re.sub(r"<\!-- Start NRT Table -->.*<\!-- End NRT Table -->", pdf_nrt_table, current_index2, flags=re.DOTALL))




# !pip3 install pdfkit
import pdfkit
options = {"enable-local-file-access": None}

# Save the a pdf with a name according to whether it's the current year or a previous year AND whether it's for all districts of a region or if it's for a subregion of desert mountain:

# ------------ If running the report for the current year ----------------------:
pdfkit.from_file("pdf.html", f"{Last_Month}_{Customer_Name_Underscores}_YTD_Purchase_and_Scrap_Report.pdf", options=options)

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      


True